# Library

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pyexasol
import scipy.interpolate as intp
import time
import datetime as dt
# 날짜 코드
## 기준연월 설정
## 연도 설정
## 날짜 설정
import psycopg2

In [2]:
start_time = time.time()

# server

In [3]:
## insider db
# wd = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
# we = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSE')
# # ws = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='SYS')

In [4]:
# exasol db
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
ws = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYS')
wz = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSZ')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSE')

# ---------- Load ----------

## 등록정보(STD_CEG_GRD_LST) 4등급만

In [5]:
## insider db
# edb_id = 'vsysd'
# edb_database = 'edb'
# edb_port = 5444
# edb_url = '172.29.135.50'
# edb_pwd = 'vsyswynn'
# conn = psycopg2.connect(dbname=edb_database, user=edb_id, password=edb_pwd, host=edb_url, port=edb_port)
# cur = conn.cursor()

# sql = "SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO, VHRNO, EXTGAS_INSP_VLD_YMD, VHCL_OWNR_CL_CD, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY, VHCTY_CL_CD, TOTL_WGHT, CRYNG_WGHT, DSPLVL, EGIN_OTPT FROM STD_CEG_GRD_LST WHERE EXHST_GAS_GRD_CD = 'A0504' AND CLS_YMD = (SELECT MAX(CLS_YMD) FROM STD_CEG_GRD_LST)"
# cur.execute(sql)
# cs_raw = pd.DataFrame(cur.fetchall())
# cs_raw.columns = [desc[0].upper() for desc in cur.description]

## 30.1s
## exasol db
cs_raw = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO, VHRNO, EXTGAS_INSP_VLD_YMD, VHCL_OWNR_CL_CD, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY, VHCTY_CL_CD, TOTL_WGHT, CRYNG_WGHT, DSPLVL, EGIN_OTPT FROM VSYSZ.STD_CEG_GRD_LST WHERE EXHST_GAS_GRD_CD = 'A0504' AND CLS_YMD = SELECT MAX(CLS_YMD) FROM VSYSZ.STD_CEG_GRD_LST;")
cs_raw_dict = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자', 
    'VHCL_MNG_NO':'차량관리번호', 
    'VHRNO':'자동차등록번호',
    'EXTGAS_INSP_VLD_YMD':'검사유효일',
    'VHCL_OWNR_CL_CD':'소유자구분',
    
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
    'VHCTY_CL_CD':'차종분류',
    'TOTL_WGHT':'총중량',
    'CRYNG_WGHT':'적재중량',
    'DSPLVL':'배기량', 
    'EGIN_OTPT':'엔진출력',
}
cs = cs_raw.rename(columns=cs_raw_dict)
print(cs.columns)
print(cs.shape)
print('data load : VSYSZ.STD_CEG_GRD_LST')

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '차량관리번호', '자동차등록번호', '검사유효일', '소유자구분', '연료',
       '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '차종분류', '총중량', '적재중량', '배기량',
       '엔진출력'],
      dtype='object')
(1924021, 26)
data load : VSYSZ.STD_CEG_GRD_LST


In [6]:
cs.isnull().sum()

차대번호        0
법정동코드       0
배출가스등급      0
배출가스인증번호    0
차량말소YN      0
제원관리번호      0
차량연식        0
차종          0
용도          0
최초등록일자      0
제작일자        0
차량관리번호      0
자동차등록번호     0
검사유효일       0
소유자구분       0
연료          0
차종유형        0
제작사명        0
차명          0
자동차형식       0
엔진형식        0
차종분류        0
총중량         0
적재중량        0
배기량         0
엔진출력        0
dtype: int64

## 등록정보(STD_CEG_GRD_LST) 5등급만

In [7]:
## 37.7s

# edb_id = 'vsysd'
# edb_database = 'edb'
# edb_port = 5444
# edb_url = '172.29.135.50'
# edb_pwd = 'vsyswynn'
# conn = psycopg2.connect(dbname=edb_database, user=edb_id, password=edb_pwd, host=edb_url, port=edb_port)
# cur = conn.cursor()

# sql = "SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO, VHRNO, EXTGAS_INSP_VLD_YMD, VHCL_OWNR_CL_CD, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY, VHCTY_CL_CD, TOTL_WGHT, CRYNG_WGHT, DSPLVL, EGIN_OTPT FROM STD_CEG_GRD_LST WHERE EXHST_GAS_GRD_CD = 'A0505' AND CLS_YMD = (SELECT MAX(CLS_YMD) FROM STD_CEG_GRD_LST)"
# cur.execute(sql)
# cs5_raw = pd.DataFrame(cur.fetchall())
# cs5_raw.columns = [desc[0].upper() for desc in cur.description]

cs5_raw = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO, VHRNO, EXTGAS_INSP_VLD_YMD, VHCL_OWNR_CL_CD, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY, VHCTY_CL_CD, TOTL_WGHT, CRYNG_WGHT, DSPLVL, EGIN_OTPT FROM VSYSZ.STD_CEG_GRD_LST WHERE EXHST_GAS_GRD_CD = 'A0505' AND CLS_YMD = (SELECT MAX(CLS_YMD) FROM VSYSZ.STD_CEG_GRD_LST);")
cs5_raw_dict = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자', 
    'VHCL_MNG_NO':'차량관리번호', 
    'VHRNO':'자동차등록번호',
    'EXTGAS_INSP_VLD_YMD':'검사유효일',
    'VHCL_OWNR_CL_CD':'소유자구분',
    
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
    'VHCTY_CL_CD':'차종분류',
    'TOTL_WGHT':'총중량',
    'CRYNG_WGHT':'적재중량',
    'DSPLVL':'배기량', 
    'EGIN_OTPT':'엔진출력',
}
cs5 = cs5_raw.rename(columns=cs5_raw_dict)
print(cs5.columns)
print(cs5.shape)
print('data load : VSYSZ.STD_CEG_GRD_LST')

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '차량관리번호', '자동차등록번호', '검사유효일', '소유자구분', '연료',
       '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '차종분류', '총중량', '적재중량', '배기량',
       '엔진출력'],
      dtype='object')
(2378360, 26)
data load : VSYSZ.STD_CEG_GRD_LST


In [8]:
cs5.isnull().sum()

차대번호        0
법정동코드       0
배출가스등급      0
배출가스인증번호    0
차량말소YN      0
제원관리번호      0
차량연식        0
차종          0
용도          0
최초등록일자      0
제작일자        0
차량관리번호      0
자동차등록번호     0
검사유효일       0
소유자구분       0
연료          0
차종유형        0
제작사명        0
차명          0
자동차형식       0
엔진형식        0
차종분류        0
총중량         0
적재중량        0
배기량         0
엔진출력        0
dtype: int64

## 등록정보(STD_CEG_CAR_MIG) 4등급만

In [9]:
## 8.6s
# car = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO, VHRNO, EXTGAS_INSP_VLD_YMD, VHCL_OWNR_CL_CD FROM STD_CEG_CAR_MIG WHERE EXHST_GAS_GRD_CD = 'A0504' OR EXHST_GAS_GRD_CD = 'A05T4';")
# car_ch_col = {
#     'VIN':'차대번호', 
#     'BSPL_STDG_CD':'법정동코드', 
#     'EXHST_GAS_GRD_CD':'배출가스등급', 
#     'EXHST_GAS_CERT_NO':'배출가스인증번호',
#     'VHCL_ERSR_YN':'차량말소YN',
#     'MANP_MNG_NO':'제원관리번호', 
#     'YRIDNW':'차량연식', 
#     'VHCTY_CD':'차종', 
#     'PURPS_CD2':'용도', 
#     'FRST_REG_YMD':'최초등록일자',
#     'VHCL_FBCTN_YMD':'제작일자', 
#     'VHCL_MNG_NO':'차량관리번호', 
#     'VHRNO':'자동차등록번호',
#     'EXTGAS_INSP_VLD_YMD':'검사유효일',
#     'VHCL_OWNR_CL_CD':'소유자구분', 
    
# }
# carr = car.rename(columns=car_ch_col)
# print(carr.shape)
# print('data load : STD_CEG_CAR_MIG')

## 제원정보(STD_CEG_CAR_SRC_MIG)

In [10]:
## 1m 56.8s
src = wd.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY, VHCTY_CL_CD, TOTL_WGHT, CRYNG_WGHT, DSPLVL, EGIN_OTPT FROM VSYSZ.STD_CEG_CAR_SRC_MIG;")
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
    'VHCTY_CL_CD':'차종분류',
    'TOTL_WGHT':'총중량',
    'CRYNG_WGHT':'적재중량',
    'DSPLVL':'배기량', 
    'EGIN_OTPT':'엔진출력',
}
srcr = src.rename(columns=src_ch_col)

print(srcr.columns)
print(srcr.shape)
print('data load : VSYSZ.STD_CEG_CAR_SRC_MIG')

c:\Users\kbjung\anaconda3\Lib\site-packages\pyexasol\callback.py:42: DtypeWarning: Columns (0,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(pipe, skip_blank_lines=False, **kwargs)


Index(['제원관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '차종분류', '총중량',
       '적재중량', '배기량', '엔진출력'],
      dtype='object')
(40076471, 12)
data load : VSYSZ.STD_CEG_CAR_SRC_MIG


## 정기검사(STD_TB_JGT_HIS)

In [11]:
## 4m 2.2s
# jgt = wz.export_to_pandas("SELECT VIN, FDRM_INSP_INSP_MTHD_CD, FDRM_INSP_KND_CD, FDRM_INSP_JGMT, FDRM_NLOD_SMO_MSTVL1, FDRM_NLOD_SMO_MSTVL2, FDRM_NLOD_SMO_MSTVL3, FDRM_NLOD_SMO_JT_YN1, FDRM_INSP_YMD, FDRM_DRVNG_DSTNC, FDRM_NLOD_SMO_PRMVL1 FROM STD_TB_JGT_HIS WHERE ROWNUM <= 10000;") # 테스트용
jgt = wd.export_to_pandas("SELECT VIN, FDRM_INSP_INSP_MTHD_CD, FDRM_INSP_KND_CD, FDRM_INSP_JGMT, FDRM_NLOD_SMO_MSTVL1, FDRM_NLOD_SMO_MSTVL2, FDRM_NLOD_SMO_MSTVL3, FDRM_NLOD_SMO_JT_YN1, FDRM_INSP_YMD, FDRM_DRVNG_DSTNC, FDRM_NLOD_SMO_PRMVL1 FROM VSYSZ.STD_TB_JGT_HIS;")
jgt_ch_col = {
    'VIN':'차대번호', 
    'FDRM_INSP_INSP_MTHD_CD':'검사방법', 
    'FDRM_INSP_KND_CD':'검사종류', 
    'FDRM_INSP_JGMT':'검사판정', 
    'FDRM_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'FDRM_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'FDRM_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'FDRM_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    'FDRM_INSP_YMD':'검사일자',
    'FDRM_DRVNG_DSTNC':'주행거리',
    'FDRM_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
}
jgtr = jgt.rename(columns=jgt_ch_col)

print(jgtr.columns)
print(jgtr.shape)
print('data load : STD_CEG_CAR_SRC_MIG')

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')
(67890730, 11)
data load : STD_CEG_CAR_SRC_MIG


## 정밀검사(STD_TB_EET_HIS_ME)

In [12]:
## 8m 15.8s
eet = wd.export_to_pandas("SELECT VIN, PRCINSP_MSRMT_MTHD_CD, PRCINSP_KND_CD, PRCINSP_JGMT, PREC_NLOD_SMO_MSTVL1, PREC_NLOD_SMO_MSTVL2, PREC_NLOD_SMO_MSTVL3, PREC_NLOD_SMO_JT_YN1, PRCINSP_YMD, PRCINSP_DRVNG_DSTNC, PREC_NLOD_SMO_PRMVL1 FROM STD_TB_EET_HIS_ME;")
eet_ch_col = {
    'VIN':'차대번호', 
    'PRCINSP_MSRMT_MTHD_CD':'검사방법', 
    'PRCINSP_KND_CD':'검사종류', 
    'PRCINSP_JGMT':'검사판정', 
    'PREC_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'PREC_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'PREC_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'PREC_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    'PRCINSP_YMD':'검사일자',
    'PRCINSP_DRVNG_DSTNC':'주행거리', 
    'PREC_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
}
eetr = eet.rename(columns=eet_ch_col)

print(eetr.columns)
print(eetr.shape)
print('data load : STD_TB_EET_HIS_ME')

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')
(106262877, 11)
data load : STD_TB_EET_HIS_ME


## 법정동코드(STD_BJCD_INFO)

In [13]:
## 1.3s
code = wd.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM, STDG_CTPV_CD, STDG_SGG_CD FROM STD_BJCD_INFO;")
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구', 
    'STDG_CTPV_CD':'시도코드',
    'STDG_SGG_CD':'시군구코드', # 오타 수정 요청 : SSG -> SGG(2023.07.13 어니컴 VSYSD에서 발견)
}
coder = code.rename(columns=code_ch_col)

print(coder.columns)
print(coder.shape)
print('data load : STD_BJCD_INFO')

Index(['법정동코드', '시도', '시군구', '시도코드', '시군구코드'], dtype='object')
(47916, 5)
data load : STD_BJCD_INFO


## 노후차 조기폐차 관리정보(수도권)(STD_DLM_TB_ERP_EARLY_ERASE_AEA)

In [14]:
## 4.5s
aea = wd.export_to_pandas("SELECT VIN, ELPDSRC_STTS_CD, ELPDSRC_LST_APRV_YN, ERSR_YMD FROM STD_DLM_TB_ERP_EARLY_ERASE_AEA;")
aea_ch_col = {
    'VIN':'차대번호', 
    'ELPDSRC_STTS_CD':'조기폐차상태코드', 
    'ELPDSRC_LST_APRV_YN':'조기폐차최종승인YN', 
    'ERSR_YMD':'말소일자', 
}
aear = aea.rename(columns=aea_ch_col)

print(aear.columns)
print(aear.shape)
print('data load : STD_DLM_TB_ERP_EARLY_ERASE_AEA')

Index(['차대번호', '조기폐차상태코드', '조기폐차최종승인YN', '말소일자'], dtype='object')
(829443, 4)
data load : STD_DLM_TB_ERP_EARLY_ERASE_AEA


## 노후차 조기폐차 관리정보(수도권외)(STD_DLM_TB_ERP_EARLY_ERASE_LGV)

In [15]:
## 3.8s
lgv = wd.export_to_pandas("SELECT VIN, ELPDSRC_STTS_CD, ELPDSRC_LST_APRV_YN, ERSR_YMD FROM STD_DLM_TB_ERP_EARLY_ERASE_LGV;")
lgv_ch_col = {
    'VIN':'차대번호', 
    'ELPDSRC_STTS_CD':'조기폐차상태코드', 
    'ELPDSRC_LST_APRV_YN':'조기폐차최종승인YN', 
    'ERSR_YMD':'말소일자',  
}
lgvr = lgv.rename(columns=lgv_ch_col)

print(lgvr.columns)
print(lgvr.shape)
print('data load : STD_DLM_TB_ERP_EARLY_ERASE_LGV')

Index(['차대번호', '조기폐차상태코드', '조기폐차최종승인YN', '말소일자'], dtype='object')
(431853, 4)
data load : STD_DLM_TB_ERP_EARLY_ERASE_LGV


## 저감장치 부착이력(STD_DLM_TB_ERP_ATT_HIS)

In [16]:
## 4.4s
att = wd.export_to_pandas("SELECT VIN, RDCDVC_SE_CD, RDCDVC_KND_CD FROM STD_DLM_TB_ERP_ATT_HIS;")
att_ch_col = {
    'VIN':'차대번호', 
    'RDCDVC_SE_CD':'저감장치구분',
    'RDCDVC_KND_CD':'저감장치종류',
}
attr = att.rename(columns=att_ch_col)

print(attr.columns)
print(attr.shape)
print('data load : STD_DLM_TB_ERP_ATT_HIS')

Index(['차대번호', '저감장치구분', '저감장치종류'], dtype='object')
(1027190, 3)
data load : STD_DLM_TB_ERP_ATT_HIS


### ----- 2024.02.05 수정 (데이터 테이블 변경)
`CEG_CAR_HISTORY_MIG => ODS_CEG_CAR_HISTORY_MIG`

## 등록이력(ODS_CEG_CAR_HISTORY_MIG)

In [17]:
## 3m 53.0s

# edb_id = 'vsysd'
# edb_database = 'edb'
# edb_port = 5444
# edb_url = '172.29.135.50'
# edb_pwd = 'vsyswynn'
# conn = psycopg2.connect(dbname=edb_database, user=edb_id, password=edb_pwd, host=edb_url, port=edb_port)
# cur = conn.cursor()
# sql = 'select VHCL_ERSR_YN, CHNG_DE, VHMNO from vsysd.ods_ceg_car_history_mig'
# cur.execute(sql)
# his = pd.DataFrame(cur.fetchall())
# his.columns = [desc[0].upper() for desc in cur.description]
# his_ch_col = {
#     'VHCL_ERSR_YN':'차량말소YN', 
#     'CHNG_DE':'변경일자',
#     'VHMNO':'차량관리번호'
# }
# hisr = his.rename(columns=his_ch_col)

his = wd.export_to_pandas("SELECT VHCL_ERSR_YN, CHNG_DE, VHMNO FROM ODS_CEG_CAR_HISTORY_MIG;")
his_ch_col = {
    'VHCL_ERSR_YN':'차량말소YN', 
    'CHNG_DE':'변경일자',
    'VHMNO':'차량관리번호', 
}
hisr = his.rename(columns=his_ch_col)

print(hisr.columns)
print(hisr.shape)
print('data load : ODS_CEG_CAR_HISTORY_MIG')

Index(['차량말소YN', '변경일자', '차량관리번호'], dtype='object')
(75825296, 3)
data load : ODS_CEG_CAR_HISTORY_MIG


In [18]:
hisr.shape

(75825296, 3)

## 비상시 및 계절제 단속발령(N_IS_ISSUE_DISCLOSURE)

In [19]:
## 9m 52.1s
isdis = wd.export_to_pandas("SELECT REGLT_NO, GNFD_NO, VIN, REG_SIDO_CD, REG_SIGNGU_CD, REGLT_AREA_CD FROM N_IS_ISSUE_DISCLOSURE;")
isdis_ch_col = {
    'REGLT_NO':'적발번호', 
    'GNFD_NO':'발령번호', 
    'REG_SIDO_CD':'등록시도코드', 
    'REG_SIGNGU_CD':'등록시군구코드', 
    'VIN':'차대번호',
    'REGLT_AREA_CD':'적발지역코드', 
}
isdisr = isdis.rename(columns=isdis_ch_col)

print(isdisr.columns)
print(isdisr.shape)
print('data load : STD_N_IS_ISSUE_DISCLOSURE')

Index(['적발번호', '발령번호', '차대번호', '등록시도코드', '등록시군구코드', '적발지역코드'], dtype='object')
(117981152, 6)
data load : STD_N_IS_ISSUE_DISCLOSURE


## 운행제한 발령정보(N_IS_ISSUE)

In [20]:
## 1.5s

# sql = "select GNFD_NO, TY_STDR_ID, DNSTY_STDR_ID from vsysd.n_is_issue"
# cur.execute(sql)
# isis = pd.DataFrame(cur.fetchall())

# isis.columns = [desc[0].upper() for desc in cur.description]
# isis_ch_col = {
#     'GNFD_NO':'발령번호', 
#     'DNSTY_STDR_ID':'농도기준아이디', 
#     'TY_STDR_ID':'유형기준아이디', 
# }
# isisr = isis.rename(columns=isis_ch_col)

isis = wd.export_to_pandas("SELECT GNFD_NO, TY_STDR_ID, DNSTY_STDR_ID FROM N_IS_ISSUE;")
isis_ch_col = {
    'GNFD_NO':'발령번호', 
    'DNSTY_STDR_ID':'농도기준아이디', 
    'TY_STDR_ID':'유형기준아이디', 
}
isisr = isis.rename(columns=isis_ch_col)

print(isisr.columns)
print(isisr.shape)
print('data load : STD_N_IS_ISSUE')

Index(['발령번호', '유형기준아이디', '농도기준아이디'], dtype='object')
(2560, 3)
data load : STD_N_IS_ISSUE


## N_IS_PENALTY

In [21]:
## 12.9s 

# sql = "select REGLT_NO, REGLT_DE from vsysd.n_is_penalty"
# cur.execute(sql)
# ispe = pd.DataFrame(cur.fetchall())
# ispe.columns = [desc[0].upper() for desc in cur.description]
# ispe_ch_col = {
#     'REGLT_NO':'적발번호', 
#     'REGLT_DE':'단속일', 
# }
# isper = ispe.rename(columns=ispe_ch_col)

ispe = wd.export_to_pandas("SELECT REGLT_NO, REGLT_DE FROM N_IS_PENALTY;")
ispe_ch_col = {
    'REGLT_NO':'적발번호', 
    'REGLT_DE':'단속일', 
}
isper = ispe.rename(columns=ispe_ch_col)

print(isper.columns)
print(isper.shape)
print('data load : N_IS_PENALTY')

Index(['적발번호', '단속일'], dtype='object')
(12074356, 2)
data load : N_IS_PENALTY


## 운행제한 단속정보(N_US_DISCLOSURE)

In [22]:
## 10.7s

# sql = 'select "NO", VIN, DISCL_TY, REGLT_AREA_CD, REG_SIDO_CD, REG_SIGNGU_CD from vsysd.n_us_disclosure'
# cur.execute(sql)
# usdis = pd.DataFrame(cur.fetchall())
# usdis.columns = [desc[0].upper() for desc in cur.description]
# usdis_ch_dict = {
#     'NO':'번호', 
#     'VIN':'차대번호', 
#     'REG_SIDO_CD':'등록시도코드', 
#     'REG_SIGNGU_CD':'등록시군구코드', 
#     'DISCL_TY':'적발유형', 
#     'REGLT_AREA_CD':'단속지역코드', 
# }
# usdisr = usdis.rename(columns=usdis_ch_dict)

usdis = wd.export_to_pandas('SELECT "NO", VIN, DISCL_TY, REGLT_AREA_CD, REG_SIDO_CD, REG_SIGNGU_CD FROM N_US_DISCLOSURE;')
usdis_ch_dict = {
    'NO':'번호', 
    'VIN':'차대번호', 
    'REG_SIDO_CD':'등록시도코드', 
    'REG_SIGNGU_CD':'등록시군구코드', 
    'DISCL_TY':'적발유형', 
    'REGLT_AREA_CD':'단속지역코드', 
}
usdisr = usdis.rename(columns=usdis_ch_dict)

print(usdisr.columns)
print(usdisr.shape)
print('data load : N_US_DISCLOSURE')

Index(['번호', '차대번호', '적발유형', '단속지역코드', '등록시도코드', '등록시군구코드'], dtype='object')
(3178626, 6)
data load : N_US_DISCLOSURE


## N_US_PENALTY

In [23]:
## 3.3s
# sql = 'select "NO", REGLT_CNT, REGLT_YM FROM from vsysd.n_us_penalty'
# cur.execute(sql)
# uspe = pd.DataFrame(cur.fetchall())
# uspe.columns = [desc[0].upper() for desc in cur.description]
# uspe_ch_dict = {
#     'NO':'번호', 
#     'REGLT_CNT':'적발건수', 
#     'REGLT_YM':'적발년월'
# }
# usper = uspe.rename(columns=uspe_ch_dict)
# cur.close()
# conn.close()

uspe = wd.export_to_pandas('SELECT "NO", REGLT_CNT, REGLT_YM FROM N_US_PENALTY;')
uspe_ch_dict = {
    'NO':'번호', 
    'REGLT_CNT':'적발건수', 
    'REGLT_YM':'적발년월'
}
usper = uspe.rename(columns=uspe_ch_dict)

print(usper.columns)
print(usper.shape)
print('data load : STD_N_US_PENALTY')

Index(['번호', '적발건수', '적발년월'], dtype='object')
(448747, 3)
data load : STD_N_US_PENALTY


## RH에서 제공한 법정동코드

In [24]:
## 1.5s
rh = we.export_to_pandas("SELECT DONG_CODE, CTPRVN_NM, SIGNGU_NM FROM STD_BD_TB_MAPDATA;")
rh = rh.rename(columns={
    'DONG_CODE':'법정동코드_rh', 
    'CTPRVN_NM':'시도', 
    'SIGNGU_NM':'시군구'
    })

print(rh.columns)
print(rh.shape)
print('data load : STD_BD_TB_MAPDATA')

Index(['법정동코드_rh', '시도', '시군구'], dtype='object')
(3799, 3)
data load : STD_BD_TB_MAPDATA


## 4등급 result(for DPF유무)

In [25]:
## 20s
rs = we.export_to_pandas("SELECT 차대번호, DPF유무_수정 FROM STD_BD_GRD4_RESULT;")

print(rs.columns)
print(rs.shape)
print('data load : STD_BD_GRD4_RESULT')

Index(['차대번호', 'DPF유무_수정'], dtype='object')
(1502235, 2)
data load : STD_BD_GRD4_RESULT


## STD_BD_KOSIS

In [26]:
## 1.5s
kosis = we.export_to_pandas("SELECT CTPV, SGG, VHCTY_CD, DY_AVRG_DRVNG_DSTNC FROM STD_BD_KOSIS;")
kosis_ch_col = {
    'CTPV':'시도', 
    'SGG':'시군구', 
    'VHCTY_CD':'차종', 
    'DY_AVRG_DRVNG_DSTNC':'일일평균주행거리', 
}
kosisr = kosis.rename(columns=kosis_ch_col)

print(kosisr.columns)
print(kosisr.shape)
print('data load : STD_BD_KOSIS')

Index(['시도', '시군구', '차종', '일일평균주행거리'], dtype='object')
(1230, 4)
data load : STD_BD_KOSIS


## 운행제한 건수 데이터

In [27]:
## 3.0s
# lmt = we.export_to_pandas("SELECT * FROM STD_BD_GRD5_LMT_NOCS;")
# lmt['운행제한건수'] = lmt[['계절제_1차', '계절제_2차', '계절제_3차', '계절제_4차', '비상시', '상시']].sum(axis=1)

# print('data load : STD_BD_GRD5_LMT_NOCS')

# ---------- 전처리 ----------

## 등록정보

In [28]:
## 중복 차대번호 제거
# carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
# cs = cs.sort_values(['최초등록일자', '차량말소YN'], ascending=[False, False]).drop_duplicates('차대번호').reset_index(drop=True) # !!! 수정(2023.12.22)

In [29]:
cs.shape

(1924021, 26)

In [30]:
## 5.8s
## 배출가스등급 코드 변환
print(cs['배출가스등급'].unique())
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
cs['배출가스등급'] = cs['배출가스등급'].replace(grd_dict)
print(cs['배출가스등급'].unique())
print()

## 차종 코드 변환
print(cs['차종'].unique())
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
cs['차종'] = cs['차종'].replace(cd_dict)
print(cs['차종'].unique())
print()

## 용도 코드 변환
print(cs['용도'].unique())
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
cs['용도'] = cs['용도'].replace(purps_dict)
print(cs['용도'].unique())
print()

## 소유자구분 코드 변환
print(cs['소유자구분'].unique())
ownr_dict = {
    'A27B':'사업자', 
    'A27F':'외국인', 
    'A27L':'법인', 
    'A27O':'기타', 
    'A27R':'주민', 
}
cs['소유자구분'] = cs['소유자구분'].replace(ownr_dict)
print(cs['소유자구분'].unique())
print()

## 연료 코드 변환
print(cs['연료'].unique())
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
cs['연료'] = cs['연료'].replace(fuel_dict)
print(cs['연료'].unique())

## 차종유형 코드 변환
print(cs['차종유형'].unique())
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
cs['차종유형'] = cs['차종유형'].replace(ty_dict)
print(cs['차종유형'].unique())

['A0504']


['4']

['A31P' 'A31V' 'A31T' 'A31S']
['승용' '승합' '화물' '특수']

['A08P' 'A08B' 'A08O']
['개인용' '영업용' '관용']

['A27R' 'A27B' 'A27F' 'A27L' 'A27O']
['주민' '사업자' '외국인' '법인' '기타']

['A90GS' 'A91DS' 'A92LP' 'A94OT' 'A92CN' 'A90AC']
['휘발유' '경유' 'LPG(액화석유가스)' '기타연료' 'CNG(압축천연가스)' '알코올']
['A30S' 'A30M' 'A30L' 'A30C']
['소형' '중형' '대형' '경형']


In [31]:
cs['최초등록일자'].head()

0    1992-03-14 00:00:00.000000
1    2010-04-26 00:00:00.000000
2    1996-12-27 00:00:00.000000
3    2008-01-29 00:00:00.000000
4    2007-11-15 00:00:00.000000
Name: 최초등록일자, dtype: object

In [32]:
## 7.0s
## 최초제작일자 타입 변경
cs['최초등록일자'] = cs['최초등록일자'].astype('str')
cs['최초등록일자'] = cs['최초등록일자'].str.split(' ').str[0]
cs['최초등록일자'] = cs['최초등록일자'].str.replace('-', '')
cs['최초등록일자'] = pd.to_numeric(cs['최초등록일자'], errors='coerce')

In [33]:
cs['최초등록일자'].head()

0    19920314
1    20100426
2    19961227
3    20080129
4    20071115
Name: 최초등록일자, dtype: int64

In [34]:
cs[['제작일자', '차량연식']].dtypes

제작일자    object
차량연식     int64
dtype: object

In [35]:
cs['제작일자'].head()

0    1111-11-11 00:00:00.000000
1    2010-04-19 00:00:00.000000
2    1111-11-11 00:00:00.000000
3    2008-01-22 00:00:00.000000
4    2007-10-24 00:00:00.000000
Name: 제작일자, dtype: object

In [36]:
cs['제작일자'] = pd.to_datetime(cs['제작일자'], errors='coerce')
cs['제작일자'] = cs['제작일자'].astype('str')
cs['제작일자'] = cs['제작일자'].str.replace('-', '')
cs['제작일자'] = pd.to_numeric(cs['제작일자'], errors='coerce')
cs['제작일자'].head()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_2144\380824528.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cs['제작일자'] = pd.to_datetime(cs['제작일자'], errors='coerce')


0           NaN
1    20100419.0
2           NaN
3    20080122.0
4    20071024.0
Name: 제작일자, dtype: float64

In [37]:
## 등록정보 말소 제거
csm = cs[cs['차량말소YN'] == 'N'].reset_index(drop=True)
csm.shape

(1318161, 26)

In [38]:
## 6.8s
## 배출가스등급 코드 변환
print(cs5['배출가스등급'].unique())
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
cs5['배출가스등급'] = cs5['배출가스등급'].replace(grd_dict)
print(cs5['배출가스등급'].unique())
print()

## 차종 코드 변환
print(cs5['차종'].unique())
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
cs5['차종'] = cs5['차종'].replace(cd_dict)
print(cs5['차종'].unique())
print()

## 용도 코드 변환
print(cs5['용도'].unique())
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
cs5['용도'] = cs5['용도'].replace(purps_dict)
print(cs5['용도'].unique())
print()

## 소유자구분 코드 변환
print(cs5['소유자구분'].unique())
ownr_dict = {
    'A27B':'사업자', 
    'A27F':'외국인', 
    'A27L':'법인', 
    'A27O':'기타', 
    'A27R':'주민', 
}
cs5['소유자구분'] = cs5['소유자구분'].replace(ownr_dict)
print(cs5['소유자구분'].unique())
print()

## 연료 코드 변환
print(cs5['연료'].unique())
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
cs5['연료'] = cs5['연료'].replace(fuel_dict)
print(cs5['연료'].unique())

## 차종유형 코드 변환
print(cs5['차종유형'].unique())
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
cs5['차종유형'] = cs5['차종유형'].replace(ty_dict)
print(cs5['차종유형'].unique())

['A0505']
['5']

['A31T' 'A31P' 'A31V' 'A31S']
['화물' '승용' '승합' '특수']

['A08P' 'A08B' 'A08O']
['개인용' '영업용' '관용']

['A27R' 'A27L' 'A27F' 'A27B' 'A27O']
['주민' '법인' '외국인' '사업자' '기타']

['A91DS' 'A90GS' 'A92LP' 'A94OT' 'A92CN' 'A90AC']
['경유' '휘발유' 'LPG(액화석유가스)' '기타연료' 'CNG(압축천연가스)' '알코올']
['A30L' 'A30M' 'A30S' 'A30C']
['대형' '중형' '소형' '경형']


In [39]:
cs5['최초등록일자'].head()

0    2001-07-28 00:00:00.000000
1    2006-10-02 00:00:00.000000
2    2004-12-14 00:00:00.000000
3    2001-03-21 00:00:00.000000
4    1989-12-11 00:00:00.000000
Name: 최초등록일자, dtype: object

In [40]:
## 8.2s
## 최초제작일자 타입 변경
cs5['최초등록일자'] = cs5['최초등록일자'].astype('str')
cs5['최초등록일자'] = cs5['최초등록일자'].str.split(' ').str[0]
cs5['최초등록일자'] = cs5['최초등록일자'].str.replace('-', '')
cs5['최초등록일자'] = pd.to_numeric(cs5['최초등록일자'], errors='coerce')

In [41]:
cs5['최초등록일자'].head()

0    20010728
1    20061002
2    20041214
3    20010321
4    19891211
Name: 최초등록일자, dtype: int64

In [42]:
cs5['제작일자'].head()

0    2001-07-23 00:00:00.000000
1    2006-06-21 00:00:00.000000
2    2004-12-04 00:00:00.000000
3    2001-02-22 00:00:00.000000
4    1111-11-11 00:00:00.000000
Name: 제작일자, dtype: object

In [43]:
cs5['제작일자'] = pd.to_datetime(cs5['제작일자'], errors='coerce')
cs5['제작일자'] = cs5['제작일자'].astype('str')
cs5['제작일자'] = cs5['제작일자'].str.replace('-', '')
cs5['제작일자'] = pd.to_numeric(cs5['제작일자'], errors='coerce')
cs5['제작일자'].head()

0    20010723.0
1    20060621.0
2    20041204.0
3    20010222.0
4           NaN
Name: 제작일자, dtype: float64

In [44]:
## 3.7s
## 등록정보 말소 제거
cs5m = cs5[cs5['차량말소YN'] == 'N'].reset_index(drop=True)
cs5m.shape

(906192, 26)

## 제원정보

In [45]:
srcr.shape

(40076471, 12)

In [46]:
## 43.2s
## 연료 코드 변환
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)

## 차종유형 코드 변환
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)

## 등록&제원 병합

In [47]:
## 0.7s
# cs = carm.merge(srcr, on='제원관리번호', how='left')

## 정기&정밀 병합

### ----- 2024.02.05 수정 시작 (데이터 타입 변경)

`이거때문에 운영계에서는 차후 3-1에 문제 발생`

In [48]:
## 2m 48.4s
ins = pd.concat([jgtr, eetr], ignore_index=True)

In [49]:
ins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174153607 entries, 0 to 174153606
Data columns (total 11 columns):
 #   Column     Dtype  
---  ------     -----  
 0   차대번호       object 
 1   검사방법       object 
 2   검사종류       object 
 3   검사판정       object 
 4   무부하매연측정치1  float64
 5   무부하매연측정치2  float64
 6   무부하매연측정치3  float64
 7   무부하매연판정1   object 
 8   검사일자       object 
 9   주행거리       float64
 10  무부하매연허용치1  float64
dtypes: float64(5), object(6)
memory usage: 14.3+ GB


In [50]:
## 1m 51.5s
ins['차대번호'] = ins['차대번호'].astype(str)

In [51]:
## 6m 20.7s
ins['검사일자'] = pd.to_datetime(ins['검사일자'], errors='coerce')

In [52]:
print(ins['차대번호'].dtype)
print(ins['검사일자'].dtype)

object
datetime64[ns]


In [53]:
## 4m 26.3s
## 최근 검사만 활용
insm = ins.sort_values('검사일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)

### ----- 2024.02.05 수정 완료 (데이터 타입 변경)

In [54]:
## 47.5s
## 검사종류 코드 변환
knd_dict = {
    'A21P01':'정밀검사', 
    'A1904':'정기검사', 
    'A21T06':'종합검사(경과)', 
    'A21T05':'종합검사', 
    'A21S01':'특정검사', 
    'A1902':'재검사(2부재검사)',
    'A21R02':'2부재검사(종합)', 
    'A21P02':'정밀검사(경과)', 
    'A1903':'정기(경과)검사', 
    'A21S04':'특정재검사(배출)', 
    'A21E01':'배출재검사', 
    'A21R01':'1부재검사(종합)',
    'A21C02':'구조변경검사', 
    'A21002':'관능재검사(임시)', 
    'A21S03':'특정재검사(관능)', 
    'A21S02':'특정검사(경과)', 
    'A1901':'재검사(1부재검사)', 
    'A21T02':'종합(정밀)',
    'A21E02':'배출재검사(구변)', 
    'A21T01':'종합(경과:정밀)', 
    'A21T04':'종합(정밀) 2부재검사', 
    'A21001':'관능재검사', 
    'A21T03':'종합(정밀) 1부재검사', 
    'A21TMP':'임시검사', 
}
insm['검사종류'] = insm['검사종류'].replace(knd_dict)

## 검사방법 코드 변환
# 14.9s
mth_dict = {
    'A18A':'무부하검사(TSI)', 
    'A18B':'무부하검사(급가속)', 
    'A18C':'무부하검사(정지가동)',
    'A2301':'무부하검사(급가속)', 
    'A2302':'무부하검사(정지가동)',
    'A2303':'무부하검사(TSI)', 
    'A2304':'부하검사(LUG DOWN)', 
    'A2305':'부하검사(ASM-Idling)', 
    'A2306':'부하검사(KD-147)', 
    'A2307':'부하검사(ASM2525)', 
}
insm['검사방법'] = insm['검사방법'].replace(mth_dict)

## 저감장치구분 코드 변환

In [55]:
## 2.5s
rdcdvc_dict = {
    'A1001':'1종', 
    'A1002':'2종', 
    'A1003':'3종', 
    'A1004':'1종+SCR', 
    'A1005':'엔진개조', 
    'A1006':'엔진교체',
    'A1007':'삼원촉매',
}
attr['저감장치구분'] = attr['저감장치구분'].replace(rdcdvc_dict)

## 저감장치 부착 유무
attr.loc[(attr['저감장치구분'] == '1종') | (attr['저감장치구분'] == '1종+SCR'), 'DPF_YN'] = '유'

# 중복 차대번호 제거
attr = attr.sort_values('DPF_YN').drop_duplicates('차대번호').reset_index(drop=True)
attr.columns

Index(['차대번호', '저감장치구분', '저감장치종류', 'DPF_YN'], dtype='object')

## 조기폐차(수도권, 수도권 외)

In [56]:
aear['조기폐차상태코드'].unique()

array(['A32B', 'A32Y', 'A32A', 'A32M', 'A32D', 'A32X', 'A32C', 'A32N',
       'A32G'], dtype=object)

In [57]:
## 11.5s
# erase_dict = {
#     'A32E':'조기폐차상태코드(추가보조금신청대상)',
#     'A32G':'조기폐차상태코드(보조금청구)',
#     'A32I':'조기폐차상태코드(신청등록)',
#     'A32K':'조기폐차상태코드(추가보조금청구승인)',
#     'A32M':'조기폐차상태코드(보조금산정)',
#     'A32N':'조기폐차상태코드(보조금청구반려(제외))',
#     'A32P':'조기폐차상태코드(보조금대상)',
#     'A32T':'조기폐차상태코드(추가보조금청구)',
#     'A32X':'조기폐차상태코드(신청취소(제외))',
#     'A32Y':'조기폐차상태코드(보조금청구승인)',
#     'A32C':'조기폐차상태코드(성능확인검사등록)',
#     'A32D':'조기폐차상태코드(기간초과)',
#     'A32A':'조기폐차상태코드(성능확인검사신청)',
#     'A32B':'조기폐차상태코드(보조금미대상)',
# }
erase_dict = {
    'A32E':'E',
    'A32G':'G',
    'A32I':'I',
    'A32K':'K',
    'A32M':'M',
    'A32N':'N',
    'A32P':'P',
    'A32T':'T',
    'A32X':'X',
    'A32Y':'Y',
    'A32C':'C',
    'A32D':'D',
    'A32A':'A',
    'A32B':'B',
}
aear['조기폐차상태코드'] = aear['조기폐차상태코드'].replace(erase_dict)
aear['조기폐차상태코드'].unique()

array(['B', 'Y', 'A', 'M', 'D', 'X', 'C', 'N', 'G'], dtype=object)

In [58]:
lgvr['조기폐차상태코드'].unique()

array(['A32Y', 'A32M', 'A32C', 'A32X', 'A32G', 'A32I', 'A32P', 'A32B',
       'A32D', 'A32N', nan], dtype=object)

In [59]:
lgvr['조기폐차상태코드'] = lgvr['조기폐차상태코드'].replace(erase_dict)
lgvr['조기폐차상태코드'].unique()

array(['Y', 'M', 'C', 'X', 'G', 'I', 'P', 'B', 'D', 'N', nan],
      dtype=object)

In [60]:
set(lgvr['조기폐차상태코드'].unique())

{'B', 'C', 'D', 'G', 'I', 'M', 'N', 'P', 'X', 'Y', nan}

### 조기폐차 신청 정보 추가

In [61]:
aear['조기폐차신청여부'] = 'Y'
lgvr['조기폐차신청여부'] = 'Y'

### 조기폐차 병합

In [62]:
elp = pd.concat([aear, lgvr], ignore_index=True)
elp.shape

(1261296, 5)

In [63]:
elp.shape, len(elp['차대번호'].unique())

((1261296, 5), 1179838)

In [64]:
elp['말소일자'].tail()

1261291   NaN
1261292   NaN
1261293   NaN
1261294   NaN
1261295   NaN
Name: 말소일자, dtype: float64

### 말소일자 타입 변경

In [65]:
# elp['말소일자'] = pd.to_datetime(elp['말소일자'], errors='coerce')
# elp['말소일자'] = elp['말소일자'].astype('str')
# elp['말소일자'] = elp['말소일자'].str.replace('-', '')
# elp['말소일자'] = pd.to_numeric(elp['말소일자'], errors='coerce')
# elp['말소일자'].head()

In [66]:
## 12.2s
elpm = elp.sort_values('조기폐차최종승인YN', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
elpm.shape

(1179838, 5)

In [67]:
elpm = elpm[elpm['조기폐차최종승인YN'] == 'Y'].reset_index(drop=True)
elpm.shape

(962063, 5)

In [68]:
elpm['말소일자'].dtype

dtype('float64')

### 최신 조기폐차 현황(# !!! 수정(2023.10.27))

In [69]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20240213'

In [70]:
elp.loc[:, ['말소일자', '조기폐차상태코드']].sort_values('말소일자', ascending=False).head()

,말소일자,조기폐차상태코드
419434,46881006.0,Y
515664,44440923.0,Y
452802,35190417.0,Y
745381,30220322.0,Y
1074212,22220801.0,Y


In [71]:
elp.loc[elp['말소일자'] >= int(today_date), ['말소일자', '조기폐차상태코드']].sort_values('말소일자', ascending=False).shape

(58, 2)

In [72]:
elp.loc[elp['말소일자'] >= int(today_date), ['말소일자', '조기폐차상태코드']].sort_values('말소일자', ascending=False).tail()

,말소일자,조기폐차상태코드
1196528,20240711.0,Y
1248613,20240628.0,Y
1186799,20240503.0,Y
1137772,20240424.0,Y
1149470,20240410.0,Y


In [73]:
date_list = pd.date_range(end=today_date, periods=1, freq='M')
date_list

DatetimeIndex(['2024-01-31'], dtype='datetime64[ns]', freq='M')

In [74]:
before_amonth = int(str(date_list[0]).split(' ')[0].replace('-', ''))
before_amonth

20240131

In [75]:
elp.loc[elp['말소일자'] <= int(before_amonth), ['말소일자', '조기폐차상태코드']].sort_values('말소일자', ascending=False).head()

,말소일자,조기폐차상태코드
1210127,20240116.0,Y
1257345,20240116.0,G
1244369,20240116.0,G
1233369,20240116.0,G
1248039,20240116.0,G


In [76]:
## 조기폐차 최신 말소일자기준 추출(수동)
elp_rct = elp[elp['말소일자'] <= int(before_amonth)].sort_values('말소일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
elp_rct.shape

(974065, 5)

In [77]:
## 조기폐차 최신 말소일자기준 추출
# elp_rct = elp[elp['말소일자'] < int(today_date)].sort_values('말소일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
# elp_rct.shape

In [78]:
elp_rct['조기폐차상태코드'].value_counts(dropna=False)

조기폐차상태코드
Y      960455
G       13123
D         225
M         120
N          76
X          53
C          11
NaN         1
B           1
Name: count, dtype: int64

In [79]:
elp_rct['말소일자'].describe()

count    9.740650e+05
mean     2.018931e+07
std      3.876358e+04
min      1.010000e+02
25%      2.017053e+07
50%      2.019102e+07
75%      2.021112e+07
max      2.024012e+07
Name: 말소일자, dtype: float64

## kosis 시도명 수정

In [80]:
kosisr.loc[kosisr['시도'] == '강원도', '시도'] = '강원특별자치도'

## 등록(말소유지)&제원

In [81]:
# carr.shape

In [82]:
# cse = carr.merge(srcr, on='제원관리번호', how='left')
cse = cs.copy()
cse.shape

(1924021, 26)

## 등록(말소유지)&제원&법정동

In [83]:
## 13.2s
cse['법정동코드'] = cse['법정동코드'].astype('str')
cse['법정동코드'] = cse['법정동코드'].str[:5] + '00000'
cse['법정동코드'] = pd.to_numeric(cse['법정동코드'], errors='coerce')

In [84]:
## 10.5s
csec = cse.merge(coder, on='법정동코드', how='left')
csec.shape

(1924021, 30)

In [85]:
csec['시도'].isnull().sum()

0

## 등록(유지)&제원&법정동&조기폐차

In [86]:
csec.shape

(1924021, 30)

In [87]:
## 12.5s
csece = csec.merge(elpm, on='차대번호', how='left')
csece.shape

(1924021, 34)

### !!! 수정(2023.10.27)

In [88]:
## 12.5s
csece_rct = csec.merge(elp_rct, on='차대번호', how='left') # !!! 수정(2023.10.27)
csece_rct.shape

(1924021, 34)

## 등록(유지)&제원&법정동&조기폐차&저감

In [89]:
## 15.5s
dfe = csece.merge(attr, on='차대번호', how='left')
dfe.shape

(1924021, 37)

## 등록&제원&정기&정밀 병합

In [90]:
insm.head()

,차대번호,검사방법,검사종류,검사판정,무부하매연측정치1,무부하매연측정치2,무부하매연측정치3,무부하매연판정1,검사일자,주행거리,무부하매연허용치1
0,KNAPH81BBLA627705,무부하검사(급가속),정기검사,Y,0.0,-9999.0,-9999.00,Y,2023-12-30,47591.0,10.0
1,KMHEC415BCA017134,무부하검사(정지가동),재검사(1부재검사),Y,-9999.0,-9999.0,-9999.00,M,2023-12-30,128187.0,-9999.0
2,KMHF341NBKA285541,무부하검사(TSI),정기검사,Y,0.0,4.0,1.06,Y,2023-12-30,170293.0,1.0
3,KMHD351UGGU306226,무부하검사(급가속),정기검사,N,38.0,-9999.0,-9999.00,N,2023-12-30,166852.0,20.0
4,WDDZF5DB0KA692245,무부하검사(정지가동),정기검사,Y,-9999.0,-9999.0,-9999.00,M,2023-12-30,52615.0,-9999.0


In [91]:
insm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29443457 entries, 0 to 29443456
Data columns (total 11 columns):
 #   Column     Dtype         
---  ------     -----         
 0   차대번호       object        
 1   검사방법       object        
 2   검사종류       object        
 3   검사판정       object        
 4   무부하매연측정치1  float64       
 5   무부하매연측정치2  float64       
 6   무부하매연측정치3  float64       
 7   무부하매연판정1   object        
 8   검사일자       datetime64[ns]
 9   주행거리       float64       
 10  무부하매연허용치1  float64       
dtypes: datetime64[ns](1), float64(5), object(5)
memory usage: 2.4+ GB


In [92]:
csm.head()

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN,제원관리번호,차량연식,차종,용도,최초등록일자,...,차종유형,제작사명,차명,자동차형식,엔진형식,차종분류,총중량,적재중량,배기량,엔진출력
0,KMHJF21JPNU189564,2729011400,4,KMY-HD-04,N,10022700070002,1992,승용,개인용,19920314,...,소형,현대자동차(주),엘란트라,JL-M-3,G4DJ,A2826,1300,0,1468,90
1,KMJWA37JBBU252574,4163034024,4,7MY-HD-14-46,N,A0810007402962210,2011,승합,개인용,20100426,...,중형,현대자동차(주),그랜드 스타렉스(GRAND STAREX),TQ-12JAB-1,D4CB,A2826,2990,0,2497,174
2,KLATF69YDTB015250,2714010800,4,결측,N,10073900030001,1996,승용,개인용,19961227,...,소형,대우자동차(주),라노스1.5,TF69Y,A15SMS,A2826,1330,0,1498,96
3,KMFZCS7JP8U380996,4783036044,4,7MY-HD-14-68,N,A0810006206523107,2008,화물,개인용,20080129,...,소형,현대자동차(주),포터Ⅱ(PORTERⅡ),HR-3SS2JKM8-2,D4CB,A2809,2950,1000,2497,126
4,KNCWJZ74ABK555695,4688032027,4,7MY-KM-14-65,N,A0110003301763210,2011,화물,개인용,20110209,...,중형,기아자동차(주),봉고Ⅲ 1.2톤,PUL4AB-12,J3,A2809,3405,1200,2902,133


In [93]:
csm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1318161 entries, 0 to 1318160
Data columns (total 26 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   차대번호      1318161 non-null  object 
 1   법정동코드     1318161 non-null  int64  
 2   배출가스등급    1318161 non-null  object 
 3   배출가스인증번호  1318161 non-null  object 
 4   차량말소YN    1318161 non-null  object 
 5   제원관리번호    1318161 non-null  object 
 6   차량연식      1318161 non-null  int64  
 7   차종        1318161 non-null  object 
 8   용도        1318161 non-null  object 
 9   최초등록일자    1318161 non-null  int64  
 10  제작일자      1156232 non-null  float64
 11  차량관리번호    1318161 non-null  object 
 12  자동차등록번호   1318161 non-null  object 
 13  검사유효일     1318161 non-null  object 
 14  소유자구분     1318161 non-null  object 
 15  연료        1318161 non-null  object 
 16  차종유형      1318161 non-null  object 
 17  제작사명      1318161 non-null  object 
 18  차명        1318161 non-null  object 
 19  자동차형식     1318161 non

In [94]:
## 2m 0.5s
csi = csm.merge(insm, on='차대번호', how='left')
csi.shape

(1318161, 36)

In [95]:
## 2m 47.ss
cs5i = cs5m.merge(insm, on='차대번호', how='left')
cs5i.shape

(906192, 36)

## 등록&제원&정기&정밀&법정동코드 병합

In [96]:
print(csi['법정동코드'].isnull().sum())
csi['법정동코드'] = csi['법정동코드'].astype('str')
csi['법정동코드'] = csi['법정동코드'].str[:5] + '00000'
csi['법정동코드'] = pd.to_numeric(csi['법정동코드'], errors='coerce')
print(csi['법정동코드'].isnull().sum())

0
0


In [97]:
df = csi.merge(coder, on='법정동코드', how='left')

In [98]:
df['시도'].isnull().sum()

0

In [99]:
print(cs5i['법정동코드'].isnull().sum())
cs5i['법정동코드'] = cs5i['법정동코드'].astype('str')
cs5i['법정동코드'] = cs5i['법정동코드'].str[:5] + '00000'
cs5i['법정동코드'] = pd.to_numeric(cs5i['법정동코드'], errors='coerce')
print(cs5i['법정동코드'].isnull().sum())

0


0


In [100]:
df_g5 = cs5i.merge(coder, on='법정동코드', how='left')

In [101]:
df_g5['시도'].isnull().sum()

0

## 주소 수정

In [102]:
df['법정동코드_mod'] = df['법정동코드'].copy()

In [103]:
df_g5['법정동코드_mod'] = df_g5['법정동코드'].copy()

## 4등급 result 파일 참고하여 DPF유무 수정

In [104]:
rdf = df.copy()
rs = rs.drop_duplicates('차대번호').reset_index(drop=True)
rdf1 = rdf.merge(rs, on='차대번호', how='left')
df = rdf1.copy()

In [105]:
df.shape

(1318161, 42)

In [106]:
rdf = df_g5.copy()
rs = rs.drop_duplicates('차대번호').reset_index(drop=True)
rdf1 = rdf.merge(rs, on='차대번호', how='left')
df_g5 = rdf1.copy()

## 연료 컬럼 추가

In [107]:
df.loc[df['연료'] == '경유', 'fuel'] = '경유'
df.loc[(df['연료'] == '휘발유') | (df['연료'] == 'LPG(액화석유가스)'), 'fuel'] = '휘발유_가스'

In [108]:
df_g5.loc[df_g5['연료'] == '경유', 'fuel'] = '경유'
df_g5.loc[(df_g5['연료'] == '휘발유') | (df_g5['연료'] == 'LPG(액화석유가스)'), 'fuel'] = '휘발유_가스'

## 차량연식 타입변환

In [109]:
df['차량연식'] = pd.to_numeric(df['차량연식'], errors='coerce')

In [110]:
df_g5['차량연식'] = pd.to_numeric(df_g5['차량연식'], errors='coerce')

# 분석

## ---------- 1-1 start ----------

### EG 분류

In [111]:
## 12.7s
grade_list = []
for f, y, cy, e in df[['fuel', '제작일자', '차량연식', 'DPF유무_수정']].values:
    if (f == '휘발유_가스') and ( (19980101 <= y <= 20001231) or (1998 <= cy <= 2000) ):
        grade_list.append('A')
    elif (f == '휘발유_가스') and ( (y <= 19971231) or (cy <= 1997) ):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '유'):
        grade_list.append('A')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) )and (e == '유'):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '무'):
        grade_list.append('C')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) ) and (e == '무'):
        grade_list.append('D')
    else:
        grade_list.append('X')
df['Grade'] = grade_list

In [112]:
# 12.8s
## STD_BD_GRD4_CAR_CURSTT
## 4등급차만 추출
df1 = df[df['배출가스등급'] == '4'].reset_index(drop=True)

In [113]:
df1['연료'].value_counts(dropna=False)

연료
경유             991313
휘발유            303261
LPG(액화석유가스)     23550
기타연료               35
알코올                 1
CNG(압축천연가스)         1
Name: count, dtype: int64

In [114]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1318161 entries, 0 to 1318160
Data columns (total 44 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   차대번호       1318161 non-null  object        
 1   법정동코드      1318161 non-null  int64         
 2   배출가스등급     1318161 non-null  object        
 3   배출가스인증번호   1318161 non-null  object        
 4   차량말소YN     1318161 non-null  object        
 5   제원관리번호     1318161 non-null  object        
 6   차량연식       1318161 non-null  int64         
 7   차종         1318161 non-null  object        
 8   용도         1318161 non-null  object        
 9   최초등록일자     1318161 non-null  int64         
 10  제작일자       1156232 non-null  float64       
 11  차량관리번호     1318161 non-null  object        
 12  자동차등록번호    1318161 non-null  object        
 13  검사유효일      1318161 non-null  object        
 14  소유자구분      1318161 non-null  object        
 15  연료         1318161 non-null  object        
 16  

In [115]:
df1.shape

(1318161, 44)

In [116]:
#  현재 날짜를 가져와서 문자열로 변환
date = datetime.today().strftime("%Y%m%d")
# 사용자가 설정한 기간
periods = 2 # !!! 수정(2023.08.23)

y_plist = list(pd.date_range(end=date, periods=periods, freq="MS").year)
mth_plist = list(pd.date_range(end=date, periods=periods, freq="MS").month)

# 연도 리스트에서 첫 번째 값을 가져와 문자열로 변환
crtr_year = str(y_plist[0])
# 월 리스트에서 첫 번째 값
tp_crtr_month = mth_plist[0] # !!! 수정(2024.01.23)
# 날짜를 표현할 때 일관성을 유지하기 위해 가져온 월을 두 자리로 표현하고, 만약 한 자리라면 앞에 0을 추가
crtr_month = f'{tp_crtr_month:0>2}' # !!! 수정(2024.01.23)
# 이 정보를 통해 현재 날짜를 기준으로 사용자가 설정한 기간 동안의 연도와 월을 계산
crtr_year, crtr_month

('2024', '01')

In [117]:
## 테이블생성일자 컬럼 추가
today_date = datetime.today().strftime("%Y%m%d")
# df1['기준연월'] = '2022.12'
df1['기준연월'] = crtr_year + '.' + crtr_month
df1['테이블생성일자'] = today_date
# RH제공 법정동코드 타입 문자열로 수정
df1['법정동코드_mod'] = df1['법정동코드_mod'].astype('str')
df1['법정동코드_mod'].head()

0    2729000000
1    4163000000
2    2714000000
3    4783000000
4    4688000000
Name: 법정동코드_mod, dtype: object

In [118]:
df1.loc[df1['시군구'] == '군위군', '시도'].unique()

array(['대구광역시'], dtype=object)

In [119]:
df1[['시도', '시군구']].isnull().sum()

시도        0
시군구    7012
dtype: int64

In [120]:
## 10.9s
## 컬럼 이름 변경
STD_BD_GRD4_CAR_CURSTT = df1[[
    '테이블생성일자', 
    '기준연월',
    '차대번호', 
    '제원관리번호', 
    '차종', 
    '용도',
    '차량연식', 
    '차종유형', 
    '연료', 
    '법정동코드', 
    '시도',
    '시군구',
    '차명', 
    '제작사명', 
    '배출가스인증번호', 
    '배출가스등급', 
    'DPF유무_수정',
    '검사방법', 
    '검사종류', 
    '검사판정', 
    '무부하매연측정치1', 
    '무부하매연판정1', 
    '무부하매연측정치2', 
    '무부하매연측정치3', 
    '법정동코드_mod', 
    'Grade', 
    ]]
ch_col_dict = {
                '테이블생성일자':'LOAD_DT',
                '기준연월':'CRTR_YM', 
                '차대번호':'VIN', 
                '제원관리번호':'MANG_MNG_NO', 
                '차종':'VHCTY_CD', 
                '용도':'PURPS_CD2', 
                '차량연식':'YRIDNW', 
                '차종유형':'VHCTY_TY', 
                '연료':'FUEL_CD', 
                '법정동코드':'STDG_CD', 
                '시도':'CTPV_NM', 
                '시군구':'SGG_NM', 
                '차명':'VHCNM', 
                '제작사명':'MNFCTR_NM', 
                '배출가스인증번호':'EXHST_GAS_CERT_NO_MOD', 
                '배출가스등급':'EXHST_GAS_GRD_CD_MOD',
                'DPF유무_수정':'DPF_MNTNG_YN', 
                '검사방법':'INSP_MTHD', 
                '검사종류':'INSP_KND', 
                '검사판정':'INSP_JGMT', 
                '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
                '무부하매연판정1':'NOLOD_SMO_JGMT_YN1', 
                '무부하매연측정치2':'NOLOD_SMO_MEVLU2', 
                '무부하매연측정치3':'NOLOD_SMO_MEVLU3', 
                '법정동코드_mod':'STDG_CD_MOD',
                'Grade':'GRD4_MLSFC',

                '제작일자':'FBCTN_YMD', 
                '차종분류':'VHCTY_CL_CD',
                }
STD_BD_GRD4_CAR_CURSTT = STD_BD_GRD4_CAR_CURSTT.rename(columns=ch_col_dict)

In [121]:
STD_BD_GRD4_CAR_CURSTT.columns

Index(['LOAD_DT', 'CRTR_YM', 'VIN', 'MANG_MNG_NO', 'VHCTY_CD', 'PURPS_CD2',
       'YRIDNW', 'VHCTY_TY', 'FUEL_CD', 'STDG_CD', 'CTPV_NM', 'SGG_NM',
       'VHCNM', 'MNFCTR_NM', 'EXHST_GAS_CERT_NO_MOD', 'EXHST_GAS_GRD_CD_MOD',
       'DPF_MNTNG_YN', 'INSP_MTHD', 'INSP_KND', 'INSP_JGMT',
       'NOLOD_SMO_MEVLU1', 'NOLOD_SMO_JGMT_YN1', 'NOLOD_SMO_MEVLU2',
       'NOLOD_SMO_MEVLU3', 'STDG_CD_MOD', 'GRD4_MLSFC'],
      dtype='object')

In [122]:
STD_BD_GRD4_CAR_CURSTT.shape

(1318161, 26)

In [123]:
STD_BD_GRD4_CAR_CURSTT.isnull().sum()

LOAD_DT                       0
CRTR_YM                       0
VIN                           0
MANG_MNG_NO                   0
VHCTY_CD                      0
PURPS_CD2                     0
YRIDNW                        0
VHCTY_TY                      0
FUEL_CD                       0
STDG_CD                       0
CTPV_NM                       0
SGG_NM                     7012
VHCNM                         0
MNFCTR_NM                     0
EXHST_GAS_CERT_NO_MOD         0
EXHST_GAS_GRD_CD_MOD          0
DPF_MNTNG_YN             326924
INSP_MTHD                353244
INSP_KND                 353247
INSP_JGMT                353246
NOLOD_SMO_MEVLU1         362663
NOLOD_SMO_JGMT_YN1       362642
NOLOD_SMO_MEVLU2         832331
NOLOD_SMO_MEVLU3         869051
STDG_CD_MOD                   0
GRD4_MLSFC                    0
dtype: int64

In [124]:
STD_BD_GRD4_CAR_CURSTT.loc[STD_BD_GRD4_CAR_CURSTT['CTPV_NM'] == '대구광역시'].isnull().sum()

LOAD_DT                      0
CRTR_YM                      0
VIN                          0
MANG_MNG_NO                  0
VHCTY_CD                     0
PURPS_CD2                    0
YRIDNW                       0
VHCTY_TY                     0
FUEL_CD                      0
STDG_CD                      0
CTPV_NM                      0
SGG_NM                       0
VHCNM                        0
MNFCTR_NM                    0
EXHST_GAS_CERT_NO_MOD        0
EXHST_GAS_GRD_CD_MOD         0
DPF_MNTNG_YN             16593
INSP_MTHD                13750
INSP_KND                 13750
INSP_JGMT                13750
NOLOD_SMO_MEVLU1         14076
NOLOD_SMO_JGMT_YN1       14076
NOLOD_SMO_MEVLU2         42345
NOLOD_SMO_MEVLU3         44313
STDG_CD_MOD                  0
GRD4_MLSFC                   0
dtype: int64

In [125]:
STD_BD_GRD4_CAR_CURSTT['VHCTY_CD'].unique()

array(['승용', '승합', '화물', '특수'], dtype=object)

In [126]:
STD_BD_GRD4_CAR_CURSTT[STD_BD_GRD4_CAR_CURSTT['VHCTY_CD'].isnull()].shape

(0, 26)

In [127]:
STD_BD_GRD4_CAR_CURSTT.loc[STD_BD_GRD4_CAR_CURSTT['VHCTY_CD'].isnull(), 'CTPV_NM'].unique()

array([], dtype=object)

### [출력] STD_BD_GRD4_CAR_CURSTT

In [128]:
# expdf = STD_BD_GRD4_CAR_CURSTT
# table_nm = 'STD_BD_GRD4_CAR_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)
# # 데이터 추가
# # 9s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

### EG 분류

In [129]:
## 12.3s
grade_list = []
for f, y, cy, e in df_g5[['fuel', '제작일자', '차량연식', 'DPF유무_수정']].values:
    if (f == '휘발유_가스') and ( (19980101 <= y <= 20001231) or (1998 <= cy <= 2000) ):
        grade_list.append('A')
    elif (f == '휘발유_가스') and ( (y <= 19971231) or (cy <= 1997) ):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '유'):
        grade_list.append('A')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) )and (e == '유'):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '무'):
        grade_list.append('C')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) ) and (e == '무'):
        grade_list.append('D')
    else:
        grade_list.append('X')
df_g5['Grade'] = grade_list

In [130]:
## 2.8s
# STD_BD_GRD4_CAR_CURSTT
## 4등급차만 추출
df1_g5 = df_g5[df_g5['배출가스등급'] == '5'].reset_index(drop=True)

In [131]:
df1_g5['연료'].value_counts(dropna=False)

연료
경유             876409
휘발유             29320
LPG(액화석유가스)       430
기타연료               25
CNG(압축천연가스)         7
알코올                 1
Name: count, dtype: int64

In [132]:
df1_g5.shape

(906192, 44)

In [133]:
## 10.5s
## 테이블생성일자 컬럼 추가
today_date = datetime.today().strftime("%Y%m%d")
# df1_g5['기준연월'] = '2022.12'
df1_g5['기준연월'] = today_date[:4] + '.' + today_date[4:6]
df1_g5['테이블생성일자'] = today_date
# RH제공 법정동코드 타입 문자열로 수정
df1_g5['법정동코드_mod'] = df1_g5['법정동코드_mod'].astype('str')
df1_g5['법정동코드_mod'].head()

0    4167000000
1    1154500000
2    4615000000
3    2632000000
4    2671000000
Name: 법정동코드_mod, dtype: object

In [134]:
df1_g5.loc[df1_g5['시군구'] == '군위군', '시도'].unique()

array(['대구광역시'], dtype=object)

In [135]:
df1_g5[['시도', '시군구']].isnull().sum()

시도        0
시군구    3206
dtype: int64

In [136]:
df1_g5['차량연식'].dtype

dtype('int64')

In [137]:
## 11.4s
df_g5['차량연식'] = df_g5['차량연식'].astype('str')
df_g5['차량연식'] = df_g5['차량연식'].str[:4]
df_g5['차량연식'] = pd.to_numeric(df_g5['차량연식'], errors='coerce')
df_g5['차량연식'].head()

0    2005
1    2001
2    1989
3    1994
4    2002
Name: 차량연식, dtype: int64

In [138]:
## 10.17s
## 컬럼 이름 변경
STD_BD_GRD5_CAR_CURSTT = df1_g5[[
    '테이블생성일자', 
    '기준연월',
    '차대번호', 
    '제원관리번호', 
    '차종', 
    '용도',
    '차량연식', 
    '차종유형', 
    '연료', 
    '법정동코드', 
    '시도',
    '시군구',
    '차명', 
    '제작사명', 
    '배출가스인증번호', 
    '배출가스등급', 
    'DPF유무_수정',
    '검사방법', 
    '검사종류', 
    '검사판정', 
    '무부하매연측정치1', 
    '무부하매연판정1', 
    '무부하매연측정치2', 
    '무부하매연측정치3', 
    '법정동코드_mod', 
    'Grade', 
    ]]
ch_col_dict = {
                '테이블생성일자':'LOAD_DT',
                '기준연월':'CRTR_YM', 
                '차대번호':'VIN', 
                '제원관리번호':'MANG_MNG_NO', 
                '차종':'VHCTY_CD', 
                '용도':'PURPS_CD2', 
                '차량연식':'YRIDNW', 
                '차종유형':'VHCTY_TY', 
                '연료':'FUEL_CD', 
                '법정동코드':'STDG_CD', 
                '시도':'CTPV_NM', 
                '시군구':'SGG_NM', 
                '차명':'VHCNM', 
                '제작사명':'MNFCTR_NM', 
                '배출가스인증번호':'EXHST_GAS_CERT_NO_MOD', 
                '배출가스등급':'EXHST_GAS_GRD_CD_MOD',
                'DPF유무_수정':'DPF_MNTNG_YN', 
                '검사방법':'INSP_MTHD', 
                '검사종류':'INSP_KND', 
                '검사판정':'INSP_JGMT', 
                '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
                '무부하매연판정1':'NOLOD_SMO_JGMT_YN1', 
                '무부하매연측정치2':'NOLOD_SMO_MEVLU2', 
                '무부하매연측정치3':'NOLOD_SMO_MEVLU3', 
                '법정동코드_mod':'STDG_CD_MOD',
                'Grade':'GRD4_MLSFC',
                '제작일자':'FBCTN_YMD', 
                '차종분류':'VHCTY_CL_CD',
                }
STD_BD_GRD5_CAR_CURSTT = STD_BD_GRD5_CAR_CURSTT.rename(columns=ch_col_dict)

In [139]:
STD_BD_GRD5_CAR_CURSTT.columns

Index(['LOAD_DT', 'CRTR_YM', 'VIN', 'MANG_MNG_NO', 'VHCTY_CD', 'PURPS_CD2',
       'YRIDNW', 'VHCTY_TY', 'FUEL_CD', 'STDG_CD', 'CTPV_NM', 'SGG_NM',
       'VHCNM', 'MNFCTR_NM', 'EXHST_GAS_CERT_NO_MOD', 'EXHST_GAS_GRD_CD_MOD',
       'DPF_MNTNG_YN', 'INSP_MTHD', 'INSP_KND', 'INSP_JGMT',
       'NOLOD_SMO_MEVLU1', 'NOLOD_SMO_JGMT_YN1', 'NOLOD_SMO_MEVLU2',
       'NOLOD_SMO_MEVLU3', 'STDG_CD_MOD', 'GRD4_MLSFC'],
      dtype='object')

In [140]:
STD_BD_GRD5_CAR_CURSTT.shape

(906192, 26)

In [141]:
STD_BD_GRD5_CAR_CURSTT.isnull().sum()

LOAD_DT                       0
CRTR_YM                       0
VIN                           0
MANG_MNG_NO                   0
VHCTY_CD                      0
PURPS_CD2                     0
YRIDNW                        0
VHCTY_TY                      0
FUEL_CD                       0
STDG_CD                       0
CTPV_NM                       0
SGG_NM                     3206
VHCNM                         0
MNFCTR_NM                     0
EXHST_GAS_CERT_NO_MOD         0
EXHST_GAS_GRD_CD_MOD          0
DPF_MNTNG_YN             906191
INSP_MTHD                336890
INSP_KND                 336987
INSP_JGMT                336937
NOLOD_SMO_MEVLU1         345244
NOLOD_SMO_JGMT_YN1       345236
NOLOD_SMO_MEVLU2         608666
NOLOD_SMO_MEVLU3         636208
STDG_CD_MOD                   0
GRD4_MLSFC                    0
dtype: int64

In [142]:
STD_BD_GRD5_CAR_CURSTT.loc[STD_BD_GRD5_CAR_CURSTT['CTPV_NM'] == '대구광역시'].isnull().sum()

LOAD_DT                      0
CRTR_YM                      0
VIN                          0
MANG_MNG_NO                  0
VHCTY_CD                     0
PURPS_CD2                    0
YRIDNW                       0
VHCTY_TY                     0
FUEL_CD                      0
STDG_CD                      0
CTPV_NM                      0
SGG_NM                       0
VHCNM                        0
MNFCTR_NM                    0
EXHST_GAS_CERT_NO_MOD        0
EXHST_GAS_GRD_CD_MOD         0
DPF_MNTNG_YN             40269
INSP_MTHD                13237
INSP_KND                 13238
INSP_JGMT                13240
NOLOD_SMO_MEVLU1         13473
NOLOD_SMO_JGMT_YN1       13473
NOLOD_SMO_MEVLU2         29565
NOLOD_SMO_MEVLU3         30891
STDG_CD_MOD                  0
GRD4_MLSFC                   0
dtype: int64

In [143]:
STD_BD_GRD5_CAR_CURSTT['VHCTY_CD'].unique()

array(['화물', '승합', '승용', '특수'], dtype=object)

In [144]:
STD_BD_GRD5_CAR_CURSTT[STD_BD_GRD5_CAR_CURSTT['VHCTY_CD'].isnull()].shape

(0, 26)

In [145]:
STD_BD_GRD5_CAR_CURSTT.loc[STD_BD_GRD5_CAR_CURSTT['VHCTY_CD'].isnull(), 'CTPV_NM'].unique()

array([], dtype=object)

### [출력] STD_BD_GRD5_CAR_CURSTT

In [146]:
# expdf = STD_BD_GRD5_CAR_CURSTT
# table_nm = 'STD_BD_GRD5_CAR_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)
# # 데이터 추가
# # 9s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [147]:
## 11.4s
## 경유차만 추출
df2 = df1[df1['연료'] == '경유'].reset_index(drop=True)
df2.shape

(991313, 46)

In [148]:
## 10.5s
## 차대번호 10자리 연식
df2['vin10'] = df2['차대번호'].str[9]
vin10_dict = {'J':1988, 'K':1989, 'L':1990, 'M':1991, 'N':1992, 'P':1993, 'R':1994, 'S':1995, 'T':1996, 'V':1997, 'W':1998, 'X':1999, 'Y':2000, '1':2001, '2':2002, '3':2003, '4':2004, '5':2005, '6':2006, '7':2007, '8':2008, '9':2009, 'A':2010, 'B':2011, 'C':2012, 'D':2013, 'E':2014, 'F':2015, 'G':2016, 'H':2017}
df2['vin10_year'] = df2['vin10'].map(vin10_dict, na_action='ignore')

In [149]:
## 배인번호_수정 문자 타입으로 변경
df2['배출가스인증번호'] = df2['배출가스인증번호'].astype('str')

In [150]:
## 12.0s
## 차대번호 17자리 샘플
df2y = df2.loc[df2['차대번호'].str.len() == 17].reset_index(drop=True)
df2n = df2.loc[df2['차대번호'].str.len() != 17].reset_index(drop=True)
df2y.shape, df2n.shape

((991303, 48), (10, 48))

In [151]:
## 10.08s
## 차대번호 연식과 연식 동일한 샘플
df3y = df2y.loc[df2y['vin10_year'] == df2y['차량연식']].reset_index(drop=True)
df3n = df2y.loc[df2y['vin10_year'] != df2y['차량연식']].reset_index(drop=True)
df3y.shape, df3n.shape

((991224, 48), (79, 48))

In [152]:
## 10.09s
grp4 = df3y[df3y['검사판정'] == 'Y'].groupby(['배출가스인증번호', '제작사명', '차명', '검사방법', '제원관리번호']).agg({'차대번호':'count'}).reset_index()
grp4 = grp4.rename(columns={'차대번호':'차량대수'})
grp4

,배출가스인증번호,제작사명,차명,검사방법,제원관리번호,차량대수
0,,(주)에이엠티코리아,에이엠티언더리프트,무부하검사(급가속),BMY10000900014111,1
1,,주식회사 한서정공,한서4륜구동다목적방역방제차,무부하검사(급가속),B1T10003500013111,2
2,,한국쓰리축공업(주),한쓰4.5톤극플러스트럭,무부하검사(급가속),B9410024600003309,1
3,,한국특장기술(주),한국특장기술27톤트럭,무부하검사(급가속),B3C10034000013310,1
4,7MY-DC-24-72,한국쓰리축공업(주),한쓰6.5톤극플러스트럭,부하검사(LUG DOWN),B9410010100003308,1
...,...,...,...,...,...,...
8223,결측,현대자동차(주),베라크루즈(VERACRUZ),부하검사(KD-147),A0810007000311307,1
8224,결측,현대자동차(주),스타렉스리무진(STAREX),부하검사(KD-147),A0810003602851306,1
8225,결측,현대자동차(주),투싼(TUCSON),부하검사(KD-147),A0810006301221206,1
8226,결측,현대자동차(주),포터Ⅱ(PORTERⅡ),무부하검사(급가속),A0810006205343107,1


In [153]:
grp4 = grp4[grp4['배출가스인증번호'] != 'nan'].reset_index(drop=True)
grp4.shape

(8228, 6)

In [154]:
grp4['차량대수'].describe()

count     8228.000000
mean        99.502552
std        615.570849
min          1.000000
25%          1.000000
50%          3.000000
75%         19.000000
max      22806.000000
Name: 차량대수, dtype: float64

In [155]:
## 100대 초과 샘플만 활용
df4 = grp4[grp4['차량대수'] > 100].reset_index(drop=True)
df4.shape

(887, 6)

In [156]:
## 8m 25.9s
df5 = pd.DataFrame()
for one, two, three, four, five in df4[['배출가스인증번호', '제원관리번호', '제작사명', '차명', '검사방법']].values:
    temp = df3y[(df3y['검사판정'] == 'Y') & (df3y['배출가스인증번호'] == one) & (df3y['제원관리번호'] == two) & (df3y['제작사명'] == three) & (df3y['차명'] == four) & (df3y['검사방법'] == five)].reset_index(drop=True)
    df5 = pd.concat([df5, temp], ignore_index=True)
df5.shape

(742975, 48)

In [157]:
## 13.3s
grp6 = df5.groupby(['배출가스인증번호', '제작사명', '차명', '검사방법', '제원관리번호']).agg({'무부하매연측정치1':[lambda x:x.describe()['25%'], lambda x:x.describe()['50%'], lambda x:x.describe()['75%']], '차대번호':'count'}).reset_index()
grp6.columns = ['배출가스인증번호', '제작사명', '차명', '검사방법', '제원관리번호', 'q1', 'q2', 'q3', '차량대수']
grp6

,배출가스인증번호,제작사명,차명,검사방법,제원관리번호,q1,q2,q3,차량대수
0,5MY-AD-13-06,아우디폭스바겐코리아(주),Phaeton 3.0 TDI,무부하검사(급가속),02420004800011305,0.0,0.0,0.75,114
1,5MY-AD-13-06,아우디폭스바겐코리아(주),Phaeton 3.0 TDI,무부하검사(급가속),02420004800021306,0.0,0.0,0.00,103
2,5MY-AD-14-08,아우디폭스바겐코리아(주),Passat 2.0 TDI,무부하검사(급가속),02420006800031206,0.0,0.0,0.00,145
3,5MY-AD-14-08,아우디폭스바겐코리아(주),Passat 2.0 TDI,무부하검사(급가속),02420006800111207,0.0,0.0,2.00,243
4,5MY-AD-14-18,아우디폭스바겐코리아(주),Golf 2.0 TDI,무부하검사(급가속),02420004900031207,0.0,0.0,3.00,239
...,...,...,...,...,...,...,...,...,...
882,결측,한불모터스(주),308 1.6 HDi MCP,무부하검사(급가속),00620000800081209,0.0,2.0,7.00,186
883,결측,한불모터스(주),Nouvelle Peugeot 308SW HDi,무부하검사(급가속),00620000800021208,0.0,3.0,8.00,145
884,결측,한불모터스(주),푸조 407 HDi FAP A6,무부하검사(급가속),00620000500251207,0.0,2.0,7.00,175
885,결측,한불모터스(주),푸조 NEW 307SW HDi,무부하검사(급가속),00620000400221207,0.0,2.0,10.50,123


In [158]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20240213'

In [159]:
grp6['테이블생성일자'] = today_date

In [160]:
STD_BD_GRD4_CAR_CURSTT_TOT = grp6[[
    '테이블생성일자',
    '차명',
    '제작사명', 
    '제원관리번호', 
    '배출가스인증번호', 
    '검사방법', 
    'q1', 
    'q2', 
    'q3',
    '차량대수',
    ]]
STD_BD_GRD4_CAR_CURSTT_TOT.shape

(887, 10)

In [161]:
chc_dict = {
    '테이블생성일자':'LOAD_DT', 
    '차명':'VHCNM',
    '제작사명':'MNFCTR_NM', 
    '제원관리번호':'MANG_MNG_NO', 
    '배출가스인증번호':'EXHST_GAS_CERT_NO_MOD', 
    '검사방법':'INSP_MTHD', 
    'q1':'LOWR_QRT',
    'q2':'MID_QRT',
    'q3':'UP_QRT',
    '차량대수':'VHCL_MKCNT',
    '차대번호':'VIN', 
}
STD_BD_GRD4_CAR_CURSTT_TOT = STD_BD_GRD4_CAR_CURSTT_TOT.rename(columns=chc_dict)
STD_BD_GRD4_CAR_CURSTT_TOT.columns

Index(['LOAD_DT', 'VHCNM', 'MNFCTR_NM', 'MANG_MNG_NO', 'EXHST_GAS_CERT_NO_MOD',
       'INSP_MTHD', 'LOWR_QRT', 'MID_QRT', 'UP_QRT', 'VHCL_MKCNT'],
      dtype='object')

### [출력] 제번별 4분위 값(STD_BD_GRD4_CAR_CURSTT_TOT)

In [162]:
# expdf = STD_BD_GRD4_CAR_CURSTT_TOT
# table_nm = 'STD_BD_GRD4_CAR_CURSTT_TOT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 1s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [163]:
## 10.1s
sidf = df5[[
    '차대번호', 
    '제원관리번호', 
    '차종', 
    '차량연식', 
    '차명', 
    '차종유형', 
    '제작사명', 
    '연료', 
    '법정동코드', 
    '배출가스인증번호', 
    '검사방법', 
    '검사종류', 
    '검사판정', 
    '주행거리', 
    '무부하매연판정1', 
    '무부하매연허용치1', 
    '무부하매연측정치1'
]]
sidf.shape

(742975, 17)

In [164]:
current_yr = int(datetime.today().strftime("%Y"))
current_yr

2024

In [165]:
sidf['차령'] = current_yr - sidf['차량연식']
sidf['SI'] = sidf['무부하매연측정치1'] / sidf['무부하매연허용치1']

C:\Users\kbjung\AppData\Local\Temp\ipykernel_2144\4139794486.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidf['차령'] = current_yr - sidf['차량연식']
C:\Users\kbjung\AppData\Local\Temp\ipykernel_2144\4139794486.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidf['SI'] = sidf['무부하매연측정치1'] / sidf['무부하매연허용치1']


In [166]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20240213'

In [167]:
sidf['테이블생성일자'] = today_date

C:\Users\kbjung\AppData\Local\Temp\ipykernel_2144\3010636050.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidf['테이블생성일자'] = today_date


In [168]:
sidf1 = sidf[[
    '테이블생성일자',
    '차대번호', 
    '제원관리번호', 
    '차명', 
    '제작사명', 
    '배출가스인증번호', 
    '검사방법',
    '주행거리', 
    '차령',
    'SI', 
    ]]

In [169]:
## 10.1s
chc_dict = {
    '테이블생성일자':'LOAD_DT',
    '차대번호':'VIN', 
    '제원관리번호':'MANG_MNG_NO', 
    '차종':'VHCTY_CD', 
    '연식':'YRIDNW', 
    '차명':'VHCNM',
    '제작사명':'MNFCTR_NM', 
    '차종유형':'VHCTY_TY', 
    '연료':'FUEL_CD',
    '법정동코드':'STDG_CD', 
    '배출가스인증번호':'EXHST_GAS_CERT_NO_MOD', 
    '검사방법':'INSP_MTHD', 
    '검사종류':'INSP_KND', 
    '검사판정':'INSP_JGMT', 
    '주행거리':'DRVNG_DSTNC',
    '무부하매연판정1':'NOLOD_SMO_JGMT_YN1',
    '무부하매연허용치1':'NOLOD_SMO_PRMT_VAL1',
    '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
    '차령':'VHCAG',
    }
STD_BD_GRD4_SI = sidf1.rename(columns=chc_dict)
STD_BD_GRD4_SI.columns

Index(['LOAD_DT', 'VIN', 'MANG_MNG_NO', 'VHCNM', 'MNFCTR_NM',
       'EXHST_GAS_CERT_NO_MOD', 'INSP_MTHD', 'DRVNG_DSTNC', 'VHCAG', 'SI'],
      dtype='object')

### [출력] SI 지수 정보(STD_BD_GRD4_SI)

In [170]:
# expdf = STD_BD_GRD4_SI
# table_nm = 'STD_BD_GRD4_SI'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)
# # 데이터 추가
# # 6s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [171]:
# 5번 데이터 셋에서 DAT용 (검토구분 계산) 테이블 생성
# grp6 : df5.groupby(['배출가스인증번호', '제작사명', '차명', '검사방법', '제원관리번호'])
grp7 = grp6.copy()

In [172]:
grp7['q2_mean'] = grp7.groupby(['배출가스인증번호', '제작사명', '차명', '검사방법'])['q2'].transform('mean')
grp7

,배출가스인증번호,제작사명,차명,검사방법,제원관리번호,q1,q2,q3,차량대수,테이블생성일자,q2_mean
0,5MY-AD-13-06,아우디폭스바겐코리아(주),Phaeton 3.0 TDI,무부하검사(급가속),02420004800011305,0.0,0.0,0.75,114,20240213,0.0
1,5MY-AD-13-06,아우디폭스바겐코리아(주),Phaeton 3.0 TDI,무부하검사(급가속),02420004800021306,0.0,0.0,0.00,103,20240213,0.0
2,5MY-AD-14-08,아우디폭스바겐코리아(주),Passat 2.0 TDI,무부하검사(급가속),02420006800031206,0.0,0.0,0.00,145,20240213,0.0
3,5MY-AD-14-08,아우디폭스바겐코리아(주),Passat 2.0 TDI,무부하검사(급가속),02420006800111207,0.0,0.0,2.00,243,20240213,0.0
4,5MY-AD-14-18,아우디폭스바겐코리아(주),Golf 2.0 TDI,무부하검사(급가속),02420004900031207,0.0,0.0,3.00,239,20240213,0.0
...,...,...,...,...,...,...,...,...,...,...,...
882,결측,한불모터스(주),308 1.6 HDi MCP,무부하검사(급가속),00620000800081209,0.0,2.0,7.00,186,20240213,2.0
883,결측,한불모터스(주),Nouvelle Peugeot 308SW HDi,무부하검사(급가속),00620000800021208,0.0,3.0,8.00,145,20240213,3.0
884,결측,한불모터스(주),푸조 407 HDi FAP A6,무부하검사(급가속),00620000500251207,0.0,2.0,7.00,175,20240213,2.0
885,결측,한불모터스(주),푸조 NEW 307SW HDi,무부하검사(급가속),00620000400221207,0.0,2.0,10.50,123,20240213,2.0


In [173]:
grp7.loc[(grp7['q2'] > grp7['q2_mean']*5) | (grp7['q2'] < grp7['q2_mean']/5), '검토구분'] = '주의'
grp7['검토구분'] = grp7['검토구분'].fillna('양호')

In [174]:
grp7['검토구분'].value_counts(dropna=False)

검토구분
양호    854
주의     33
Name: count, dtype: int64

In [175]:
len(grp7.loc[grp7['검토구분'] == '양호', '배출가스인증번호'].unique())

67

In [176]:
len(grp7.loc[grp7['검토구분'] == '주의', '배출가스인증번호'].unique())

6

In [177]:
grp7.loc[grp7['검토구분'] == '주의', '배출가스인증번호'].unique()

array(['5MY-HD-24-59', '6MY-HD-14-63', '6MY-KM-14-61', '7MY-HD-24-29',
       '7MY-SM-14-36', '8MY-KM-14-29'], dtype=object)

In [178]:
grp7[grp7['배출가스인증번호'] == '9MY-KM-14-22']

,배출가스인증번호,제작사명,차명,검사방법,제원관리번호,q1,q2,q3,차량대수,테이블생성일자,q2_mean,검토구분
831,9MY-KM-14-22,기아자동차(주),쏘렌토,부하검사(KD-147),A0110004400101209,2.0,6.0,10.0,2308,20240213,6.0,양호
832,9MY-KM-14-22,기아자동차(주),쏘렌토,부하검사(KD-147),A0110004400111209,3.0,6.0,10.0,15359,20240213,6.0,양호
833,9MY-KM-14-22,기아자동차(주),쏘렌토,부하검사(KD-147),A0110004400121209,2.0,6.0,10.0,1582,20240213,6.0,양호
834,9MY-KM-14-22,기아자동차(주),쏘렌토,부하검사(KD-147),A0110004400231210,2.0,6.0,10.0,883,20240213,6.0,양호
835,9MY-KM-14-22,기아자동차(주),쏘렌토,부하검사(KD-147),A0110004400241210,2.0,6.0,10.0,6455,20240213,6.0,양호
836,9MY-KM-14-22,기아자동차(주),쏘렌토,부하검사(KD-147),A0110004400251210,2.0,6.0,10.0,668,20240213,6.0,양호


In [179]:
# grp7.to_excel('샘플_4등급경유차_배출가스인증번호_검토(2023.10.27).xlsx', index=False)

In [180]:
STD_BD_DAT_GRD4_CERT_NO_RVW = grp7[[
    '배출가스인증번호',
    '검사방법',
    '검토구분',
    '제작사명',
    '차명',
    '제원관리번호',
    'q1',
    'q2',
    'q3',
    '테이블생성일자',
]]

In [181]:
STD_BD_DAT_GRD4_CERT_NO_RVW.head()

,배출가스인증번호,검사방법,검토구분,제작사명,차명,제원관리번호,q1,q2,q3,테이블생성일자
0,5MY-AD-13-06,무부하검사(급가속),양호,아우디폭스바겐코리아(주),Phaeton 3.0 TDI,02420004800011305,0.0,0.0,0.75,20240213
1,5MY-AD-13-06,무부하검사(급가속),양호,아우디폭스바겐코리아(주),Phaeton 3.0 TDI,02420004800021306,0.0,0.0,0.00,20240213
2,5MY-AD-14-08,무부하검사(급가속),양호,아우디폭스바겐코리아(주),Passat 2.0 TDI,02420006800031206,0.0,0.0,0.00,20240213
3,5MY-AD-14-08,무부하검사(급가속),양호,아우디폭스바겐코리아(주),Passat 2.0 TDI,02420006800111207,0.0,0.0,2.00,20240213
4,5MY-AD-14-18,무부하검사(급가속),양호,아우디폭스바겐코리아(주),Golf 2.0 TDI,02420004900031207,0.0,0.0,3.00,20240213


In [182]:
cdict = {
    '배출가스인증번호':'EXHST_GAS_CERT_NO',
    '검사방법':'INSP_MTHD',
    '검토구분':'RVW_SE',
    '제작사명':'RPRS_MNFCTR_NM',
    '차명':'RPRS_VHCNM', 
    '제원관리번호':'MANG_MNG_NO',
    'q1':'LOWR_QRT',
    'q2':'MID_QRT',
    'q3':'UP_QRT',
    '테이블생성일자':'LOAD_DT',
}
STD_BD_DAT_GRD4_CERT_NO_RVW = STD_BD_DAT_GRD4_CERT_NO_RVW.rename(columns=cdict)
STD_BD_DAT_GRD4_CERT_NO_RVW.columns

Index(['EXHST_GAS_CERT_NO', 'INSP_MTHD', 'RVW_SE', 'RPRS_MNFCTR_NM',
       'RPRS_VHCNM', 'MANG_MNG_NO', 'LOWR_QRT', 'MID_QRT', 'UP_QRT',
       'LOAD_DT'],
      dtype='object')

### [출력] STD_BD_DAT_GRD4_CERT_NO_RVW

In [183]:
# expdf = STD_BD_DAT_GRD4_CERT_NO_RVW
# table_nm = 'STD_BD_DAT_GRD4_CERT_NO_RVW'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 1s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [184]:
sidf.shape

(742975, 20)

In [185]:
sidf['차량연식'].dtype

dtype('int64')

### !!! 수정 시작(2023.11.03) 

In [186]:
### !!! 수정 (2023.11.03) 
## 11.3s
grp_sidf = sidf.groupby(['배출가스인증번호', '제작사명', '차명', '검사방법', '제원관리번호']).agg({'차종':lambda x:x.value_counts().index[0], '연료':lambda x:x.value_counts().index[0], '차량연식':lambda x:x.nsmallest(1), 'SI':'mean'}).reset_index()
grp_sidf = grp_sidf.rename(columns={'차량연식':'최초연식', 'SI':'열화도'})
grp_sidf

,배출가스인증번호,제작사명,차명,검사방법,제원관리번호,차종,연료,최초연식,열화도
0,5MY-AD-13-06,아우디폭스바겐코리아(주),Phaeton 3.0 TDI,무부하검사(급가속),02420004800011305,승용,경유,2006,0.039326
1,5MY-AD-13-06,아우디폭스바겐코리아(주),Phaeton 3.0 TDI,무부하검사(급가속),02420004800021306,승용,경유,2007,0.028355
2,5MY-AD-14-08,아우디폭스바겐코리아(주),Passat 2.0 TDI,무부하검사(급가속),02420006800031206,승용,경유,2007,0.028115
3,5MY-AD-14-08,아우디폭스바겐코리아(주),Passat 2.0 TDI,무부하검사(급가속),02420006800111207,승용,경유,2008,0.081009
4,5MY-AD-14-18,아우디폭스바겐코리아(주),Golf 2.0 TDI,무부하검사(급가속),02420004900031207,승용,경유,2008,0.112931
...,...,...,...,...,...,...,...,...,...
882,결측,한불모터스(주),308 1.6 HDi MCP,무부하검사(급가속),00620000800081209,승용,경유,2010,0.191412
883,결측,한불모터스(주),Nouvelle Peugeot 308SW HDi,무부하검사(급가속),00620000800021208,승용,경유,2009,0.223986
884,결측,한불모터스(주),푸조 407 HDi FAP A6,무부하검사(급가속),00620000500251207,승용,경유,2008,0.181953
885,결측,한불모터스(주),푸조 NEW 307SW HDi,무부하검사(급가속),00620000400221207,승용,경유,2008,0.217644


### !!! 수정 끝(2023.11.03) 

In [187]:
df71 = grp_sidf.merge(grp7[['배출가스인증번호', '제원관리번호', '제작사명', '차명', '검사방법', '검토구분']], on=['배출가스인증번호', '제원관리번호', '제작사명', '차명', '검사방법'],how='left')
df71.shape

(887, 10)

In [188]:
df71['테이블생성일자'] = today_date

In [189]:
STD_BD_DAT_GRD4_SI = df71[[
    '배출가스인증번호', 
    '검사방법', 
    '검토구분', 
    '제작사명', 
    '차명', 
    '차종', 
    '연료', 
    '최초연식', 
    '열화도', 
    '테이블생성일자', 
]]

In [190]:
cdict = {
    '배출가스인증번호':'EXHST_GAS_CERT_NO', 
    '검사방법':'INSP_MTHD', 
    '검토구분':'RVW_SE', 
    '제작사명':'RPRS_MNFCTR_NM', 
    '차명':'RPRS_VHCNM', 
    '차종':'RPRS_VHCTY_CD', 
    '연료':'RPRS_FUEL', 
    '최초연식':'FRST_YRIDNW', 
    '열화도':'SI', 
    '테이블생성일자':'LOAD_DT', 
}
STD_BD_DAT_GRD4_SI = STD_BD_DAT_GRD4_SI.rename(columns=cdict)
STD_BD_DAT_GRD4_SI.columns

Index(['EXHST_GAS_CERT_NO', 'INSP_MTHD', 'RVW_SE', 'RPRS_MNFCTR_NM',
       'RPRS_VHCNM', 'RPRS_VHCTY_CD', 'RPRS_FUEL', 'FRST_YRIDNW', 'SI',
       'LOAD_DT'],
      dtype='object')

In [191]:
STD_BD_DAT_GRD4_SI.columns

Index(['EXHST_GAS_CERT_NO', 'INSP_MTHD', 'RVW_SE', 'RPRS_MNFCTR_NM',
       'RPRS_VHCNM', 'RPRS_VHCTY_CD', 'RPRS_FUEL', 'FRST_YRIDNW', 'SI',
       'LOAD_DT'],
      dtype='object')

### [출력] STD_BD_DAT_GRD4_SI

In [192]:
# expdf = STD_BD_DAT_GRD4_SI
# table_nm = 'STD_BD_DAT_GRD4_SI'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 1s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [193]:
df5.shape

(742975, 48)

In [194]:
STD_BD_GRD4_CAR_CURSTT_MOD = df5[[
    '테이블생성일자', 
    '차대번호', 
    '배출가스인증번호', 
    '제원관리번호', 
    '제작사명', 
    '차명', 
    '검사방법', 
    '무부하매연측정치1', 
]]

In [195]:
## 10.1s
cdict = {
        '테이블생성일자':'LOAD_DT',
        '차대번호':'VIN', 
        '배출가스인증번호':'EXHST_GAS_CERT_NO_MOD', 
        '제원관리번호':'MANG_MNG_NO', 
        '제작사명':'MNFCTR_NM', 
        '차명':'VHCNM', 
        '검사방법':'INSP_MTHD', 
        '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
}
STD_BD_GRD4_CAR_CURSTT_MOD = STD_BD_GRD4_CAR_CURSTT_MOD.rename(columns=ch_col_dict)
STD_BD_GRD4_CAR_CURSTT_MOD.columns

Index(['LOAD_DT', 'VIN', 'EXHST_GAS_CERT_NO_MOD', 'MANG_MNG_NO', 'MNFCTR_NM',
       'VHCNM', 'INSP_MTHD', 'NOLOD_SMO_MEVLU1'],
      dtype='object')

### [출력] STD_BD_GRD4_CAR_CURSTT_MOD

In [196]:
# expdf = STD_BD_GRD4_CAR_CURSTT_MOD
# table_nm = 'STD_BD_GRD4_CAR_CURSTT_MOD'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 1s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

## 1-1 code end ##################################################################

## ---------- 1-2 start ----------

In [197]:
# carr.shape

In [198]:
## 11.1s
# 전체 4등급 등록&제원 병합
# cse = carr.merge(srcr, on='제원관리번호', how='left')
cse = cs.copy()
cse.shape

(1924021, 26)

In [199]:
## 13.4s
cse['법정동코드'] = cse['법정동코드'].astype('str')
cse['법정동코드'] = cse['법정동코드'].str[:5] + '00000'
cse['법정동코드'] = pd.to_numeric(cse['법정동코드'])

In [200]:
## 10.6s
# 시도, 시군구 추가
csec = cse.merge(coder, on='법정동코드', how='left')
csec.shape

(1924021, 30)

In [201]:
csec['시도'].isnull().sum()

0

In [202]:
## 12.6s
# 조기폐차 추가
dfe = csec.merge(elpm, on='차대번호', how='left')
dfe.shape

(1924021, 34)

In [203]:
## 11.8s
df1 = dfe[dfe['연료'] == '경유'].reset_index(drop=True)
df1.shape

(1426749, 34)

#### !!! 수정 시작(2023.10.12)

In [204]:
#### !!! 수정 (2023.10.12) 
## 11.0s
# 조기폐차 해당 차량 추출
idx = df1.loc[df1['조기폐차최종승인YN'] == 'Y'].index
df1_ey = df1.loc[idx]
df1_en = df1.loc[list(set(df1.index) - set(idx))]
df1_ey.shape[0] + df1_en.shape[0]

1426749

In [205]:
# 기준연월 추가
df1_ey['말소일자'] = df1_ey['말소일자'].astype('str')
df1_ey['기준연월'] = df1_ey['말소일자'].str[:4] + '.' + df1_ey['말소일자'].str[4:6]
df1_ey[['말소일자', '기준연월']].head()

,말소일자,기준연월
112,20231213.0,2023.12
113,20230412.0,2023.04
140,20230524.0,2023.05
143,20230720.0,2023.07
169,20230804.0,2023.08


In [206]:
## 15.8s
# 다시 병합
df1 = pd.concat([df1_ey, df1_en], ignore_index=True)
df1.shape

(1426749, 35)

In [207]:
df1['조기폐차상태코드'].unique()
#### !!! 수정 끝(2023.10.12) 

array(['Y', 'D', 'X', 'M', nan], dtype=object)

#### !!! 수정 끝(2023.10.12) 

####  !!! 수정 시작(2023.10.24) 

In [208]:
####  !!! 수정 시작(2023.10.24) 
df1.loc[df1['말소일자'] < '20230201', '말소일자']

7596    20201210.0
Name: 말소일자, dtype: object

In [209]:
df1.loc[df1['말소일자'] > today_date, '말소일자']

2812     20301026.0
3831     20241130.0
7071     20260624.0
7320     20251025.0
19414    20320511.0
26136    20240424.0
27775    20240725.0
33425    20321026.0
34577    20240831.0
34802    20320601.0
36566    20320419.0
40134    20240726.0
40694    20320526.0
41739    20240711.0
44567    20320915.0
52142    20320526.0
53174    20320526.0
57286    20240410.0
61501    20241024.0
61950    20320418.0
Name: 말소일자, dtype: object

In [210]:
## 11.8s
# 조기폐차 이상치 말소일자 제거
df1 = df1[(df1['말소일자'] >= '20230201') & (df1['말소일자'] <= today_date) | (df1['말소일자'].isnull())].reset_index(drop=True)
df1.shape
####  !!! 수정 끝(2023.10.24)

(1426728, 35)

####  !!! 수정 끝(2023.10.24)

In [211]:
## 10.7s
STD_BD_GRD4_ELPDSRC_CURSTT = df1[[
    '기준연월',
    '차대번호', 
    '법정동코드', 
    '차종', 
    '용도', 
    '연료', 
    '차종유형', 
    '시도',
    '시군구', 
    '조기폐차상태코드', 
    '조기폐차최종승인YN',
]]
today_date = datetime.today().strftime("%Y%m%d")
STD_BD_GRD4_ELPDSRC_CURSTT['테이블생성일자'] = today_date
STD_BD_GRD4_ELPDSRC_CURSTT = STD_BD_GRD4_ELPDSRC_CURSTT[[
    '기준연월',
    '차대번호', 
    '법정동코드', 
    '차종', 
    '용도', 
    '연료', 
    '차종유형', 
    '시도', 
    '시군구', 
    '조기폐차상태코드', 
    '조기폐차최종승인YN', 
    '테이블생성일자', 
]]
chc_dict = {
    '기준연월':'CRTR_YM', 
    '차대번호':'VIN', 
    '법정동코드':'STDG_CD', 
    '차종':'VHCTY_CD', 
    '용도':'PURPS_CD2', 
    '연료':'FUEL_CD', 
    '차종유형':'VHCTY_TY', 
    '시도':'CTPV', 
    '시군구':'SGG', 
    '조기폐차상태코드':'ELPDSRC_STTS_CD',
    '조기폐차최종승인YN':'ELPDSRC_LAST_APRV_YN', # !!! 수정
    '테이블생성일자':'LOAD_DT', 
}
STD_BD_GRD4_ELPDSRC_CURSTT = STD_BD_GRD4_ELPDSRC_CURSTT.rename(columns=chc_dict)

C:\Users\kbjung\AppData\Local\Temp\ipykernel_2144\4186908792.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  STD_BD_GRD4_ELPDSRC_CURSTT['테이블생성일자'] = today_date


In [212]:
STD_BD_GRD4_ELPDSRC_CURSTT.columns

Index(['CRTR_YM', 'VIN', 'STDG_CD', 'VHCTY_CD', 'PURPS_CD2', 'FUEL_CD',
       'VHCTY_TY', 'CTPV', 'SGG', 'ELPDSRC_STTS_CD', 'ELPDSRC_LAST_APRV_YN',
       'LOAD_DT'],
      dtype='object')

In [213]:
STD_BD_GRD4_ELPDSRC_CURSTT.ELPDSRC_STTS_CD.unique()

array(['Y', 'D', 'X', 'M', nan], dtype=object)

In [214]:
STD_BD_GRD4_ELPDSRC_CURSTT.head()

,CRTR_YM,VIN,STDG_CD,VHCTY_CD,PURPS_CD2,FUEL_CD,VHCTY_TY,CTPV,SGG,ELPDSRC_STTS_CD,ELPDSRC_LAST_APRV_YN,LOAD_DT
0,2023.12,KPACE4AB19P057583,4141000000,화물,개인용,경유,소형,경기도,군포시,Y,Y,20240213
1,2023.04,KMJWAH7JP8U055888,4215000000,승합,개인용,경유,중형,강원도,강릉시,Y,Y,20240213
2,2023.05,KMHSH81VP8U385464,4159000000,승용,개인용,경유,중형,경기도,화성시,Y,Y,20240213
3,2023.07,KMHDT41TP8U558251,1132000000,승용,개인용,경유,중형,서울특별시,도봉구,Y,Y,20240213
4,2023.08,KMFWBH7JP8U054633,4180000000,화물,개인용,경유,소형,경기도,연천군,Y,Y,20240213


In [215]:
STD_BD_GRD4_ELPDSRC_CURSTT['CTPV'].unique()

array(['경기도', '강원도', '서울특별시', '제주특별자치도', '부산광역시', '인천광역시', '충청남도', '경상북도',
       '울산광역시', '전라북도', '전라남도', '경상남도', '충청북도', '대전광역시', '강원특별자치도',
       '광주광역시', '세종특별자치시', '대구광역시'], dtype=object)

In [216]:
## 10.1s
elpd_dict = {'강원도':'강원특별자치도'}
STD_BD_GRD4_ELPDSRC_CURSTT['CTPV'] = STD_BD_GRD4_ELPDSRC_CURSTT['CTPV'].replace(elpd_dict)
STD_BD_GRD4_ELPDSRC_CURSTT['CTPV'].unique()

array(['경기도', '강원특별자치도', '서울특별시', '제주특별자치도', '부산광역시', '인천광역시', '충청남도',
       '경상북도', '울산광역시', '전라북도', '전라남도', '경상남도', '충청북도', '대전광역시', '광주광역시',
       '세종특별자치시', '대구광역시'], dtype=object)

In [217]:
STD_BD_GRD4_ELPDSRC_CURSTT['RGN'] = STD_BD_GRD4_ELPDSRC_CURSTT['CTPV'].copy()

In [218]:
## 11.1s
ctpv_dict = {
    '경상북도':'경북',
    '충청남도':'충북',
    '경기도':'경기', 
    '강원도':'강원',
    '강원특별자치도':'강원', 
    '경상남도':'경남', 
    '서울특별시':'서울',
    '전라북도':'전북',
    '부산광역시':'부산',
    '전라남도':'전남',
    '대전광역시':'대전',
    '충청북도':'충북',
    '인천광역시':'인천',
    '울산광역시':'울산',
    '대구광역시':'대구',
    '제주특별자치도':'제주',
    '광주광역시':'광주',
    '세종특별자치시':'세종',
            }
STD_BD_GRD4_ELPDSRC_CURSTT['RGN'] = STD_BD_GRD4_ELPDSRC_CURSTT['RGN'].replace(ctpv_dict)
STD_BD_GRD4_ELPDSRC_CURSTT['RGN'].unique()

array(['경기', '강원', '서울', '제주', '부산', '인천', '충북', '경북', '울산', '전북', '전남',
       '경남', '대전', '광주', '세종', '대구'], dtype=object)

### [출력] STD_BD_GRD4_ELPDSRC_CURSTT

In [219]:
# expdf = STD_BD_GRD4_ELPDSRC_CURSTT
# table_nm = 'STD_BD_GRD4_ELPDSRC_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

#### !!! 수정 시작(2023.10.27)  

In [220]:
#### !!! 수정 시작(2023.10.27)  
csece_rct['조기폐차상태코드'].value_counts(dropna=False)

조기폐차상태코드
NaN    1853188
Y        68869
G         1872
D           78
N            7
X            3
C            2
M            2
Name: count, dtype: int64

In [221]:
csece_rct['연료'].value_counts(dropna=False)

연료
경유             1426749
휘발유             458650
LPG(액화석유가스)      38571
기타연료                39
CNG(압축천연가스)         11
알코올                  1
Name: count, dtype: int64

In [222]:
csece_rct_dgl = csece_rct[csece_rct['연료'] == '경유'].reset_index(drop=True)
csece_rct_dgl.shape

(1426749, 34)

In [223]:
csece_rct_dgl['조기폐차상태코드'].value_counts(dropna=False)

조기폐차상태코드
NaN    1355916
Y        68869
G         1872
D           78
N            7
X            3
C            2
M            2
Name: count, dtype: int64

In [224]:
csece_rct_dgl.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '차량관리번호', '자동차등록번호', '검사유효일', '소유자구분', '연료',
       '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '차종분류', '총중량', '적재중량', '배기량',
       '엔진출력', '시도', '시군구', '시도코드', '시군구코드', '조기폐차상태코드', '조기폐차최종승인YN', '말소일자',
       '조기폐차신청여부'],
      dtype='object')

In [225]:
## 10.1s
STD_BD_GRD4_NOW_ELPDSRC_CURSTT = csece_rct_dgl[[
    '차대번호', 
    '법정동코드', 
    # '차종', 
    # '용도', 
    '연료', 
    # '차종유형', 
    '시도', 
    '시군구', 
    '조기폐차상태코드', 
    '조기폐차최종승인YN', 
]]

In [226]:
STD_BD_GRD4_NOW_ELPDSRC_CURSTT['테이블생성일자'] = today_date

C:\Users\kbjung\AppData\Local\Temp\ipykernel_2144\3872026400.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  STD_BD_GRD4_NOW_ELPDSRC_CURSTT['테이블생성일자'] = today_date


In [227]:
STD_BD_GRD4_NOW_ELPDSRC_CURSTT = STD_BD_GRD4_NOW_ELPDSRC_CURSTT[[
    '차대번호', 
    '법정동코드', 
    # '차종', 
    # '용도', 
    '연료', 
    # '차종유형', 
    '시도', 
    '시군구', 
    '조기폐차상태코드', 
    '조기폐차최종승인YN',
    '테이블생성일자', 
]]

In [228]:
chc_dict = {
    # '기준연월':'CRTR_YM', 
    '차대번호':'VIN', 
    '법정동코드':'STDG_CD', 
    # '차종':'VHCTY_CD', 
    # '용도':'PURPS_CD2', 
    '연료':'FUEL_CD', 
    # '차종유형':'VHCTY_TY', 
    '시도':'CTPV', 
    '시군구':'SGG', 
    '조기폐차상태코드':'ELPDSRC_STTS_CD',
    '조기폐차최종승인YN':'ELPDSRC_LAST_APRV_YN', 
    '테이블생성일자':'LOAD_DT', 
}

In [229]:
STD_BD_GRD4_NOW_ELPDSRC_CURSTT = STD_BD_GRD4_NOW_ELPDSRC_CURSTT.rename(columns=chc_dict)
STD_BD_GRD4_NOW_ELPDSRC_CURSTT.columns

Index(['VIN', 'STDG_CD', 'FUEL_CD', 'CTPV', 'SGG', 'ELPDSRC_STTS_CD',
       'ELPDSRC_LAST_APRV_YN', 'LOAD_DT'],
      dtype='object')

In [230]:
STD_BD_GRD4_NOW_ELPDSRC_CURSTT.head()

,VIN,STDG_CD,FUEL_CD,CTPV,SGG,ELPDSRC_STTS_CD,ELPDSRC_LAST_APRV_YN,LOAD_DT
0,KMJWA37JBBU252574,4163000000,경유,경기도,양주시,NaN,NaN,20240213
1,KMFZCS7JP8U380996,4783000000,경유,경상북도,고령군,NaN,NaN,20240213
2,KNCWJZ74ABK555695,4688000000,경유,전라남도,장성군,NaN,NaN,20240213
3,KMFZCZ7JABU648198,5178000000,경유,강원특별자치도,철원군,NaN,NaN,20240213
4,VF33HRHRJ8S005333,4613000000,경유,전라남도,여수시,NaN,NaN,20240213


In [231]:
# erase_dict = {
#     'A32E':'조기폐차상태코드(추가보조금신청대상)',
#     'A32G':'조기폐차상태코드(보조금청구)',
#     'A32I':'조기폐차상태코드(신청등록)',
#     'A32K':'조기폐차상태코드(추가보조금청구승인)',
#     'A32M':'조기폐차상태코드(보조금산정)',
#     'A32N':'조기폐차상태코드(보조금청구반려(제외))',
#     'A32P':'조기폐차상태코드(보조금대상)',
#     'A32T':'조기폐차상태코드(추가보조금청구)',
#     'A32X':'조기폐차상태코드(신청취소(제외))',
#     'A32Y':'조기폐차상태코드(보조금청구승인)',
#     'A32C':'조기폐차상태코드(성능확인검사등록)',
#     'A32D':'조기폐차상태코드(기간초과)',
#     'A32A':'조기폐차상태코드(성능확인검사신청)',
#     'A32B':'조기폐차상태코드(보조금미대상)',
# }
STD_BD_GRD4_NOW_ELPDSRC_CURSTT.ELPDSRC_STTS_CD.value_counts(dropna=False)

ELPDSRC_STTS_CD
NaN    1355916
Y        68869
G         1872
D           78
N            7
X            3
C            2
M            2
Name: count, dtype: int64

### [출력] STD_BD_GRD4_NOW_ELPDSRC_CURSTT

In [232]:
# expdf = STD_BD_GRD4_NOW_ELPDSRC_CURSTT
# table_nm = 'STD_BD_GRD4_NOW_ELPDSRC_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [233]:
# STD_BD_GRD4_NOW_ELPDSRC_CURSTT.to_csv(f'STD_BD_GRD4_NOW_ELPDSRC_CURSTT({today_date}).csv', index=False)

#### !!! 수정 끝(2023.10.27) 

In [234]:
#### !!! 수정 끝(2023.10.27) 
csm.shape

(1318161, 26)

In [235]:
# 11.8s
## 등록&제원&저감이력 병합
csa = csm.merge(attr[['차대번호', 'DPF_YN']], on='차대번호', how='left')
csa.shape

(1318161, 27)

In [236]:
## 12.2s
csa['법정동코드'] = csa['법정동코드'].astype('str')
csa['법정동코드'] = csa['법정동코드'].str[:5] + '00000'
csa['법정동코드'] = pd.to_numeric(csa['법정동코드'])

In [237]:
## 10.5s
## 지역정보 병합
df = csa.merge(coder, on='법정동코드', how='left')
df.shape

(1318161, 31)

In [238]:
df['시도'].isnull().sum()

0

In [239]:
df['법정동코드_mod'] = df['법정동코드'].copy()

In [240]:
## 14.2
## 4등급 result 파일 참고하여 DPF유무 수정
rdf = df.copy()
rs = rs.drop_duplicates('차대번호').reset_index(drop=True)
rdf1 = rdf.merge(rs, on='차대번호', how='left')
rdf1.loc[(rdf1['DPF_YN'] == '유') | (rdf1['DPF유무_수정'] == '유'), 'DPF_YN'] = '유'
rdf1.loc[(rdf1['DPF유무_수정'] == '무'), 'DPF_YN'] = '무'
rdf1.loc[(rdf1['DPF유무_수정'] == '확인불가'), 'DPF_YN'] = '확인불가'
df = rdf1.drop('DPF유무_수정', axis=1)
df.shape

(1318161, 32)

In [241]:
cs5m.shape

(906192, 26)

In [242]:
# 11.7s
## 등록&제원&저감이력 병합
cs5a = cs5m.merge(attr[['차대번호', 'DPF_YN']], on='차대번호', how='left')
cs5a.shape

(906192, 27)

In [243]:
## 11.5s
cs5a['법정동코드'] = cs5a['법정동코드'].astype('str')
cs5a['법정동코드'] = cs5a['법정동코드'].str[:5] + '00000'
cs5a['법정동코드'] = pd.to_numeric(cs5a['법정동코드'])

In [244]:
## 10.4s
## 지역정보 병합
df_g5 = cs5a.merge(coder, on='법정동코드', how='left')
df_g5.shape

(906192, 31)

In [245]:
df_g5['시도'].isnull().sum()

0

In [246]:
df_g5['법정동코드_mod'] = df_g5['법정동코드'].copy()

In [247]:
## 13.6s
## 4등급 result 파일 참고하여 DPF유무 수정
rdf = df_g5.copy()
rs = rs.drop_duplicates('차대번호').reset_index(drop=True)
rdf1 = rdf.merge(rs, on='차대번호', how='left')
rdf1.loc[(rdf1['DPF_YN'] == '유') | (rdf1['DPF유무_수정'] == '유'), 'DPF_YN'] = '유'
rdf1.loc[(rdf1['DPF유무_수정'] == '무'), 'DPF_YN'] = '무'
rdf1.loc[(rdf1['DPF유무_수정'] == '확인불가'), 'DPF_YN'] = '확인불가'
df_g5 = rdf1.drop('DPF유무_수정', axis=1)
df_g5.shape

(906192, 32)

In [248]:
## 10.7s
## 연료 컬럼 추가
df1 = df.copy()
df1.loc[df1['연료'] == '경유', 'fuel'] = '경유'
df1.loc[(df1['연료'] == '휘발유') | (df1['연료'] == 'LPG(액화석유가스)'), 'fuel'] = '휘발유_가스'
df1.shape

(1318161, 33)

In [249]:
## 12.8s
# 분석
## EG 분류
grade_list = []
for f, y, cy, e in df1[['fuel', '제작일자', '차량연식', 'DPF_YN']].values:
    if (f == '휘발유_가스') and ( (19980101 <= y <= 20001231) or (1998 <= cy <= 2000) ):
        grade_list.append('A')
    elif (f == '휘발유_가스') and ( (y <= 19971231) or (cy <= 1997) ):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '유'):
        grade_list.append('A')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) )and (e == '유'):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '무'):
        grade_list.append('C')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) ) and (e == '무'):
        grade_list.append('D')
    else:
        grade_list.append('X')
df1['Grade'] = grade_list

In [250]:
df1.groupby('Grade', dropna=False)['차대번호'].count()

Grade
A    182218
B    351012
C    733017
D     29220
X     22694
Name: 차대번호, dtype: int64

In [251]:
STD_BD_GRD4_MLSFC_RSLT = df1[[
    '차대번호', 
    '제원관리번호',
    '차종', 
    '용도', 
    '차량연식', 
    '차종유형', 
    '연료', 
    '법정동코드', 
    '시도', 
    '시군구', 
    'DPF_YN',
    'Grade',
    '법정동코드_mod',
    ]]

In [252]:
today_date = datetime.today().strftime("%Y%m%d")
STD_BD_GRD4_MLSFC_RSLT['테이블생성일자'] = today_date
STD_BD_GRD4_MLSFC_RSLT = STD_BD_GRD4_MLSFC_RSLT[[
    '테이블생성일자', 
    '차대번호', 
    '제원관리번호', 
    '차종', 
    '용도', 
    '차량연식', 
    '차종유형', 
    '연료', 
    '법정동코드', 
    '시도',
    '시군구', 
    'DPF_YN', 
    'Grade', 
    '법정동코드_mod',
    ]]

C:\Users\kbjung\AppData\Local\Temp\ipykernel_2144\4082969194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  STD_BD_GRD4_MLSFC_RSLT['테이블생성일자'] = today_date


In [253]:
# RH법정동코드 문자형으로 변환
STD_BD_GRD4_MLSFC_RSLT['법정동코드_mod'] = STD_BD_GRD4_MLSFC_RSLT['법정동코드_mod'].astype('str')

In [254]:
## 10.4s
ch_col_dict = {
                '테이블생성일자':'LOAD_DT',
                '차대번호':'VIN', 
                '제원관리번호':'MANG_MNG_NO',
                '차종':'VHCTY_CD', 
                '용도':'PURPS_CD2',
                '차량연식':'YRIDNW', 
                '차종유형':'VHCTY_TY', 
                '연료':'FUEL_CD', 
                '법정동코드':'STDG_CD', 
                '시도':'CTPV_NM',
                '시군구':'SGG_NM',
                'DPF_YN':'DPF_MNTNG_YN',
                'Grade':'GRD4_MLSFC', 
                '법정동코드_mod':'STDG_CD_MOD'
                }
STD_BD_GRD4_MLSFC_RSLT = STD_BD_GRD4_MLSFC_RSLT.rename(columns=ch_col_dict)

STD_BD_GRD4_MLSFC_RSLT.columns

Index(['LOAD_DT', 'VIN', 'MANG_MNG_NO', 'VHCTY_CD', 'PURPS_CD2', 'YRIDNW',
       'VHCTY_TY', 'FUEL_CD', 'STDG_CD', 'CTPV_NM', 'SGG_NM', 'DPF_MNTNG_YN',
       'GRD4_MLSFC', 'STDG_CD_MOD'],
      dtype='object')

### [출력] STD_BD_GRD4_MLSFC_RSLT

In [255]:
# expdf = STD_BD_GRD4_MLSFC_RSLT
# table_nm = 'STD_BD_GRD4_MLSFC_RSLT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [256]:
# 12.8s
## 4등급 등급세분류
dat_mlsfc = df1.copy()
dat_mlsfc['시군구_수정'] = dat_mlsfc['시군구'].str.split(' ').str[0]

In [257]:
dat_mlsfc['시군구_수정'].isnull().sum()

7012

In [258]:
dat_mlsfc.loc[dat_mlsfc['시군구_수정'].isnull(), '시도'].unique()

array(['세종특별자치시'], dtype=object)

In [259]:
dat_mlsfc['연료'].isnull().sum()

0

In [260]:
dat_mlsfc.loc[dat_mlsfc['연료'].isnull(), '연료'] = '결측' # !!! 수정(2023.11.13)
dat_mlsfc.loc[dat_mlsfc['연료'].isnull(), '연료'].unique()

array([], dtype=object)

In [261]:
dat_mlsfc.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '차량관리번호', '자동차등록번호', '검사유효일', '소유자구분', '연료',
       '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '차종분류', '총중량', '적재중량', '배기량',
       '엔진출력', 'DPF_YN', '시도', '시군구', '시도코드', '시군구코드', '법정동코드_mod', 'fuel',
       'Grade', '시군구_수정'],
      dtype='object')

In [262]:
dat_mlsfc[dat_mlsfc['연료'].isnull()].shape

(0, 35)

In [263]:
dat_mlsfc[dat_mlsfc['연료'].isnull()].head()

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN,제원관리번호,차량연식,차종,용도,최초등록일자,...,엔진출력,DPF_YN,시도,시군구,시도코드,시군구코드,법정동코드_mod,fuel,Grade,시군구_수정


In [264]:
len(set(dat_mlsfc.loc[dat_mlsfc['연료'].isnull(), '제원관리번호'].to_list()) - set(srcr['제원관리번호'].to_list()))

0

In [265]:
srcr.loc[srcr['제원관리번호'] == '99934703006081306', '연료'].values[0]

'9999'

In [266]:
srcr_list = []
for one in dat_mlsfc.loc[dat_mlsfc['연료'].isnull(), '제원관리번호'].to_list():
    temp = srcr.loc[srcr['제원관리번호'] == one, '연료'].values[0]
    srcr_list.append(temp)
len(srcr_list), set(srcr_list)

(0, set())

In [267]:
# dat_mlsfc.loc[dat_mlsfc['연료'].isnull()].to_excel('샘플_STD_BD_DAT_GRD4_MLSFC_연료없는차량(2023.11.13).xlsx', index=False)

In [268]:
dat_idx = dat_mlsfc.loc[dat_mlsfc['연료'].isnull()].index
len(dat_idx)

0

In [269]:
dat_mlsfc.loc[dat_mlsfc['연료'].isnull(), '연료'] = '결측' # !!! 수정(2023.12.22)
dat_mlsfc.loc[dat_idx, '연료'].unique()

array([], dtype=object)

In [270]:
dat_mlsfc.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '차량관리번호', '자동차등록번호', '검사유효일', '소유자구분', '연료',
       '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '차종분류', '총중량', '적재중량', '배기량',
       '엔진출력', 'DPF_YN', '시도', '시군구', '시도코드', '시군구코드', '법정동코드_mod', 'fuel',
       'Grade', '시군구_수정'],
      dtype='object')

In [271]:
grp1 = dat_mlsfc.groupby(['연료', '시도', '시군구_수정', '차종', '차종유형', '용도', '차량연식', 'Grade'], dropna=False)['차대번호'].count().unstack('Grade').reset_index() # !!! 수정(2023.11.13)
grp1

Grade,연료,시도,시군구_수정,차종,차종유형,용도,차량연식,A,B,C,D,X
0,CNG(압축천연가스),전라남도,해남군,화물,대형,개인용,2005,NaN,NaN,NaN,NaN,1.0
1,LPG(액화석유가스),강원특별자치도,강릉시,승용,대형,개인용,1998,1.0,NaN,NaN,NaN,NaN
2,LPG(액화석유가스),강원특별자치도,강릉시,승용,대형,개인용,2002,NaN,NaN,NaN,NaN,1.0
3,LPG(액화석유가스),강원특별자치도,강릉시,승용,중형,개인용,1992,NaN,1.0,NaN,NaN,NaN
4,LPG(액화석유가스),강원특별자치도,강릉시,승용,중형,개인용,1994,NaN,3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
38253,휘발유,충청북도,충주시,화물,소형,개인용,1991,NaN,5.0,NaN,NaN,NaN
38254,휘발유,충청북도,충주시,화물,소형,개인용,1992,NaN,5.0,NaN,NaN,NaN
38255,휘발유,충청북도,충주시,화물,소형,개인용,1993,NaN,1.0,NaN,NaN,NaN
38256,휘발유,충청북도,충주시,화물,소형,개인용,1994,NaN,1.0,NaN,NaN,NaN


In [272]:
grp1['연료'].unique()

array(['CNG(압축천연가스)', 'LPG(액화석유가스)', '경유', '기타연료', '알코올', '휘발유'],
      dtype=object)

In [273]:
grp1[['A', 'B', 'C', 'D', 'X']].sum()

Grade
A    182218.0
B    351012.0
C    733017.0
D     29220.0
X     22694.0
dtype: float64

In [274]:
# 연도 설정
# grp1['연도'] = '2022'
today_date = datetime.today().strftime("%Y%m%d")
grp1['연도'] = today_date[:4]
grp1['테이블생성일자'] = today_date

In [275]:
dat_mlsfc.shape

(1318161, 35)

In [276]:
dat_mlsfc['Grade'].value_counts(dropna=False).reset_index().sort_values('Grade')

,Grade,count
2,A,182218
1,B,351012
0,C,733017
3,D,29220
4,X,22694


In [277]:
dat_mlsfc[['연료', '시도', '시군구_수정', '차종', '차종유형', '용도', 'Grade']].isnull().sum()

연료           0
시도           0
시군구_수정    7012
차종           0
차종유형         0
용도           0
Grade        0
dtype: int64

In [278]:
grp1.head()

Grade,연료,시도,시군구_수정,차종,차종유형,용도,차량연식,A,B,C,D,X,연도,테이블생성일자
0,CNG(압축천연가스),전라남도,해남군,화물,대형,개인용,2005,NaN,NaN,NaN,NaN,1.0,2024,20240213
1,LPG(액화석유가스),강원특별자치도,강릉시,승용,대형,개인용,1998,1.0,NaN,NaN,NaN,NaN,2024,20240213
2,LPG(액화석유가스),강원특별자치도,강릉시,승용,대형,개인용,2002,NaN,NaN,NaN,NaN,1.0,2024,20240213
3,LPG(액화석유가스),강원특별자치도,강릉시,승용,중형,개인용,1992,NaN,1.0,NaN,NaN,NaN,2024,20240213
4,LPG(액화석유가스),강원특별자치도,강릉시,승용,중형,개인용,1994,NaN,3.0,NaN,NaN,NaN,2024,20240213


In [279]:
grp1[['A', 'B', 'C', 'D', 'X']].sum()

Grade
A    182218.0
B    351012.0
C    733017.0
D     29220.0
X     22694.0
dtype: float64

In [280]:
## 컬럼 변경
STD_BD_DAT_GRD4_MLSFC = grp1[[
    '연도', 
    '연료', 
    '시도', 
    '시군구_수정', 
    '차종', 
    '차종유형', 
    '용도',
    '차량연식', # !!! 수정(2023.12.08)
    'A', 
    'B', 
    'C', 
    'D', 
    'X',
    '테이블생성일자',
]]
cdict = {
    '연도':'YR', 
    '연료':'FUEL_CD', 
    '시도':'CTPV', 
    '시군구_수정':'SGG', 
    '차종':'VHCTY_CD', 
    '차종유형':'VHCTY_TY', 
    '용도':'PURPS_CD2', 
    '차량연식':'YRIDNW', # !!! 수정(2023.12.08)
    'A':'A_MKCNT', 
    'B':'B_MKCNT', 
    'C':'C_MKCNT', 
    'D':'D_MKCNT', 
    'X':'X_MKCNT', 
    '테이블생성일자':'LOAD_DT', 
}
STD_BD_DAT_GRD4_MLSFC = grp1.rename(columns=cdict)

In [281]:
STD_BD_DAT_GRD4_MLSFC.head()

Grade,FUEL_CD,CTPV,SGG,VHCTY_CD,VHCTY_TY,PURPS_CD2,YRIDNW,A_MKCNT,B_MKCNT,C_MKCNT,D_MKCNT,X_MKCNT,YR,LOAD_DT
0,CNG(압축천연가스),전라남도,해남군,화물,대형,개인용,2005,NaN,NaN,NaN,NaN,1.0,2024,20240213
1,LPG(액화석유가스),강원특별자치도,강릉시,승용,대형,개인용,1998,1.0,NaN,NaN,NaN,NaN,2024,20240213
2,LPG(액화석유가스),강원특별자치도,강릉시,승용,대형,개인용,2002,NaN,NaN,NaN,NaN,1.0,2024,20240213
3,LPG(액화석유가스),강원특별자치도,강릉시,승용,중형,개인용,1992,NaN,1.0,NaN,NaN,NaN,2024,20240213
4,LPG(액화석유가스),강원특별자치도,강릉시,승용,중형,개인용,1994,NaN,3.0,NaN,NaN,NaN,2024,20240213


### [출력] STD_BD_DAT_GRD4_MLSFC

In [282]:
# expdf = STD_BD_DAT_GRD4_MLSFC
# table_nm = 'STD_BD_DAT_GRD4_MLSFC'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [283]:
# STD_BD_DAT_GRD4_MLSFC.to_csv('STD_BD_DAT_GRD4_MLSFC_20231211.csv', index=False)

In [284]:
## 13.5s
## 4등급차량 상세정보
# cst = carr.merge(srcr, on='제원관리번호', how='left')
cst = cs.copy()
csat = cst.merge(attr, on='차대번호', how='left')

In [285]:
## 13.3s
csat['법정동코드'] = csat['법정동코드'].astype('str')
csat['법정동코드'] = csat['법정동코드'].str[:5] + '00000'
csat['법정동코드'] = pd.to_numeric(csat['법정동코드'])

In [286]:
csact = csat.merge(coder, on='법정동코드', how='left')

In [287]:
csact.shape

(1924021, 33)

In [288]:
csact['시도'].isnull().sum()

0

In [289]:
## 경기도 양주시
# csact.loc[csact['법정동코드'] == 4163055000, ['시도', '시군구']] = ['경기도', '양주시']

###  !!! 수정 시작(2023.11.27) 

In [290]:
###  !!! 수정 시작(2023.11.27) 
## 1m 03.3s
csacit = csact.merge(insm[['차대번호', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연판정1']], on='차대번호', how='left')
csacit.shape

(1924021, 37)

In [291]:
## 12.6s
## 조기폐차 정보추가
dft = csacit.merge(elpm, on='차대번호', how='left')
dft.shape

(1924021, 41)

###  !!! 수정 끝(2023.11.27) 

In [292]:
###  !!! 수정 끝(2023.11.27) 
## 16.8s
### 4등급 result 파일 참고하여 DPF유무 수정
rdf = dft.copy()
rs = rs.drop_duplicates('차대번호').reset_index(drop=True)
rdf1 = rdf.merge(rs, on='차대번호', how='left')
rdf1.loc[(rdf1['DPF_YN'] == '유') | (rdf1['DPF유무_수정'] == '유'), 'DPF_YN'] = '유'
rdf1.loc[(rdf1['DPF유무_수정'] == '무'), 'DPF_YN'] = '무'
rdf1.loc[(rdf1['DPF유무_수정'] == '확인불가'), 'DPF_YN'] = '확인불가'
dft = rdf1.drop('DPF유무_수정', axis=1)

In [293]:
# 4등급 연월, 시도, 시군구별 차량대수
## 등록 & 제원 정보 병합(말소 유지)
# csersr = carr.merge(srcr, on='제원관리번호', how='left')
csersr = cs.copy()

In [294]:
## 3m 9.0s
## 1\. 차량관리번호 기준 병합
ersr = csersr.merge(hisr, on='차량관리번호', how='left')

In [295]:
ersr

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN_x,제원관리번호,차량연식,차종,용도,최초등록일자,...,차명,자동차형식,엔진형식,차종분류,총중량,적재중량,배기량,엔진출력,차량말소YN_y,변경일자
0,KMHJF21JPNU189564,2729011400,4,KMY-HD-04,N,10022700070002,1992,승용,개인용,19920314,...,엘란트라,JL-M-3,G4DJ,A2826,1300,0,1468,90,N,20220425.0
1,KMJWA37JBBU252574,4163034024,4,7MY-HD-14-46,N,A0810007402962210,2011,승합,개인용,20100426,...,그랜드 스타렉스(GRAND STAREX),TQ-12JAB-1,D4CB,A2826,2990,0,2497,174,N,20230711.0
2,KMJWA37JBBU252574,4163034024,4,7MY-HD-14-46,N,A0810007402962210,2011,승합,개인용,20100426,...,그랜드 스타렉스(GRAND STAREX),TQ-12JAB-1,D4CB,A2826,2990,0,2497,174,N,20220425.0
3,KMJWA37JBBU252574,4163034024,4,7MY-HD-14-46,N,A0810007402962210,2011,승합,개인용,20100426,...,그랜드 스타렉스(GRAND STAREX),TQ-12JAB-1,D4CB,A2826,2990,0,2497,174,N,20190723.0
4,KMJWA37JBBU252574,4163034024,4,7MY-HD-14-46,N,A0810007402962210,2011,승합,개인용,20100426,...,그랜드 스타렉스(GRAND STAREX),TQ-12JAB-1,D4CB,A2826,2990,0,2497,174,N,20221026.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5226996,KMJHD17HP8C002053,5113011200,4,7MY-HD-24-28,N,A0810005602262207,2008,승합,개인용,20080912,...,이-카운티,HCS15-8LFb-C2,D4GA,A2826,5215,0,3933,150,N,20230609.0
5226997,KMJHD17HP8C002053,5113011200,4,7MY-HD-24-28,N,A0810005602262207,2008,승합,개인용,20080912,...,이-카운티,HCS15-8LFb-C2,D4GA,A2826,5215,0,3933,150,N,20230611.0
5226998,KMJHD17HP8C002053,5113011200,4,7MY-HD-24-28,N,A0810005602262207,2008,승합,개인용,20080912,...,이-카운티,HCS15-8LFb-C2,D4GA,A2826,5215,0,3933,150,N,20200210.0
5226999,KMJHD17HP8C002053,5113011200,4,7MY-HD-24-28,N,A0810005602262207,2008,승합,개인용,20080912,...,이-카운티,HCS15-8LFb-C2,D4GA,A2826,5215,0,3933,150,N,20200526.0


In [296]:
## 11.2s
## 2\. 차량말소YN 만 추출
errm = ersr[(ersr['차량말소YN_x'] == 'Y') & (ersr['차량말소YN_y'] == 'Y')].reset_index(drop=True)

In [297]:
errm

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN_x,제원관리번호,차량연식,차종,용도,최초등록일자,...,차명,자동차형식,엔진형식,차종분류,총중량,적재중량,배기량,엔진출력,차량말소YN_y,변경일자
0,VF3WC5FWF8W002348,4580032028,4,결측,Y,00620000700031207,2008,승용,개인용,20071115,...,Nouvelle Peugeot 207,WA,5FW,A2826,1645,0,1598,120,Y,20220908.0
1,KMHCF31FPTU722143,2726010100,4,결측,Y,10021000370001,1996,승용,개인용,19960830,...,쏘나타III2.0DOHC,Y3-DG-S-A-4,G4CP,A2826,1660,0,1997,146,Y,20210625.0
2,KMHCF31FPTU722143,2726010100,4,결측,Y,10021000370001,1996,승용,개인용,19960830,...,쏘나타III2.0DOHC,Y3-DG-S-A-4,G4CP,A2826,1660,0,1997,146,Y,20210625.0
3,KMHWP81VP2U429938,2823710800,4,XMY-HD-55,Y,10080401640000,2002,승용,개인용,20020328,...,스타렉스장축9인,HA1V6ML-1,L6AT,A2821,2540,0,2972,135,Y,20230410.0
4,KNCSE03429K375543,4783025321,4,8MY-KM-13-19,Y,A0110003115513108,2009,화물,개인용,20090106,...,봉고Ⅲ 1톤,SEL42K-L-A,J3,A2809,2970,1000,2902,126,Y,20200221.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640041,KNHMB76438S239472,4122011800,4,9MY-KM-14-41,Y,A0110003600592207,2008,승합,개인용,20080314,...,그랜드 카니발,MBB43E-LS-SR,J3,A2826,2960,0,2902,192,Y,20230613.0
640042,KMHSJ81XBAU555611,2826012200,4,결측,Y,00034703000011323,2010,승용,개인용,20230303,...,싼타페(SANTAFE),CM7XBA,D4HB,A2819,2295,0,2199,200,Y,20230418.0
640043,KMFZSY7JACU767880,4374025000,4,결측,Y,A0810006210843111,2012,화물,개인용,20220923,...,포터Ⅱ (PORTERⅡ),HREJA-DX-D,D4CB,A2809,3250,1000,2497,126,Y,20230726.0
640044,KMFZCY7JAAU633300,4374025000,4,결측,Y,A0810006208493109,2010,화물,개인용,20220923,...,포터Ⅱ(PORTERⅡ),HREJA-HI-A,D4CB,A2809,3270,1000,2497,126,Y,20230726.0


In [298]:
## 11.1s
## 3\. 변경일자 최신으로 차대번호 중복 제거
errm = errm.sort_values('변경일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)

In [299]:
## 4\. 변경일자 2019.01.01 이상만 추출
errm = errm[errm['변경일자'] >= 20190101].reset_index(drop=True)

In [300]:
errm

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN_x,제원관리번호,차량연식,차종,용도,최초등록일자,...,차명,자동차형식,엔진형식,차종분류,총중량,적재중량,배기량,엔진출력,차량말소YN_y,변경일자
0,KMHJP81VP7U695662,5121010800,4,5MY-HD-24-59,Y,A0810006301241206,2007,승용,개인용,20070619,...,투싼(TUCSON),JM-1VAF7-1,D4EA,A2819,1985,0,1991,146,Y,20231101.0
1,KMHJN81VP7U725237,4715011400,4,5MY-HD-24-59,Y,A0810006301201206,2007,승용,개인용,20070817,...,투싼(TUCSON),JM-3VAF7-1,D4EA,A2819,1965,0,1991,146,Y,20231101.0
2,KMHSJ81VP8U275598,5115011000,4,6MY-HD-24-100,Y,A0810006800831207,2008,승용,개인용,20071017,...,싼타페(SANTAFE),CM-R2DBF8-1,D4EA,A2819,2345,0,1991,151,Y,20231101.0
3,KMFZCS7JP9U479327,4136036024,4,7MY-HD-14-68,Y,A0810006207343108,2009,화물,개인용,20090204,...,포터Ⅱ(PORTERⅡ),HR-3SS2JKM9-1,D4CB,A2809,2950,1000,2497,126,Y,20231101.0
4,KNAJE55539K627591,3120012200,4,6MY-KM-14-61,Y,A0110003400581208,2009,승용,개인용,20081217,...,스포티지,JE553J-HG,D4EA,A2819,1960,0,1991,151,Y,20231101.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449104,KMHJF31KPYU988484,4136025328,4,RMY-HD-20,Y,10044101040013,2000,승용,개인용,20000121,...,아반떼오토메틱,JS-DL15-M1,G4FK,A2826,1460,0,1495,95,Y,20190620.0
449105,KLATF48YDXB490959,4831010600,4,결측,Y,10073900180002,1999,승용,개인용,19991006,...,라노스해치백1.5,TF48Y-1,A15SMS,A2826,1320,0,1498,94,Y,20190604.0
449106,KMFZCX7JAAU567310,4715012000,4,7MY-HD-14-68,Y,A0810006208223109,2010,화물,개인용,20091009,...,포터Ⅱ(PORTERⅡ),HRTJA-DX-A,D4CB,A2809,2905,1000,2497,126,Y,20190521.0
449107,KMHEN41BP1A477461,4182025024,4,WMY-HD-32,Y,10106100460000,2001,승용,개인용,20010428,...,NEW EF쏘나타,YS-20DI-A1,G4JP,A2826,1785,0,1997,133,Y,20190515.0


In [301]:
## 10.08s
## 5\. 법정동 코드 정보 병합
errm['법정동코드'] = errm['법정동코드'].astype('str')
errm['법정동코드'] = errm['법정동코드'].str[:5] + '00000'
errm['법정동코드'] = pd.to_numeric(errm['법정동코드'])
errc = errm.merge(coder, on='법정동코드', how='left')

In [302]:
errc['시도'].isnull().sum()

0

In [303]:
## 12.2s
# 5등급 연월, 시도, 시군구별 차량대수
## 등록 & 제원 정보 병합(말소 유지)
# csersr = carr.merge(srcr, on='제원관리번호', how='left')
cs5ersr = cs5.copy()

In [304]:
## 2m 47.3s
## 1\. 차량관리번호 기준 병합
ersr5 = cs5ersr.merge(hisr, on='차량관리번호', how='left')

In [305]:
## 11.9s
## 2\. 차량말소YN 만 추출
errm5 = ersr5[(ersr5['차량말소YN_x'] == 'Y') & (ersr5['차량말소YN_y'] == 'Y')].reset_index(drop=True)

In [306]:
## 15.6s
## 3\. 변경일자 최신으로 차대번호 중복 제거
errm5 = errm5.sort_values('변경일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)

In [307]:
## 10.9s
## 4\. 변경일자 2019.01.01 이상만 추출
errm5 = errm5[errm5['변경일자'] >= 20190101].reset_index(drop=True)

In [308]:
## 12.2s
## 5\. 법정동 코드 정보 병합
errm5['법정동코드'] = errm5['법정동코드'].astype('str')
errm5['법정동코드'] = errm5['법정동코드'].str[:5] + '00000'
errm5['법정동코드'] = pd.to_numeric(errm5['법정동코드'])
errc5 = errm5.merge(coder, on='법정동코드', how='left')

In [309]:
errc5['시도'].isnull().sum()

0

In [310]:
## 15.4s
dfte = dft.merge(errc[['차대번호', '변경일자']], on='차대번호', how='left')
dftem = dfte.merge(df1[['차대번호', 'Grade']], on='차대번호', how='left')
dfte.shape, dftem.shape

((1924021, 42), (1924075, 43))

In [311]:
dftem.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '차량관리번호', '자동차등록번호', '검사유효일', '소유자구분', '연료',
       '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '차종분류', '총중량', '적재중량', '배기량',
       '엔진출력', '저감장치구분', '저감장치종류', 'DPF_YN', '시도', '시군구', '시도코드', '시군구코드',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연판정1', '조기폐차상태코드',
       '조기폐차최종승인YN', '말소일자', '조기폐차신청여부', '변경일자', 'Grade'],
      dtype='object')

In [312]:
dftem.groupby('Grade', dropna=False)['차대번호'].count()

Grade
A      182222
B      351064
C      733019
D       29220
X       22696
NaN    605854
Name: 차대번호, dtype: int64

In [313]:
dftem.loc[dftem['차량말소YN'] == 'Y', 'Grade'].unique()

array([nan, 'C', 'B', 'A', 'X'], dtype=object)

In [314]:
dftem.loc[dftem['연료'].isnull(), '연료'] = '해당없음' # !!! 수정(2023.12.22)
dftem.loc[dftem['연료'].isnull(), '연료'].unique()

array([], dtype=object)

In [315]:
## 13.3s
## 컬럼 추출
today_date = datetime.today().strftime("%Y%m%d")
dftem['테이블생성일자'] = today_date
STD_BD_DAT_GRD4_DTL_INFO = dftem[[
    '자동차등록번호',
    '차대번호',
    'Grade',
    '차종',
    '차종유형',
    '용도',
    '연료',
    '시도',
    '시군구',
    '차량연식',
    'DPF_YN',
    '저감장치종류',
    '최초등록일자',
    '무부하매연측정치1', # !!! 수정(2023.11.27)
    '무부하매연측정치2', # !!! 수정(2023.11.27)
    '무부하매연측정치3', # !!! 수정(2023.11.27)
    '무부하매연판정1', # !!! 수정(2023.11.27)
    '조기폐차신청여부',
    '조기폐차상태코드',
    '변경일자',
    '차량말소YN', 
    '테이블생성일자', 
    # '법정동코드',
    # '배출가스등급',
    # '배출가스인증번호',
    # '제원관리번호',
    # '제작일자',
    # '차량관리번호',
    # '제작사명',
    # '차명',
    # '자동차형식',
    # '엔진형식',
    # '저감장치구분',
    # '조기폐차최종승인YN',
]]

In [316]:
## 10.8s
## 컬럼 변경
cdict = {
    '자동차등록번호':'VHRNO',
    '차대번호':'VIN',
    'Grade':'GRD4_MLSFC',
    '차종':'VHCTY_CD',
    '차종유형':'VHCTY_TY',
    '용도':'PURPS_CD2',
    '연료':'FUEL_CD',
    '시도':'CTPV',
    '시군구':'SGG',
    '차량연식':'YRIDNW',
    'DPF_YN':'DPF_MNTNG_YN',
    '저감장치종류':'RDCDVC_KND',
    '최초등록일자':'FRST_REG_YMD',
    '무부하매연측정치1':'NLOD_SMO_MEVLU1', # !!! 수정(2023.11.27)
    '무부하매연측정치2':'NLOD_SMO_MEVLU2', # !!! 수정(2023.11.27)
    '무부하매연측정치3':'NLOD_SMO_MEVLU3', # !!! 수정(2023.11.27)
    '무부하매연판정1':'FDRM_NLOD_SMO_JT_YN1', # !!! 수정(2023.11.27)
    '조기폐차신청여부':'ELPDSRC_APLY_YN',
    '조기폐차상태코드':'ELPDSRC_STTS_CD',
    '변경일자':'CHNG_DE',
    '차량말소YN':'VHCL_ERSR_YN',
    '테이블생성일자':'LOAD_DT', 
}
STD_BD_DAT_GRD4_DTL_INFO = STD_BD_DAT_GRD4_DTL_INFO.rename(columns=cdict)

STD_BD_DAT_GRD4_DTL_INFO.columns

Index(['VHRNO', 'VIN', 'GRD4_MLSFC', 'VHCTY_CD', 'VHCTY_TY', 'PURPS_CD2',
       'FUEL_CD', 'CTPV', 'SGG', 'YRIDNW', 'DPF_MNTNG_YN', 'RDCDVC_KND',
       'FRST_REG_YMD', 'NLOD_SMO_MEVLU1', 'NLOD_SMO_MEVLU2', 'NLOD_SMO_MEVLU3',
       'FDRM_NLOD_SMO_JT_YN1', 'ELPDSRC_APLY_YN', 'ELPDSRC_STTS_CD', 'CHNG_DE',
       'VHCL_ERSR_YN', 'LOAD_DT'],
      dtype='object')

In [317]:
STD_BD_DAT_GRD4_DTL_INFO.head()

,VHRNO,VIN,GRD4_MLSFC,VHCTY_CD,VHCTY_TY,PURPS_CD2,FUEL_CD,CTPV,SGG,YRIDNW,...,FRST_REG_YMD,NLOD_SMO_MEVLU1,NLOD_SMO_MEVLU2,NLOD_SMO_MEVLU3,FDRM_NLOD_SMO_JT_YN1,ELPDSRC_APLY_YN,ELPDSRC_STTS_CD,CHNG_DE,VHCL_ERSR_YN,LOAD_DT
0,대구2라1800,KMHJF21JPNU189564,B,승용,소형,개인용,휘발유,대구광역시,달서구,1992,...,19920314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,20240213
1,72도7909,KMJWA37JBBU252574,C,승합,중형,개인용,경유,경기도,양주시,2011,...,20100426,6.00,NaN,NaN,Y,NaN,NaN,NaN,N,20240213
2,30누6268,KLATF69YDTB015250,B,승용,소형,개인용,휘발유,대구광역시,동구,1996,...,19961227,0.01,9.0,50.00,Y,NaN,NaN,NaN,N,20240213
3,80러6589,KMFZCS7JP8U380996,C,화물,소형,개인용,경유,경상북도,고령군,2008,...,20080129,11.00,NaN,NaN,Y,NaN,NaN,NaN,N,20240213
4,25서3808,VF3WC5FWF8W002348,NaN,승용,중형,개인용,휘발유,전라북도,부안군,2008,...,20071115,0.07,16.0,1.09,Y,NaN,NaN,20220908.0,Y,20240213


### [출력] STD_BD_DAT_GRD4_DTL_INFO

In [318]:
# expdf = STD_BD_DAT_GRD4_DTL_INFO
# table_nm = 'STD_BD_DAT_GRD4_DTL_INFO'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [319]:
df.shape

(1318161, 32)

In [320]:
## 15.6s
## 시도, 연도별 차량 대수
dfm = df.copy()
dfm['최초등록일자'] = dfm['최초등록일자'].astype('str')
dfm['최초등록일자_년'] = dfm['최초등록일자'].str[:4]
dfm['최초등록일자_월'] = dfm['최초등록일자'].str[4:6]
dfm['최초등록일자_일'] = dfm['최초등록일자'].str[6:8]
dfm['최초등록일자'] = dfm['최초등록일자_년'] + dfm['최초등록일자_월'] + dfm['최초등록일자_일']
dfm['최초등록일자'] = pd.to_numeric(dfm['최초등록일자'], errors='coerce')

### 시군구명 앞쪽 지역명만 남기기(dfm)
dfm['시군구_수정'] = dfm['시군구'].str.split(' ').str[0]

In [321]:
dfm[['시도', '시군구_수정', '연료', '최초등록일자', '최초등록일자_년', '최초등록일자_월']].isnull().sum()

시도             0
시군구_수정      7012
연료             0
최초등록일자         0
최초등록일자_년       0
최초등록일자_월       0
dtype: int64

In [322]:
dfm['시도'].unique()

array(['대구광역시', '경기도', '경상북도', '전라남도', '강원특별자치도', '광주광역시', '경상남도',
       '서울특별시', '대전광역시', '충청북도', '충청남도', '세종특별자치시', '부산광역시', '전라북도',
       '인천광역시', '울산광역시', '제주특별자치도'], dtype=object)

In [323]:
dfm[dfm['시도'] == '세종특별자치시'].shape

(7012, 36)

In [324]:
dfm.loc[(dfm['최초등록일자_년'] == '2023') & (dfm['최초등록일자_월'] == '09'), ['최초등록일자', '최초등록일자_년', '최초등록일자_월']]

,최초등록일자,최초등록일자_년,최초등록일자_월
1318129,20230926,2023,09
1318130,20230926,2023,09
1318131,20230926,2023,09
1318132,20230926,2023,09
1318133,20230926,2023,09


In [325]:
dfm.loc[(dfm['최초등록일자_년'] == '2023') & (dfm['최초등록일자_월'] == '10'), ['최초등록일자', '최초등록일자_년', '최초등록일자_월']]

,최초등록일자,최초등록일자_년,최초등록일자_월
1318135,20231027,2023,10


In [326]:
dfm.loc[dfm['최초등록일자'].isnull(), ['최초등록일자', '최초등록일자_년', '최초등록일자_월']]

,최초등록일자,최초등록일자_년,최초등록일자_월


In [327]:
### 연료 지역별 차량대수
num_car_by_local1 = dfm.groupby(['연료', '시도', '시군구_수정'], dropna=False)['차대번호'].count().reset_index()
num_car_by_local1 = num_car_by_local1.rename(columns={'차대번호':'차량대수'})

In [328]:
num_car_by_local1['차량대수'].sum()

1318161

In [329]:
num_car_by_local1.isnull().sum()

연료        0
시도        0
시군구_수정    3
차량대수      0
dtype: int64

In [330]:
num_car_by_local1.loc[num_car_by_local1['연료'].isnull(), '차량대수'].sum()

0

###  !!! 수정 시작(2023.12.11) 

In [331]:
###  !!! 수정 시작(2023.12.11) 
# max_date = str(dfm['최초등록일자'].max())
# max_year = max_date[:4]
# max_month = max_date[4:6]

# date = '20220601'
# max_year = '2022'
# max_month = '06'
today_date = datetime.today().strftime("%Y%m%d")
date = today_date
# max_year = today_date[:4]
# max_month = today_date[4:6]
# max_year, max_month

In [332]:
periods = 2 # !!! 수정(2023.08.23)
y_plist = list(pd.date_range(end=date, periods=periods, freq="MS").year) # !!! 수정(2023.08.23)
mth_plist = list(pd.date_range(end=date, periods=periods, freq="MS").month) # !!! 수정(2023.08.23)
y_plist, mth_plist

([2024, 2024], [1, 2])

In [333]:
max_year = str(y_plist[0])
tp_max_month = mth_plist[0] # !!! 수정(2024.01.23)
max_month = f'{tp_max_month:0>2}' # !!! 수정(2024.01.23)
max_year, max_month

('2024', '01')

###  !!! 수정 끝(2023.12.11) 

In [334]:
###  !!! 수정 끝(2023.12.11) 
num_car_by_local1[['연도', '월']] = [max_year, max_month]

###  !!! 수정(2023.10.24) 

In [335]:
###  !!! 수정(2023.10.24) 
dfm['최초등록일자'].max()

20231220

In [336]:
### 연료 지역별 등록차량대수
num_car_by_local2 = dfm.groupby(['연료', '시도', '시군구_수정', '최초등록일자_년', '최초등록일자_월'], dropna=False)['차대번호'].count().reset_index()
num_car_by_local2 = num_car_by_local2.rename(columns={'차대번호':'등록차량대수', '최초등록일자_년':'연도', '최초등록일자_월':'월'})

In [337]:
errc.shape

(449109, 32)

In [338]:
errc[['시도', '시군구', '연료', '변경일자']].isnull().sum()

시도         0
시군구     2729
연료         0
변경일자       0
dtype: int64

In [339]:
### 연료 지역별 말소 대수
errc['변경일자'] = errc['변경일자'].astype('str')
errc['변경일자_년'] = errc['변경일자'].str[:4]
errc['변경일자_월'] = errc['변경일자'].str[4:6]
errc['변경일자_일'] = errc['변경일자'].str[6:8]

In [340]:
### 시군구명 앞쪽 지역명만 남기기(errc)
errc['시군구_수정'] = errc['시군구'].str.split(' ').str[0]

In [341]:
errc.loc[(errc['변경일자_년'] == '2023') & (errc['변경일자_월'] == '09'), ['변경일자', '변경일자_년', '변경일자_월']]

,변경일자,변경일자_년,변경일자_월
15923,20230927.0,2023,09
15924,20230927.0,2023,09
15925,20230927.0,2023,09
15926,20230927.0,2023,09
15927,20230927.0,2023,09
...,...,...,...
30890,20230901.0,2023,09
30891,20230901.0,2023,09
30892,20230901.0,2023,09
30893,20230901.0,2023,09


In [342]:
errc.loc[(errc['변경일자_년'] == '2023') & (errc['변경일자_월'] == '10'), ['변경일자', '변경일자_년', '변경일자_월']]

,변경일자,변경일자_년,변경일자_월
280,20231031.0,2023,10
281,20231031.0,2023,10
282,20231031.0,2023,10
283,20231031.0,2023,10
284,20231031.0,2023,10
...,...,...,...
15918,20231004.0,2023,10
15919,20231004.0,2023,10
15920,20231004.0,2023,10
15921,20231004.0,2023,10


###  !!! 수정(2023.10.24) 

In [343]:
###  !!! 수정(2023.10.24) 
## 10.8s
### 시군구명 앞쪽 지역명만 남기기(errc)
errc['시군구_수정'] = errc['시군구'].str.split(' ').str[0]
grp_erase = errc.loc[errc['변경일자_년'] == max_year].groupby(['변경일자_년', '변경일자_월', '연료', '시도', '시군구_수정'], dropna=False)['차대번호'].count().reset_index()
grp_erase = grp_erase.rename(columns={'차대번호':'말소차량대수', '변경일자_년':'연도', '변경일자_월':'월'})
grp_erase = grp_erase.sort_values(['시도', '시군구_수정'])
grp_erase

,연도,월,연료,시도,시군구_수정,말소차량대수


In [344]:
errc['시군구_수정'].isnull().sum()

2729

In [345]:
errc.loc[errc['시군구_수정'].isnull(), '시도'].unique()

array(['세종특별자치시'], dtype=object)

In [346]:
grp_erase.loc[grp_erase['시군구_수정'].isnull()].head()

,연도,월,연료,시도,시군구_수정,말소차량대수


In [347]:
num_car_by_local1['차량대수'].sum()

1318161

In [348]:
num_car_by_local1.columns

Index(['연료', '시도', '시군구_수정', '차량대수', '연도', '월'], dtype='object')

###  !!! 수정 시작(2023.12.11) 

In [349]:
###  !!! 수정 시작(2023.12.11) 
periods = 13 # !!! 수정(2023.08.23)
y_plist = list(pd.date_range(end=date, periods=periods, freq="MS").year) # !!! 수정(2023.08.23)
mth_plist = list(pd.date_range(end=date, periods=periods, freq="MS").month) # !!! 수정(2023.08.23)
y_plist = y_plist[:-1]
mth_plist = mth_plist[:-1]
y_plist, mth_plist

([2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2024],
 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1])

In [350]:
# yr_list, mth_list, fuel_list, ctpv_list, sgg_list = [], [], [], [], []
# sl = num_car_by_local1.drop_duplicates(['시도', '시군구_수정', '연료']).reset_index(drop=True)
# for ctpv, sgg in sl[['시도', '시군구_수정']].values:
#     for fuel in sl['연료'].unique():
#         for yr, mth in zip(y_plist, mth_plist):
#             mthm = f'{mth:0>2}'
#             yr_list.append(str(yr))
#             mth_list.append(mthm)
#             fuel_list.append(fuel)
#             ctpv_list.append(ctpv)
#             sgg_list.append(sgg)
# base = pd.DataFrame({'연도':yr_list, '월':mth_list, '연료':fuel_list, '시도':ctpv_list, '시군구_수정':sgg_list})

In [351]:
sl = num_car_by_local1.drop_duplicates(['시도', '시군구_수정', '연료']).reset_index(drop=True)

In [352]:
sl.shape

(710, 6)

In [353]:
sl.shape[0] * 12

8520

In [354]:
slt = sl
for _ in range(periods - 2):
    slt = pd.concat([slt, sl], ignore_index=False)
slt.shape

(8520, 6)

In [355]:
slt = slt.sort_values(['시도', '시군구_수정', '연료'])
slt = slt[['시도', '시군구_수정', '연료']]
slt.head(20)

,시도,시군구_수정,연료
1,강원특별자치도,강릉시,LPG(액화석유가스)
1,강원특별자치도,강릉시,LPG(액화석유가스)
1,강원특별자치도,강릉시,LPG(액화석유가스)
1,강원특별자치도,강릉시,LPG(액화석유가스)
1,강원특별자치도,강릉시,LPG(액화석유가스)
1,강원특별자치도,강릉시,LPG(액화석유가스)
1,강원특별자치도,강릉시,LPG(액화석유가스)
1,강원특별자치도,강릉시,LPG(액화석유가스)
1,강원특별자치도,강릉시,LPG(액화석유가스)
1,강원특별자치도,강릉시,LPG(액화석유가스)


In [356]:
y_plist01 = [str(x) for x in y_plist]
mth_plist01 = [f'{x:0>2}' for x in mth_plist]

In [357]:
ym = pd.DataFrame({'연도':y_plist01, '월':mth_plist01})
ym

,연도,월
0,2023,02
1,2023,03
2,2023,04
3,2023,05
4,2023,06
5,2023,07
6,2023,08
7,2023,09
8,2023,10
9,2023,11


In [358]:
ymt = ym
for _ in range(slt.shape[0]//ym.shape[0] - 1):
    ymt = pd.concat([ymt, ym], ignore_index=False)
ymt.shape

(8520, 2)

In [359]:
slt = slt.reset_index(drop=True)
ymt = ymt.reset_index(drop=True)
slt.shape, ymt.shape

((8520, 3), (8520, 2))

In [360]:
base = pd.concat([ymt, slt], axis=1)
base.head(20)

,연도,월,시도,시군구_수정,연료
0,2023,02,강원특별자치도,강릉시,LPG(액화석유가스)
1,2023,03,강원특별자치도,강릉시,LPG(액화석유가스)
2,2023,04,강원특별자치도,강릉시,LPG(액화석유가스)
3,2023,05,강원특별자치도,강릉시,LPG(액화석유가스)
4,2023,06,강원특별자치도,강릉시,LPG(액화석유가스)
5,2023,07,강원특별자치도,강릉시,LPG(액화석유가스)
6,2023,08,강원특별자치도,강릉시,LPG(액화석유가스)
7,2023,09,강원특별자치도,강릉시,LPG(액화석유가스)
8,2023,10,강원특별자치도,강릉시,LPG(액화석유가스)
9,2023,11,강원특별자치도,강릉시,LPG(액화석유가스)


In [361]:
base['연료'].isnull().sum()

0

In [362]:
276 / 12

23.0

In [363]:
base1 = base.merge(num_car_by_local1, on=['연도', '월', '연료', '시도', '시군구_수정'], how='left')
base2 = base1.merge(num_car_by_local2, on=['연도', '월', '연료', '시도', '시군구_수정'], how='left')
base3 = base2.merge(grp_erase, on=['연도', '월', '연료', '시도', '시군구_수정'], how='left')
base3[['차량대수', '등록차량대수', '말소차량대수']] = base3[['차량대수', '등록차량대수', '말소차량대수']].fillna(0)

In [364]:
## 11.5s
# 시간에 따른 '차량대수'의 변화를 계산하고, 그 결과를 DataFrame에 반영해 시계열 데이터에서 전월 대비 차량의 순 증가, 감소 확인 가능
n = len(base3['월'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소차량대수'] - base3.loc[(i+1)*n - (j-1), '등록차량대수']

In [365]:
base3

,연도,월,시도,시군구_수정,연료,차량대수,등록차량대수,말소차량대수
0,2023,02,강원특별자치도,강릉시,LPG(액화석유가스),60.0,0.0,0.0
1,2023,03,강원특별자치도,강릉시,LPG(액화석유가스),60.0,0.0,0.0
2,2023,04,강원특별자치도,강릉시,LPG(액화석유가스),60.0,0.0,0.0
3,2023,05,강원특별자치도,강릉시,LPG(액화석유가스),60.0,0.0,0.0
4,2023,06,강원특별자치도,강릉시,LPG(액화석유가스),60.0,0.0,0.0
...,...,...,...,...,...,...,...,...
8515,2023,09,충청북도,충주시,휘발유,1372.0,0.0,0.0
8516,2023,10,충청북도,충주시,휘발유,1372.0,0.0,0.0
8517,2023,11,충청북도,충주시,휘발유,1372.0,0.0,0.0
8518,2023,12,충청북도,충주시,휘발유,1372.0,0.0,0.0


In [366]:
base3[base3['차량대수'] < 0].shape

(0, 8)

In [367]:
today_date

'20240213'

In [368]:
base3[(base3['연도'] != today_date[:4]) | (base3['월'] != today_date[4:6])].shape

(8520, 8)

In [369]:
base3.groupby(['연도', '월'])['차량대수'].sum()

연도    월 
2023  02    1318147.0
      03    1318147.0
      04    1318149.0
      05    1318149.0
      06    1318149.0
      07    1318149.0
      08    1318149.0
      09    1318154.0
      10    1318155.0
      11    1318155.0
      12    1318161.0
2024  01    1318161.0
Name: 차량대수, dtype: float64

####  !!! 수정(2023.10.27) 

In [370]:
####  !!! 수정(2023.10.27) 
# 수동
base3 = base3[(base3['연도'] != today_date[:4]) | (base3['월'] != today_date[4:6])].reset_index(drop=True)
base3

,연도,월,시도,시군구_수정,연료,차량대수,등록차량대수,말소차량대수
0,2023,02,강원특별자치도,강릉시,LPG(액화석유가스),60.0,0.0,0.0
1,2023,03,강원특별자치도,강릉시,LPG(액화석유가스),60.0,0.0,0.0
2,2023,04,강원특별자치도,강릉시,LPG(액화석유가스),60.0,0.0,0.0
3,2023,05,강원특별자치도,강릉시,LPG(액화석유가스),60.0,0.0,0.0
4,2023,06,강원특별자치도,강릉시,LPG(액화석유가스),60.0,0.0,0.0
...,...,...,...,...,...,...,...,...
8515,2023,09,충청북도,충주시,휘발유,1372.0,0.0,0.0
8516,2023,10,충청북도,충주시,휘발유,1372.0,0.0,0.0
8517,2023,11,충청북도,충주시,휘발유,1372.0,0.0,0.0
8518,2023,12,충청북도,충주시,휘발유,1372.0,0.0,0.0


###  !!!수정 끝(2023.12.11) 

In [371]:
###  !!!수정 끝(2023.12.11) 
base3.groupby(['연도', '월'], dropna=False)['차량대수'].sum()

연도    월 
2023  02    1318147.0
      03    1318147.0
      04    1318149.0
      05    1318149.0
      06    1318149.0
      07    1318149.0
      08    1318149.0
      09    1318154.0
      10    1318155.0
      11    1318155.0
      12    1318161.0
2024  01    1318161.0
Name: 차량대수, dtype: float64

In [372]:
num_car_by_local2.groupby(['연도', '월'], dropna=False)['등록차량대수'].sum()

연도    월 
1111  11    3
1965  02    1
1975  01    1
1986  05    1
1987  01    1
           ..
2022  11    1
2023  04    2
      09    5
      10    1
      12    6
Name: 등록차량대수, Length: 423, dtype: int64

In [373]:
grp_erase.groupby(['연도', '월'], dropna=False)['말소차량대수'].sum()

Series([], Name: 말소차량대수, dtype: int64)

In [374]:
today_date = datetime.today().strftime("%Y%m%d")
base3['테이블생성일자'] = today_date
base3['기준연월'] = base3['연도'] + '.' + base3['월']
base4 = base3[[
    '테이블생성일자', 
    '기준연월',
    '연도',
    '월', 
    '연료', 
    '시도', 
    '시군구_수정', 
    '차량대수',
]]
chc_col = {
    '테이블생성일자':'LOAD_DT', 
    '기준연월':'CRTR_YM',
    '연도':'YR', 
    '월':'MM', 
    '연료':'FUEL_CD', 
    '시도':'CTPV', 
    '시군구_수정':'SGG', 
    '차량대수':'VHCL_MKCNT', 
}
STD_BD_GRD4_RGN_CURSTT = base4.rename(columns=chc_col)

STD_BD_GRD4_RGN_CURSTT.columns

Index(['LOAD_DT', 'CRTR_YM', 'YR', 'MM', 'FUEL_CD', 'CTPV', 'SGG',
       'VHCL_MKCNT'],
      dtype='object')

In [375]:
STD_BD_GRD4_RGN_CURSTT.tail()

,LOAD_DT,CRTR_YM,YR,MM,FUEL_CD,CTPV,SGG,VHCL_MKCNT
8515,20240213,2023.09,2023,09,휘발유,충청북도,충주시,1372.0
8516,20240213,2023.10,2023,10,휘발유,충청북도,충주시,1372.0
8517,20240213,2023.11,2023,11,휘발유,충청북도,충주시,1372.0
8518,20240213,2023.12,2023,12,휘발유,충청북도,충주시,1372.0
8519,20240213,2024.01,2024,01,휘발유,충청북도,충주시,1372.0


In [376]:
STD_BD_GRD4_RGN_CURSTT.CRTR_YM.unique()

array(['2023.02', '2023.03', '2023.04', '2023.05', '2023.06', '2023.07',
       '2023.08', '2023.09', '2023.10', '2023.11', '2023.12', '2024.01'],
      dtype=object)

### [출력] STD_BD_GRD4_RGN_CURSTT

In [377]:
# expdf = STD_BD_GRD4_RGN_CURSTT
# table_nm = 'STD_BD_GRD4_RGN_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [378]:
# # 0s
# STD_BD_GRD4_RGN_CURSTT.to_csv('STD_BD_GRD4_RGN_CURSTT.csv', index=False)

In [379]:
df_g5.shape

(906192, 32)

In [380]:
dfm_g5 = df_g5.copy()

In [381]:
dfm_g5['최초등록일자'].head()

0    20041214
1    20010321
2    19891211
3    19940531
4    20020124
Name: 최초등록일자, dtype: int64

In [382]:
## 시도, 연도별 차량 대수
# dfm_g5 = df_g5.copy()
dfm_g5['최초등록일자'] = dfm_g5['최초등록일자'].astype('str')
dfm_g5['최초등록일자_년'] = dfm_g5['최초등록일자'].str[:4]
dfm_g5['최초등록일자_월'] = dfm_g5['최초등록일자'].str[4:6]
dfm_g5['최초등록일자_일'] = dfm_g5['최초등록일자'].str[6:8]
dfm_g5['최초등록일자'] = dfm_g5['최초등록일자_년'] + dfm_g5['최초등록일자_월'] + dfm_g5['최초등록일자_일']
dfm_g5['최초등록일자'] = pd.to_numeric(dfm_g5['최초등록일자'], errors='coerce')

### 시군구명 앞쪽 지역명만 남기기(dfm_g5)
dfm_g5['시군구_수정'] = dfm_g5['시군구'].str.split(' ').str[0]

In [383]:
dfm_g5[['시도', '시군구_수정', '연료', '최초등록일자', '최초등록일자_년', '최초등록일자_월']].isnull().sum()

시도             0
시군구_수정      3206
연료             0
최초등록일자         0
최초등록일자_년       0
최초등록일자_월       0
dtype: int64

In [384]:
dfm_g5['시도'].unique()

array(['경기도', '서울특별시', '전라남도', '부산광역시', '경상북도', '충청남도', '제주특별자치도', '경상남도',
       '대전광역시', '울산광역시', '대구광역시', '전라북도', '광주광역시', '충청북도', '인천광역시',
       '세종특별자치시', '강원특별자치도'], dtype=object)

In [385]:
dfm_g5[dfm_g5['시도'] == '세종특별자치시'].shape

(3205, 36)

In [386]:
dfm_g5.loc[(dfm_g5['최초등록일자_년'] == '2023') & (dfm_g5['최초등록일자_월'] == '09'), ['최초등록일자', '최초등록일자_년', '최초등록일자_월']]

,최초등록일자,최초등록일자_년,최초등록일자_월


In [387]:
dfm_g5.loc[(dfm_g5['최초등록일자_년'] == '2023') & (dfm_g5['최초등록일자_월'] == '10'), ['최초등록일자', '최초등록일자_년', '최초등록일자_월']]

,최초등록일자,최초등록일자_년,최초등록일자_월


In [388]:
dfm_g5.loc[dfm_g5['최초등록일자'].isnull(), ['최초등록일자', '최초등록일자_년', '최초등록일자_월']]

,최초등록일자,최초등록일자_년,최초등록일자_월


In [389]:
### 연료 지역별 차량대수
num_car_by_local1 = dfm_g5.groupby(['연료', '시도', '시군구_수정'], dropna=False)['차대번호'].count().reset_index()
num_car_by_local1 = num_car_by_local1.rename(columns={'차대번호':'차량대수'})

In [390]:
num_car_by_local1['차량대수'].sum()

906192

In [391]:
num_car_by_local1.isnull().sum()

연료        0
시도        0
시군구_수정    4
차량대수      0
dtype: int64

In [392]:
num_car_by_local1.loc[num_car_by_local1['연료'].isnull(), '차량대수'].sum()

0

###  !!! 수정 시작(2023.12.11) 

In [393]:
###  !!! 수정 시작(2023.12.11) 
# max_date = str(dfm['최초등록일자'].max())
# max_year = max_date[:4]
# max_month = max_date[4:6]

# date = '20220601'
# max_year = '2022'
# max_month = '06'
today_date = datetime.today().strftime("%Y%m%d")
date = today_date
# max_year = today_date[:4]
# max_month = today_date[4:6]
# max_year, max_month

In [394]:
periods = 2 # !!! 수정(2023.08.23)
y_plist = list(pd.date_range(end=date, periods=periods, freq="MS").year) # !!! 수정(2023.08.23)
mth_plist = list(pd.date_range(end=date, periods=periods, freq="MS").month) # !!! 수정(2023.08.23)
y_plist, mth_plist

([2024, 2024], [1, 2])

In [395]:
max_year = str(y_plist[0])
tp_max_month = mth_plist[0] # !!! 수정(2024.01.23)
max_month = f'{tp_max_month:0>2}' # !!! 수정(2024.01.23)
max_year, max_month

('2024', '01')

###  !!! 수정 끝(2023.12.11) 

In [396]:
###  !!! 수정 끝(2023.12.11) 
num_car_by_local1[['연도', '월']] = [max_year, max_month]

###  !!! 수정(2023.10.24) 

In [397]:
###  !!! 수정(2023.10.24) 
dfm['최초등록일자'].max()

20231220

In [398]:
### 연료 지역별 등록차량대수
num_car_by_local2 = dfm_g5.groupby(['연료', '시도', '시군구_수정', '최초등록일자_년', '최초등록일자_월'], dropna=False)['차대번호'].count().reset_index()
num_car_by_local2 = num_car_by_local2.rename(columns={'차대번호':'등록차량대수', '최초등록일자_년':'연도', '최초등록일자_월':'월'})

In [399]:
errc5.shape

(1136422, 32)

In [400]:
errc5[['시도', '시군구', '연료', '변경일자']].isnull().sum()

시도         0
시군구     6753
연료         0
변경일자       0
dtype: int64

In [401]:
### 연료 지역별 말소 대수
errc5['변경일자'] = errc5['변경일자'].astype('str')
errc5['변경일자_년'] = errc5['변경일자'].str[:4]
errc5['변경일자_월'] = errc5['변경일자'].str[4:6]
errc5['변경일자_일'] = errc5['변경일자'].str[6:8]

In [402]:
### 시군구명 앞쪽 지역명만 남기기(errc5)
errc5['시군구_수정'] = errc5['시군구'].str.split(' ').str[0]

In [403]:
errc5.loc[(errc5['변경일자_년'] == '2023') & (errc5['변경일자_월'] == '09'), ['변경일자', '변경일자_년', '변경일자_월']]

,변경일자,변경일자_년,변경일자_월
14422,20230927.0,2023,09
14423,20230927.0,2023,09
14424,20230927.0,2023,09
14425,20230927.0,2023,09
14426,20230927.0,2023,09
...,...,...,...
27190,20230901.0,2023,09
27191,20230901.0,2023,09
27192,20230901.0,2023,09
27193,20230901.0,2023,09


In [404]:
errc5.loc[(errc5['변경일자_년'] == '2023') & (errc5['변경일자_월'] == '10'), ['변경일자', '변경일자_년', '변경일자_월']]

,변경일자,변경일자_년,변경일자_월
291,20231031.0,2023,10
292,20231031.0,2023,10
293,20231031.0,2023,10
294,20231031.0,2023,10
295,20231031.0,2023,10
...,...,...,...
14417,20231004.0,2023,10
14418,20231004.0,2023,10
14419,20231004.0,2023,10
14420,20231004.0,2023,10


###  !!! 수정(2023.10.24) 

In [405]:
###  !!! 수정(2023.10.24) 
### 시군구명 앞쪽 지역명만 남기기(errc5)
errc5['시군구_수정'] = errc5['시군구'].str.split(' ').str[0]
grp_erase = errc5.loc[errc5['변경일자_년'] == max_year].groupby(['변경일자_년', '변경일자_월', '연료', '시도', '시군구_수정'], dropna=False)['차대번호'].count().reset_index()
grp_erase = grp_erase.rename(columns={'차대번호':'말소차량대수', '변경일자_년':'연도', '변경일자_월':'월'})
grp_erase = grp_erase.sort_values(['시도', '시군구_수정'])
grp_erase

,연도,월,연료,시도,시군구_수정,말소차량대수


In [406]:
errc5['시군구_수정'].isnull().sum()

6753

In [407]:
errc5.loc[errc5['시군구_수정'].isnull(), '시도'].unique()

array(['세종특별자치시'], dtype=object)

In [408]:
grp_erase.loc[grp_erase['시군구_수정'].isnull()].head()

,연도,월,연료,시도,시군구_수정,말소차량대수


In [409]:
num_car_by_local1['차량대수'].sum()

906192

In [410]:
num_car_by_local1.columns

Index(['연료', '시도', '시군구_수정', '차량대수', '연도', '월'], dtype='object')

### !!! 수정 시작(2023.12.11)

In [411]:
### !!! 수정 시작(2023.12.11)
periods = 13 # !!! 수정(2023.08.23)
y_plist = list(pd.date_range(end=date, periods=periods, freq="MS").year) # !!! 수정(2023.08.23)
mth_plist = list(pd.date_range(end=date, periods=periods, freq="MS").month) # !!! 수정(2023.08.23)
y_plist = y_plist[:-1]
mth_plist = mth_plist[:-1]
y_plist, mth_plist

([2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2024],
 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1])

In [412]:
# yr_list, mth_list, fuel_list, ctpv_list, sgg_list = [], [], [], [], []
# sl = num_car_by_local1.drop_duplicates(['시도', '시군구_수정', '연료']).reset_index(drop=True)
# for ctpv, sgg in sl[['시도', '시군구_수정']].values:
#     for fuel in sl['연료'].unique():
#         for yr, mth in zip(y_plist, mth_plist):
#             mthm = f'{mth:0>2}'
#             yr_list.append(str(yr))
#             mth_list.append(mthm)
#             fuel_list.append(fuel)
#             ctpv_list.append(ctpv)
#             sgg_list.append(sgg)
# base = pd.DataFrame({'연도':yr_list, '월':mth_list, '연료':fuel_list, '시도':ctpv_list, '시군구_수정':sgg_list})

In [413]:
sl = num_car_by_local1.drop_duplicates(['시도', '시군구_수정', '연료']).reset_index(drop=True)

In [414]:
sl.shape

(647, 6)

In [415]:
sl.shape[0] * 12

7764

In [416]:
slt = sl
for _ in range(periods - 2):
    slt = pd.concat([slt, sl], ignore_index=False)
slt.shape

(7764, 6)

In [417]:
slt = slt.sort_values(['시도', '시군구_수정', '연료'])
slt = slt[['시도', '시군구_수정', '연료']]
slt.head(20)

,시도,시군구_수정,연료
6,강원특별자치도,강릉시,LPG(액화석유가스)
6,강원특별자치도,강릉시,LPG(액화석유가스)
6,강원특별자치도,강릉시,LPG(액화석유가스)
6,강원특별자치도,강릉시,LPG(액화석유가스)
6,강원특별자치도,강릉시,LPG(액화석유가스)
6,강원특별자치도,강릉시,LPG(액화석유가스)
6,강원특별자치도,강릉시,LPG(액화석유가스)
6,강원특별자치도,강릉시,LPG(액화석유가스)
6,강원특별자치도,강릉시,LPG(액화석유가스)
6,강원특별자치도,강릉시,LPG(액화석유가스)


In [418]:
y_plist01 = [str(x) for x in y_plist]
mth_plist01 = [f'{x:0>2}' for x in mth_plist]

In [419]:
ym = pd.DataFrame({'연도':y_plist01, '월':mth_plist01})
ym

,연도,월
0,2023,02
1,2023,03
2,2023,04
3,2023,05
4,2023,06
5,2023,07
6,2023,08
7,2023,09
8,2023,10
9,2023,11


In [420]:
ymt = ym
for _ in range(slt.shape[0]//ym.shape[0] - 1):
    ymt = pd.concat([ymt, ym], ignore_index=False)
ymt.shape

(7764, 2)

In [421]:
slt = slt.reset_index(drop=True)
ymt = ymt.reset_index(drop=True)
slt.shape, ymt.shape

((7764, 3), (7764, 2))

In [422]:
base = pd.concat([ymt, slt], axis=1)
base.head(20)

,연도,월,시도,시군구_수정,연료
0,2023,02,강원특별자치도,강릉시,LPG(액화석유가스)
1,2023,03,강원특별자치도,강릉시,LPG(액화석유가스)
2,2023,04,강원특별자치도,강릉시,LPG(액화석유가스)
3,2023,05,강원특별자치도,강릉시,LPG(액화석유가스)
4,2023,06,강원특별자치도,강릉시,LPG(액화석유가스)
5,2023,07,강원특별자치도,강릉시,LPG(액화석유가스)
6,2023,08,강원특별자치도,강릉시,LPG(액화석유가스)
7,2023,09,강원특별자치도,강릉시,LPG(액화석유가스)
8,2023,10,강원특별자치도,강릉시,LPG(액화석유가스)
9,2023,11,강원특별자치도,강릉시,LPG(액화석유가스)


In [423]:
base['연료'].isnull().sum()

0

In [424]:
276 / 12

23.0

In [425]:
base1 = base.merge(num_car_by_local1, on=['연도', '월', '연료', '시도', '시군구_수정'], how='left')
base2 = base1.merge(num_car_by_local2, on=['연도', '월', '연료', '시도', '시군구_수정'], how='left')
base3 = base2.merge(grp_erase, on=['연도', '월', '연료', '시도', '시군구_수정'], how='left')
base3[['차량대수', '등록차량대수', '말소차량대수']] = base3[['차량대수', '등록차량대수', '말소차량대수']].fillna(0)

In [426]:
n = len(base3['월'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소차량대수'] - base3.loc[(i+1)*n - (j-1), '등록차량대수']

In [427]:
base3

,연도,월,시도,시군구_수정,연료,차량대수,등록차량대수,말소차량대수
0,2023,02,강원특별자치도,강릉시,LPG(액화석유가스),1.0,0.0,0.0
1,2023,03,강원특별자치도,강릉시,LPG(액화석유가스),1.0,0.0,0.0
2,2023,04,강원특별자치도,강릉시,LPG(액화석유가스),1.0,0.0,0.0
3,2023,05,강원특별자치도,강릉시,LPG(액화석유가스),1.0,0.0,0.0
4,2023,06,강원특별자치도,강릉시,LPG(액화석유가스),1.0,0.0,0.0
...,...,...,...,...,...,...,...,...
7759,2023,09,충청북도,충주시,휘발유,77.0,0.0,0.0
7760,2023,10,충청북도,충주시,휘발유,77.0,0.0,0.0
7761,2023,11,충청북도,충주시,휘발유,77.0,0.0,0.0
7762,2023,12,충청북도,충주시,휘발유,77.0,0.0,0.0


In [428]:
base3[base3['차량대수'] < 0].shape

(0, 8)

In [429]:
today_date

'20240213'

In [430]:
base3[(base3['연도'] != today_date[:4]) | (base3['월'] != today_date[4:6])].shape

(7764, 8)

In [431]:
base3.groupby(['연도', '월'])['차량대수'].sum()

연도    월 
2023  02    906190.0
      03    906190.0
      04    906191.0
      05    906191.0
      06    906192.0
      07    906192.0
      08    906192.0
      09    906192.0
      10    906192.0
      11    906192.0
      12    906192.0
2024  01    906192.0
Name: 차량대수, dtype: float64

#### !!! 수정(2023.10.27)

In [432]:
#### !!! 수정(2023.10.27)
# 수동
base3 = base3[(base3['연도'] != today_date[:4]) | (base3['월'] != today_date[4:6])].reset_index(drop=True)
base3

,연도,월,시도,시군구_수정,연료,차량대수,등록차량대수,말소차량대수
0,2023,02,강원특별자치도,강릉시,LPG(액화석유가스),1.0,0.0,0.0
1,2023,03,강원특별자치도,강릉시,LPG(액화석유가스),1.0,0.0,0.0
2,2023,04,강원특별자치도,강릉시,LPG(액화석유가스),1.0,0.0,0.0
3,2023,05,강원특별자치도,강릉시,LPG(액화석유가스),1.0,0.0,0.0
4,2023,06,강원특별자치도,강릉시,LPG(액화석유가스),1.0,0.0,0.0
...,...,...,...,...,...,...,...,...
7759,2023,09,충청북도,충주시,휘발유,77.0,0.0,0.0
7760,2023,10,충청북도,충주시,휘발유,77.0,0.0,0.0
7761,2023,11,충청북도,충주시,휘발유,77.0,0.0,0.0
7762,2023,12,충청북도,충주시,휘발유,77.0,0.0,0.0


### !!!수정 끝(2023.12.11)

In [433]:
### !!!수정 끝(2023.12.11)
base3.groupby(['연도', '월'], dropna=False)['차량대수'].sum()

연도    월 
2023  02    906190.0
      03    906190.0
      04    906191.0
      05    906191.0
      06    906192.0
      07    906192.0
      08    906192.0
      09    906192.0
      10    906192.0
      11    906192.0
      12    906192.0
2024  01    906192.0
Name: 차량대수, dtype: float64

In [434]:
num_car_by_local2.groupby(['연도', '월'], dropna=False)['등록차량대수'].sum()

연도    월 
1111  11    1
1920  09    1
1942  03    1
      06    1
1943  12    3
           ..
2022  04    2
      07    1
2023  02    1
      04    1
      06    1
Name: 등록차량대수, Length: 634, dtype: int64

In [435]:
grp_erase.groupby(['연도', '월'], dropna=False)['말소차량대수'].sum()

Series([], Name: 말소차량대수, dtype: int64)

In [436]:
today_date = datetime.today().strftime("%Y%m%d")
base3['테이블생성일자'] = today_date
base3['기준연월'] = base3['연도'] + '.' + base3['월']
base4 = base3[[
    '테이블생성일자', 
    '기준연월',
    '연도',
    '월', 
    '연료', 
    '시도', 
    '시군구_수정', 
    '차량대수',
]]
chc_col = {
    '테이블생성일자':'LOAD_DT', 
    '기준연월':'CRTR_YM',
    '연도':'YR', 
    '월':'MM', 
    '연료':'FUEL_CD', 
    '시도':'CTPV', 
    '시군구_수정':'SGG', 
    '차량대수':'VHCL_MKCNT', 
}
STD_BD_GRD5_RGN_CURSTT = base4.rename(columns=chc_col)

STD_BD_GRD5_RGN_CURSTT.columns

Index(['LOAD_DT', 'CRTR_YM', 'YR', 'MM', 'FUEL_CD', 'CTPV', 'SGG',
       'VHCL_MKCNT'],
      dtype='object')

### [출력] STD_BD_GRD5_RGN_CURSTT

In [437]:
# expdf = STD_BD_GRD5_RGN_CURSTT
# table_nm = 'STD_BD_GRD5_RGN_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [438]:
## 4등급 연도, 시도, 차종별 차량 대수
### 현재 차량 대수
num_car_by_local1 = dfm.groupby(['시도', '차종'], dropna=False)['차대번호'].count().reset_index()
num_car_by_local1 = num_car_by_local1.rename(columns={'차대번호':'차량대수'})
num_car_by_local1['연도'] = max_year

In [439]:
# !!! 수정(2023.10.24)
### 등록 차량 대수
num_car_by_local2 = dfm.groupby(['시도', '차종', '최초등록일자_년'], dropna=False)['차대번호'].count().reset_index()
num_car_by_local2 = num_car_by_local2.rename(columns={'차대번호':'등록차량대수', '최초등록일자_년':'연도'})

In [440]:
# !!! 수정(2023.10.24)
### 말소 차량 대수
grp_erase = errc.groupby(['변경일자_년', '시도', '차종'], dropna=False)['차대번호'].count().reset_index()
grp_erase = grp_erase.rename(columns={'차대번호':'말소차량대수', '변경일자_년':'연도'})
grp_erase = grp_erase.sort_values(['시도'])

In [441]:
y_plist = list(pd.date_range(end=date, periods=4, freq="YS").year)

y_plist

[2021, 2022, 2023, 2024]

In [442]:
yr_list = []
fuel_list = []
ctpv_list = []
cd_list = []
for ctpv in num_car_by_local1['시도'].unique():
    for cd in ['승용', '승합', '화물', '특수']:
        for yrm in y_plist:
            yr_list.append(str(yrm))
            ctpv_list.append(ctpv)
            cd_list.append(cd)
base = pd.DataFrame({'연도':yr_list, '시도':ctpv_list, '차종':cd_list})

In [443]:
base1 = base.merge(num_car_by_local1, on=['연도', '시도', '차종'], how='left')
base2 = base1.merge(num_car_by_local2, on=['연도', '시도', '차종'], how='left')
base3 = base2.merge(grp_erase, on=['연도', '시도', '차종'], how='left')
base3[['차량대수', '등록차량대수', '말소차량대수']] = base3[['차량대수', '등록차량대수', '말소차량대수']].fillna(0)

In [444]:
n = len(base3['연도'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소차량대수'] - base3.loc[(i+1)*n - (j-1), '등록차량대수']

In [445]:
today_date

'20240213'

In [446]:
base3['연도'].unique()

array(['2021', '2022', '2023', '2024'], dtype=object)

In [447]:
today_date = datetime.today().strftime("%Y%m%d")
base3['테이블생성일자'] = today_date
base4 = base3[[
    '테이블생성일자', 
    '연도', 
    '시도', 
    '차종', 
    '차량대수', 
]]
chc_col = {
    '테이블생성일자':'LOAD_DT',
    '연도':'CRTR_Y', 
    '시도':'CTPV', 
    '차종':'VHCTY_CD', 
    '차량대수':'VHCL_MKCNT', 
}
STD_BD_GRD4_RGN_CURSTT_MOD = base4.rename(columns=chc_col)

In [448]:
base4.groupby('연도')['차량대수'].sum()

연도
2021    1530671.0
2022    1448646.0
2023    1318161.0
2024    1318161.0
Name: 차량대수, dtype: float64

### [출력] STD_BD_GRD4_RGN_CURSTT_MOD

In [449]:
# expdf = STD_BD_GRD4_RGN_CURSTT_MOD
# table_nm = 'STD_BD_GRD4_RGN_CURSTT_MOD'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [450]:
STD_BD_GRD4_RGN_CURSTT_MOD.CRTR_Y.unique()

array(['2021', '2022', '2023', '2024'], dtype=object)

In [451]:
## 5등급 연도, 시도, 차종별 차량 대수
### 현재 차량 대수
num_car_by_local1 = dfm_g5.groupby(['시도', '차종'], dropna=False)['차대번호'].count().reset_index()
num_car_by_local1 = num_car_by_local1.rename(columns={'차대번호':'차량대수'})
num_car_by_local1['연도'] = max_year
num_car_by_local1

,시도,차종,차량대수,연도
0,강원특별자치도,승용,8293,2024
1,강원특별자치도,승합,2534,2024
2,강원특별자치도,특수,404,2024
3,강원특별자치도,화물,24537,2024
4,경기도,승용,55502,2024
...,...,...,...,...
63,충청남도,화물,34807,2024
64,충청북도,승용,7509,2024
65,충청북도,승합,2751,2024
66,충청북도,특수,425,2024


In [452]:
num_car_by_local1['연도'].dtype

dtype('O')

In [453]:
# !!! 수정(2023.10.24)
### 등록 차량 대수
num_car_by_local2 = dfm_g5.groupby(['시도', '차종', '최초등록일자_년'], dropna=False)['차대번호'].count().reset_index()
num_car_by_local2 = num_car_by_local2.rename(columns={'차대번호':'등록차량대수', '최초등록일자_년':'연도'})
num_car_by_local2

,시도,차종,연도,등록차량대수
0,강원특별자치도,승용,1977,1
1,강원특별자치도,승용,1978,10
2,강원특별자치도,승용,1979,11
3,강원특별자치도,승용,1980,12
4,강원특별자치도,승용,1981,10
...,...,...,...,...
2429,충청북도,화물,2013,2
2430,충청북도,화물,2014,1
2431,충청북도,화물,2015,1
2432,충청북도,화물,2016,1


In [454]:
num_car_by_local2['연도'].dtype

dtype('O')

In [455]:
# !!! 수정(2023.10.24)
### 말소 차량 대수
grp_erase = errc5.groupby(['변경일자_년', '시도', '차종'], dropna=False)['차대번호'].count().reset_index()
grp_erase = grp_erase.rename(columns={'차대번호':'말소차량대수', '변경일자_년':'연도'})
grp_erase = grp_erase.sort_values(['시도'])
grp_erase

,연도,시도,차종,말소차량대수
0,2019,강원도,승용,5708
206,2022,강원도,특수,82
207,2022,강원도,화물,4132
139,2021,강원도,화물,8119
138,2021,강원도,특수,108
...,...,...,...,...
342,2023,충청북도,특수,80
203,2021,충청북도,화물,8865
202,2021,충청북도,특수,188
132,2020,충청북도,승용,6209


In [456]:
grp_erase['연도'].dtype

dtype('O')

In [457]:
y_plist = list(pd.date_range(end=date, periods=4, freq="YS").year)

y_plist

[2021, 2022, 2023, 2024]

In [458]:
yr_list = []
fuel_list = []
ctpv_list = []
cd_list = []
for ctpv in num_car_by_local1['시도'].unique():
    for cd in ['승용', '승합', '화물', '특수']:
        for yrm in y_plist:
            yr_list.append(str(yrm))
            ctpv_list.append(ctpv)
            cd_list.append(cd)
base = pd.DataFrame({'연도':yr_list, '시도':ctpv_list, '차종':cd_list})
base

,연도,시도,차종
0,2021,강원특별자치도,승용
1,2022,강원특별자치도,승용
2,2023,강원특별자치도,승용
3,2024,강원특별자치도,승용
4,2021,강원특별자치도,승합
...,...,...,...
267,2024,충청북도,화물
268,2021,충청북도,특수
269,2022,충청북도,특수
270,2023,충청북도,특수


In [459]:
base1 = base.merge(num_car_by_local1, on=['연도', '시도', '차종'], how='left')
base1

,연도,시도,차종,차량대수
0,2021,강원특별자치도,승용,NaN
1,2022,강원특별자치도,승용,NaN
2,2023,강원특별자치도,승용,NaN
3,2024,강원특별자치도,승용,8293.0
4,2021,강원특별자치도,승합,NaN
...,...,...,...,...
267,2024,충청북도,화물,22672.0
268,2021,충청북도,특수,NaN
269,2022,충청북도,특수,NaN
270,2023,충청북도,특수,NaN


In [460]:
base2 = base1.merge(num_car_by_local2, on=['연도', '시도', '차종'], how='left')
base2

,연도,시도,차종,차량대수,등록차량대수
0,2021,강원특별자치도,승용,NaN,NaN
1,2022,강원특별자치도,승용,NaN,NaN
2,2023,강원특별자치도,승용,NaN,NaN
3,2024,강원특별자치도,승용,8293.0,NaN
4,2021,강원특별자치도,승합,NaN,NaN
...,...,...,...,...,...
267,2024,충청북도,화물,22672.0,NaN
268,2021,충청북도,특수,NaN,NaN
269,2022,충청북도,특수,NaN,NaN
270,2023,충청북도,특수,NaN,NaN


In [461]:
base3 = base2.merge(grp_erase, on=['연도', '시도', '차종'], how='left')
base3[['차량대수', '등록차량대수', '말소차량대수']] = base3[['차량대수', '등록차량대수', '말소차량대수']].fillna(0)
base3

,연도,시도,차종,차량대수,등록차량대수,말소차량대수
0,2021,강원특별자치도,승용,0.0,0.0,0.0
1,2022,강원특별자치도,승용,0.0,0.0,0.0
2,2023,강원특별자치도,승용,0.0,0.0,1238.0
3,2024,강원특별자치도,승용,8293.0,0.0,0.0
4,2021,강원특별자치도,승합,0.0,0.0,0.0
...,...,...,...,...,...,...
267,2024,충청북도,화물,22672.0,0.0,0.0
268,2021,충청북도,특수,0.0,0.0,188.0
269,2022,충청북도,특수,0.0,0.0,98.0
270,2023,충청북도,특수,0.0,0.0,80.0


In [462]:
n = len(base3['연도'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소차량대수'] - base3.loc[(i+1)*n - (j-1), '등록차량대수']

In [463]:
today_date

'20240213'

In [464]:
base3['연도'].unique()

array(['2021', '2022', '2023', '2024'], dtype=object)

In [465]:
base3 = base3[base3['연도'] != '2024'].reset_index(drop=True)
base3.shape

(204, 6)

In [466]:
today_date = datetime.today().strftime("%Y%m%d")
base3['테이블생성일자'] = today_date
base4 = base3[[
    '테이블생성일자', 
    '연도', 
    '시도', 
    '차종', 
    '차량대수', 
]]
chc_col = {
    '테이블생성일자':'LOAD_DT',
    '연도':'CRTR_Y', 
    '시도':'CTPV', 
    '차종':'VHCTY_CD', 
    '차량대수':'VHCL_MKCNT', 
}
STD_BD_GRD5_RGN_CURSTT_MOD = base4.rename(columns=chc_col)

In [467]:
base4.groupby('연도')['차량대수'].sum()

연도
2021    1186337.0
2022    1034568.0
2023     906192.0
Name: 차량대수, dtype: float64

### [출력] STD_BD_GRD5_RGN_CURSTT_MOD

In [468]:
# expdf = STD_BD_GRD5_RGN_CURSTT_MOD
# table_nm = 'STD_BD_GRD5_RGN_CURSTT_MOD'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [469]:
STD_BD_GRD5_RGN_CURSTT_MOD.CRTR_Y.unique()

array(['2021', '2022', '2023'], dtype=object)

In [470]:
## 4등급 차량현황(그룹)
# - 연도, 월, 시도, 시군구, 연료, 차종, 차종유형, 용도
### 현재 차량 대수
# num_car_by_local1 = dfm.groupby(['시도', '시군구_수정', '연료', '차종', '차종유형', '용도'], dropna=False, as_index=False).agg({'차대번호':'count', 'DPF_YN':'count', '조기폐차최종승인YN':'count'})
# num_car_by_local1 = num_car_by_local1.rename(columns={'차대번호':'차량대수', 'DPF_YN':'저감장치부착대수', '조기폐차최종승인YN':'조기폐차대수'})
# num_car_by_local1['저감장치미부착대수'] = num_car_by_local1['차량대수'] - num_car_by_local1['저감장치부착대수']

# max_date = str(dfm['최초등록일자'].max())
# max_year = max_date[:4]
# max_month = max_date[4:6]
# num_car_by_local1[['연도', '월']] = [max_year, max_month]

# if len(num_car_by_local1['월'].unique()) != 1:
#     # 오름차순 정렬
#     num_car_by_local1 = num_car_by_local1.sort_values(['시도', '시군구_수정', '연료', '차종', '차종유형', '용도', '연도', '월']).reset_index(drop=True)
#     num_car_by_local1['차량대수_전월'] = num_car_by_local1['차량대수'].shift(1)
#     num_car_by_local1['감소율'] = (num_car_by_local1['차량대수_전월'] - num_car_by_local1['차량대수']) / num_car_by_local1['차량대수_전월']
#     for n in range(num_car_by_local1.shape[0] // len(num_car_by_local1['월'].unique())):
#         num_car_by_local1.loc[n*3, '감소율'] = np.nan
# else:
#     num_car_by_local1['감소율'] = np.nan

# today_date = datetime.today().strftime("%Y%m%d")
# num_car_by_local1['테이블생성일자'] = today_date

In [471]:
## 16.4s
# cse = carr.merge(srcr, on='제원관리번호', how='left')
cse = cs.copy()
ce = cse.merge(elpm, on='차대번호', how='left')
cea = ce.merge(attr, on='차대번호', how='left')

In [472]:
## 13.3s
cea['법정동코드'] = cea['법정동코드'].astype('str')
cea['법정동코드'] = cea['법정동코드'].str[:5] + '00000'
cea['법정동코드'] = pd.to_numeric(cea['법정동코드'], errors='coerce')

In [473]:
## 10.8s
dfe = cea.merge(coder, on='법정동코드', how='left')

In [474]:
dfe['시도'].isnull().sum()

0

In [475]:
dfe['시도'].unique()

array(['대구광역시', '경기도', '경상북도', '전라북도', '전라남도', '강원특별자치도', '광주광역시', '경상남도',
       '서울특별시', '인천광역시', '대전광역시', '충청북도', '충청남도', '세종특별자치시', '부산광역시',
       '울산광역시', '강원도', '제주특별자치도'], dtype=object)

In [476]:
# dfe.loc[dfe['법정동코드'] == 4163055000, ['시도', '시군구']] = ['경기도', '양주시']

In [477]:
dfe['시군구'].isnull().sum()

10678

In [478]:
## 12.8s
dfe['시군구_수정'] = dfe['시군구'].str.split(' ').str[0]

In [479]:
dfe['시군구_수정'].isnull().sum()

10678

In [480]:
elpm

,차대번호,조기폐차상태코드,조기폐차최종승인YN,말소일자,조기폐차신청여부
0,KMFFA17XPNU206384,Y,Y,20100325.0,Y
1,KPBFH2AD15P155529,Y,Y,20210408.0,Y
2,KPDHADMD1YP219104,Y,Y,20210513.0,Y
3,KPAKA4AD13P327273,Y,Y,20210521.0,Y
4,KPDKFDNF14P000955,Y,Y,20210504.0,Y
...,...,...,...,...,...
962058,KNCUA28121S081047,Y,Y,20180725.0,Y
962059,KPBFA2AD14P124370,Y,Y,20180612.0,Y
962060,KMJFD37BPYU478777,Y,Y,20180614.0,Y
962061,KMFWVH7JP5U670422,Y,Y,20180607.0,Y


In [481]:
errm

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN_x,제원관리번호,차량연식,차종,용도,최초등록일자,...,차명,자동차형식,엔진형식,차종분류,총중량,적재중량,배기량,엔진출력,차량말소YN_y,변경일자
0,KMHJP81VP7U695662,5121000000,4,5MY-HD-24-59,Y,A0810006301241206,2007,승용,개인용,20070619,...,투싼(TUCSON),JM-1VAF7-1,D4EA,A2819,1985,0,1991,146,Y,20231101.0
1,KMHJN81VP7U725237,4715000000,4,5MY-HD-24-59,Y,A0810006301201206,2007,승용,개인용,20070817,...,투싼(TUCSON),JM-3VAF7-1,D4EA,A2819,1965,0,1991,146,Y,20231101.0
2,KMHSJ81VP8U275598,5115000000,4,6MY-HD-24-100,Y,A0810006800831207,2008,승용,개인용,20071017,...,싼타페(SANTAFE),CM-R2DBF8-1,D4EA,A2819,2345,0,1991,151,Y,20231101.0
3,KMFZCS7JP9U479327,4136000000,4,7MY-HD-14-68,Y,A0810006207343108,2009,화물,개인용,20090204,...,포터Ⅱ(PORTERⅡ),HR-3SS2JKM9-1,D4CB,A2809,2950,1000,2497,126,Y,20231101.0
4,KNAJE55539K627591,3120000000,4,6MY-KM-14-61,Y,A0110003400581208,2009,승용,개인용,20081217,...,스포티지,JE553J-HG,D4EA,A2819,1960,0,1991,151,Y,20231101.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449104,KMHJF31KPYU988484,4136000000,4,RMY-HD-20,Y,10044101040013,2000,승용,개인용,20000121,...,아반떼오토메틱,JS-DL15-M1,G4FK,A2826,1460,0,1495,95,Y,20190620.0
449105,KLATF48YDXB490959,4831000000,4,결측,Y,10073900180002,1999,승용,개인용,19991006,...,라노스해치백1.5,TF48Y-1,A15SMS,A2826,1320,0,1498,94,Y,20190604.0
449106,KMFZCX7JAAU567310,4715000000,4,7MY-HD-14-68,Y,A0810006208223109,2010,화물,개인용,20091009,...,포터Ⅱ(PORTERⅡ),HRTJA-DX-A,D4CB,A2809,2905,1000,2497,126,Y,20190521.0
449107,KMHEN41BP1A477461,4182000000,4,WMY-HD-32,Y,10106100460000,2001,승용,개인용,20010428,...,NEW EF쏘나타,YS-20DI-A1,G4JP,A2826,1785,0,1997,133,Y,20190515.0


In [482]:
# coder
# errm (얘가 없음) 
# errc = errm.merge(coder, on='법정동코드', how='left')

In [483]:
errc

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN_x,제원관리번호,차량연식,차종,용도,최초등록일자,...,차량말소YN_y,변경일자,시도,시군구,시도코드,시군구코드,변경일자_년,변경일자_월,변경일자_일,시군구_수정
0,KMHJP81VP7U695662,5121000000,4,5MY-HD-24-59,Y,A0810006301241206,2007,승용,개인용,20070619,...,Y,20231101.0,강원특별자치도,속초시,51,210,2023,11,01,속초시
1,KMHJN81VP7U725237,4715000000,4,5MY-HD-24-59,Y,A0810006301201206,2007,승용,개인용,20070817,...,Y,20231101.0,경상북도,김천시,47,150,2023,11,01,김천시
2,KMHSJ81VP8U275598,5115000000,4,6MY-HD-24-100,Y,A0810006800831207,2008,승용,개인용,20071017,...,Y,20231101.0,강원특별자치도,강릉시,51,150,2023,11,01,강릉시
3,KMFZCS7JP9U479327,4136000000,4,7MY-HD-14-68,Y,A0810006207343108,2009,화물,개인용,20090204,...,Y,20231101.0,경기도,남양주시,41,360,2023,11,01,남양주시
4,KNAJE55539K627591,3120000000,4,6MY-KM-14-61,Y,A0110003400581208,2009,승용,개인용,20081217,...,Y,20231101.0,울산광역시,북구,31,200,2023,11,01,북구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449104,KMHJF31KPYU988484,4136000000,4,RMY-HD-20,Y,10044101040013,2000,승용,개인용,20000121,...,Y,20190620.0,경기도,남양주시,41,360,2019,06,20,남양주시
449105,KLATF48YDXB490959,4831000000,4,결측,Y,10073900180002,1999,승용,개인용,19991006,...,Y,20190604.0,경상남도,거제시,48,310,2019,06,04,거제시
449106,KMFZCX7JAAU567310,4715000000,4,7MY-HD-14-68,Y,A0810006208223109,2010,화물,개인용,20091009,...,Y,20190521.0,경상북도,김천시,47,150,2019,05,21,김천시
449107,KMHEN41BP1A477461,4182000000,4,WMY-HD-32,Y,10106100460000,2001,승용,개인용,20010428,...,Y,20190515.0,경기도,가평군,41,820,2019,05,15,가평군


In [484]:
ere = errc.merge(elpm, on='차대번호', how='left')

In [485]:
ere

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN_x,제원관리번호,차량연식,차종,용도,최초등록일자,...,시도코드,시군구코드,변경일자_년,변경일자_월,변경일자_일,시군구_수정,조기폐차상태코드,조기폐차최종승인YN,말소일자,조기폐차신청여부
0,KMHJP81VP7U695662,5121000000,4,5MY-HD-24-59,Y,A0810006301241206,2007,승용,개인용,20070619,...,51,210,2023,11,01,속초시,NaN,NaN,NaN,NaN
1,KMHJN81VP7U725237,4715000000,4,5MY-HD-24-59,Y,A0810006301201206,2007,승용,개인용,20070817,...,47,150,2023,11,01,김천시,NaN,NaN,NaN,NaN
2,KMHSJ81VP8U275598,5115000000,4,6MY-HD-24-100,Y,A0810006800831207,2008,승용,개인용,20071017,...,51,150,2023,11,01,강릉시,NaN,NaN,NaN,NaN
3,KMFZCS7JP9U479327,4136000000,4,7MY-HD-14-68,Y,A0810006207343108,2009,화물,개인용,20090204,...,41,360,2023,11,01,남양주시,Y,Y,20231101.0,Y
4,KNAJE55539K627591,3120000000,4,6MY-KM-14-61,Y,A0110003400581208,2009,승용,개인용,20081217,...,31,200,2023,11,01,북구,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449104,KMHJF31KPYU988484,4136000000,4,RMY-HD-20,Y,10044101040013,2000,승용,개인용,20000121,...,41,360,2019,06,20,남양주시,NaN,NaN,NaN,NaN
449105,KLATF48YDXB490959,4831000000,4,결측,Y,10073900180002,1999,승용,개인용,19991006,...,48,310,2019,06,04,거제시,NaN,NaN,NaN,NaN
449106,KMFZCX7JAAU567310,4715000000,4,7MY-HD-14-68,Y,A0810006208223109,2010,화물,개인용,20091009,...,47,150,2019,05,21,김천시,NaN,NaN,NaN,NaN
449107,KMHEN41BP1A477461,4182000000,4,WMY-HD-32,Y,10106100460000,2001,승용,개인용,20010428,...,41,820,2019,05,15,가평군,NaN,NaN,NaN,NaN


In [486]:
## 11.7s
erea = ere.merge(attr, on='차대번호', how='left')
erea.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN_x', '제원관리번호', '차량연식',
       '차종', '용도', '최초등록일자', '제작일자', '차량관리번호', '자동차등록번호', '검사유효일', '소유자구분',
       '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '차종분류', '총중량', '적재중량',
       '배기량', '엔진출력', '차량말소YN_y', '변경일자', '시도', '시군구', '시도코드', '시군구코드',
       '변경일자_년', '변경일자_월', '변경일자_일', '시군구_수정', '조기폐차상태코드', '조기폐차최종승인YN',
       '말소일자', '조기폐차신청여부', '저감장치구분', '저감장치종류', 'DPF_YN'],
      dtype='object')

In [487]:
erea.shape

(449109, 43)

In [488]:
temp = erea.copy()

In [489]:
temp.loc[np.nan, 'test'] = 'Y'
temp['test'].head()

0.0    NaN
1.0    NaN
2.0    NaN
3.0    NaN
4.0    NaN
Name: test, dtype: object

In [490]:
erea['DPF_YN'].value_counts(dropna=False)

DPF_YN
NaN    449087
유          22
Name: count, dtype: int64

In [491]:
erea['DPF_YN'].dtype

dtype('O')

In [492]:
## 12.9s
rdf = dfe.copy()
rs = rs.drop_duplicates('차대번호').reset_index(drop=True)

In [493]:
## 13.1s
rdf1 = rdf.merge(rs, on='차대번호', how='left')

In [494]:
rdf1['DPF_YN'].value_counts(dropna=False)

DPF_YN
NaN    1923941
유           80
Name: count, dtype: int64

In [495]:
rdf1['DPF유무_수정'].value_counts(dropna=False)

DPF유무_수정
무       879464
NaN     772234
유       259890
확인불가     12433
Name: count, dtype: int64

In [496]:
rdf1.loc[(rdf1['DPF_YN'] == '유') | (rdf1['DPF유무_수정'] == '유'), 'DPF_YN'] = '유'
rdf1.loc[(rdf1['DPF유무_수정'] == '무'), 'DPF_YN'] = '무'
rdf1.loc[(rdf1['DPF유무_수정'] == '확인불가'), 'DPF_YN'] = '확인불가'

In [497]:
rdf1['DPF_YN'].value_counts(dropna=False)

DPF_YN
무       879464
NaN     772212
유       259912
확인불가     12433
Name: count, dtype: int64

In [498]:
dfe = rdf1.drop('DPF유무_수정', axis=1)

In [499]:
dfe['연도'] = max_year
dfe['월'] = max_month

In [500]:
## 12.7s
dfe['최초등록일자'] = dfe['최초등록일자'].astype('str')
dfe['최초등록일자_년'] = dfe['최초등록일자'].str[:4]
dfe['최초등록일자_월'] = dfe['최초등록일자'].str[4:6]
dfe['최초등록일자_일'] = dfe['최초등록일자'].str[6:8]

In [501]:
dfe['DPF_YN'].unique()

array([nan, '무', '유', '확인불가'], dtype=object)

In [502]:
erea['DPF_YN'].unique()

array([nan, '유'], dtype=object)

In [503]:
## 10.2s
dfe.loc[dfe['DPF_YN'] == '유', '저감장치부착유무'] = 'Y'
erea.loc[erea['DPF_YN'] == '유', '저감장치부착유무'] = 'Y'

In [504]:
## 12.7s
dfe['말소일자'] = dfe['말소일자'].astype('str')
dfe['말소일자_년'] = dfe['말소일자'].str[:4]
dfe['말소일자_월'] = dfe['말소일자'].str[4:6]
dfe['말소일자_일'] = dfe['말소일자'].str[6:8]

In [505]:
## 10.6s
erea['말소일자'] = erea['말소일자'].astype('str')
erea['말소일자_년'] = erea['말소일자'].str[:4]
erea['말소일자_월'] = erea['말소일자'].str[4:6]
erea['말소일자_일'] = erea['말소일자'].str[6:8]

In [506]:
## 11.1s
dfe[dfe['차량말소YN'] == 'N'].shape

(1318161, 47)

In [507]:
## 12.2s
# !!! 수정(2023.10.24)
# 현재 연도 차량 대수
grp1 = dfe[dfe['차량말소YN'] == 'N'].groupby(['연도', '월', '시도', '시군구_수정', '연료', '차종', '차종유형', '용도', '차량연식'], dropna=False).agg({'차대번호':'count', '저감장치부착유무':'count'}).reset_index() # !!! 수정(2023.11.27)
grp1 = grp1.rename(columns={'차대번호':'차량대수', '저감장치부착유무':'저감대수'})
grp1

,연도,월,시도,시군구_수정,연료,차종,차종유형,용도,차량연식,차량대수,저감대수
0,2024,01,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,1,0
1,2024,01,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,2002,1,0
2,2024,01,강원특별자치도,강릉시,LPG(액화석유가스),승용,중형,개인용,1992,1,0
3,2024,01,강원특별자치도,강릉시,LPG(액화석유가스),승용,중형,개인용,1994,3,0
4,2024,01,강원특별자치도,강릉시,LPG(액화석유가스),승용,중형,개인용,1996,3,0
...,...,...,...,...,...,...,...,...,...,...,...
38253,2024,01,충청북도,충주시,휘발유,화물,소형,개인용,1991,5,0
38254,2024,01,충청북도,충주시,휘발유,화물,소형,개인용,1992,5,0
38255,2024,01,충청북도,충주시,휘발유,화물,소형,개인용,1993,1,0
38256,2024,01,충청북도,충주시,휘발유,화물,소형,개인용,1994,1,0


In [508]:
grp1['차량대수'].sum()

1318161

In [509]:
## 12.3s
# !!! 수정(2023.10.24)
# 연도별 등록대수
grp2 = dfe[dfe['차량말소YN'] == 'N'].groupby(['최초등록일자_년', '최초등록일자_월', '시도', '시군구_수정', '연료', '차종', '차종유형', '용도', '차량연식'], dropna=False).agg({'차대번호':'count', '저감장치부착유무':'count'}).reset_index() # !!! 수정(2023.11.27)
grp2 = grp2.rename(columns={'차대번호':'등록대수', '저감장치부착유무':'등록저감대수', '최초등록일자_년':'연도', '최초등록일자_월':'월'})
grp2

,연도,월,시도,시군구_수정,연료,차종,차종유형,용도,차량연식,등록대수,등록저감대수
0,1111,11,강원특별자치도,고성군,휘발유,승용,중형,개인용,1996,1,0
1,1111,11,서울특별시,도봉구,휘발유,승용,소형,개인용,1993,1,0
2,1111,11,서울특별시,양천구,경유,승용,대형,개인용,2007,1,0
3,1965,02,서울특별시,동작구,휘발유,승용,소형,개인용,2002,1,0
4,1975,01,서울특별시,서초구,휘발유,승용,대형,개인용,1968,1,0
...,...,...,...,...,...,...,...,...,...,...,...
238010,2023,09,경기도,광주시,경유,화물,소형,개인용,2010,1,0
238011,2023,09,광주광역시,서구,경유,화물,소형,개인용,2010,2,0
238012,2023,09,전라남도,영암군,경유,화물,소형,개인용,2010,2,0
238013,2023,10,대전광역시,서구,경유,화물,대형,개인용,2006,1,0


In [510]:
grp2['등록대수'].sum()

1318161

In [511]:
erea.shape

(449109, 47)

In [512]:
# !!! 수정(2023.10.24)
# 연도별 말소대수
grp3 = erea.groupby(['변경일자_년', '변경일자_월', '시도', '시군구_수정', '연료', '차종', '차종유형', '용도', '차량연식'], dropna=False).agg({'차대번호':'count', '저감장치부착유무':'count'}).reset_index() # !!! 수정(2023.11.27)
grp3 = grp3.rename(columns={'차대번호':'말소대수', '저감장치부착유무':'말소저감대수', '변경일자_년':'연도', '변경일자_월':'월'})
grp3

,연도,월,시도,시군구_수정,연료,차종,차종유형,용도,차량연식,말소대수,말소저감대수
0,2019,04,강원도,철원군,경유,화물,소형,개인용,2006,1,0
1,2019,05,경기도,가평군,휘발유,승용,중형,개인용,2001,1,0
2,2019,05,경상북도,김천시,경유,화물,소형,개인용,2010,1,0
3,2019,06,강원도,강릉시,LPG(액화석유가스),승용,중형,개인용,2000,1,0
4,2019,06,강원도,강릉시,경유,승용,중형,개인용,2006,1,0
...,...,...,...,...,...,...,...,...,...,...,...
210654,2023,11,충청북도,청주시,휘발유,승용,경형,개인용,1993,1,0
210655,2023,11,충청북도,청주시,휘발유,승용,소형,개인용,1993,1,0
210656,2023,11,충청북도,청주시,휘발유,승용,중형,개인용,1992,1,0
210657,2023,11,충청북도,청주시,휘발유,승용,중형,개인용,1994,2,0


In [513]:
grp3['말소대수'].sum()

449109

In [514]:
# !!! 수정(2023.10.24)
# 연도별 조기폐차 대수
grp4 = dfe.groupby(['말소일자_년', '말소일자_월', '시도', '시군구_수정', '연료', '차종', '차종유형', '용도', '차량연식'], dropna=False).agg({'조기폐차최종승인YN':'count'}).reset_index() # !!! 수정(2023.11.27)
grp4 = grp4.rename(columns={'말소일자_년':'연도', '말소일자_월':'월', '조기폐차최종승인YN':'조기폐차'})
grp4

,연도,월,시도,시군구_수정,연료,차종,차종유형,용도,차량연식,조기폐차
0,2020,12,경기도,김포시,경유,승용,중형,개인용,2006,1
1,2023,02,강원도,인제군,경유,승용,중형,개인용,2010,1
2,2023,02,강원도,인제군,경유,화물,소형,개인용,2009,1
3,2023,02,경기도,고양시,경유,승용,대형,개인용,2007,8
4,2023,02,경기도,고양시,경유,승용,대형,개인용,2008,4
...,...,...,...,...,...,...,...,...,...,...
67766,nan,,충청북도,충주시,휘발유,화물,소형,개인용,1991,0
67767,nan,,충청북도,충주시,휘발유,화물,소형,개인용,1992,0
67768,nan,,충청북도,충주시,휘발유,화물,소형,개인용,1993,0
67769,nan,,충청북도,충주시,휘발유,화물,소형,개인용,1994,0


In [515]:
div = 5
y_plist = list(pd.date_range(end=date, periods=div, freq="MS").year)
mth_plist = list(pd.date_range(end=date, periods=div, freq="MS").month)

y_plist, mth_plist

([2023, 2023, 2023, 2024, 2024], [10, 11, 12, 1, 2])

In [516]:
grp1['연료'].unique()

array(['LPG(액화석유가스)', '경유', '휘발유', '기타연료', '알코올', 'CNG(압축천연가스)'],
      dtype=object)

In [517]:
grp1['차종'].unique()

array(['승용', '승합', '화물', '특수'], dtype=object)

In [518]:
grp1['차종유형'].unique()

array(['대형', '중형', '경형', '소형'], dtype=object)

In [519]:
grp1['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

In [520]:
grp1.loc[(grp1['시도'] == '서울특별시') & (grp1['시군구_수정'] == '강남구') & (grp1['연료'] == '경유') & (grp1['차종'] == '승용') & (grp1['차종유형'] == '중형') & (grp1['용도'] == '개인용'), '차량연식'].unique()

array([2006, 2007, 2008, 2009, 2010, 2011, 2012], dtype=int64)

In [521]:
st1 = grp1[['시도', '시군구_수정', '연료', '차종', '차종유형', '용도', '차량연식']].drop_duplicates(['시도', '시군구_수정', '연료', '차종', '차종유형', '용도', '차량연식']).reset_index(drop=True)
st1

,시도,시군구_수정,연료,차종,차종유형,용도,차량연식
0,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998
1,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,2002
2,강원특별자치도,강릉시,LPG(액화석유가스),승용,중형,개인용,1992
3,강원특별자치도,강릉시,LPG(액화석유가스),승용,중형,개인용,1994
4,강원특별자치도,강릉시,LPG(액화석유가스),승용,중형,개인용,1996
...,...,...,...,...,...,...,...
38253,충청북도,충주시,휘발유,화물,소형,개인용,1991
38254,충청북도,충주시,휘발유,화물,소형,개인용,1992
38255,충청북도,충주시,휘발유,화물,소형,개인용,1993
38256,충청북도,충주시,휘발유,화물,소형,개인용,1994


In [522]:
temp = st1.copy()
for _ in range(4):
    st1 = pd.concat([st1, temp])
st1 = st1.sort_values(['시도', '시군구_수정', '연료', '차종', '차종유형', '용도', '차량연식'])
print(st1.shape)
st1 = st1.reset_index(drop=True)
st1

(191290, 7)


,시도,시군구_수정,연료,차종,차종유형,용도,차량연식
0,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998
1,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998
2,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998
3,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998
4,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998
...,...,...,...,...,...,...,...
191285,충청북도,충주시,휘발유,화물,소형,개인용,1995
191286,충청북도,충주시,휘발유,화물,소형,개인용,1995
191287,충청북도,충주시,휘발유,화물,소형,개인용,1995
191288,충청북도,충주시,휘발유,화물,소형,개인용,1995


In [523]:
st2 = pd.DataFrame({'연도':y_plist, '월':mth_plist})
st2[['연도', '월']] = st2[['연도', '월']].astype('str')
st2

,연도,월
0,2023,10
1,2023,11
2,2023,12
3,2024,1
4,2024,2


In [524]:
st2[['연도', '월']].dtypes

연도    object
월     object
dtype: object

In [525]:
## 1m 49.6s 
temp = st2.copy()
for _ in range(1, st1.shape[0] // div):
    st2 = pd.concat([st2, temp])
print(st2.shape)
st2 = st2.reset_index(drop=True)
st2

(191290, 2)


,연도,월
0,2023,10
1,2023,11
2,2023,12
3,2024,1
4,2024,2
...,...,...
191285,2023,10
191286,2023,11
191287,2023,12
191288,2024,1


In [526]:
base = pd.concat([st2, st1], axis=1)
base

,연도,월,시도,시군구_수정,연료,차종,차종유형,용도,차량연식
0,2023,10,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998
1,2023,11,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998
2,2023,12,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998
3,2024,1,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998
4,2024,2,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998
...,...,...,...,...,...,...,...,...,...
191285,2023,10,충청북도,충주시,휘발유,화물,소형,개인용,1995
191286,2023,11,충청북도,충주시,휘발유,화물,소형,개인용,1995
191287,2023,12,충청북도,충주시,휘발유,화물,소형,개인용,1995
191288,2024,1,충청북도,충주시,휘발유,화물,소형,개인용,1995


In [527]:
# # 18s
# # 4개월 차량 통계 기본 데이터셋
# ctpv_list, sgg_list, fuel_list, vhcty_list, ty_list, purps_list, yr_list, month_list, car_year_list = [], [], [], [], [], [], [], [], [] # !!! 수정(2023.11.27)
# ctpv_sgg = grp1.drop_duplicates(['시도', '시군구_수정']).reset_index(drop=True)
# for ctpv, sgg in ctpv_sgg[['시도', '시군구_수정']].values:
#     for fuel in grp1['연료'].unique():
#         for vhcty in grp1['차종'].unique():
#             for ty in grp1['차종유형'].unique():
#                 for purps in grp1['용도'].unique():
#                     for car_year in grp1.loc[(grp1['시도'] == ctpv) & (grp1['시군구_수정'] == sgg) & (grp1['연료'] == fuel) & (grp1['차종'] == vhcty) & (grp1['차종유형'] == ty) & (grp1['용도'] == purps), '차량연식'].unique(): # !!! 수정(2023.11.27)
#                         for yr, month in zip(y_plist, mth_plist):
#                             ctpv_list.append(ctpv)
#                             sgg_list.append(sgg)
#                             fuel_list.append(fuel)
#                             vhcty_list.append(vhcty)
#                             ty_list.append(ty)
#                             purps_list.append(purps)
#                             yr_list.append(str(yr))
#                             month_list.append(f'{month:0>2}')
#                             car_year_list.append(car_year)
# base = pd.DataFrame({'연도':yr_list, '월':month_list, '시도':ctpv_list, '시군구_수정':sgg_list, '연료':fuel_list, '차종':vhcty_list, '차종유형':ty_list, '용도':purps_list, '차량연식':car_year_list}) # !!! 수정(2023.11.27)
# base

In [528]:
base1 = base.merge(grp1, on=['연도', '월', '시도', '시군구_수정', '연료', '차종', '차종유형', '용도', '차량연식'], how='left') # !!! 수정(2023.11.27)
base2 = base1.merge(grp2, on=['연도', '월', '시도', '시군구_수정', '연료', '차종', '차종유형', '용도', '차량연식'], how='left') # !!! 수정(2023.11.27)
base3 = base2.merge(grp3, on=['연도', '월', '시도', '시군구_수정', '연료', '차종', '차종유형', '용도', '차량연식'], how='left') # !!! 수정(2023.11.27)
base4 = base3.merge(grp4, on=['연도', '월', '시도', '시군구_수정', '연료', '차종', '차종유형', '용도', '차량연식'], how='left') # !!! 수정(2023.11.27)
base4

,연도,월,시도,시군구_수정,연료,차종,차종유형,용도,차량연식,차량대수,저감대수,등록대수,등록저감대수,말소대수,말소저감대수,조기폐차
0,2023,10,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023,11,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023,12,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024,1,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024,2,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191285,2023,10,충청북도,충주시,휘발유,화물,소형,개인용,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191286,2023,11,충청북도,충주시,휘발유,화물,소형,개인용,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191287,2023,12,충청북도,충주시,휘발유,화물,소형,개인용,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191288,2024,1,충청북도,충주시,휘발유,화물,소형,개인용,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [529]:
base4.loc[base4['차량대수'] < 0, ['연도', '월', '시도', '시군구_수정', '차량대수']]

,연도,월,시도,시군구_수정,차량대수


In [530]:
# !!! 수정 시작(2023.11.09)

base4[['차량대수', '조기폐차', '저감대수', '등록대수', '등록저감대수', '말소대수', '말소저감대수']] = base4[['차량대수', '조기폐차', '저감대수', '등록대수', '등록저감대수', '말소대수', '말소저감대수']].fillna(0)
# base4[['차량대수_1', '등록대수_1', '말소대수_1', '저감대수_1', '등록저감대수_1', '말소저감대수_1']] = base4[['차량대수', '등록대수', '말소대수', '저감대수', '등록저감대수', '말소저감대수']].shift(-1)
# base4.loc[[x for x in range(3, base4.shape[0], 4)], ['차량대수_1', '저감대수_1']]  = base4.loc[[x for x in range(3, base4.shape[0], 4)], ['차량대수', '저감대수']].values
# base4.loc[[x for x in range(3, base4.shape[0], 4)], ['등록대수_1', '말소대수_1', '등록저감대수_1', '말소저감대수_1']] = 0
# base4['차량대수'] = base4['차량대수_1'] - base4['등록대수_1'] + base4['말소대수_1']
base4

,연도,월,시도,시군구_수정,연료,차종,차종유형,용도,차량연식,차량대수,저감대수,등록대수,등록저감대수,말소대수,말소저감대수,조기폐차
0,2023,10,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023,11,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2023,12,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2024,1,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024,2,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191285,2023,10,충청북도,충주시,휘발유,화물,소형,개인용,1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0
191286,2023,11,충청북도,충주시,휘발유,화물,소형,개인용,1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0
191287,2023,12,충청북도,충주시,휘발유,화물,소형,개인용,1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0
191288,2024,1,충청북도,충주시,휘발유,화물,소형,개인용,1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [531]:
base4['월'].unique()

array(['10', '11', '12', '1', '2'], dtype=object)

In [532]:
# 2m 4.9s
n = len(base4['월'].unique())
for i in range(base4.shape[0] // n):
    for j in range(2, n+1):
        base4.loc[(i+1)*n - j, '차량대수'] = base4.loc[(i+1)*n - (j-1), '차량대수'] + base4.loc[(i+1)*n - (j-1), '말소대수'] - base4.loc[(i+1)*n - (j-1), '등록대수']
        base4.loc[(i+1)*n - j, '저감대수'] = base4.loc[(i+1)*n - (j-1), '저감대수'] + base4.loc[(i+1)*n - (j-1), '말소저감대수'] - base4.loc[(i+1)*n - (j-1), '등록저감대수']
        
# !!! 수정 끝(2023.11.09)

In [533]:
base4

,연도,월,시도,시군구_수정,연료,차종,차종유형,용도,차량연식,차량대수,저감대수,등록대수,등록저감대수,말소대수,말소저감대수,조기폐차
0,2023,10,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023,11,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2023,12,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2024,1,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024,2,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191285,2023,10,충청북도,충주시,휘발유,화물,소형,개인용,1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0
191286,2023,11,충청북도,충주시,휘발유,화물,소형,개인용,1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0
191287,2023,12,충청북도,충주시,휘발유,화물,소형,개인용,1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0
191288,2024,1,충청북도,충주시,휘발유,화물,소형,개인용,1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [534]:
base4[(base4['연도'] == '2023') & (base4['시도'] == '경기도') & (base4['시군구_수정'] == '고양시') & (base4['연료'] == '경유') & (base4['차종'] == '승용') & (base4['차종유형'] == '중형') & (base4['용도'] == '개인용')]

,연도,월,시도,시군구_수정,연료,차종,차종유형,용도,차량연식,차량대수,저감대수,등록대수,등록저감대수,말소대수,말소저감대수,조기폐차
12770,2023,10,경기도,고양시,경유,승용,중형,개인용,2005,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12771,2023,11,경기도,고양시,경유,승용,중형,개인용,2005,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12772,2023,12,경기도,고양시,경유,승용,중형,개인용,2005,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12775,2023,10,경기도,고양시,경유,승용,중형,개인용,2006,0.0,0.0,0.0,0.0,16.0,0.0,3.0
12776,2023,11,경기도,고양시,경유,승용,중형,개인용,2006,0.0,0.0,0.0,0.0,0.0,0.0,2.0
12777,2023,12,경기도,고양시,경유,승용,중형,개인용,2006,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12780,2023,10,경기도,고양시,경유,승용,중형,개인용,2007,1.0,0.0,0.0,0.0,22.0,0.0,3.0
12781,2023,11,경기도,고양시,경유,승용,중형,개인용,2007,0.0,0.0,0.0,0.0,1.0,0.0,1.0
12782,2023,12,경기도,고양시,경유,승용,중형,개인용,2007,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12785,2023,10,경기도,고양시,경유,승용,중형,개인용,2008,0.0,0.0,0.0,0.0,41.0,0.0,27.0


In [535]:
base5 = base4[['연도', '월', '시도', '시군구_수정', '연료', '차종', '차종유형', '용도', '차량연식', '차량대수', '조기폐차', '저감대수']] # !!! 수정(2023.11.27)

In [536]:
base5.loc[32175:32179, ['차량대수']].shift()

,차량대수
32175,NaN
32176,0.0
32177,0.0
32178,0.0
32179,0.0


In [537]:
base5['감소대수'] = base5['차량대수'].shift() - base5['차량대수']

C:\Users\kbjung\AppData\Local\Temp\ipykernel_2144\746243287.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base5['감소대수'] = base5['차량대수'].shift() - base5['차량대수']


In [538]:
base5.loc[32175:32179, ['연도', '월', '차량대수', '감소대수']]

,연도,월,차량대수,감소대수
32175,2023,10,0.0,0.0
32176,2023,11,0.0,0.0
32177,2023,12,0.0,0.0
32178,2024,1,0.0,0.0
32179,2024,2,0.0,0.0


In [539]:
base5['감소율'] = base5['감소대수'] / base5['차량대수'].shift()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_2144\3327424748.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base5['감소율'] = base5['감소대수'] / base5['차량대수'].shift()


In [540]:
base5.loc[32175:32179, ['연도', '월', '차량대수', '감소대수', '감소율']]

,연도,월,차량대수,감소대수,감소율
32175,2023,10,0.0,0.0,NaN
32176,2023,11,0.0,0.0,NaN
32177,2023,12,0.0,0.0,NaN
32178,2024,1,0.0,0.0,NaN
32179,2024,2,0.0,0.0,NaN


In [541]:
base5.loc[(base5['감소율'] == -np.inf) | (base5['감소율'] == np.inf), '감소율'] = 0
base5['감소율'] = base5['감소율'].fillna(0)
base5['저감장치미부착대수'] = base5['차량대수'] - base5['저감대수']

C:\Users\kbjung\AppData\Local\Temp\ipykernel_2144\2245260143.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base5['감소율'] = base5['감소율'].fillna(0)
C:\Users\kbjung\AppData\Local\Temp\ipykernel_2144\2245260143.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base5['저감장치미부착대수'] = base5['차량대수'] - base5['저감대수']


In [542]:
div

5

In [543]:
base5.loc[base5['차량대수'] < 0, '차량대수'] = 0
base5.loc[base5['저감장치미부착대수'] < 0, '저감장치미부착대수'] = 0
base5.loc[[x for x in range(0, base5.shape[0], div)], '감소율'] = 0 # !!! 수정(2023.11.09)
base5.loc[[x for x in range(0, base5.shape[0], div)], '감소대수'] = 0 # !!! 수정(2023.11.09)
base5 = base5.rename(columns={'조기폐차':'조기폐차대수', '저감대수':'저감장치부착대수'})

In [544]:
base5.loc[32175:32179, ['연도', '월', '차량대수', '감소대수', '감소율']]

,연도,월,차량대수,감소대수,감소율
32175,2023,10,0.0,0.0,0.0
32176,2023,11,0.0,0.0,0.0
32177,2023,12,0.0,0.0,0.0
32178,2024,1,0.0,0.0,0.0
32179,2024,2,0.0,0.0,0.0


In [545]:
base5.head()

,연도,월,시도,시군구_수정,연료,차종,차종유형,용도,차량연식,차량대수,조기폐차대수,저감장치부착대수,감소대수,감소율,저감장치미부착대수
0,2023,10,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,0.0
1,2023,11,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,0.0
2,2023,12,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,0.0
3,2024,1,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,0.0
4,2024,2,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,0.0


In [546]:
base5.isnull().sum()

연도             0
월              0
시도             0
시군구_수정       895
연료             0
차종             0
차종유형           0
용도             0
차량연식           0
차량대수           0
조기폐차대수         0
저감장치부착대수       0
감소대수           0
감소율            0
저감장치미부착대수      0
dtype: int64

In [547]:
base5['감소대수'] = base5['감소대수'].fillna(0) # !!! 수정 (2023.11.09)
base5

,연도,월,시도,시군구_수정,연료,차종,차종유형,용도,차량연식,차량대수,조기폐차대수,저감장치부착대수,감소대수,감소율,저감장치미부착대수
0,2023,10,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,0.0
1,2023,11,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,0.0
2,2023,12,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,0.0
3,2024,1,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,0.0
4,2024,2,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191285,2023,10,충청북도,충주시,휘발유,화물,소형,개인용,1995,0.0,0.0,0.0,0.0,0.0,0.0
191286,2023,11,충청북도,충주시,휘발유,화물,소형,개인용,1995,0.0,0.0,0.0,0.0,0.0,0.0
191287,2023,12,충청북도,충주시,휘발유,화물,소형,개인용,1995,0.0,0.0,0.0,0.0,0.0,0.0
191288,2024,1,충청북도,충주시,휘발유,화물,소형,개인용,1995,0.0,0.0,0.0,0.0,0.0,0.0


In [548]:
base5[['연도', '월']].dtypes

연도    object
월     object
dtype: object

In [549]:
base5[['연도', '월']] = base5[['연도', '월']].astype('int')

In [550]:
base5[['연도', '월']].dtypes

연도    int32
월     int32
dtype: object

In [551]:
base5['월'].unique()

array([10, 11, 12,  1,  2])

In [552]:
# 현재 월 -1 까지만 추출(수동)
base5 = base5[(base5['연도'] != int(today_date[:4])) | (base5['월'] != int(today_date[4:6]))].reset_index(drop=True) # !!! 수정(2024.01.03)

In [553]:
base5['월'].unique()

array([10, 11, 12,  1])

In [554]:
today_date = datetime.today().strftime("%Y%m%d")
base5['테이블생성일자'] = today_date

In [555]:
base5.groupby(['연도', '월'])['차량대수'].sum()

연도    월 
2023  10    276.0
      11      0.0
      12      0.0
2024  1       0.0
Name: 차량대수, dtype: float64

In [556]:
base5['연료'].unique()

array(['LPG(액화석유가스)', '경유', '휘발유', '기타연료', '알코올', 'CNG(압축천연가스)'],
      dtype=object)

In [557]:
base5.shape

(153032, 16)

In [558]:
base5 = base5[base5['연료'].notnull()]
base5.shape

(153032, 16)

In [559]:
base5[base5['연료'].isnull()].shape

(0, 16)

In [560]:
base5[(base5['연도'] == '2023') & (base5['시도'] == '경기도') & (base5['시군구_수정'] == '고양시') & (base5['연료'] == '경유') & (base5['차종'] == '승용') & (base5['차종유형'] == '중형') & (base5['용도'] == '개인용')]

,연도,월,시도,시군구_수정,연료,차종,차종유형,용도,차량연식,차량대수,조기폐차대수,저감장치부착대수,감소대수,감소율,저감장치미부착대수,테이블생성일자


In [561]:
## 10.1s
STD_BD_DAT_GRD4_CAR_CURSTT = base5[[
    '연도',
    '월',
    '시도',
    '시군구_수정',
    '연료',
    '차종',
    '차종유형', 
    '용도',
    '차량연식', # !!! 수정(2023.11.27)
    '차량대수',
    '감소율',
    '저감장치부착대수',
    '저감장치미부착대수',
    '조기폐차대수',
    '테이블생성일자',
]]
chc_col = {
    '연도':'YR',
    '월':'MM',
    '시도':'CTPV',
    '시군구_수정':'SGG',
    '연료':'FUEL_CD',
    '차종':'VHCTY_CD',
    '차종유형':'VHCTY_TY', 
    '용도':'PURPS_CD2',
    '차량연식':'YRIDNW', # !!! 수정(2023.11.27)
    '차량대수':'VHCL_MKCNT',
    '감소율':'DEC_RT',
    '저감장치부착대수':'RDCDVC_EXTRNS_MKCNT',
    '저감장치미부착대수':'RDCDVC_UNAT_MKCNT',
    '조기폐차대수':'ELPDSRC_MKCNT',
    '테이블생성일자':'LOAD_DT', 
}
STD_BD_DAT_GRD4_CAR_CURSTT = STD_BD_DAT_GRD4_CAR_CURSTT.rename(columns=chc_col)

STD_BD_DAT_GRD4_CAR_CURSTT.columns

Index(['YR', 'MM', 'CTPV', 'SGG', 'FUEL_CD', 'VHCTY_CD', 'VHCTY_TY',
       'PURPS_CD2', 'YRIDNW', 'VHCL_MKCNT', 'DEC_RT', 'RDCDVC_EXTRNS_MKCNT',
       'RDCDVC_UNAT_MKCNT', 'ELPDSRC_MKCNT', 'LOAD_DT'],
      dtype='object')

In [562]:
STD_BD_DAT_GRD4_CAR_CURSTT.head()

,YR,MM,CTPV,SGG,FUEL_CD,VHCTY_CD,VHCTY_TY,PURPS_CD2,YRIDNW,VHCL_MKCNT,DEC_RT,RDCDVC_EXTRNS_MKCNT,RDCDVC_UNAT_MKCNT,ELPDSRC_MKCNT,LOAD_DT
0,2023,10,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,20240213
1,2023,11,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,20240213
2,2023,12,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,20240213
3,2024,1,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,1998,0.0,0.0,0.0,0.0,0.0,20240213
4,2023,10,강원특별자치도,강릉시,LPG(액화석유가스),승용,대형,개인용,2002,0.0,0.0,0.0,0.0,0.0,20240213


### [출력] STD_BD_DAT_GRD4_CAR_CURSTT

In [563]:
# expdf = STD_BD_DAT_GRD4_CAR_CURSTT
# table_nm = 'STD_BD_DAT_GRD4_CAR_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

## 1-2 code end ##################################################################

## ---------- 1-3 start ----------

In [564]:
## 10.6s
## 지역정보 병합
df = csi.merge(coder, on='법정동코드', how='left')
df.shape

(1318161, 40)

In [565]:
df['시도'].isnull().sum()

0

In [566]:
# df.loc[df['법정동코드'] == 4163055000, ['시도', '시군구']] = ['경기도', '양주시']

In [567]:
# ### rh 법정동코드 참고하여 법정동코드 수정
# rdf = df.copy()
# rdf['법정동코드'] = rdf['법정동코드'].astype('str')
# rdf['법정동코드_mod'] = rdf['법정동코드'].str[:5] + '00000'
# rdf['법정동코드_mod'] = pd.to_numeric(rdf['법정동코드_mod'])
# # 2817000000 인천광역시 남구 -> 인천광역시 미추홀구 2817700000
# # 4119500000 경기도 부천시 원미구 -> 경기도 부천시 4119000000
# # 4119700000 경기도 부천시 소사구 -> 경기도 부천시 4119000000
# # 4119900000 경기도 부천시 오정구 -> 경기도 부천시 4119000000
# # 4173000000 경기도 여주군 -> 경기도 여주시 4167000000
# # 4371000000 충청북도 청원군 -> 충청북도 충주시 4311000000
# rdf.loc[rdf['법정동코드_mod'] == 2817000000, '법정동코드_mod'] = 2817700000
# rdf.loc[rdf['법정동코드_mod'] == 4119500000, '법정동코드_mod'] = 4119000000
# rdf.loc[rdf['법정동코드_mod'] == 4119700000, '법정동코드_mod'] = 4119000000
# rdf.loc[rdf['법정동코드_mod'] == 4119900000, '법정동코드_mod'] = 4119000000
# rdf.loc[rdf['법정동코드_mod'] == 4173000000, '법정동코드_mod'] = 4167000000
# rdf.loc[rdf['법정동코드_mod'] == 4371000000, '법정동코드_mod'] = 4311000000
# df = rdf.copy()

In [568]:
df['법정동코드_mod'] = df['법정동코드'].copy()

In [569]:
# rdf = df.copy()
# rdf['법정동코드'] = rdf['법정동코드'].astype('str')
# rdf.loc[rdf['시도'].isnull() == True, '법정동코드'] = rdf.loc[rdf['시도'].isnull() == True, '법정동코드'].str[:5] + '00000'
# rdf['법정동코드'] = pd.to_numeric(rdf['법정동코드'])

# rdfy = rdf[rdf['시도'].isnull() == False]
# rdfn = rdf[rdf['시도'].isnull() == True]

# rdfn = rdfn.drop(['시도', '시군구'], axis=1)
# rdfnm = rdfn.merge(coder, on='법정동코드', how='left')

# df = pd.concat([rdfy, rdfnm], ignore_index=False)

In [570]:
## 16.3s
## 일일평균주행거리 계산
df['검사일자'] = df['검사일자'].fillna(0)
df['검사일자'] = df['검사일자'].astype('str')
df['검사일자'] = df['검사일자'].str.split('.').str[0]
df['최초등록일자'] = pd.to_datetime(df['최초등록일자'], errors='coerce')
df['검사일자'] = pd.to_datetime(df['검사일자'], errors='coerce')
df['기간차이'] = df['검사일자'] - df['최초등록일자']

C:\Users\kbjung\AppData\Local\Temp\ipykernel_2144\1675057448.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['검사일자'] = pd.to_datetime(df['검사일자'], errors='coerce')


In [571]:
# 23.2s
df['기간차이'] = df['기간차이'].astype('str')
df['기간차이'] = df['기간차이'].str.split(' ').str[0]
df['기간차이'] = pd.to_numeric(df['기간차이'], errors='coerce')
df['일일평균주행거리'] = df['주행거리'] / (df['기간차이'])

In [572]:
## 10.6s
### 빈 값 kosis로 대체
df1y = df[df['일일평균주행거리'].isnull() == False]
df1n = df[df['일일평균주행거리'].isnull() == True]

In [573]:
## 11.0s
df1n = df1n.drop('일일평균주행거리', axis=1)
df1nm = df1n.merge(kosisr, on=['시도', '시군구', '차종'], how='left')

In [574]:
## 10.65s
df2y = df1nm[df1nm['일일평균주행거리'].isnull() == False]
df2n = df1nm[df1nm['일일평균주행거리'].isnull() == True]
df2n = df2n.drop('일일평균주행거리', axis=1)
df2nm = df2n.merge(kosisr.drop_duplicates(['시도', '차종'])[['시도', '차종', '일일평균주행거리']], on=['시도', '차종'], how='left')

In [575]:
df3y = df2nm[df2nm['일일평균주행거리'].isnull() == False]
df3n = df2nm[df2nm['일일평균주행거리'].isnull() == True]

In [576]:
for ctpv, sgg, cd in df3n.loc[df3n['일일평균주행거리'].isnull() == True, ['시도', '시군구', '차종']].values:
    try:
        df3n.loc[(df3n['일일평균주행거리'].isnull() == True) & (df3n['시도'] == ctpv) & (df3n['시군구'] == sgg), '일일평균주행거리'] = kosisr.loc[(kosisr['시도'] == ctpv) & (kosisr['시군구'] == '소계') & (kosisr['차종'] == '합계'), '일일평균주행거리'].values[0]
    except:
        df3n.loc[(df3n['일일평균주행거리'].isnull() == True) & (df3n['시도'].isnull() == True) & (df3n['시군구'].isnull() == True), '일일평균주행거리'] = kosisr.loc[(kosisr['시도'] == '서울특별시') & (kosisr['시군구'] == '소계') & (kosisr['차종'] == '합계'), '일일평균주행거리'].values[0]

In [577]:
## 11.1s
df2nm = pd.concat([df3y, df3n], ignore_index=True)
df1nm = pd.concat([df2y, df2nm], ignore_index=True)
df = pd.concat([df1y, df1nm], ignore_index=True)
df.shape

(1318161, 43)

In [578]:
## 12.3s
### 저감장치 부착 유무 정보 병합
df1 = df.merge(attr[['차대번호', 'DPF_YN']], on='차대번호', how='left')
df1.shape

(1318161, 44)

In [579]:
## 15.2s
## 4등급 result 파일 참고하여 DPF유무 수정
rdf = df1.copy()
rs = rs.drop_duplicates('차대번호').reset_index(drop=True)
rdf1 = rdf.merge(rs, on='차대번호', how='left')
rdf1.loc[(rdf1['DPF_YN'] == '유') | (rdf1['DPF유무_수정'] == '유'), 'DPF_YN'] = '유'
rdf1.loc[(rdf1['DPF유무_수정'] == '무'), 'DPF_YN'] = '무'
rdf1.loc[(rdf1['DPF유무_수정'] == '확인불가'), 'DPF_YN'] = '확인불가'
df1 = rdf1.drop('DPF유무_수정', axis=1)

In [580]:
df1.shape

(1318161, 44)

In [581]:
## 11.6s
## 4등급 차량(휘발유, 경유, LPG) 추출
etc = df1[(df1['연료'] == '알코올') | (df1['연료'] == 'CNG(압축천연가스)') | (df1['연료'] == '기타연료')].reset_index(drop=True)
dgl = df1[(df1['연료'] == '휘발유') | (df1['연료'] == '경유') | (df1['연료'] == 'LPG(액화석유가스)')].reset_index(drop=True)

In [582]:
dgl.shape

(1318124, 44)

In [583]:
### 차령 정보 계산 및 데이터 추가
current_yr = int(datetime.today().strftime("%Y"))
dgl['차령'] = current_yr - dgl['차량연식']

In [584]:
## 12.3s
## 시군구명 앞쪽만 사용
# - 시군구명 앞지역명만 추출
dgl['시군구_수정'] = dgl['시군구'].str.split(' ').str[0]

In [585]:
## 11.0s
# 자동차-엔진 가열(Hot-start) 배출
## 배출량 식
# $$E_{i, j} = VKT \times {EF_i \over {1000}} \times DF \times (1 - {R \over {100}})$$
# $E_{i, j}$ : 자동차 $j$의 도로주행시 발생한 오염물질 $i$의 배출량($kg/yr$)
# $VKT$ : 주행거리($km/yr$)
# $EF_i$ : 차종별, 연료별, 연식별, 차속별 배출계수($g/km$)
# $DF$ : 열화계수
# $R$ : 저감장치 부착 효율(%)
df2 = dgl.copy()

In [586]:
## 주행거리(VKT)
# - [현재 설정] "용도별_차종별_시군구별_자동차주행거리" 자료(KOSIS)(2021)의 1일 평균주행거리 * 365 = 주행거리(km/yr)
df2['VKT'] = df2['일일평균주행거리'] * 365

In [587]:
## DF(열화계수)
# - 필요한 정보 : 연료, 차종, 차종유형, 연식, 차령
# - 조건 수정(2023.04.20, 최)
#     - 특수 소형 = 승용 소형
#     - 승합 대형 = 화물 대형

DF_col = ['연료', '차종', '차종유형', '차량연식', '차령']

In [588]:
## 24.9s
### ❗ DF(열화계수) 코드
# - 연료, 차종, 차종유형 설정에 해당되지 않는 차량의 열화계수 값 설정
#     - 현재 설정값 : np.nan
#     - 고려 설정값 : 1

# about 5.0s
DF_CO_list = []
DF_HC_list = []
DF_PM_list = []
DF_NOx_list = []
# '연료', '차종', '차종유형', '차량연식', '차령'
for fuel, car_type, car_size, car_birth, car_age in df2[DF_col].values:    
    if fuel == '경유':
        if (car_type == '승용') or ( (car_type == '특수') and (car_size == '소형') ): # 최 확인(2023.04.20) : "특수 소형" -> "승용 소형" 조건으로 계산
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
        elif car_type == '승합':
            if car_size == '소형' or car_size == '중형':
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.05
                    DF_HC = 1 + (car_age - warranty)*0.05
                    DF_PM = 1 + (car_age - warranty)*0.05
                    DF_NOx = 1 + (car_age - warranty)*0.02
                    if DF_CO > 1.5:
                        DF_CO = 1.5
                    if DF_HC > 1.5:
                        DF_HC = 1.5
                    if DF_PM > 1.5:
                        DF_PM = 1.5
                    if DF_NOx > 1.2:
                        DF_NOx = 1.2
            elif car_size == '대형':
                warranty = 3
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.05
                    DF_HC = 1 + (car_age - warranty)*0.05
                    DF_PM = 1 + (car_age - warranty)*0.05
                    DF_NOx = 1 + (car_age - warranty)*0.02
                    if DF_CO > 1.5:
                        DF_CO = 1.5
                    if DF_HC > 1.5:
                        DF_HC = 1.5
                    if DF_PM > 1.5:
                        DF_PM = 1.5
                    if DF_NOx > 1.2:
                        DF_NOx = 1.2
        elif car_type == '화물':
            # car_size : 경, 소, 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
        elif car_type == '특수':
            # car_size : 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
    elif (fuel == '휘발유') or (fuel == 'LPG(액화석유가스)'):
        if car_type == '승용':
            # 경, 소, 중, 대
            if car_birth <= 2000:
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
            elif car_birth >= 2001:
                warranty = 10
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
        elif car_type == '승합':
            if car_size == '경형':
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
            elif (car_size == '소형') or (car_size == '중형'):
                if car_birth <= 2005:
                    warranty = 5
                    if car_age <= warranty:
                        DF_CO = 1
                        DF_HC = 1
                        DF_PM = 1
                        DF_NOx = 1
                    else:
                        DF_CO = 1 + (car_age - warranty)*0.1
                        DF_HC = 1 + (car_age - warranty)*0.1
                        DF_PM = 1 + (car_age - warranty)*0.1
                        DF_NOx = 1 + (car_age - warranty)*0.1
                        if DF_CO > 2.0:
                            DF_CO = 2.0
                        if DF_HC > 2.0:
                            DF_HC = 2.0
                        if DF_PM > 2.0:
                            DF_PM = 2.0
                        if DF_NOx > 2.0:
                            DF_NOx = 2.0
                elif car_birth >= 2006:
                    warranty = 10
                    if car_age <= warranty:
                        DF_CO = 1
                        DF_HC = 1
                        DF_PM = 1
                        DF_NOx = 1
                    else:
                        DF_CO = 1 + (car_age - warranty)*0.1
                        DF_HC = 1 + (car_age - warranty)*0.1
                        DF_PM = 1 + (car_age - warranty)*0.1
                        DF_NOx = 1 + (car_age - warranty)*0.1
                        if DF_CO > 2.0:
                            DF_CO = 2.0
                        if DF_HC > 2.0:
                            DF_HC = 2.0
                        if DF_PM > 2.0:
                            DF_PM = 2.0
                        if DF_NOx > 2.0:
                            DF_NOx = 2.0
        elif (car_type == '화물') or ( (car_type == '승합') and (car_size == '대형')): # 최 확인(2023.04.20) : 습합 대형 -> 화물 대형 조건으로 계산
            # 경, 소, 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.1
                DF_HC = 1 + (car_age - warranty)*0.1
                DF_PM = 1 + (car_age - warranty)*0.1
                DF_NOx = 1 + (car_age - warranty)*0.1
                if DF_CO > 2.0:
                    DF_CO = 2.0
                if DF_HC > 2.0:
                    DF_HC = 2.0
                if DF_PM > 2.0:
                    DF_PM = 2.0
                if DF_NOx > 2.0:
                    DF_NOx = 2.0
        elif car_type == '특수':
            # 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.1
                DF_HC = 1 + (car_age - warranty)*0.1
                DF_PM = 1 + (car_age - warranty)*0.1
                DF_NOx = 1 + (car_age - warranty)*0.1
                if DF_CO > 2.0:
                    DF_CO = 2.0
                if DF_HC > 2.0:
                    DF_HC = 2.0
                if DF_PM > 2.0:
                    DF_PM = 2.0
                if DF_NOx > 2.0:
                    DF_NOx = 2.0
    else:
        # 설정 고민
            # 고려 : 1
        # DF_CO, DF_HC, DF_PM, DF_NOx = np.nan, np.nan, np.nan, np.nan
        DF_CO, DF_HC, DF_PM, DF_NOx = 1, 1, 1, 1

    # 열화계수(DF) 리스트에 저장
    DF_CO_list.append(DF_CO)
    DF_HC_list.append(DF_HC)
    DF_PM_list.append(DF_PM)
    DF_NOx_list.append(DF_NOx)

df2['DF_CO'] = DF_CO_list
df2['DF_HC'] = DF_HC_list
df2['DF_PM'] = DF_PM_list
df2['DF_NOx'] = DF_NOx_list
check_DF_col = ['DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx']

In [589]:
## 11.1s
## 저감장치 부착 효율(R) 계산
# - DPF유무_수정 : 유 -> DPF로 가정하고 진행
# - 휘발성 유기 화합물(Volatile Organic Compounds:VOC) : 생활주변에서 흔히 사용하는 탄화수소류가 거의 해당됨.
# - 저감장치별 물질 제거 효율(%)

#     장치종류|CO|VOC|PM
#     :-:|:-:|:-:|:-:
#     DPF|99.5|90|83.6
#     pDPF|94.6|89.3|56
#     DOC|85.4|72|35

df2.loc[df2['DPF_YN'] == '유', ['R_CO', 'R_HC', 'R_PM']] = 99.5, 90, 83.6
check_R_col = ['R_CO', 'R_HC', 'R_PM']
df2[check_R_col] = df2[check_R_col].fillna(0)

In [590]:
## ❗ 배출계수(EFi)
# - 연료, 차종, 차종유형 설정에 해당되지 않는 차량의 배출계수 값 설정
#     - 현재 설정값 : np.nan
#     - 고려 설정값 : 1
# - 조건 수정(2023.04.20, 최)
#     - 특수 중형 = 승합 중형
#     - 화물 중형 = 승합 중형
#     - 화물 대형 = 승합 대형
#     - 특수 중형 = 승용 중형

EFi_col = ['차종', '차종유형', '연료', '차량연식']

In [591]:
# 참고 : KOSIS 차량속도(2017) 일반국도 평균 (https://kosis.kr/statHtml/statHtml.do?orgId=210&tblId=DT_21002_J008)
V = 54.1
# 국가 대기오염물질 배출량 산정방법 편람(V)(2022) 부록 참고(for PM-2.5)
k = 0.92

In [592]:
## 38.0s
# about 20.1s
EFi_CO_list = []
EFi_HC_list = []
EFi_NOx_list = []
EFi_PM10_list = []
EFi_PM2_5_list = []
EFi_NH3_list = []

# 차종, 차종유형, 연료, 연식
for car_type, car_size, fuel, car_birth in df2[EFi_col].values:
    EFi_COm = 0
    EFi_HCm = 0
    EFi_NOxm = 0
    EFi_PM10m = 0
    EFi_PM2_5m = 0
    EFi_NH3m = 0
    for V in [35, 70, 100]:
        if fuel == '휘발유':
            if (car_type == '승용') and (car_size == '경형'):
                if car_birth <= 1996:
                    if V <= 65:
                        EFi_CO = 59.783 * (V**-1.0007)
                    else:
                        EFi_CO = 0.0874 * V - 3.5618
                    EFi_HC = 7.6244 * (V**-0.8364)
                    EFi_NOx = 2.6754 * (V**-0.3236)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1997 <= car_birth <= 1999:
                    if V <= 65:
                        EFi_CO = 59.783 * (V**-1.0007)
                    else:
                        EFi_CO = 0.0874 * V - 3.5618
                    EFi_HC = 8.6275 * (V**-1.0722)
                    EFi_NOx = 3.2294 * (V**-0.5763)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 2000 <= car_birth <= 2002:
                    if V <= 65:
                        EFi_CO = 60.556 * (V**-1.2501)
                    else:
                        EFi_CO = -0.0006 * V + 0.5753
                    EFi_HC = 5.1835 * (V**-1.1889)
                    EFi_NOx = 1.7525 * (V**-0.6481)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V > 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    if V <= 65:
                        EFi_CO = 60.556 * (V**-1.2501)
                    else:
                        EFi_CO = -0.0006 * V + 0.5753
                    EFi_HC = 0.7446 * (V**-0.9392)
                    EFi_NOx = 0.3403 * (V**-0.5455)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    if V <= 45:
                        EFi_CO = 4.9952 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0001 * V**2 + 0.0229 * V - 0.5701
                    EFi_HC = 0.2958 * (V**-0.7830)
                    EFi_NOx = 0.4819 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth <= 2011:
                    if V <= 45:
                        EFi_CO = 4.5956 * (V**-0.8461)
                    else:
                        EFi_CO = -9.2000*(10**-5) * (V**2) + 2.1068*(10**-2) * V - 5.2449*(10**-1)
                    EFi_HC = 0.2662 * (V**-0.7830)
                    EFi_NOx = 0.4476 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2012 <= car_birth <= 2013:
                    if V <= 45:
                        EFi_CO = 4.4517 * (V**-0.8461)
                    else:
                        EFi_CO = -8.9120*(10**-5) * (V**2) + 2.0408*(10**-2)*V - 5.0807*(10**-1)
                    EFi_HC = 0.2556 * (V**-0.7830)
                    EFi_NOx = 0.4353 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif car_birth == 2014:
                    if V <= 45:
                        EFi_CO = 4.3079 * (V**-0.8461)
                    else:
                        EFi_CO = -8.6240*(10**-5) * (V**2) + 1.9749*(10**-2)*V - 4.9165*(10**-1)
                    EFi_HC = 0.2449 * (V**-0.7830)
                    EFi_NOx = 0.4230 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2015 <= car_birth:
                    if V <= 45:
                        EFi_CO = 4.164 * (V**-0.8461)
                    else:
                        EFi_CO = -8.3360*(10**-5) * (V**2) + 1.9089*(10**-2)*V - 4.7524*(10**-1)
                    EFi_HC = 0.2343 * (V**-0.7830)
                    EFi_NOx = 0.4106 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
            elif (car_type == '승용') and (car_size == '소형'):
                if car_birth <= 1986:
                    EFi_CO = 247.00 * (V**-0.6651)
                    EFi_HC = 15.953 * (V**-0.5059)
                    EFi_NOx = 3.1140 * (V**-0.2278)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1987 <= car_birth <= 1990:
                    EFi_CO = 36.169 * (V**-0.7587)
                    EFi_HC = 15.607 * (V**-1.0423)
                    EFi_NOx = 6.2007 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1991 <= car_birth <= 1999:
                    EFi_CO = 111.67 * (V**-1.1566)
                    EFi_HC = 32.017 * (V**-1.4171)
                    EFi_NOx = 7.5244 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 2000 <= car_birth <= 2002:
                    EFi_CO = 22.356 * (V**-0.9068)
                    EFi_HC = 0.8428 * (V**-0.8829)
                    EFi_NOx = 1.2613 * (V**-0.3873)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V > 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 1.4898 * (V**-0.3837)
                    EFi_HC = 0.1738 * (V**-0.7268)
                    EFi_NOx = 0.1563 * (V**-0.2671)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    EFi_CO = 1.0000*(10**-4)*(V**2) - 7.1000*(10**-3)*V + 2.2450*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0633 * (V**-1.0484)
                    else:
                        EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                    EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth <= 2011:
                    EFi_CO = 9.2000*(10**-5)*(V**2) - 6.5320*(10**-3)*V + 2.0654*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0570 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                    EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2012 <= car_birth <= 2013:
                    EFi_CO = 8.9120*(10**-5)*(V**2) - 6.3275*(10**-3)*V + 2.0007*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0547 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                    EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif car_birth == 2014:
                    EFi_CO = 8.6240*(10**-5)*(V**2) - 6.1230*(10**-3)*V + 1.9361*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0524 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                    EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2015 <= car_birth:
                    EFi_CO = 8.3360*(10**-5)*(V**2) - 5.9186*(10**-3)*V + 1.8714*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0501 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.09840*(10**-3)
                    EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
            elif ( (car_type == '승용') and (car_size == '중형') ) or ( (car_type == '특수') and (car_size == '중형') ): # 최이사님 확인(2023.04.20) : 특수 중형 -> 승용 중형 조건으로 계산
                if car_birth <= 1986:
                    EFi_CO = 247.00 * (V**-0.6651)
                    EFi_HC = 15.953 * (V**-0.5059)
                    EFi_NOx = 3.1140 * (V**-0.2278)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1987 <= car_birth <= 1990:
                    EFi_CO = 36.169 * (V**-0.7587)
                    EFi_HC = 15.607 * (V**-1.0423)
                    EFi_NOx = 6.2007 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1991 <= car_birth <= 1999:
                    EFi_CO = 51.555 * (V**-0.9531)
                    EFi_HC = 31.816 * (V**-1.4804)
                    EFi_NOx = 7.5244 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 2000 <= car_birth <= 2002:
                    EFi_CO = 29.921 * (V**-0.8868)
                    EFi_HC = 7.9374 * (V**-1.3041)
                    EFi_NOx = 1.8525 * (V**-0.4192)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V > 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 2.4938 * (V**-0.6106)
                    EFi_HC = 0.4262 * (V**-1.0122)
                    EFi_NOx = 0.1818 * (V**-0.4316)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    EFi_CO = 2.2900*(10**-5)*(V**2) - 1.6300*(10**-3)*V + 5.8300*(10**-2)
                    if V <= 65.4:
                        EFi_HC = 0.0633 * (V**-1.0484)
                    else:
                        EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                    EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth <= 2011:
                    EFi_CO = 2.1068*(10**-5)*(V**2) - 1.4996*(10**-3)*V + 5.3636*(10**-2)
                    if V <= 65.4:
                        EFi_HC = 0.0570 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                    EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2012 <= car_birth <= 2013:
                    EFi_CO = 2.0408*(10**-5)*(V**2) - 1.4527*(10**-3)*V + 5.1957*(10**-2)
                    if V <= 65.4:
                        EFi_HC = 0.0547 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                    EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif car_birth == 2014:
                    EFi_CO = 1.9749*(10**-5)*(V**2) - 1.4057*(10**-3)*V + 5.0278*(10**-2)
                    if V <= 65.4:
                        EFi_HC = 0.0524 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                    EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2015 <= car_birth:
                    EFi_CO = 1.9089*(10**-5)*(V**2) - 1.3588*(10**-3)*V + 4.8599*(10**-2)
                    if V <= 65.4:
                        EFi_HC = 0.0501 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.0984*(10**-3)
                    EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
            elif (car_type == '승용') and (car_size == '대형'):
                if car_birth <= 1986:
                    EFi_CO = 247.00 * (V**-0.6651)
                    EFi_HC = 15.953 * (V**-0.5059)
                    EFi_NOx = 3.1140 * (V**-0.2278)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1987 <= car_birth <= 1990:
                    EFi_CO = 36.169 * (V**-0.7587)
                    EFi_HC = 15.607 * (V**-1.0423)
                    EFi_NOx = 6.2007 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1991 <= car_birth <= 1999:
                    EFi_CO = 51.555 * (V**-0.9531)
                    EFi_HC = 31.816 * (V**-1.4804)
                    EFi_NOx = 7.5244 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 2000 <= car_birth <= 2002:
                    EFi_CO = 29.921 * (V**-0.8868)
                    EFi_HC = 7.9374 * (V**-1.3041)
                    EFi_NOx = 1.8525 * (V**-0.4192)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V > 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 2.4938 * (V**-0.6106)
                    EFi_HC = 0.4262 * (V**-1.0122)
                    EFi_NOx = 0.1818 * (V**-0.4316)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    if V <= 65.4:
                        EFi_CO = 1.4082 * (V**-0.7728)
                    else:
                        EFi_CO = 8.0000*(10**-5)*(V**2) - 1.2700*(10**-2)*V + 5.7510*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0633 * (V**-1.0484)
                    else:
                        EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                    EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth <= 2011:
                    if V <= 65.4:
                        EFi_CO = 1.2955 * (V**-0.7728)
                    else:
                        EFi_CO = 7.3600*(10**-5)*(V**2) - 1.1684*(10**-2)*V + 5.2909*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0570 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                    EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2012 <= car_birth <= 2013:
                    if V <= 65.4:
                        EFi_CO = 1.2550 * (V**-0.7728)
                    else:
                        EFi_CO = 7.1296*(10**-5)*(V**2) - 1.1318*(10**-2)*V + 5.1253*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0547 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                    EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif car_birth == 2014:
                    if V <= 65.4:
                        EFi_CO = 1.2144 * (V**-0.7728)
                    else:
                        EFi_CO = 6.8992*(10**-5)*(V**2) - 1.0952*(10**-2)*V + 4.9597*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0524 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                    EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2015 <= car_birth:
                    if V <= 65.4:
                        EFi_CO = 1.1739 * (V**-0.7728)
                    else:
                        EFi_CO = 6.6688*(10**-5)*(V**2) - 1.0587*(10**-2)*V + 4.7940*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0501 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.0984*(10**-3)
                    EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
            elif ( (car_type == '승합') and (car_size == '경형') ) or ( (car_type == '화물') and (car_size == '경형') ):
                if car_birth <= 1996:
                    if V <= 45:
                        EFi_CO = 11.249 * (V**-0.6579)
                    else:
                        EFi_CO = 0.0003 * (V**2) + 0.0002 * V + 0.4136
                    EFi_HC = 7.6244 * (V**-0.8364)
                    EFi_NOx = 2.6754 * (V**-0.3236)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1997 <= car_birth <= 1999:
                    if V <= 45:
                        EFi_CO = 16.965 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0003 * (V**2) + 0.0777 * V - 1.9363
                    EFi_HC = 3.0285 * (V**-0.7830)
                    EFi_NOx = 1.9923 * (V**-0.3889)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 2000 <= car_birth <= 2002:
                    if V <= 45:
                        EFi_CO = 9.9433 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0002 * (V**2) + 0.0455 * V - 1.1349
                    EFi_HC = 1.8928 * (V**-0.7830)
                    EFi_NOx = 1.2352 * (V**-0.3889)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V >= 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    if V <= 45:
                        EFi_CO = 9.9433 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0002 * (V**2) + 0.0455 * V - 1.1349
                    EFi_HC = 0.9227 * (V**-0.7830)
                    EFi_NOx = 3.8859 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    if V <= 45:
                        EFi_CO = 4.4952 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0001 * (V**2) + 0.0229 * V - 0.5701
                    EFi_HC = 0.2958 * (V**-0.7830)
                    EFi_NOx = 0.4819 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth:
                    if V <= 45:
                        EFi_CO = 4.4952 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0001 * (V**2) + 0.0229 * V - 0.5701
                    EFi_HC = 0.2958 * (V**-0.7830)
                    EFi_NOx = 0.4819 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V >= 90:
                        EFi_NH3 = 0.022
            elif (car_type == '승합') and (car_size == '소형'):
                if car_birth <= 1990:
                    EFi_CO = 36.169 * (V**-0.7587)
                    EFi_HC = 15.607 * (V**-1.0423)
                    EFi_NOx = 6.2007 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 1991 <= car_birth <= 1996:
                    EFi_CO = 39.402 * (V**-0.8879)
                    EFi_HC = 23.400 * (V**-1.4041)
                    EFi_NOx = 7.5244 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 77.088 * (V**-1.2078)
                    EFi_HC = 18.731 * (V**-1.5356)
                    EFi_NOx = 4.4260 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif car_birth == 2000:
                    EFi_CO = 41.669 * (V**-1.2078)
                    EFi_HC = 14.190 * (V**-1.5356)
                    EFi_NOx = 3.4578 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V >= 90:
                        EFi_NH3 = 0.084
                elif 2001 <= car_birth <= 2002:
                    EFi_CO = 41.669 * (V**-1.2078)
                    EFi_HC = 11.920 * (V**-1.5356)
                    EFi_NOx = 3.0649 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V >= 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 37.573 * (V**-1.2078)
                    EFi_HC = 3.1786 * (V**-1.5356)
                    EFi_NOx = 1.4931 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    EFi_CO = 32.899 * (V**-1.2078)
                    EFi_HC = 2.7387 * (V**-1.5356)
                    EFi_NOx = 1.1808 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth:
                    EFi_CO = 32.899 * (V**-1.2078)
                    EFi_HC = 2.7387 * (V**-1.5356)
                    EFi_NOx = 1.1808 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.08
                    elif V >= 90:
                        EFi_NH3 = 0.022
            elif ( (car_type == '승합') and (car_size == '중형') or (car_size == '대형') ) or ( (car_type == '화물') and (car_size == '소형') ):
                if car_birth <= 1990:
                    EFi_CO = 36.169 * (V**-0.7587)
                    EFi_HC = 15.607 * (V**-1.0423)
                    EFi_NOx = 6.2007 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 1991 <= car_birth <= 1996:
                    EFi_CO = 39.402 * (V**-0.8879)
                    EFi_HC = 23.400 * (V**-1.4041)
                    EFi_NOx = 7.5244 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 77.088 * (V**-1.2078)
                    EFi_HC = 18.731 * (V**-1.5356)
                    EFi_NOx = 4.4260 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 2000 <= car_birth <= 2002:
                    EFi_CO = 41.669 * (V**-1.2078)
                    EFi_HC = 14.190 * (V**-1.5356)
                    EFi_NOx = 3.4578 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V >= 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 36.578 * (V**-1.2078)
                    EFi_HC = 3.0337 * (V**-1.5356)
                    EFi_NOx = 2.0104 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    EFi_CO = 14.202 * (V**-1.2078)
                    EFi_HC = 1.2233 * (V**-1.5356)
                    EFi_NOx = 0.2493 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth:
                    EFi_CO = 14.202 * (V**-1.2078)
                    EFi_HC = 1.2233 * (V**-1.5356)
                    EFi_NOx = 0.2493 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V >= 90:
                        EFi_NH3 = 0.022
            elif (car_type == '화물') and (car_size == '중형'):
                if 1993 <= car_birth <= 1998:
                    if V <= 65:
                        EFi_CO = 70
                        EFi_HC = 7
                        EFi_NOx = 4.5
                    else:
                        EFi_CO = 55
                        EFi_HC = 3.5
                        EFi_NOx = 7.5
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif car_birth == 1999:
                    if V <= 65:
                        EFi_CO = 70
                        EFi_HC = 7
                        EFi_NOx = 4.5
                    else:
                        EFi_CO = 55
                        EFi_HC = 3.5
                        EFi_NOx = 7.5
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 2000 <= car_birth <= 2002: # CO, HC, NOx 배출계수 누락(⭕)
                    if V <= 65:
                        EFi_CO = 70
                        EFi_HC = 7
                        EFi_NOx = 4.5
                    else:
                        EFi_CO = 55
                        EFi_HC = 3.5
                        EFi_NOx = 7.5
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V > 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2008: # CO, HC, NOx 배출계수 누락(⭕)
                    if V <= 65:
                        EFi_CO = 70
                        EFi_HC = 7
                        EFi_NOx = 4.5
                    else:
                        EFi_CO = 55
                        EFi_HC = 3.5
                        EFi_NOx = 7.5
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth:
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
            elif (car_type == '화물') and (car_size == '대형'):
                if 1993 <= car_birth:
                    if V <= 65:
                        EFi_CO = 70
                        EFi_HC = 7
                        EFi_NOx = 4.5
                    else:
                        EFi_CO = 55
                        EFi_HC = 3.5
                        EFi_NOx = 7.5
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                else:
                    EFi_CO = 1
                    EFi_HC = 1
                    EFi_NOx = 1
                    EFi_PM10 = 1
                    EFi_PM2_5 = 1
                EFi_NH3 = 0.002
        elif fuel == '경유':
            if (car_type == '승용') and (car_size == '경형'):
                if car_birth <= 2005:
                    EFi_CO = 0.7392 * (V**-0.7524)
                    EFi_HC = 0.0989 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0839 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0839 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2006 <= car_birth <= 2010:
                    EFi_CO = 0.5775 * (V**-0.7524)
                    EFi_HC = 0.0825 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0420 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0420 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2011 <= car_birth <= 2015:
                    EFi_CO = 0.5141 * (V**-0.6792)
                    EFi_HC = 0.3713 * (V**-0.7513)
                    EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)      # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416) # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
                elif 2016 <= car_birth:
                    EFi_CO = 0.4574 * (V**-0.5215)
                    EFi_HC = 0.1300 * (V**-0.7265)
                    EFi_NOx = 2.7702 * (V**-0.3869)
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)     # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416) # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
            elif (car_type == '승용') and (car_size == '소형'):
                if car_birth <= 2004:
                    EFi_CO = 5.9672 * (V**-0.9534)
                    EFi_HC = 0.6523 * (V**-1.0167)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    if V <= 65:
                        EFi_PM10 = 0.3861 * (V**-0.5093)
                        EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                    else:
                        EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                        EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_NH3 = 0.001
                elif car_birth == 2005:
                    EFi_CO = 0.7392 * (V**-0.7524)
                    EFi_HC = 0.0989 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0839 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0839 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2006 <= car_birth <= 2010:
                    EFi_CO = 0.5775 * (V**-0.7524)
                    EFi_HC = 0.0825 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0420 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0420 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2011 <= car_birth <= 2015:
                    EFi_CO = 0.5141 * (V**-0.6792)
                    EFi_HC = 0.3713 * (V**-0.7513)
                    EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)        # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)   # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
                elif 2016 <= car_birth:
                    EFi_CO = 0.4574 * (V**-0.5215)
                    EFi_HC = 0.1300 * (V**-0.7265)
                    EFi_NOx = 2.7702 * (V**-0.3869)
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)           # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)      # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
            elif (car_type == '승용') and (car_size == '중형'):
                if car_birth <= 2004:
                    EFi_CO = 5.9672 * (V**-0.9534)
                    EFi_HC = 0.6523 * (V**-1.0167)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    if V <= 65:
                        EFi_PM10 = 0.3861 * (V**-0.5093)
                        EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                    else:
                        EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                        EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_NH3 = 0.001
                elif car_birth == 2005:
                    EFi_CO = 0.6930 * (V**-0.7524)
                    EFi_HC = 0.1865 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0723 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0723 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2006 <= car_birth <= 2010:
                    EFi_CO = 0.5414 * (V**-0.7524)
                    EFi_HC = 0.0927 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0396 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0396 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2011 <= car_birth <= 2015:
                    EFi_CO = 0.5141 * (V**-0.6792)
                    EFi_HC = 0.3713 * (V**-0.7513)
                    EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)            # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)      # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
                elif 2016 <= car_birth:
                    EFi_CO = 0.4574 * (V**-0.5215)
                    EFi_HC = 0.1300 * (V**-0.7265)
                    EFi_NOx = 2.7702 * (V**-0.3869)
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)              # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)         # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
            elif (car_type == '승용') and (car_size == '대형'):
                if car_birth <= 2004:
                    EFi_CO = 5.9672 * (V**-0.9534)
                    EFi_HC = 0.6523 * (V**-1.0167)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    if V <= 65:
                        EFi_PM10 = 0.3861 * (V**-0.5093)
                        EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                    else:
                        EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                        EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_NH3 = 0.001
                elif car_birth == 2005:
                    EFi_CO = 0.9609 * (V**-0.7524)
                    EFi_HC = 0.1865 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0723 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0723 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2006 <= car_birth <= 2010:
                    EFi_CO = 0.7507 * (V**-0.7524)
                    EFi_HC = 0.1554 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0361 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0361 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2011 <= car_birth <= 2015:
                    EFi_CO = 0.5141 * (V**-0.6792)
                    EFi_HC = 0.3713 * (V**-0.7513)
                    EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)              # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)         # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
                elif 2016 <= car_birth:
                    EFi_CO = 0.4574 * (V**-0.5215)
                    EFi_HC = 0.1300 * (V**-0.7265)
                    EFi_NOx = 2.7702 * (V**-0.3869)
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)              # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)         # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
            elif (car_type == '승합') and (car_size == '소형'):
                if car_birth <= 1990:
                    if V <= 65.4:
                        EFi_CO = 3.4539 * (V**-0.4266)
                    else:
                        EFi_CO = 0.0051 * V + 0.2212
                    EFi_HC = 0.9835 * (V**-0.5096)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 1.1412 * (V**-0.4324)
                    EFi_PM2_5 = k * 1.1412 * (V**-0.4324)
                elif 1991 <= car_birth <= 1995:
                    if V <= 65.4:
                        EFi_CO = 3.4539 * (V**-0.4266)
                    else:
                        EFi_CO = 0.0051 * V + 0.2212
                    EFi_HC = 1.6313 * (V**-0.7298)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.5999 * (V**-0.3294)
                    EFi_PM2_5 = k * 0.5999 * (V**-0.3294)
                elif 1996 <= car_birth <= 1997:
                    if V <= 65.4:
                        EFi_CO = 3.4539 * (V**-0.4266)
                    else:
                        EFi_CO = 0.0051 * V + 0.2212
                    EFi_HC = 1.1293 * (V**-0.6588)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.6408 * (V**-0.3596)
                    EFi_PM2_5 = k * 0.6408 * (V**-0.3596)
                elif 1998 <= car_birth <= 1999:
                    EFi_CO = 3.7564 * (V**-0.5175)
                    EFi_HC = 1.1293 * (V**-0.6588)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.5168 * (V**-0.3596)
                    EFi_PM2_5 = k * 0.5168 * (V**-0.3596)
                elif 2000 <= car_birth <= 2003:
                    EFi_CO = 3.7564 * (V**-0.5175)
                    EFi_HC = 1.1293 * (V**-0.6588)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.2894 * (V**-0.3596)
                    EFi_PM2_5 = k * 0.2894 * (V**-0.3596)
                elif 2004 <= car_birth <= 2007:
                    EFi_CO = 3.2797 * (V**-0.8887)
                    EFi_HC = 0.1807 * (V**-0.6588)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.2067 * (V**-0.3596)
                    EFi_PM2_5 = k * 0.2067 * (V**-0.3596)
                elif 2008 <= car_birth <= 2011:
                    if V <= 65.4:
                        EFi_CO = 4.222 * (V**-1.4035)
                    else:
                        EFi_CO = 0.01166 * (V**0.09222)         # EFi_CO = 0.01166 * (V**0.09222) (⭕)
                    if V <= 97.3:                               # 97.3 초과에 대한 내용 없음.(편람 p.371)
                        EFi_HC = 0.829 * (V**-1.0961)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.3111 * (V**-0.5125)
                    EFi_PM2_5 = k * 0.3111 * (V**-0.5125)
                elif 2012 <= car_birth:
                    if V <= 65.4:
                        EFi_CO = 4.222 * (V**-1.4035)
                    else:
                        EFi_CO = 0.01166 * (V**0.09222)         # EFi_CO = 0.01166 * (V**0.09222) (⭕)
                    if V <= 97.3:                               # 97.3 초과에 대한 내용 없음.(편람 p.371)
                        EFi_HC = 0.829 * (V**-1.0961)
                    if V <= 65.4:
                        EFi_NOx = 2.0217 * (V**-0.2645)
                    else:
                        EFi_NOx = 0.0271 * (V**0.7596)
                    EFi_PM10 = 0.1119 * (V**-0.5125)
                    EFi_PM2_5 = k * 0.1119 * (V**-0.5125)
                EFi_NH3 = 0.001
            elif (car_type == '승합') and (car_size == '중형'):
                if car_birth <= 1995:
                    EFi_CO = 32.550 * (V**-0.4944)
                    EFi_HC = 15.753 * (V**-0.5912)
                    if V < 80:
                        EFi_NOx = 40.692 * (V**-0.5590)
                    else:
                        EFi_NOx = -0.0023 * (V**2) + 0.5381 * V - 23.590
                    EFi_PM10 = 5.4886 * (V**-0.5911)
                    EFi_PM2_5 = k * 5.4886 * (V**-0.5911)
                elif 1996 <= car_birth <= 1997:
                    EFi_CO = 16.410 * (V**-0.3790)
                    EFi_HC = 4.2324 * (V**-0.3926)
                    if V < 80:
                        EFi_NOx = 22.804 * (V**-0.4660)
                    else:
                        EFi_NOx = -0.0021 * (V**2) + 0.4430 * V - 18.730
                    EFi_PM10 = 1.6593 * (V**-0.3935)
                    EFi_PM2_5 = k * 1.6593 * (V**-0.3935)
                elif 1998 <= car_birth <= 2000:
                    EFi_CO = 16.410 * (V**-0.3790)
                    EFi_HC = 4.2324 * (V**-0.3926)
                    if V < 80:
                        EFi_NOx = 25.708 * (V**-0.4772)
                    else:
                        EFi_NOx = 0.0019 * (V**2) - 0.2628 * V + 12.145
                    EFi_PM10 = 1.6593 * (V**-0.3935)
                    EFi_PM2_5 = k * 1.6593 * (V**-0.3935)
                elif 2001 <= car_birth <= 2004:
                    EFi_CO = 16.378 * (V**-0.5340)
                    EFi_HC = 5.8477 * (V**-0.5466)
                    if V < 80:
                        EFi_NOx = 25.436 * (V**-0.4656)
                    else:
                        EFi_NOx = 0.0008 * (V**2) - 0.0482 * V + 1.8424
                    EFi_PM10 = 1.2848 * (V**-0.4715)
                    EFi_PM2_5 = k * 1.2848 * (V**-0.4715)
                elif 2005 <= car_birth <= 2007:
                    EFi_CO = 15.256 * (V**-0.7448)
                    EFi_HC = 2.0502 * (V**-0.6504)
                    EFi_NOx = 15.001 * (V**-0.4528)
                    EFi_PM10 = 0.2979 * (V**-0.4008)
                    EFi_PM2_5 = k * 1.0457 * (V**-0.4527)
                elif 2008 <= car_birth <= 2010:
                    EFi_CO = 8.1771 * (V**-0.7725)
                    EFi_HC = 1.2991 * (V**-0.6538)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                        EFi_PM10 = 0.0539 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                        EFi_PM10 = 2.0000*(10**-4) * V - 7.5600*(10**-3)
                        EFi_PM2_5 = k * 2.0000*(10**-4) * V - 7.5600*(10**-3)
                elif 2011 <= car_birth <= 2014:
                    EFi_CO = 4.5201 * (V**-0.7279)
                    EFi_HC = 1.6826 * (V**-0.8045)
                    if V <= 64.7:
                        EFi_NOx = 17.2485 * (V**-0.4040)
                        EFi_PM10 = 0.0469 * (V**-0.4674)
                        EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                    else:
                        EFi_NOx = 1.1797 * (V**0.2308)
                        EFi_PM10 = 1.6800*(10**-4) * V - 5.1600*(10**-3)
                        EFi_PM2_5 = k * 1.6800*(10**-4) * V - 5.1600*(10**-3)
                elif 2015 <= car_birth:
                    EFi_CO = 7.4065 * (V**-0.5995)
                    EFi_HC = 2.4562 * (V**-1.3145)
                    EFi_NOx = 42.7393 * (V**-1.2949)
                    if V <= 64.7:
                        EFi_PM10 = 0.0081 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                    else:
                        EFi_PM10 = 3.0000*(10**-5) * V - 0.0011
                        EFi_PM2_5 = k * 3.0000*(10**-5) * V - 0.0011
                EFi_NH3 = 0.001
            elif (car_type == '승합') and (car_size == '대형'):
                if car_birth <= 1995:
                    EFi_CO = 28.205 * (V**-0.5337)
                    EFi_HC = 6.1146 * (V**-0.4979)
                    EFi_NOx = 41.346 * (V**-0.3645)
                    EFi_PM10 = 5.2158 * (V**-0.5048)
                    EFi_PM2_5 = k * 5.2158 * (V**-0.5048)
                    EFi_NH3 = 0.003
                elif 1996 <= car_birth <= 1997:
                    EFi_CO = 23.205 * (V**-0.5425)
                    EFi_HC = 6.5657 * (V**-0.5431)
                    EFi_NOx = 42.1379 * (V**-0.3786)
                    EFi_PM10 = 2.4911 * (V**-0.4149)
                    EFi_PM2_5 = k * 2.4911 * (V**-0.4149)
                    EFi_NH3 = 0.003
                elif 1998 <= car_birth <= 2000:
                    EFi_CO = 23.205 * (V**-0.5425)
                    EFi_HC = 6.5657 * (V**-0.5431)
                    EFi_NOx = 42.1379 * (V**-0.3786)
                    EFi_PM10 = 1.4432 * (V**-0.3870)
                    EFi_PM2_5 = k * 1.4432 * (V**-0.3870)
                    EFi_NH3 = 0.003
                elif car_birth == 2001:
                    EFi_CO = 21.348 * (V**-0.5806)
                    EFi_HC = 6.6390 * (V**-0.5760)
                    EFi_NOx = 36.7191 * (V**-0.3548)
                    EFi_PM10 = 0.9375 * (V**-0.3910)
                    EFi_PM2_5 = k * 0.9375 * (V**-0.3910)
                    EFi_NH3 = 0.003
                elif 2002 <= car_birth <= 2004:
                    EFi_CO = 21.348 * (V**-0.5806)
                    EFi_HC = 6.6390 * (V**-0.5760)
                    EFi_NOx = 36.7191 * (V**-0.3548)
                    EFi_PM10 = 1.1507 * (V**-0.4804)
                    EFi_PM2_5 = k * 1.1507 * (V**-0.4804)
                    EFi_NH3 = 0.003
                elif 2005 <= car_birth <= 2007:
                    EFi_CO = 9.6452 * (V**-0.5291)
                    EFi_HC = 3.2339 * (V**-0.7436)
                    EFi_NOx = 30.5870 * (V**-0.3548)
                    if V <= 80:
                        EFi_PM10 = 0.4657 * (V**-0.5634)
                        EFi_PM2_5 = k * 0.4657 * (V**-0.5634)
                    else:
                        EFi_PM10 = 0.0014 * (V**0.7970)
                        EFi_PM2_5 = k * 0.0014 * (V**0.7970)
                    EFi_NH3 = 0.003
                elif 2008 <= car_birth <= 2010:
                    EFi_CO = 6.8493 * (V**-0.6506)
                    EFi_HC = 1.7177 * (V**-0.6781)
                    EFi_NOx = 40.7564 * (V**-0.4757)
                    EFi_PM10 = 0.2418 * (V**-0.4727)
                    EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
                    EFi_NH3 = 0.003
                elif 2011 <= car_birth <= 2014:
                    EFi_CO = 5.4607 * (V**-0.2990)
                    EFi_HC = 0.8863 * (V**-0.6933)
                    EFi_NOx = 40.3729 * (V**-0.5386)
                    EFi_PM10 = 0.2125 * (V**-0.4650)
                    EFi_PM2_5 = k * 0.2125 * (V**-0.4650)
                    EFi_NH3 = 0.003
                elif 2015 <= car_birth:
                    EFi_CO = 11.4415 * (V**-0.8036)
                    EFi_HC = 0.6774 * (V**-0.8321)
                    EFi_NOx = 112.1229 * (V**-1.6393)
                    EFi_PM10 = 0.0363 * (V**-0.4727)
                    EFi_PM2_5 = k * 0.0363 * (V**-0.4727)
                    EFi_NH3 = 0.007
            elif (car_type == '화물') and (car_size == '소형'):
                if car_birth <= 1990:
                    EFi_CO = 4.5854 * (V**-0.3613)
                    EFi_HC = 0.4840 * (V**-0.2756)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.8117 * (V**-0.4071)
                    EFi_PM2_5 = k * 0.8117 * (V**-0.4071)
                elif 1991 <= car_birth <= 1995:
                    EFi_CO = 3.4774 * (V**-0.3483)
                    EFi_HC = 0.4844 * (V**-0.3288)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.6188 * (V**-0.4540)
                    EFi_PM2_5 = k * 0.6188 * (V**-0.4540)
                elif 1996 <= car_birth <= 1997:
                    EFi_CO = 3.3934 * (V**-0.3837)
                    EFi_HC = 0.4955 * (V**-0.3393)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.7037 * (V**-0.5357)
                    EFi_PM2_5 = k * 0.7037 * (V**-0.5357)
                elif 1998 <= car_birth <= 1999:
                    EFi_CO = 4.0896 * (V**-0.6083)
                    EFi_HC = 0.6122 * (V**-0.5684)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.6157 * (V**-0.5357)
                    EFi_PM2_5 = k * 0.6157 * (V**-0.5357)
                elif 2000 <= car_birth <= 2003:
                    EFi_CO = 4.0896 * (V**-0.6083)
                    EFi_HC = 0.6122 * (V**-0.5684)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.4838 * (V**-0.5357)
                    EFi_PM2_5 = k * 0.4838 * (V**-0.5357)
                elif 2004 <= car_birth <= 2007:
                    EFi_CO = 3.2797 * (V**-0.8887)
                    EFi_HC = 0.1807 * (V**-0.6588)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.2067 * (V**-0.3596)
                    EFi_PM2_5 = k * 0.2067 * (V**-0.3596)
                elif 2008 <= car_birth <= 2011:
                    if V <= 65.4:
                        EFi_CO = 4.222 * (V**-1.4035)
                    else:
                        EFi_CO = 0.01166 * (V**0.09222)
                    if V <= 97.3:                               # 97.3 초과에 대한 내용 없음.(편람 p.374)
                        EFi_HC = 0.829 * (V**-1.0961)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.3111 * (V**-0.5125)
                    EFi_PM2_5 = k * 0.3111 * (V**-0.5125)
                elif 2012 <= car_birth <= 2016:
                    if V <= 65.4:
                        EFi_CO = 4.222 * (V**-1.4035)
                        EFi_NOx = 2.0217 * (V**-0.2645)
                    else:
                        EFi_CO = 0.01166 * (V**0.09222)
                        EFi_NOx = 0.0271 * (V**0.7596)
                    if V <= 97.3:                               # 97.3 초과에 대한 내용 없음.(편람 p.374)
                        EFi_HC = 0.829 * (V**-1.0961)
                    EFi_PM10 = 0.1119 * (V**-0.5125)
                    EFi_PM2_5 = k * 0.1119 * (V**-0.5125)
                EFi_NH3 = 0.001
            elif (car_type == '화물') and (car_size == '중형'):
                if car_birth <= 1995:
                    EFi_CO = 16.769 * (V**-0.3772)
                    EFi_HC = 6.7755 * (V**-0.5003)
                    EFi_NOx = 24.915 * (V**-0.3942)
                    EFi_PM10 = 3.6772 * (V**-0.5514)
                    EFi_PM2_5 = k * 3.6772 * (V**-0.5514)
                elif 1996 <= car_birth <= 2000:
                    EFi_CO = 21.057 * (V**-0.4958)
                    EFi_HC = 6.7532 * (V**-0.5711)
                    EFi_NOx = 25.022 * (V**-0.4240)
                    EFi_PM10 = 3.5285 * (V**-0.5962)
                    EFi_PM2_5 = k * 3.5285 * (V**-0.5962)
                elif car_birth == 2001:
                    EFi_CO = 23.501 * (V**-0.6100)        # EFi_CO = 23.501 * (V**-0.6100) (⭕)     
                    EFi_HC = 6.8738 * (V**-0.5913)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 1.4444 * (V**-0.4824)
                    EFi_PM2_5 = k * 1.4444 * (V**-0.4824)
                elif 2002 <= car_birth <= 2004:
                    EFi_CO = 23.501 * (V**-0.6100)        # EFi_CO = 23.501 * (V**-0.6100) (⭕)
                    EFi_HC = 6.8738 * (V**-0.5913)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 1.0432 * (V**-0.4992)
                    EFi_PM2_5 = k * 1.0432 * (V**-0.4992)
                elif 2005 <= car_birth <= 2007:
                    EFi_CO = 15.256 * (V**-0.7448)
                    EFi_HC = 2.0502 * (V**-0.6504)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 0.2979 * (V**-0.4008)
                    EFi_PM2_5 = k * 0.2979 * (V**-0.4008)
                elif 2008 <= car_birth <= 2010:
                    EFi_CO = 8.1771 * (V**-0.7725)
                    EFi_HC = 1.2991 * (V**-0.6538)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                        EFi_PM10 = 0.0539 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                        EFi_PM10 = 0.0002 * V - 0.00756
                        EFi_PM2_5 = k * 0.0002 * V - 0.00756
                elif 2011 <= car_birth <= 2014:
                    EFi_CO = 4.5201 * (V**-0.7279)
                    EFi_HC = 1.6826 * (V**-0.8045)
                    if V <= 64.7:
                        EFi_NOx = 17.2485 * (V**-0.4040)
                        EFi_PM10 = 0.0469 * (V**-0.4674)
                        EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                    else:
                        EFi_NOx = 1.1797 * (V**0.2308)
                        EFi_PM10 = 0.000168 * V - 0.00516
                        EFi_PM2_5 = k * 0.000168 * V - 0.00516
                elif 2015 <= car_birth:
                    EFi_CO = 7.4065 * (V**-0.5995)
                    EFi_HC = 2.4562 * (V**-1.3145)
                    EFi_NOx = 42.7393 * (V**-1.2949)
                    if V <= 64.7:
                        EFi_PM10 = 0.0081 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                    else:
                        EFi_PM10 = 3.0000 * (10**-5) * V - 0.0011
                        EFi_PM2_5 = k * 3.0000 * (10**-5) * V - 0.0011
                EFi_NH3 = 0.001
            elif car_type == '특수':
                if car_birth <= 1995:
                    EFi_CO = 16.769 * (V**-0.3772)
                    EFi_HC = 6.7755 * (V**-0.5003)
                    EFi_NOx = 24.915 * (V**-0.3942)
                    EFi_PM10 = 3.6772 * (V**-0.5514)
                    EFi_PM2_5 = k * 3.6772 * (V**-0.5514)
                    EFi_NH3 = 0.003
                elif 1996 <= car_birth <= 2000:
                    EFi_CO = 21.057 * (V**-0.4958)
                    EFi_HC = 6.7532 * (V**-0.5711)
                    EFi_NOx = 25.022 * (V**-0.4240)
                    EFi_PM10 = 3.5285 * (V**-0.5962)
                    EFi_PM2_5 = k * 3.5285 * (V**-0.5962)
                    EFi_NH3 = 0.003
                elif car_birth == 2001:
                    EFi_CO = 23.501 * (V**-0.6100)             # EFi_CO = 23.501 * (V**-0.6100) (⭕)
                    EFi_HC = 6.8738 * (V**-0.5913)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 1.4444 * (V**-0.4824)
                    EFi_PM2_5 = k * 1.4444 * (V**-0.4824)
                    EFi_NH3 = 0.003
                elif 2002 <= car_birth <= 2004:
                    EFi_CO = 23.501 * (V**-0.6100)             # EFi_CO = 23.501 * (V**-0.6100) (⭕)
                    EFi_HC = 6.8738 * (V**-0.5913)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 1.0432 * (V**-0.4992)
                    EFi_PM2_5 = k * 1.0432 * (V**-0.4992)
                    EFi_NH3 = 0.003
                elif 2005 <= car_birth <= 2007:
                    EFi_CO = 15.256 * (V**-0.7448)
                    EFi_HC = 2.0502 * (V**-0.6504)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 0.2979 * (V**-0.4008)
                    EFi_PM2_5 = k * 0.2979 * (V**-0.4008)
                    EFi_NH3 = 0.003
                elif 2008 <= car_birth <= 2009:
                    EFi_CO = 8.1771 * (V**-0.7725)
                    EFi_HC = 1.2991 * (V**-0.6538)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                        EFi_PM10 = 0.0539 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                        EFi_PM10 = 0.0002 * V - 0.00756
                        EFi_PM2_5 = k * 0.0002 * V - 0.00756
                    EFi_NH3 = 0.003
                elif car_birth == 2010:
                    EFi_CO = 8.1771 * (V**-0.7725)
                    EFi_HC = 1.2991 * (V**-0.6538)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                        EFi_PM10 = 0.0539 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                        EFi_PM10 = 0.0002 * V - 0.00756
                        EFi_PM2_5 = k * 0.0002 * V - 0.00756
                    EFi_NH3 = 0.011
                elif 2011 <= car_birth <= 2014:
                    EFi_CO = 4.5201 * (V**-0.7279)
                    EFi_HC = 1.6826 * (V**-0.8045)
                    if V <= 64.7:
                        EFi_NOx = 17.2485 * (V**-0.4040)
                        EFi_PM10 = 0.0469 * (V**-0.4674)
                        EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                    else:
                        EFi_NOx = 1.1797 * (V**0.2308)
                        EFi_PM10 = 0.000168 * V - 0.00516
                        EFi_PM2_5 = k * 0.000168 * V - 0.00516
                    EFi_NH3 = 0.011
                elif 2015 <= car_birth:
                    EFi_CO = 7.4065 * (V**-0.5995)
                    EFi_HC = 2.4562 * (V**-1.3145)
                    EFi_NOx = 42.7393 * (V**-1.2949)
                    if V <= 64.7:
                        EFi_PM10 = 0.0081 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                    else:
                        EFi_PM10 = 3.0000 * (10**-5) * V - 0.0011
                        EFi_PM2_5 = k * 3.0000 * (10**-5) * V - 0.0011
                    EFi_NH3 = 0.007
            elif (car_type == '화물') and (car_size == '대형'):
                if car_birth <= 1995:
                    EFi_CO = 30.402 * (V**-0.4685)
                    EFi_HC = 15.75 * (V**-0.582)
                    EFi_NOx = 117.49 * (V**-0.365)
                    EFi_PM10 = 7.6212 * (V**-0.4183)
                    EFi_PM2_5 = k * 7.6212 * (V**-0.4183)
                    EFi_NH3 = 0.003
                elif 1996 <= car_birth <= 1997:
                    EFi_CO = 18.101 * (V**-0.3454)
                    EFi_HC = 10.301 * (V**-0.5856)
                    EFi_NOx = 94.319 * (V**-0.4061)
                    EFi_PM10 = 6.0264 * (V**-0.4627)
                    EFi_PM2_5 = k * 6.0264 * (V**-0.4627)
                    EFi_NH3 = 0.003
                elif 1998 <= car_birth <= 2000:
                    EFi_CO = 18.101 * (V**-0.3454)
                    EFi_HC = 10.301 * (V**-0.5856)
                    EFi_NOx = 94.319 * (V**-0.4061)
                    EFi_PM10 = 4.873 * (V**-0.4382)
                    EFi_PM2_5 = k * 4.873 * (V**-0.4382)
                    EFi_NH3 = 0.003
                elif 2001 <= car_birth <= 2004:
                    EFi_CO = 28.399 * (V**-0.5999)
                    EFi_HC = 10.031 * (V**-0.5828)
                    EFi_NOx = 85.301 * (V**-0.4023)
                    EFi_PM10 = 3.7541 * (V**-0.4055)
                    EFi_PM2_5 = k * 3.7541 * (V**-0.4055)
                    EFi_NH3 = 0.003
                elif 2005 <= car_birth <= 2007:
                    EFi_CO = 52.136 * (V**-0.8618)
                    EFi_HC = 3.7878 * (V**-0.5425)
                    EFi_NOx = 107.5 * (V**-0.5679)
                    EFi_PM10 = 2.6847 * (V**-0.6112)
                    EFi_PM2_5 = k * 2.6847 * (V**-0.6112)
                    EFi_NH3 = 0.003
                elif car_birth == 2008:                    # 2008 <= car_birth <= 2009 -> 2008 == car_birth: 변경 (⭕)
                    EFi_CO = 6.8493 * (V**-0.6506)
                    EFi_HC = 1.7177 * (V**-0.6781)
                    EFi_NOx = 40.7564 * (V**-0.4757)
                    EFi_PM10 = 0.2418 * (V**-0.4727)
                    EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
                    EFi_NH3 = 0.003
                elif 2009 <= car_birth <= 2010:
                    EFi_CO = 6.8493 * (V**-0.6506)
                    EFi_HC = 1.7177 * (V**-0.6781)
                    EFi_NOx = 40.7564 * (V**-0.4757)
                    EFi_PM10 = 0.2418 * (V**-0.4727)
                    EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
                    EFi_NH3 = 0.011
                elif 2011 <= car_birth <= 2014:
                    EFi_CO = 5.4607 * (V**-0.2990)
                    EFi_HC = 0.8863 * (V**-0.6933)
                    EFi_NOx = 40.3729 * (V**-0.5386)
                    EFi_PM10 = 0.2125 * (V**-0.4650)
                    EFi_PM2_5 = k * 0.2125 * (V**-0.4650)
                    EFi_NH3 = 0.011
                elif 2015 <= car_birth:
                    EFi_CO = 4.3762 * (V**-0.4550)
                    EFi_HC = 0.3627 * (V**-0.7071)
                    EFi_NOx = 18.0405 * (V**-1.0986)
                    EFi_PM10 = 0.0363 * (V**-0.4727)
                    EFi_PM2_5 = k * 0.0363 * (V**-0.4727)
                    EFi_NH3 = 0.007
        elif fuel == 'LPG':
            if (car_type == '승용') and (car_size == '경형'):
                if car_birth <= 1996:
                    if V <= 45:
                        EFi_CO = 22.498 * (V**-0.6579)
                    else:
                        EFi_CO = 0.0006 * (V**2) + 0.0004 * V + 0.8272
                    EFi_HC = 12.961 * (V**-0.8364)
                    EFi_NOx = 4.0131 * (V**-0.3236)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth <= 1999:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 2.2714 * (V**-0.7830)
                    EFi_NOx = 1.8528 * (V**-0.3889)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2002:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 2.2714 * (V**-0.7830)
                    EFi_NOx = 5.8289 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2003 <= car_birth <= 2005:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 1.1073 * (V**-0.7830)
                    EFi_NOx = 5.8289 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth <= 2007:
                    if V <= 45:
                        EFi_CO = 8.9904 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0002 * (V**2) + 0.0457 * V - 1.1403
                    EFi_HC = 0.3549 * (V**-0.7830)
                    EFi_NOx = 0.7228 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2008:
                    if V <= 79.6:
                        EFi_CO = 0.7693 * (V**-0.7666)
                        EFi_HC = 0.1063 * (V**-1.0745)
                    else:
                        EFi_CO = 5.0000 * (10**-16) * (V**7.2766)
                        EFi_HC = 1.0000 * (10**-15) * (V**6.2696)
                    EFi_NOx = -4.0000 * (10**-6) * (V**2) + 6.0000 * (10**-4) * V + 5.5000 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2009 <= car_birth <= 2011:
                    if V <= 79.6:
                        EFi_CO = 0.7059 * (V**-0.7666)
                        EFi_HC = 0.0974 * (V**-1.0745)
                    else:
                        EFi_CO = 4.5878 * (10**-16) * (V**7.2766)
                        EFi_HC = 9.1667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.7333 * (10**-6) * (V**2) + 5.6000 * (10**-4) * V + 5.1333 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2012 <= car_birth <= 2013:
                    if V <= 79.6:
                        EFi_CO = 0.6830 * (V**-0.7666)
                        EFi_HC = 0.0943 * (V**-1.0745)
                    else:
                        EFi_CO = 4.4393 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.8667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.6373 * (10**-6) * (V**2) + 5.4560 * (10**-4) * V + 5.0013 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2014:
                    if V <= 79.6:
                        EFi_CO = 0.6602 * (V**-0.7666)
                        EFi_HC = 0.0911 * (V**-1.0745)
                    else:
                        EFi_CO = 4.2909 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.5667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.5413 * (10**-6) * (V**2) + 5.3120 * (10**-4) * V + 4.8693 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2015 <= car_birth:
                    if V <= 79.6:
                        EFi_CO = 0.6374 * (V**-0.7666)
                        EFi_HC = 0.0879 * (V**-1.0745)
                    else:
                        EFi_CO = 4.1425 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.2667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.4453 * (10**-6) * (V**2) + 5.1680 * (10**-4) * V + 4.7373 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
            elif (car_type == '승용') and (car_size == '소형'):
                if car_birth <= 1990:
                    EFi_CO = 72.338 * (V**-0.7587)
                    EFi_HC = 26.532 * (V**-1.0423)
                    EFi_NOx = 9.3011 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1991 <= car_birth == 1996:
                    EFi_CO = 72.338 * (V**-0.7587)
                    EFi_HC = 101.79 * (V**-1.6823)
                    EFi_NOx = 11.287 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 44.956 * (V**-1.0085)
                    EFi_HC = 11.173 * (V**-1.3927)
                    EFi_NOx = 7.5371 * (V**-0.7864)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2002:
                    EFi_CO = 44.956 * (V**-1.0085)
                    EFi_HC = 11.173 * (V**-1.3927)
                    EFi_NOx = 4.7108 * (V**-0.7864)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 44.956 * (V**-1.0085)
                    EFi_HC = 3.2821 * (V**-1.3927)
                    EFi_NOx = 4.7108 * (V**-0.7864)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth <= 2007:
                    EFi_CO = 39.362 * (V**-1.0085)
                    EFi_HC = 2.8981 * (V**-1.3927)
                    EFi_NOx = 1.8419 * (V**-0.7864)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2008:
                    if V <= 79.6:
                        EFi_CO = 0.7693 * (V**-0.7666)
                        EFi_HC = 0.1063 * (V**-1.0745)
                    else:
                        EFi_CO = 5.0000 * (10**-16) * (V**7.2766)
                        EFi_HC = 1.0000 * (10**-15) * (V**6.2696)
                    EFi_NOx = -4.0000 * (10**-6) * (V**2) + 6.0000 * (10**-4) * V + 5.5000 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2009 <= car_birth <= 2011:
                    if V <= 79.6:
                        EFi_CO = 0.7059 * (V**-0.7666)
                        EFi_HC = 0.0974 * (V**-1.0745)
                    else:
                        EFi_CO = 4.5878 * (10**-16) * (V**7.2766)
                        EFi_HC = 9.1667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.7333 * (10**-6) * (V**2) + 5.6000 * (10**-4) * V + 5.1333 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2012 <= car_birth <= 2013:
                    if V <= 79.6:
                        EFi_CO = 0.6830 * (V**-0.7666)
                        EFi_HC = 0.0943 * (V**-1.0745)
                    else:
                        EFi_CO = 4.4393 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.8667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.6373 * (10**-6) * (V**2) + 5.4560 * (10**-4) * V + 5.0013 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2014:
                    if V <= 79.6:
                        EFi_CO = 0.6602 * (V**-0.7666)
                        EFi_HC = 0.0911 * (V**-1.0745)
                    else:
                        EFi_CO = 4.2909 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.5667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.5413 * (10**-6) * (V**2) + 5.3120 * (10**-4) * V + 4.8693 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2015 <= car_birth:
                    if V <= 79.6:
                        EFi_CO = 0.6374 * (V**-0.7666)
                        EFi_HC = 0.0879 * (V**-1.0745)
                    else:
                        EFi_CO = 4.1425 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.2667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.4453 * (10**-6) * (V**2) + 5.1680 * (10**-4) * V + 4.7373 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                if V <= 60:
                    EFi_NH3 = 0.0095
                elif 60 < V <= 90:
                    EFi_NH3 = 0.0082
                else:
                    EFi_NH3 = 0.022
            elif ( (car_type == '승용') and ( (car_size == '중형') or (car_size == '대형') ) or ( (car_type == '특수') and (car_size == '중형') ) ): # 최이사님 확인(2023.04.20) : 특수 중형 -> 승용 중형 조건으로 계산
                if car_birth <= 1990:
                    EFi_CO = 72.338 * (V**-0.7587)
                    EFi_HC = 26.532 * (V**-1.0423)
                    EFi_NOx = 9.3011 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1991 <= car_birth == 1996:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 26.520 * (V**-1.4041)
                    EFi_NOx = 11.287 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 4.7595 * (V**-0.9512)
                    EFi_NOx = 12.562 * (V**-0.8606)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2005:
                    EFi_CO = 17.829 * (V**-0.6778)
                    EFi_HC = 6.3668 * (V**-1.2849)
                    EFi_NOx = 8.8952 * (V**-0.8119)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth <= 2007:
                    EFi_CO = 73.022 * (V**-1.2078)
                    EFi_HC = 4.4166 * (V**-1.5356)
                    EFi_NOx = 2.0280 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2008:
                    if V <= 79.6:
                        EFi_CO = 0.7693 * (V**-0.7666)
                        EFi_HC = 0.1063 * (V**-1.0745)
                    else:
                        EFi_CO = 5.0000 * (10**-16) * (V**7.2766)
                        EFi_HC = 1.0000 * (10**-15) * (V**6.2696)
                    EFi_NOx = -4.0000 * (10**-6) * (V**2) + 6.0000 * (10**-4) * V + 5.5000 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2009 <= car_birth <= 2011:
                    if V <= 79.6:
                        EFi_CO = 0.7059 * (V**-0.7666)
                        EFi_HC = 0.0974 * (V**-1.0745)
                    else:
                        EFi_CO = 4.5878 * (10**-16) * (V**7.2766)
                        EFi_HC = 9.1667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.7333 * (10**-6) * (V**2) + 5.6000 * (10**-4) * V + 5.1333 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2012 <= car_birth <= 2013:
                    if V <= 79.6:
                        EFi_CO = 0.6830 * (V**-0.7666)
                        EFi_HC = 0.0943 * (V**-1.0745)
                    else:
                        EFi_CO = 4.4393 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.8667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.6373 * (10**-6) * (V**2) + 5.4560 * (10**-4) * V + 5.0013 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2014:
                    if V <= 79.6:
                        EFi_CO = 0.6602 * (V**-0.7666)
                        EFi_HC = 0.0911 * (V**-1.0745)
                    else:
                        EFi_CO = 4.2909 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.5667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.5413 * (10**-6) * (V**2) + 5.3120 * (10**-4) * V + 4.8693 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2015 <= car_birth:
                    if V <= 79.6:
                        EFi_CO = 0.6374 * (V**-0.7666)
                        EFi_HC = 0.0879 * (V**-1.0745)
                    else:
                        EFi_CO = 4.1425 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.2667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.4453 * (10**-6) * (V**2) + 5.1680 * (10**-4) * V + 4.7373 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                if V <= 60:
                    EFi_NH3 = 0.0095
                elif 60 < V <= 90:
                    EFi_NH3 = 0.0082
                else:
                    EFi_NH3 = 0.022
            elif ( (car_type == '승합') and (car_size == '경형') ) or ( (car_type == '화물') and (car_size == '경형') ):
                if car_birth <= 1996:
                    if V <= 45:
                        EFi_CO = 22.498 * (V**-0.6579)
                    else:
                        EFi_CO = 0.0006 * (V**2) + 0.0004 * V + 0.8272
                    EFi_HC = 12.961 * (V**-0.8364)
                    EFi_NOx = 4.0131 * (V**-0.3236)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth == 1999:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 2.2714 * (V**-0.7830)
                    EFi_NOx = 1.8528 * (V**-0.3889)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2002:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 2.2714 * (V**-0.7830)
                    EFi_NOx = 5.8289 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2003 <= car_birth <= 2005:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 1.1073 * (V**-0.7830)
                    EFi_NOx = 5.8289 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth:
                    if V <= 45:
                        EFi_CO = 8.9904 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0002 * (V**2) + 0.0457 * V - 1.1403
                    EFi_HC = 0.3549 * (V**-0.7830)
                    EFi_NOx = 0.7228 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                if V <= 60:
                    EFi_NH3 = 0.0095
                elif 60 < V <= 90:
                    EFi_NH3 = 0.0082
                else:
                    EFi_NH3 = 0.022
            elif (car_type == '승합') and (car_size == '소형'):
                if car_birth <= 1990:
                    EFi_CO = 72.338 * (V**-0.7587)
                    EFi_HC = 26.532 * (V**-1.0423)
                    EFi_NOx = 9.3011 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1991 <= car_birth == 1996:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 26.520 * (V**-1.4041)
                    EFi_NOx = 11.287 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 4.7595 * (V**-0.9512)
                    EFi_NOx = 12.562 * (V**-0.8606)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2005:
                    EFi_CO = 17.829 * (V**-0.6778)
                    EFi_HC = 6.3668 * (V**-1.2849)
                    EFi_NOx = 8.8952 * (V**-0.8119)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth:
                    if V <= 65.4:
                        EFi_CO = 6.1701 * (V**-1.0719)
                        EFi_HC = 2.3782 * (V**-1.9979)
                    else:
                        EFi_CO = 3.0000 * (10**-10) * (V**4.5809)
                        EFi_HC = 5.0000 * (10**-10) * (V**3.4895)
                    if V <= 97.3:
                        EFi_NOx = 0.4758 * (V**-1.0665)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                if V <= 60:
                    EFi_NH3 = 0.0095
                elif 60 < V <= 90:
                    EFi_NH3 = 0.0082
                else:
                    EFi_NH3 = 0.022
            # 최이사님 확인(2023.04.20) : 화물 중형 -> 승합 중형 조건, 화물 대형 -> 승합 대형 조건으로 계산
            elif ( (car_type == '승합') and ( (car_size == '중형') or (car_size == '대형') ) ) or ( (car_type == '화물') and ((car_size == '소형') or (car_size == '중형') or (car_size == '대형')) ):
                if car_birth <= 1990:
                    EFi_CO = 72.338 * (V**-0.7587)
                    EFi_HC = 26.532 * (V**-1.0423)
                    EFi_NOx = 9.3011 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1991 <= car_birth == 1996:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 26.520 * (V**-1.4041)
                    EFi_NOx = 11.287 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 4.7595 * (V**-0.9512)
                    EFi_NOx = 12.562 * (V**-0.8606)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2005:
                    EFi_CO = 17.829 * (V**-0.6778)
                    EFi_HC = 6.3668 * (V**-1.2849)
                    EFi_NOx = 8.8952 * (V**-0.8119)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth:
                    if V <= 65.4:
                        EFi_CO = 16.491 * (V**-1.4713)
                        EFi_HC = 1.9218 * (V**-1.8418)
                    else:
                        EFi_CO = 3.0000 * (10**-13) * (V**6.0619)
                        EFi_HC = 3.0000 * (10**-9) * (V**3.0639)
                    if V <= 97.3:
                        EFi_NOx = 0.1246 * (V**-0.441)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                if V <= 60:
                    EFi_NH3 = 0.0095
                elif 60 < V <= 90:
                    EFi_NH3 = 0.0082
                else:
                    EFi_NH3 = 0.022
        # else:
        #     # 값 고민 : 1
        #     # EFi_CO, EFi_HC, EFi_NOx, EFi_PM10, EFi_PM2_5 = np.nan, np.nan, np.nan, np.nan, np.nan
        #     EFi_CO, EFi_HC, EFi_NOx, EFi_PM10, EFi_PM2_5 = 1, 1, 1, 1, 1

        if V < 60:
            EFi_COm += EFi_CO*0.34
            EFi_HCm += EFi_HC*0.34
            EFi_NOxm += EFi_NOx*0.34
            EFi_PM10m += EFi_PM10*0.34
            EFi_PM2_5m += EFi_PM2_5*0.34
            EFi_NH3m += EFi_NH3*0.34
        elif 60 <= V < 90:
            EFi_COm += EFi_CO*0.33
            EFi_HCm += EFi_HC*0.33
            EFi_NOxm += EFi_NOx*0.33
            EFi_PM10m += EFi_PM10*0.33
            EFi_PM2_5m += EFi_PM2_5*0.33
            EFi_NH3m += EFi_NH3*0.33
        elif V > 90:
            EFi_COm += EFi_CO*0.33
            EFi_HCm += EFi_HC*0.33
            EFi_NOxm += EFi_NOx*0.33
            EFi_PM10m += EFi_PM10*0.33
            EFi_PM2_5m += EFi_PM2_5*0.33
            EFi_NH3m += EFi_NH3*0.33

    # 배출계수(EFi) 리스트에 저장
    EFi_CO_list.append(EFi_COm)
    EFi_HC_list.append(EFi_HCm)
    EFi_NOx_list.append(EFi_NOxm)
    EFi_PM10_list.append(EFi_PM10m)
    EFi_PM2_5_list.append(EFi_PM2_5m)
    EFi_NH3_list.append(EFi_NH3m)

In [593]:
## 11.3s
df2['EFi_CO'] = EFi_CO_list
df2['EFi_HC'] = EFi_HC_list
df2['EFi_NOx'] = EFi_NOx_list
df2['EFi_PM10'] = EFi_PM10_list
df2['EFi_PM2_5'] = EFi_PM2_5_list
check_EFi_col = ['EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5']

In [594]:
## 10.1s
## 배출량 계산
# E = VKT * (EFi / 1000) * DF * (1 - R / 100)
df2['E_HOT_CO'] = df2['VKT'] * (df2['EFi_CO'] / 1000) * df2['DF_CO'] * (1 - df2['R_CO'] / 100)
df2['E_HOT_HC'] = df2['VKT'] * (df2['EFi_HC'] / 1000) * df2['DF_HC'] * (1 - df2['R_HC'] / 100)
df2['E_HOT_NOx'] = df2['VKT'] * (df2['EFi_NOx'] / 1000) * df2['DF_NOx'] * (1 - 0 / 100)
df2['E_HOT_PM10'] = df2['VKT'] * (df2['EFi_PM10'] / 1000) * df2['DF_PM'] * (1 - df2['R_PM'] / 100)
df2['E_HOT_PM2_5'] = df2['VKT'] * (df2['EFi_PM2_5'] / 1000) * df2['DF_PM'] * (1 - df2['R_PM'] / 100)
check_E_HOT_col = ['E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5']

In [595]:
# 자동차-엔진 미가열(Cold-start) 배출
## 배출량 식
# $$E_{COLD :i, j} = \beta_{i, j} \times N_j \times M_j \times e^{HOT}_j \times (e^{COLD} / e^{HOT} \vert_{i, j} - 1)$$

# - $E_{COLD :i, j}$ : 차종 $j$에서 배출되는 오염물질 $i$의 엔진미가열 배출량
# - $\beta_{i, j}$ : 차종 $j$의 엔진미가열 상태의 주행거리 분율
# - $N_j$ : 차종 $j$의 수
# - $M_j$ : 차종 $j$의 주행거리
# - $e^{HOT}_j$ : 차종 $j$의 엔진가열 상태에서의 배출계수
# - $e^{COLD} / e^{HOT} \vert_{i, j}$ : 차종 $j$의 엔진가열상태 대비 엔진미가열 상태에서의 배출 비율

## 베타($\beta$)
# - 1회 평균주행거리(1 trip length)와 대기온도, 자동차 이용 패턴을 고려
#     - 1회 평균주행거리 정보는 모든 차종에 대하여 각각 적용하는 방안은 수집자료의 한계로 국내 연구결과를 바탕으로 수도권 지역 승용차의 1회 평균 주행거리 12.35km를 적용
#     - 이때 과대산정 가능성을 염두하여, 도시지역의 택시를 제외한 승용차(경형, 소형, 중형, 대형), 승합차(경형, 소형), RV(소형, 중형)에 적용

#     - 1회 주행거리 : 12.35 km (2002년 기준)
#     - 미가열 배출 적용 차종 : 승용차(경형, 소형, 중형, 대형) / 승합차(경형, 소형) / RV(소형, 중형)
#     - 미가열 배출 적용 지역 : 도시지역(고속도로 구간 제외)

# - 산정식

# -|Factor $\beta$의 산정식
# -|-
# $Estimated$ $l_{trip}$|$0.647-0.025 \times l_{trip} - (0.00974 - 0.000385 \times l_{trip}) \times T_a$
# $Measured$ $l_{trip}$|$0.698-0.051 \times l_{trip} - (0.01051 - 0.000770 \times l_{trip}) \times T_a$

# - $T_a$ : 대기온도
# - $Estimated$ $l_{trip}$ : 유럽평균(1985) 약 11.8km / 유럽평균(1998) 약 12.4km(Andre et al., 1998)
l_trip = 12.35 # 국립환경과학원(2007), 도로 이동오염원 대기오염 배출량 산정방법 개선 및 장래 배출량 예측방법 연구
Ta = 12.4 # e-나라지표 (https://www.index.go.kr/unity/potal/main/EachDtlPageDetail.do?idx_cd=1400) 2002년 년 평균 기온
df2['Beta(Estimated)'] = 0.647 - 0.025 * l_trip - (0.00974 - 0.000385 * l_trip) * Ta
# df2['Beta(Measured)'] = 0.698 - 0.051 * l_trip - (0.01051 - 0.000770 * l_trip) * Ta

In [596]:
## 13.9s
## $e^{COLD} / e^{HOT}$
# 휘발유(자동 제어)
df2.loc[(df2['연료'] == '휘발유') & (df2['차량연식'] < 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_CO'] = 3.7 - 0.09 * Ta
df2.loc[(df2['연료'] == '휘발유') & (df2['차량연식'] >= 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_CO'] = 9.04 - 0.09 * Ta
df2.loc[(df2['연료'] == '휘발유') & (df2['차량연식'] < 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_NOx'] = 1.14 - 0.006 * Ta
df2.loc[(df2['연료'] == '휘발유') & (df2['차량연식'] >= 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_NOx'] = 3.66 - 0.006 * Ta
df2.loc[(df2['연료'] == '휘발유') & (df2['차량연식'] < 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_HC'] = 2.8 - 0.06 * Ta
df2.loc[(df2['연료'] == '휘발유') & (df2['차량연식'] >= 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_HC'] = 12.59 - 0.06 * Ta
df2.loc[df2['연료'] == '휘발유', 'eCOLD/eHOT_PM'] = 1

In [597]:
## 11.9s
# 경유
df2.loc[(df2['연료'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_CO'] = 1.9 - 0.03 * Ta
df2.loc[(df2['연료'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_NOx'] = 1.3 - 0.013 * Ta
df2.loc[(df2['연료'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_HC'] = 3.1 - 0.09 * Ta
df2.loc[(df2['연료'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_PM'] = 3.1 - 0.1 * Ta

In [598]:
## 11.9s
# LPG(액화석유가스)
df2.loc[(df2['연료'] == 'LPG(액화석유가스)') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_CO'] = 3.66 - 0.09 * Ta
df2.loc[(df2['연료'] == 'LPG(액화석유가스)') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_NOx'] = 0.98 - 0.006 * Ta
df2.loc[(df2['연료'] == 'LPG(액화석유가스)') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_HC'] = 2.24 - 0.06 * Ta
df2.loc[(df2['연료'] == 'LPG(액화석유가스)') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_PM'] = 1
check_ecoldhot_col = ['eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM']
df2[check_ecoldhot_col] = df2[check_ecoldhot_col].fillna(1)

In [599]:
## 10.1s
## 배출량 계산
### 🔺 계산식 고민
# $$E_{COLD :i, j} = \beta_{i, j} \times N_j \times M_j \times e^{HOT}_j \times (e^{COLD} / e^{HOT} \vert_{i, j} - 1)$$
# - $M_j$ : 차종 $j$의 주행거리
#     - 1년 주행거리로 계산? 
#         - VKT
#     - 1회 주행거리로 계산?
#         - l_trip

#         E      =             B          *     M      *    e(HOT)     * ( e(COLD)/e(HOT)      - 1 )
df2['E_COLD_CO'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_CO'] * (df2['eCOLD/eHOT_CO'] - 1)
df2['E_COLD_NOx'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_NOx'] * (df2['eCOLD/eHOT_NOx'] - 1)
df2['E_COLD_HC'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_HC'] * (df2['eCOLD/eHOT_HC'] - 1)
df2['E_COLD_PM10'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_PM10'] * (df2['eCOLD/eHOT_PM'] - 1)
df2['E_COLD_PM2_5'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_PM2_5'] * (df2['eCOLD/eHOT_PM'] - 1)
check_E_cold_col = ['E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5']

In [600]:
# 자동차-휘발유 증발 배출
## 배출량 식
# - 3가지
#     - 주간증발손실(Diurnal loss)
#     - 고온증발손실(Hot and warm soak)
#     - 주행손실(Running loss)
# - 현재('19 배출량 기준) 휘발유 증발 배출원
#     - 주간증발손실, 주행손실 고려
#     - 우리나라 휘발유 차량 대부분이 방지설비로 카본 캐니스터를 장착하고 있으므로 $e^{S,HOT}$, $e^{S,WARM}$, 고온증발손실(hot and warm soak) 배출은 없는 것으로 가정

# $$E_{EVA, VOC:j} = 365 \times N_j \times (e^d + S^c + S^{fi}) + R$$
# $S^c = (1-q)(p \times x \times e^{S, HOT} + w \times x \times e^{S,WARM})$

# $S^{fi} = q \times e^{fi} \times x$
# $$R = m_j(p \times e^{R,HOT} + w \times e^{R,WARM})$$

# - $E_{EVA, VOC:j}$ : 차종 j의 증발손실에 의한 VOC 배출량(g/yr)
# - $N_j$ : 차종 j의 휘발유 사용 차량 등록대수(대)
# - $e^d$ : 금속탱크를 가지고 있는 휘발유 차량의 일중 VOC 배출량(g/day-대)
# - $S^c$ : Carburetor 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
# - $S^{fi}$ : Fuel injection 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
# - $R$ : Running loss에 의한 휘발유 차량의 연중 배출량(g/yr)
# - $q$ : Fuel injection 휘발유 차량의 비율
# - $p$ : 가열 엔진 상태로 끝나는 trip의 비율
# - $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
# - $w$ : 미가열 엔진 상태료 끝나는 trip의 비율
# - $e^{S,HOT}$ : Hot soak emission의 평균 배출계수
# - $e^{S,WARM}$ : Cold and warm soak emission의 평균 배출계수
# - $e^{fi}$ : Fuel injection 휘발유 차량의 평균 hot and warm soak 배출계수
# - $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수
# - $e^{R,WARM}$ : 휘발유 차량의 평균 warm running loss 배출계수
# - $m_j$ : 차종 j의 연간 총 주행거리
# - $V_j$ : 차종 j의 1대당 평균 연간 주행거리

In [601]:
## 배출계수
### 🔺 수정 중
### $e^d$ Diurnal(g/day)
# - $e^d$ : 금속탱크를 가지고 있는 휘발유 차량의 일중 VOC 배출량(g/day-대)
#### 🔺 RVP(증기압)
# - 참고 COPERT 3(p.80) Table 6.10: Gasoline fuel specifications
#     - summer : 60
#     - winter : 70
#     - 평균 : 65
# - 수정 사항(2023.04.20, 최)
#     - 국내 과학기술원, 환경부 자료 찾아서 사용 추천
RVP = 65
#### ❗ t_a(기온)
# - t_a, t_min, t_rise
#     - 참고 COPERT 3(p.72) Table 5.32: Summary of emission factors for estimating evaporative emissions of gasoline vehicles(all RVp in kPa, all temperatures in °C)
#         - t_a = (t_max + t_min) / 2
#         - t_rise = t_max - t_min
#     - 기상자료개방포털(2002년) 연평균 기온
#         - https://data.kma.go.kr/stcs/grnd/grndTaList.do?pgmNo=70
#         - 7.5°C(최저), 12.4°C(평균), 17.9°C(최고)
t_min = 7.5 # 평균 최저 온도
t_max = 17.9
t_a = (t_min + t_max) / 2
t_rise = 12.4 # 평균 상승 온도

df2.loc[df2['차량연식'] < 1991, 'e_d'] = 9.1 * np.exp( 0.0158 * (RVP - 61.2) + 0.0574 * (t_min - 22.5) + 0.0614 * (t_rise - 11.7) )
df2.loc[df2['차량연식'] >= 1991, 'e_d'] = 0.2 * (9.1 * np.exp( 0.0158 * (RVP - 61.2) + 0.0574 * (t_min - 22.5) + 0.0614 * (t_rise - 11.7) ))

In [602]:
### $e^{R,HOT}$ Hot running loss(g/km)
# - $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수
df2.loc[df2['차량연식'] < 1991, 'e_RHOT'] = 0.136 * np.exp( - 5.967 + 0.04259 * RVP + 0.1773 * t_a)
df2.loc[df2['차량연식'] >= 1991, 'e_RHOT'] = 0.1 * (0.136 * np.exp( - 5.967 + 0.04259 * RVP + 0.1773 * t_a))

In [603]:
### $e^{S,HOT}$ Hot soak(g/procedure)
# - $e^{S,HOT}$ : Hot soak emission의 평균 배출계수
df2.loc[df2['차량연식'] < 1991, 'e_SHOT'] = 3.0042 * np.exp(0.02 * RVP)
df2.loc[df2['차량연식'] >= 1991, 'e_SHOT'] = 0.3 * np.exp(-2.14 + 0.02302 * RVP + 0.09408 * t_a)

In [604]:
### $e^{S,WARM}$ Waram soak(g/procedure)
# - $e^{S,WARM}$ : Cold and warm soak emission의 평균 배출계수
df2.loc[df2['차량연식'] < 1991, 'e_SWARM'] = np.exp(-1.644 + 0.01993 * RVP + 0.07521 * t_a)
df2.loc[df2['차량연식'] >= 1991, 'e_SWARM'] = 0.2 * np.exp(-2.41 + 0.02302 * RVP + 0.09408 * t_a)

In [605]:
### $e^{fi}$ Warm and hot soak for Fuel injected vehicle(g/procedure)
# - $e^{fi}$ : Fuel injection 휘발유 차량의 평균 hot and warm soak 배출계수
df2.loc[df2['차량연식'] < 1991, 'e_fi'] = 0.7
df2.loc[df2['차량연식'] >= 1991, 'e_fi'] = 0

In [606]:
### $e^{R,WARM}$ Warm running loss(g/km)
# - $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수
df2.loc[df2['차량연식'] < 1991, 'e_RWARM'] = 0.1 * np.exp(-5.967 + 0.04259 * RVP + 0.1773 * t_a)
df2.loc[df2['차량연식'] >= 1991, 'e_RWARM'] = 0.1 * (0.1 * np.exp(-5.967 + 0.04259 * RVP + 0.1773 * t_a))

In [607]:
## $R$
# $R = m_j(p \times e^{R,HOT} + w \times e^{R,WARM})$
# - Running loss에 의한 휘발유 차량의 연중 배출량(g/yr)
# - $m_j$ : 차종 j의 연간 총 주행거리
# - $p$ : 가열 엔진 상태로 끝나는 trip의 비율
# - $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수
# - $w$ : 미가열 엔진 상태로 끝나는 trip의 비율
# - $e^{R,WARM}$ : 휘발유 차량의 평균 warm running loss 배출계수
### ❓ 확인 중
# - 수정 사항(2023.04.20, 최)
    # - 찾는 중
    # - p:w = 0.2:0.8(2023.06.29, 최)
p = 0.2 # 가열 엔진 상태로 끝나는 trip의 비율
w = 0.8 # 미가열 엔진 상태로 끝나는 trip의 비율
df2['R'] = df2['VKT'] * (p * df2['e_RHOT'] + w * df2['e_RWARM'])

In [608]:
## $S^{fi}$
# $S^{fi} = q \times e^{fi} \times x$
# - Fuel injection 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
# - $q$ : Fuel injection 휘발유 차량의 비율
# - $e^{fi}$ : Fuel injection 휘발유 차량의 평균 hot and warm soak 배출계수
# - $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
# - $V_j$ : 차종 j의 1대당 평균 연간 주행거리
not_injection = df2[(df2['연료'] == '휘발유') & (df2['차량연식'] < 1991)].shape[0]
injection = df2[(df2['연료'] == '휘발유') & (df2['차량연식'] >= 1991)].shape[0]

# Fuel injection 휘발유 차량의 비율
q = injection / (injection + not_injection)

In [609]:
# x = df2['VKT'] / (365 * l_trip)
df2['S_fi'] = q * df2['e_fi'] * ( df2['VKT'] / (365 * l_trip) )

In [610]:
## $S^c$
# $S^c = (1-q)(p \times x \times e^{S, HOT} + w \times x \times e^{S,WARM})$
# - Carburetor 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
# - $q$ : Fuel injection 휘발유 차량의 비율
# - $p$ : 가열 엔진 상태로 끝나는 trip의 비율
# - $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
# - $e^{S,HOT}$ : Hot soak emission의 평균 배출계수
# - $w$ : 미가열 엔진 상태료 끝나는 trip의 비율
# - $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
# - $V_j$ : 차종 j의 1대당 평균 연간 주행거리
# - $e^{S,WARM}$ : Cold and warm soak emission의 평균 배출계수
# x = df2['VKT'] / (365 * l_trip)
df2['S_c'] = (1-q) * (p * (df2['VKT'] / (365 * l_trip)) * df2['e_SHOT'] + w * (df2['VKT'] / (365 * l_trip)) * df2['e_SWARM'])

In [611]:
## $E_{EVA,VOC}$
# $E_{EVA, VOC:j} = 365 \times N_j \times (e^d + S^c + S^{fi}) + R$
# - $N_j$ : 차종 j의 휘발유 사용 차량 등록대수(대)
# - $e^d$ : 금속탱크를 가지고 있는 휘발유 차량의 일중 VOC 배출량(g/day-대)
# - $S^c$ : Carburetor 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
# - $S^{fi}$ : Fuel injection 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
# - $R$ : Running loss에 의한 휘발유 차량의 연중 배출량(g/yr)
df2['E_EVA_VOC'] = 365 * (df2['e_d'] + df2['S_c'] + df2['S_fi']) + df2['R']

In [612]:
# 배출량 합계
# $E_{total}(kg) = E_{HOT}(kg) + E_{COLD}(g) + E_{EVAP}(g)$
# E_COLD_NOx 음수 -> 0으로 처리(2023.04.24 from 최이사님)
df2.loc[df2['E_COLD_NOx'] < 0, 'E_COLD_NOx'] = 0

In [613]:
# 단위 변환(g -> kg)
df2['E_COLD_CO'] = df2['E_COLD_CO'] / 1000
df2['E_COLD_HC'] = df2['E_COLD_HC'] / 1000
df2['E_COLD_NOx'] = df2['E_COLD_NOx'] / 1000
df2['E_COLD_PM10'] = df2['E_COLD_PM10'] / 1000
df2['E_COLD_PM2_5'] = df2['E_COLD_PM2_5'] / 1000
df2['E_EVA_VOC'] = df2['E_EVA_VOC'] / 1000

In [614]:
## 11.6s
fuel = '휘발유'
df2.loc[df2['연료'] == fuel, 'E_CO_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_CO'] + df2.loc[df2['연료'] == fuel , 'E_COLD_CO']
df2.loc[df2['연료'] == fuel, 'E_HC_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_HC'] + df2.loc[df2['연료'] == fuel , 'E_COLD_HC'] + df2.loc[df2['연료'] == fuel , 'E_EVA_VOC']
df2.loc[df2['연료'] == fuel, 'E_NOx_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_NOx'] + df2.loc[df2['연료'] == fuel , 'E_COLD_NOx']
df2.loc[df2['연료'] == fuel, 'E_PM10_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_PM10'] + df2.loc[df2['연료'] == fuel , 'E_COLD_PM10']
df2.loc[df2['연료'] == fuel, 'E_PM2_5_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_PM2_5'] + df2.loc[df2['연료'] == fuel , 'E_COLD_PM2_5']

In [615]:
## 11.6s
fuel = '경유'
df2.loc[df2['연료'] == fuel, 'E_CO_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_CO'] + df2.loc[df2['연료'] == fuel , 'E_COLD_CO']
df2.loc[df2['연료'] == fuel, 'E_HC_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_HC'] + df2.loc[df2['연료'] == fuel , 'E_COLD_HC']
df2.loc[df2['연료'] == fuel, 'E_NOx_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_NOx'] + df2.loc[df2['연료'] == fuel , 'E_COLD_NOx']
df2.loc[df2['연료'] == fuel, 'E_PM10_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_PM10'] + df2.loc[df2['연료'] == fuel , 'E_COLD_PM10']
df2.loc[df2['연료'] == fuel, 'E_PM2_5_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_PM2_5'] + df2.loc[df2['연료'] == fuel , 'E_COLD_PM2_5']

In [616]:
## 12.2s
fuel = 'LPG(액화석유가스)'
df2.loc[df2['연료'] == fuel, 'E_CO_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_CO'] + df2.loc[df2['연료'] == fuel , 'E_COLD_CO']
df2.loc[df2['연료'] == fuel, 'E_HC_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_HC'] + df2.loc[df2['연료'] == fuel , 'E_COLD_HC']
df2.loc[df2['연료'] == fuel, 'E_NOx_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_NOx'] + df2.loc[df2['연료'] == fuel , 'E_COLD_NOx']
df2.loc[df2['연료'] == fuel, 'E_PM10_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_PM10'] + df2.loc[df2['연료'] == fuel , 'E_COLD_PM10']
df2.loc[df2['연료'] == fuel, 'E_PM2_5_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_PM2_5'] + df2.loc[df2['연료'] == fuel , 'E_COLD_PM2_5']
check_E_col = ['E_CO_total', 'E_HC_total', 'E_NOx_total', 'E_PM10_total', 'E_PM2_5_total']

In [617]:
## 배출량
# - 연료 : 휘발유, 경유, LPG(액화석유가스)
# - 물질 : CO, HC, NOx, PM10, PM2.5

## 시도/시군구별 배출량 합계
grp1 = df2.groupby(['시도', '시군구_수정']).agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum'}).reset_index() # !!! 수정(2023.11.13)
grp1

,시도,시군구_수정,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
0,강원특별자치도,강릉시,65659.775250,9435.086106,122005.034592,1384.541257,1270.170462
1,강원특별자치도,고성군,15887.809038,2097.189418,21222.777501,272.181619,249.962876
2,강원특별자치도,동해시,31025.299689,4120.594116,51351.768822,565.043273,518.513815
3,강원특별자치도,삼척시,20555.480408,2777.734924,38862.821181,471.779415,435.141055
4,강원특별자치도,속초시,29363.529426,3960.588852,37068.246239,397.481306,364.717588
...,...,...,...,...,...,...,...
226,충청북도,증평군,20339.509055,2805.336647,28820.541528,357.281634,328.022393
227,충청북도,진천군,51195.647927,6864.307225,72586.493870,810.876157,746.366442
228,충청북도,청원군,188.145895,31.445589,40.861656,0.057823,0.053197
229,충청북도,청주시,313767.861098,42731.473837,492640.043216,5291.510159,4858.108749


In [618]:
periods = 2 # !!! 수정(2023.08.23)
y_plist = list(pd.date_range(end=date, periods=periods, freq="MS").year)
mth_plist = list(pd.date_range(end=date, periods=periods, freq="MS").month)

crtr_year = str(y_plist[0])
tp_crtr_month = mth_plist[0] # !!! 수정(2024.01.23)
crtr_month = f'{tp_crtr_month:0>2}' # !!! 수정(2024.01.23)
crtr_year, crtr_month

('2024', '01')

In [619]:
# 연도 설정
today_date = datetime.today().strftime("%Y%m%d")
grp1['연도'] = crtr_year
# grp1['연도'] = '2022' # 하드코딩
grp1 = grp1[['연도', '시도', '시군구_수정', 'E_CO_total', 'E_HC_total', 'E_NOx_total', 'E_PM10_total', 'E_PM2_5_total']]
grp1['테이블생성일자'] = today_date

C:\Users\kbjung\AppData\Local\Temp\ipykernel_2144\1659578264.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grp1['테이블생성일자'] = today_date


In [620]:
# 기준연월 추가
# grp1['기준연월'] = '2022.12'
grp1['기준연월'] = crtr_year + '.' + crtr_month # !!! 수정(2024.01.23)
chc_dict = {
    '테이블생성일자':'LOAD_DT', 
    '기준연월':'CRTR_YM', 
    '연도':'YR', 
    '시도':'CTPV', 
    '시군구_수정':'SGG', 
    'E_CO_total':'CO_EXHST_MSS', 
    'E_HC_total':'HC_EXHST_MSS', 
    'E_NOx_total':'NOx_EXHST_MSS',
    'E_PM10_total':'PM10_EXHST_MSS', 
    'E_PM2_5_total':'PM2_5_EXHST_MSS', 
}
STD_BD_GRD4_EXHST_GAS_MSS_TOT = grp1.rename(columns=chc_dict)

C:\Users\kbjung\AppData\Local\Temp\ipykernel_2144\399832553.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grp1['기준연월'] = crtr_year + '.' + crtr_month # !!! 수정(2024.01.23)


In [621]:
STD_BD_GRD4_EXHST_GAS_MSS_TOT.head()

,YR,CTPV,SGG,CO_EXHST_MSS,HC_EXHST_MSS,NOx_EXHST_MSS,PM10_EXHST_MSS,PM2_5_EXHST_MSS,LOAD_DT,CRTR_YM
0,2024,강원특별자치도,강릉시,65659.775250,9435.086106,122005.034592,1384.541257,1270.170462,20240213,2024.01
1,2024,강원특별자치도,고성군,15887.809038,2097.189418,21222.777501,272.181619,249.962876,20240213,2024.01
2,2024,강원특별자치도,동해시,31025.299689,4120.594116,51351.768822,565.043273,518.513815,20240213,2024.01
3,2024,강원특별자치도,삼척시,20555.480408,2777.734924,38862.821181,471.779415,435.141055,20240213,2024.01
4,2024,강원특별자치도,속초시,29363.529426,3960.588852,37068.246239,397.481306,364.717588,20240213,2024.01


In [622]:
STD_BD_GRD4_EXHST_GAS_MSS_TOT.columns

Index(['YR', 'CTPV', 'SGG', 'CO_EXHST_MSS', 'HC_EXHST_MSS', 'NOx_EXHST_MSS',
       'PM10_EXHST_MSS', 'PM2_5_EXHST_MSS', 'LOAD_DT', 'CRTR_YM'],
      dtype='object')

In [623]:
STD_BD_GRD4_EXHST_GAS_MSS_TOT.iloc[:, 3:8].sum()

CO_EXHST_MSS       1.950190e+07
HC_EXHST_MSS       2.661797e+06
NOx_EXHST_MSS      2.784552e+07
PM10_EXHST_MSS     3.099679e+05
PM2_5_EXHST_MSS    2.845254e+05
dtype: float64

In [624]:
STD_BD_GRD4_EXHST_GAS_MSS_TOT.iloc[:, 3:8].sum().sum()

50603707.88613959

### [출력] STD_BD_GRD4_EXHST_GAS_MSS_TOT

In [625]:
# expdf = STD_BD_GRD4_EXHST_GAS_MSS_TOT
# table_nm = 'STD_BD_GRD4_EXHST_GAS_MSS_TOT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [626]:
df2.shape

(1318124, 89)

In [627]:
## 10.3s
## 출력(4등급)
STD_BD_GRD4_EXHST_GAS_MSS = df2[[
    '차대번호',
    '자동차등록번호',
    '제원관리번호',
    '차종', 
    '용도', 
    '최초등록일자',
    '차량연식',
    '제작일자',
    '검사유효일',
    '차명',
    '차종분류',
    '차종유형', 
    '자동차형식', 
    '제작사명',
    '연료', 
    '엔진형식',
    '총중량', 
    '적재중량', 
    '엔진출력', 
    '배기량', 
    '시도', 
    '시군구',
    '소유자구분',
    '차량말소YN',
    '배출가스인증번호',
    '배출가스등급',
    'DPF_YN',
    '법정동코드', 
    'E_CO_total', 
    'E_HC_total', 
    'E_NOx_total', 
    'E_PM10_total',
    'E_PM2_5_total', 
    '법정동코드_mod',
    ]]

# 날짜 설정
# STD_BD_GRD4_EXHST_GAS_MSS['기준연월'] = '2022.12'
today_date = datetime.today().strftime("%Y%m%d")
STD_BD_GRD4_EXHST_GAS_MSS['기준연월'] = crtr_year + '.' + crtr_month # !!! 수정(2024.01.23)

C:\Users\kbjung\AppData\Local\Temp\ipykernel_2144\1820448429.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  STD_BD_GRD4_EXHST_GAS_MSS['기준연월'] = crtr_year + '.' + crtr_month # !!! 수정(2024.01.23)


In [628]:
## 12.3s
STD_BD_GRD4_EXHST_GAS_MSS['테이블생성일자'] = today_date
# RH법정동코드 문자열타입으로 변경
STD_BD_GRD4_EXHST_GAS_MSS['법정동코드_mod'] = STD_BD_GRD4_EXHST_GAS_MSS['법정동코드_mod'].astype('str')
# 기준연월 추가 고민
STD_BD_GRD4_EXHST_GAS_MSS = STD_BD_GRD4_EXHST_GAS_MSS[[
    '테이블생성일자', 
    '기준연월', 
    '차대번호',
    '자동차등록번호',
    '제원관리번호',
    '차종', 
    '용도', 
    '최초등록일자',
    '차량연식',
    '제작일자',
    '검사유효일',
    '차명',
    '차종분류',
    '차종유형', 
    '자동차형식', 
    '제작사명',
    '연료', 
    '엔진형식',
    '총중량', 
    '적재중량', 
    '엔진출력', 
    '배기량', 
    '법정동코드', 
    '시도', 
    '시군구',
    '소유자구분',
    '차량말소YN',
    '배출가스인증번호',
    '배출가스등급',
    'DPF_YN',
    'E_CO_total', 
    'E_HC_total', 
    'E_NOx_total', 
    'E_PM10_total',
    'E_PM2_5_total', 
    '법정동코드_mod',
    ]]
ch_col_dict = {
                '테이블생성일자':'LOAD_DT', 
                '기준연월':'CRTR_YM',
                '차대번호':'VIN', 
                '자동차등록번호':'VHRNO', 
                '제원관리번호':'MANG_MNG_NO', 
                '차종':'VHCTY_CD', 
                '용도':'PURPS_CD2', 
                '최초등록일자':'FRST_REG_YMD', 
                '차량연식':'YRIDNW', 
                '제작일자':'FBCTN_YMD', 
                '검사유효일':'EXHST_GAS_INSP_VLD_YMD',
                '차명':'VHCNM', 
                '차종분류':'VHCTY_CL_CD', 
                '차종유형':'VHCTY_TY', 
                '자동차형식':'CAR_FRM', 
                '제작사명':'MNFCTR_NM', 
                '연료':'FUEL_CD', 
                '엔진형식':'EGIN_TY',
                '총중량':'TOTL_WGHT', 
                '적재중량':'CRYNG_WGHT', 
                '엔진출력':'EGIN_OTPT',
                '배기량':'DSPLVL', 
                '법정동코드':'STDG_CD', 
                '시도':'CTPV_NM', 
                '시군구':'SGG_NM', 
                '소유자구분':'OWNR_SE', 
                '차량말소YN':'ERSR_YN',
                '배출가스인증번호':'EXHST_GAS_CERT_NO', 
                '배출가스등급':'EXHST_GAS_GRD_CD', 
                'DPF유무_수정':'DPF_MNTNG_YN', 
                'Grade':'GRD4_MLSFC',
                'E_CO_total':'CO_EXHST_MSS',
                'E_HC_total':'HC_EXHST_MSS',
                'E_NOx_total':'NOx_EXHST_MSS', 
                'E_PM10_total':'PM10_EXHST_MSS',
                'E_PM2_5_total':'PM2_5_EXHST_MSS',
                '법정동코드_mod':'STDG_CD_MOD',
                }
                
STD_BD_GRD4_EXHST_GAS_MSS = STD_BD_GRD4_EXHST_GAS_MSS.rename(columns=ch_col_dict)

C:\Users\kbjung\AppData\Local\Temp\ipykernel_2144\28703130.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  STD_BD_GRD4_EXHST_GAS_MSS['테이블생성일자'] = today_date
C:\Users\kbjung\AppData\Local\Temp\ipykernel_2144\28703130.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  STD_BD_GRD4_EXHST_GAS_MSS['법정동코드_mod'] = STD_BD_GRD4_EXHST_GAS_MSS['법정동코드_mod'].astype('str')


In [629]:
STD_BD_GRD4_EXHST_GAS_MSS.columns

Index(['LOAD_DT', 'CRTR_YM', 'VIN', 'VHRNO', 'MANG_MNG_NO', 'VHCTY_CD',
       'PURPS_CD2', 'FRST_REG_YMD', 'YRIDNW', 'FBCTN_YMD',
       'EXHST_GAS_INSP_VLD_YMD', 'VHCNM', 'VHCTY_CL_CD', 'VHCTY_TY', 'CAR_FRM',
       'MNFCTR_NM', 'FUEL_CD', 'EGIN_TY', 'TOTL_WGHT', 'CRYNG_WGHT',
       'EGIN_OTPT', 'DSPLVL', 'STDG_CD', 'CTPV_NM', 'SGG_NM', 'OWNR_SE',
       'ERSR_YN', 'EXHST_GAS_CERT_NO', 'EXHST_GAS_GRD_CD', 'DPF_YN',
       'CO_EXHST_MSS', 'HC_EXHST_MSS', 'NOx_EXHST_MSS', 'PM10_EXHST_MSS',
       'PM2_5_EXHST_MSS', 'STDG_CD_MOD'],
      dtype='object')

### [출력] STD_BD_GRD4_EXHST_GAS_MSS

In [630]:
# expdf = STD_BD_GRD4_EXHST_GAS_MSS
# table_nm = 'STD_BD_GRD4_EXHST_GAS_MSS'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 13s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [631]:
df2[['시도', '시군구_수정', '연료', '차종', '차종유형', '용도']].isnull().sum()

시도           0
시군구_수정    7012
연료           0
차종           0
차종유형         0
용도           0
dtype: int64

In [632]:
## 10.5s
## 배출량 현황
grp2 = df2.groupby(['시도', '시군구_수정', '연료', '차종', '차종유형', '용도']).agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum'}).reset_index()
grp2 = grp2.rename(columns={'E_CO_total':'E_CO_total_sum', 'E_HC_total':'E_HC_total_sum', 'E_NOx_total':'E_NOx_total_sum', 'E_PM10_total':'E_PM10_total_sum', 'E_PM2_5_total':'E_PM2_5_total_sum'})

In [633]:
# 연도 설정
# grp2['연도'] = '2022'
today_date = datetime.today().strftime("%Y%m%d")
grp2['연도'] = today_date[:4]
grp2['테이블생성일자'] = today_date

In [634]:
STD_BD_DAT_GRD4_EXHST_MSS_CURSTT = grp2[[
    '연도',
    '시도',
    '시군구_수정',
    '연료',
    '차종',
    '차종유형',
    '용도',
    'E_CO_total_sum',
    'E_HC_total_sum',
    'E_NOx_total_sum',
    'E_PM10_total_sum',
    'E_PM2_5_total_sum',
    '테이블생성일자',
]]
cdict = {
    '연도':'YR',
    '시도':'CTPV',
    '시군구_수정':'SGG',
    '연료':'FUEL_CD',
    '차종':'VHCTY_CD',
    '차종유형':'VHCTY_TY',
    '용도':'PURPS_CD2',
    'E_CO_total_sum':'CO_EXHST_MSS_SM',
    'E_HC_total_sum':'HC_EXHST_MSS_SM',
    'E_NOx_total_sum':'NOx_EXHST_MSS_SM',
    'E_PM10_total_sum':'PM10_EXHST_MSS_SM',
    'E_PM2_5_total_sum':'PM2_5_EXHST_MSS_SM',
    '테이블생성일자':'LOAD_DT',
}
STD_BD_DAT_GRD4_EXHST_MSS_CURSTT = STD_BD_DAT_GRD4_EXHST_MSS_CURSTT.rename(columns=cdict)

In [635]:
STD_BD_DAT_GRD4_EXHST_MSS_CURSTT.columns

Index(['YR', 'CTPV', 'SGG', 'FUEL_CD', 'VHCTY_CD', 'VHCTY_TY', 'PURPS_CD2',
       'CO_EXHST_MSS_SM', 'HC_EXHST_MSS_SM', 'NOx_EXHST_MSS_SM',
       'PM10_EXHST_MSS_SM', 'PM2_5_EXHST_MSS_SM', 'LOAD_DT'],
      dtype='object')

In [636]:
STD_BD_DAT_GRD4_EXHST_MSS_CURSTT[['CO_EXHST_MSS_SM', 'HC_EXHST_MSS_SM', 'NOx_EXHST_MSS_SM', 'PM10_EXHST_MSS_SM', 'PM2_5_EXHST_MSS_SM']].sum()

CO_EXHST_MSS_SM       1.950190e+07
HC_EXHST_MSS_SM       2.661797e+06
NOx_EXHST_MSS_SM      2.784552e+07
PM10_EXHST_MSS_SM     3.099679e+05
PM2_5_EXHST_MSS_SM    2.845254e+05
dtype: float64

### [출력] STD_BD_DAT_GRD4_EXHST_MSS_CURSTT

In [637]:
# expdf = STD_BD_DAT_GRD4_EXHST_MSS_CURSTT
# table_nm = 'STD_BD_DAT_GRD4_EXHST_MSS_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 30.7s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [638]:
df2.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '차량관리번호', '자동차등록번호', '검사유효일', '소유자구분', '연료',
       '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '차종분류', '총중량', '적재중량', '배기량',
       '엔진출력', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1', '시도', '시군구', '시도코드', '시군구코드',
       '법정동코드_mod', '기간차이', '일일평균주행거리', 'DPF_YN', '차령', '시군구_수정', 'VKT',
       'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'R_CO', 'R_HC', 'R_PM', 'EFi_CO',
       'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5', 'E_HOT_CO', 'E_HOT_HC',
       'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5', 'Beta(Estimated)',
       'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM',
       'E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5',
       'e_d', 'e_RHOT', 'e_SHOT', 'e_SWARM', 'e_fi', 'e_RWARM', 'R', 'S_fi',
       'S_c', 'E_EVA_VOC', 'E_CO_total', 'E_HC_total', 'E_NOx_total',
       'E_PM10_

In [639]:
# !!! 수정 시작(2023.11.03)

In [640]:
dgl.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '차량관리번호', '자동차등록번호', '검사유효일', '소유자구분', '연료',
       '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '차종분류', '총중량', '적재중량', '배기량',
       '엔진출력', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1', '시도', '시군구', '시도코드', '시군구코드',
       '법정동코드_mod', '기간차이', '일일평균주행거리', 'DPF_YN', '차령', '시군구_수정'],
      dtype='object')

In [641]:
dgl['연료'].value_counts(dropna=False)

연료
경유             991313
휘발유            303261
LPG(액화석유가스)     23550
Name: count, dtype: int64

In [642]:
## 12.1s
diesel = dgl[dgl['연료'] == '경유'].reset_index(drop=True)
gas_lpg = dgl[(dgl['연료'] == '휘발유') | (dgl['연료'] == 'LPG(액화석유가스)')].reset_index(drop=True)
diesel.shape, gas_lpg.shape

((991313, 46), (326811, 46))

In [643]:
## 10.5s
grp_diesel = diesel.groupby(['시도', '시군구_수정', '연료', '차종', '차종유형', '용도', '검사방법']).agg({'무부하매연측정치1':'mean', '차대번호':'count'}).reset_index() # !!! 수정(2023.11.15)
grp_diesel = grp_diesel.rename(columns={'무부하매연측정치1':'매연(평균)', '차대번호':'차량대수'}) # !!! 수정(2023.11.15)
grp_diesel

,시도,시군구_수정,연료,차종,차종유형,용도,검사방법,매연(평균),차량대수
0,강원특별자치도,강릉시,경유,승용,대형,개인용,무부하검사(급가속),6.881057,227
1,강원특별자치도,강릉시,경유,승용,대형,개인용,부하검사(KD-147),6.722222,90
2,강원특별자치도,강릉시,경유,승용,소형,개인용,무부하검사(급가속),13.666667,9
3,강원특별자치도,강릉시,경유,승용,소형,개인용,부하검사(KD-147),15.391304,25
4,강원특별자치도,강릉시,경유,승용,중형,개인용,무부하검사(급가속),4.247292,554
...,...,...,...,...,...,...,...,...,...
7528,충청북도,충주시,경유,화물,중형,개인용,부하검사(KD-147),7.000000,12
7529,충청북도,충주시,경유,화물,중형,개인용,부하검사(LUG DOWN),16.125000,8
7530,충청북도,충주시,경유,화물,중형,관용,무부하검사(급가속),0.000000,1
7531,충청북도,충주시,경유,화물,중형,영업용,무부하검사(급가속),5.111111,18


In [644]:
## 10.2s
grp_gas_lpg = gas_lpg.groupby(['시도', '시군구_수정', '연료', '차종', '차종유형', '용도', '검사방법']).agg({'무부하매연측정치1':'mean', '무부하매연측정치2':'mean', '무부하매연측정치3':'mean', '차대번호':'count'}).reset_index() # !!! 수정(2023.11.15)
grp_gas_lpg = grp_gas_lpg.rename(columns={'무부하매연측정치1':'CO(평균)', '무부하매연측정치2':'HC(평균)', '무부하매연측정치3':'NOx(평균)', '차대번호':'차량대수'}) # !!! 수정(2023.11.15)
grp_gas_lpg

,시도,시군구_수정,연료,차종,차종유형,용도,검사방법,CO(평균),HC(평균),NOx(평균),차량대수
0,강원특별자치도,강릉시,LPG(액화석유가스),승용,중형,개인용,무부하검사(TSI),0.000000,334.000000,1.080000,1
1,강원특별자치도,강릉시,LPG(액화석유가스),승합,중형,개인용,무부하검사(정지가동),0.000000,0.000000,1.010000,1
2,강원특별자치도,강릉시,LPG(액화석유가스),화물,경형,개인용,부하검사(ASM-Idling),0.440000,126.000000,1315.000000,1
3,강원특별자치도,강릉시,LPG(액화석유가스),화물,소형,개인용,부하검사(ASM2525),0.000000,33.000000,272.000000,3
4,강원특별자치도,강릉시,LPG(액화석유가스),화물,중형,개인용,무부하검사(정지가동),3.460000,159.000000,0.910000,1
...,...,...,...,...,...,...,...,...,...,...,...
5161,충청북도,충주시,휘발유,승용,중형,개인용,부하검사(ASM-Idling),0.450741,49.407407,390.296296,27
5162,충청북도,충주시,휘발유,승용,중형,개인용,부하검사(ASM2525),1.364706,123.647059,453.882353,17
5163,충청북도,충주시,휘발유,승합,중형,개인용,부하검사(ASM2525),0.000000,8.000000,53.000000,1
5164,충청북도,충주시,휘발유,화물,소형,개인용,부하검사(ASM-Idling),2.040000,124.000000,473.000000,1


In [645]:
grp3 = pd.concat([grp_diesel, grp_gas_lpg], ignore_index=False)
grp3

,시도,시군구_수정,연료,차종,차종유형,용도,검사방법,매연(평균),차량대수,CO(평균),HC(평균),NOx(평균)
0,강원특별자치도,강릉시,경유,승용,대형,개인용,무부하검사(급가속),6.881057,227,NaN,NaN,NaN
1,강원특별자치도,강릉시,경유,승용,대형,개인용,부하검사(KD-147),6.722222,90,NaN,NaN,NaN
2,강원특별자치도,강릉시,경유,승용,소형,개인용,무부하검사(급가속),13.666667,9,NaN,NaN,NaN
3,강원특별자치도,강릉시,경유,승용,소형,개인용,부하검사(KD-147),15.391304,25,NaN,NaN,NaN
4,강원특별자치도,강릉시,경유,승용,중형,개인용,무부하검사(급가속),4.247292,554,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5161,충청북도,충주시,휘발유,승용,중형,개인용,부하검사(ASM-Idling),NaN,27,0.450741,49.407407,390.296296
5162,충청북도,충주시,휘발유,승용,중형,개인용,부하검사(ASM2525),NaN,17,1.364706,123.647059,453.882353
5163,충청북도,충주시,휘발유,승합,중형,개인용,부하검사(ASM2525),NaN,1,0.000000,8.000000,53.000000
5164,충청북도,충주시,휘발유,화물,소형,개인용,부하검사(ASM-Idling),NaN,1,2.040000,124.000000,473.000000


In [646]:
# grp3 = df2.groupby(['시도', '시군구_수정', '연료', '차종', '차종유형', '용도']).agg({'E_CO_total':'mean', 'E_HC_total':'mean', 'E_NOx_total':'mean', 'E_PM10_total':'mean', 'E_PM2_5_total':'mean'}).reset_index()
# grp3 = grp3.rename(columns={'E_CO_total':'E_CO_total_mean', 'E_HC_total':'E_HC_total_mean', 'E_NOx_total':'E_NOx_total_mean', 'E_PM10_total':'E_PM10_total_mean', 'E_PM2_5_total':'E_PM2_5_total_mean'})
# grp3

In [647]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20240213'

In [648]:
# 연도 설정
grp3['연도'] = today_date[:4]
# grp3['연도'] = '2022'
grp3['테이블생성일자'] = today_date

In [649]:
grp3.head()

,시도,시군구_수정,연료,차종,차종유형,용도,검사방법,매연(평균),차량대수,CO(평균),HC(평균),NOx(평균),연도,테이블생성일자
0,강원특별자치도,강릉시,경유,승용,대형,개인용,무부하검사(급가속),6.881057,227,NaN,NaN,NaN,2024,20240213
1,강원특별자치도,강릉시,경유,승용,대형,개인용,부하검사(KD-147),6.722222,90,NaN,NaN,NaN,2024,20240213
2,강원특별자치도,강릉시,경유,승용,소형,개인용,무부하검사(급가속),13.666667,9,NaN,NaN,NaN,2024,20240213
3,강원특별자치도,강릉시,경유,승용,소형,개인용,부하검사(KD-147),15.391304,25,NaN,NaN,NaN,2024,20240213
4,강원특별자치도,강릉시,경유,승용,중형,개인용,무부하검사(급가속),4.247292,554,NaN,NaN,NaN,2024,20240213


In [650]:
grp3[['매연(평균)', 'CO(평균)', 'HC(평균)', 'NOx(평균)']] = grp3[['매연(평균)', 'CO(평균)', 'HC(평균)', 'NOx(평균)']].round(2) # !!! 수정(2023.12.06)
grp3.head()

,시도,시군구_수정,연료,차종,차종유형,용도,검사방법,매연(평균),차량대수,CO(평균),HC(평균),NOx(평균),연도,테이블생성일자
0,강원특별자치도,강릉시,경유,승용,대형,개인용,무부하검사(급가속),6.88,227,NaN,NaN,NaN,2024,20240213
1,강원특별자치도,강릉시,경유,승용,대형,개인용,부하검사(KD-147),6.72,90,NaN,NaN,NaN,2024,20240213
2,강원특별자치도,강릉시,경유,승용,소형,개인용,무부하검사(급가속),13.67,9,NaN,NaN,NaN,2024,20240213
3,강원특별자치도,강릉시,경유,승용,소형,개인용,부하검사(KD-147),15.39,25,NaN,NaN,NaN,2024,20240213
4,강원특별자치도,강릉시,경유,승용,중형,개인용,무부하검사(급가속),4.25,554,NaN,NaN,NaN,2024,20240213


In [651]:
list(grp3.columns)

['시도',
 '시군구_수정',
 '연료',
 '차종',
 '차종유형',
 '용도',
 '검사방법',
 '매연(평균)',
 '차량대수',
 'CO(평균)',
 'HC(평균)',
 'NOx(평균)',
 '연도',
 '테이블생성일자']

In [652]:
STD_BD_DAT_GRD4_MEVLU = grp3[[
    '연도',
    '시도',
    '시군구_수정',
    '연료',
    '차종',
    '차종유형',
    '용도',
    '검사방법',
    '매연(평균)',
    'CO(평균)',
    'HC(평균)',
    'NOx(평균)',
    '차량대수', # !!! 수정(2023.11.15)
    '테이블생성일자', 
]]

In [653]:
STD_BD_DAT_GRD4_MEVLU

,연도,시도,시군구_수정,연료,차종,차종유형,용도,검사방법,매연(평균),CO(평균),HC(평균),NOx(평균),차량대수,테이블생성일자
0,2024,강원특별자치도,강릉시,경유,승용,대형,개인용,무부하검사(급가속),6.88,NaN,NaN,NaN,227,20240213
1,2024,강원특별자치도,강릉시,경유,승용,대형,개인용,부하검사(KD-147),6.72,NaN,NaN,NaN,90,20240213
2,2024,강원특별자치도,강릉시,경유,승용,소형,개인용,무부하검사(급가속),13.67,NaN,NaN,NaN,9,20240213
3,2024,강원특별자치도,강릉시,경유,승용,소형,개인용,부하검사(KD-147),15.39,NaN,NaN,NaN,25,20240213
4,2024,강원특별자치도,강릉시,경유,승용,중형,개인용,무부하검사(급가속),4.25,NaN,NaN,NaN,554,20240213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5161,2024,충청북도,충주시,휘발유,승용,중형,개인용,부하검사(ASM-Idling),NaN,0.45,49.41,390.30,27,20240213
5162,2024,충청북도,충주시,휘발유,승용,중형,개인용,부하검사(ASM2525),NaN,1.36,123.65,453.88,17,20240213
5163,2024,충청북도,충주시,휘발유,승합,중형,개인용,부하검사(ASM2525),NaN,0.00,8.00,53.00,1,20240213
5164,2024,충청북도,충주시,휘발유,화물,소형,개인용,부하검사(ASM-Idling),NaN,2.04,124.00,473.00,1,20240213


In [654]:
cdict = {
    '연도':'YR',
    '시도':'CTPV',
    '시군구_수정':'SGG',
    '연료':'FUEL_CD',
    '차종':'VHCTY_CD',
    '차종유형':'VHCTY_TY',
    '용도':'PURPS_CD2',
    '검사방법':'INSP_MTHD',
    '매연(평균)':'SMO_MEVLU_AVRG',
    'CO(평균)':'CO_MEVLU_AVRG',
    'HC(평균)':'HC_MEVLU_AVRG',
    'NOx(평균)':'NOx_MEVLU_AVRG',
    '차량대수':'VHCL_MKCNT', # !!! 수정(2023.11.15)
    '테이블생성일자':'LOAD_DT',
}
STD_BD_DAT_GRD4_MEVLU = STD_BD_DAT_GRD4_MEVLU.rename(columns=cdict)
STD_BD_DAT_GRD4_MEVLU.columns

Index(['YR', 'CTPV', 'SGG', 'FUEL_CD', 'VHCTY_CD', 'VHCTY_TY', 'PURPS_CD2',
       'INSP_MTHD', 'SMO_MEVLU_AVRG', 'CO_MEVLU_AVRG', 'HC_MEVLU_AVRG',
       'NOx_MEVLU_AVRG', 'VHCL_MKCNT', 'LOAD_DT'],
      dtype='object')

In [655]:
# !!! 수정 끝(2023.11.03)

### [출력] STD_BD_DAT_GRD4_MEVLU

In [656]:
# expdf = STD_BD_DAT_GRD4_MEVLU
# table_nm = 'STD_BD_DAT_GRD4_MEVLU'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 30.7s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

## 1-3 code end ##################################################################

## ---------- 3-1 start ----------

`3-1 부분에서 정기정밀 병합 데이터의 type 변경을 확인, 적용`

In [657]:
## 2023.02.13 수정 (데이터 테이블 변경)
## 5m 39.6s
## 전체 등록정보(VSYSZ.STD_CEG_GRD_LST)
# exasol db
car = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_MNG_NO FROM VSYSZ.STD_CEG_GRD_LST;")
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_MNG_NO':'차량관리번호'
}
carr = car.rename(columns=car_ch_col)

In [658]:
## 2m 46.0s
## 중복 차대번호 제거
carr['최초등록일자'] = pd.to_numeric(carr['최초등록일자'], errors='coerce')
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)

In [659]:
## 11.6s
carr['배출가스등급'].unique()

array(['A0505', 'A0504', 'A0501', 'A0503', 'A0502', '9999'], dtype=object)

In [660]:
## 24.4s
## 배출가스등급 코드 변환
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
carr['배출가스등급'] = carr['배출가스등급'].replace(grd_dict)
carr['배출가스등급'].unique()

array(['5', '4', '1', '3', '2', '9999'], dtype=object)

In [661]:
## 11.3s
carr['차종'].unique()

array(['A31T', 'A31P', 'A31V', 'A31S'], dtype=object)

In [662]:
## 19.2s
## 차종 코드 변환
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
carr['차종'].unique()

array(['화물', '승용', '승합', '특수'], dtype=object)

In [663]:
carr['용도'].unique()

array(['A08P', 'A08B', 'A08O'], dtype=object)

In [664]:
## 16.9s
## 용도 코드 변환
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
carr['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

### ----- 2024.02.05 (테이블 변경 : STD_CEG_GRD_LST => VSYSZ.STD_CEG_GRD_LST)

In [665]:
### ----- 2024.02.05 (테이블 변경 : STD_CEG_GRD_LST => VSYSZ.STD_CEG_GRD_LST)
## 10m 8.4s
cs_raw = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO, VHRNO, EXTGAS_INSP_VLD_YMD, VHCL_OWNR_CL_CD , FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY, VHCTY_CL_CD, TOTL_WGHT, CRYNG_WGHT, DSPLVL, EGIN_OTPT FROM VSYSZ.STD_CEG_GRD_LST;")
cs_raw_dict = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자', 
    'VHCL_MNG_NO':'차량관리번호', 
    'VHRNO':'자동차등록번호',
    'EXTGAS_INSP_VLD_YMD':'검사유효일',
    'VHCL_OWNR_CL_CD':'소유자구분',
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
    'VHCTY_CL_CD':'차종분류',
    'TOTL_WGHT':'총중량',
    'CRYNG_WGHT':'적재중량',
    'DSPLVL':'배기량', 
    'EGIN_OTPT':'엔진출력',
}
cs = cs_raw.rename(columns=cs_raw_dict)
print(cs.columns)
print(cs.shape)
print('data load : VSYSZ.STD_CEG_GRD_LST')

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '차량관리번호', '자동차등록번호', '검사유효일', '소유자구분', '연료',
       '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '차종분류', '총중량', '적재중량', '배기량',
       '엔진출력'],
      dtype='object')
(31498871, 26)
data load : VSYSZ.STD_CEG_GRD_LST


In [666]:
cs.shape

(31498871, 26)

In [667]:
### ----- 2024.02.05 수정3
cs['차량말소YN'].unique()

array(['Y', 'N'], dtype=object)

In [668]:
cs['배출가스등급'].unique()

array(['A0505', 'A0504', 'A0501', 'A0503', 'A0502', '9999'], dtype=object)

In [669]:
## 37.6s
## 배출가스등급 코드 변환
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
cs['배출가스등급'] = cs['배출가스등급'].replace(grd_dict)
cs['배출가스등급'].unique()

array(['5', '4', '1', '3', '2', '9999'], dtype=object)

In [670]:
cs['차종'].unique()

array(['A31T', 'A31P', 'A31V', 'A31S'], dtype=object)

In [671]:
## 24.4s
## 차종 코드 변환
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
cs['차종'] = cs['차종'].replace(cd_dict)
cs['차종'].unique()

array(['화물', '승용', '승합', '특수'], dtype=object)

In [672]:
cs['용도'].unique()

array(['A08P', 'A08B', 'A08O'], dtype=object)

In [673]:
## 19.8s
## 용도 코드 변환
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
cs['용도'] = cs['용도'].replace(purps_dict)
cs['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

In [674]:
cs['소유자구분'].unique()

array(['A27R', 'A27L', 'A27B', 'A27O', 'A27F'], dtype=object)

In [675]:
## 23.7s
## 소유자구분 코드 변환
ownr_dict = {
    'A27B':'사업자', 
    'A27F':'외국인', 
    'A27L':'법인', 
    'A27O':'기타', 
    'A27R':'주민', 
}
cs['소유자구분'] = cs['소유자구분'].replace(ownr_dict)
cs['소유자구분'].unique()

array(['주민', '법인', '사업자', '기타', '외국인'], dtype=object)

In [676]:
cs['연료'].unique()

array(['A91DS', 'A92LP', 'A90GS', 'A92CN', 'A90GH', 'A93EV', '9999',
       'A92LH', 'A94OT', 'A91DH', 'A93HD', 'A92CH', 'A90AC', 'A91KS',
       'A93SH', 'A92LN'], dtype=object)

In [677]:
## 22.4s
## 연료 코드 변환
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
cs['연료'] = cs['연료'].replace(fuel_dict)
cs['연료'].unique()

array(['경유', 'LPG(액화석유가스)', '휘발유', 'CNG(압축천연가스)', '휘발유 하이브리드', '전기',
       '9999', 'LPG 하이브리드', '기타연료', '경유 하이브리드', '수소', 'CNG 하이브리드', '알코올',
       '등유', '태양열', 'LNG(액화천연가스)'], dtype=object)

In [678]:
cs['차종유형'].unique()

array(['A30S', 'A30L', 'A30M', 'A30C', '9999'], dtype=object)

In [679]:
## 13.8s
## 차종유형 코드 변환
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
cs['차종유형'] = cs['차종유형'].replace(ty_dict)
cs['차종유형'].unique()

array(['소형', '대형', '중형', '경형', '9999'], dtype=object)

In [680]:
carr['차량말소YN'].unique()

array(['Y', 'N'], dtype=object)

### ----- 2024.02.05 수정 (등록정보 말소 제거 조건 변경)

`carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)` => `carm = carr[carr['차량말소YN'] == 'Y'].reset_index(drop=True)`

In [681]:
### ----- 2024.02.05 수정3 (등록정보 말소 제거 변경)
# `carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)` => `carm = carr[carr['차량말소YN'] == 'Y'].reset_index(drop=True)`

## 12.4s
## 등록정보 말소 제거 : 말소된 차량 정보만 도출
carm = carr[carr['차량말소YN'] == 'Y'].reset_index(drop=True)

In [682]:
carm['차량말소YN'].unique()

array(['Y'], dtype=object)

In [683]:
## 48.7s
## 등록&제원 병합
cs = carm.merge(srcr, on='제원관리번호', how='left')
cs.shape

(25526563, 22)

### ----- 2024.02.05 수정 (등록정보 말소 제거 조건 변경)

`csm = cs[cs['차량말소YN'] == 'N'].reset_index(drop=True)` => `csm = cs[cs['차량말소YN'] == 'Y'].reset_index(drop=True)`

In [684]:
### ----- 2024.02.05 수정3 (등록정보 말소 제거 변경)
# `csm = cs[cs['차량말소YN'] == 'N'].reset_index(drop=True)` => `csm = cs[cs['차량말소YN'] == 'Y'].reset_index(drop=True)`

## 30.8s
## 등록정보 말소 제거 : 말소된 차량 정보만 도출
csm = cs[cs['차량말소YN'] == 'Y'].reset_index(drop=True)

In [685]:
csm.shape

(25526563, 22)

In [686]:
cs['차량말소YN'].unique()

array(['Y'], dtype=object)

In [687]:
## 59.8s
csm['법정동코드'] = csm['법정동코드'].astype('str')
csm['법정동코드'] = csm['법정동코드'].str[:5] + '00000'
csm['법정동코드'] = pd.to_numeric(csm['법정동코드'])

In [688]:
## 24.5s
## 등록&제원&법정동코드 병합(df)
df = csm.merge(coder, on='법정동코드', how='left')

In [689]:
## 56.7s
cs['법정동코드'] = cs['법정동코드'].astype('str')
cs['법정동코드'] = cs['법정동코드'].str[:5] + '00000'
cs['법정동코드'] = pd.to_numeric(cs['법정동코드'])

In [690]:
## 19.3s
## 등록&제원&법정동코드 병합(df)
df = cs.merge(coder, on='법정동코드', how='left')

In [691]:
df['시도'].isnull().sum()

0

In [692]:
# ### 매칭 안되는 지역 처리
# # 주소 수정
# df.loc[df['법정동코드'] == 5172035031, ['시도', '시군구']] = ['강원특별자치도', '홍천군'] # 강원특별자치도 홍천군
# df.loc[df['법정동코드'] == 5180031023, ['시도', '시군구']] = ['강원특별자치도', '양구군'] # 
# df.loc[df['법정동코드'] == 5180031031, ['시도', '시군구']] = ['강원특별자치도', '양구군'] # 
# df.loc[df['법정동코드'] == 5172035030, ['시도', '시군구']] = ['강원특별자치도', '홍천군'] # 
# df.loc[df['법정동코드'] == 5180031028, ['시도', '시군구']] = ['강원특별자치도', '양구군'] # 

# df.loc[df['법정동코드'] == 5172035021, ['시도', '시군구']] = ['강원특별자치도', '홍천군'] # 
# df.loc[df['법정동코드'] == 5180031025, ['시도', '시군구']] = ['강원특별자치도', '양구군'] # 
# df.loc[df['법정동코드'] == 4165052000, ['시도', '시군구']] = ['경기도', '포천시'] # 경기도 포천시 선단동
# df.loc[df['법정동코드'] == 5172035023, ['시도', '시군구']] = ['강원특별자치도', '홍천군'] # 
# df.loc[df['법정동코드'] == 5180031027, ['시도', '시군구']] = ['강원특별자치도', '양구군'] # 

# df.loc[df['법정동코드'] == 5172035024, ['시도', '시군구']] = ['강원특별자치도', '홍천군'] # 
# df.loc[df['법정동코드'] == 5175037022, ['시도', '시군구']] = ['강원특별자치도', '영월군'] # 
# df.loc[df['법정동코드'] == 4163055000, ['시도', '시군구']] = ['경기도', '양주시'] # 경기도 양주시 회천3동
# df.loc[df['법정동코드'] == 5180031033, ['시도', '시군구']] = ['강원특별자치도', '양구군'] # 경기도 양주시 회천3동

In [693]:
# rdf = df.copy()
# rdf['법정동코드'] = rdf['법정동코드'].astype('str')
# rdf.loc[rdf['시도'].isnull() == True, '법정동코드'] = rdf.loc[rdf['시도'].isnull() == True, '법정동코드'].str[:5] + '00000'
# rdf['법정동코드'] = pd.to_numeric(rdf['법정동코드'])

# rdfy = rdf[rdf['시도'].isnull() == False]
# rdfn = rdf[rdf['시도'].isnull() == True]

# rdfn = rdfn.drop(['시도', '시군구'], axis=1)
# rdfnm = rdfn.merge(coder, on='법정동코드', how='left')

# df = pd.concat([rdfy, rdfnm], ignore_index=False)

In [694]:
## 3m 26.5s
## 등록(말소 유지) & 제원 병합
cse = carr.merge(srcr, on='제원관리번호', how='left')

### ----- 2024.02.05 수정 (등록말소 & 제원 병합 셀 주석처리)

`등록(말소 유지) & 제원 병합`이 겹침

In [695]:
### ----- 2024.02.05 수정 (등록말소 & 제원 병합 셀 주석처리)
# `등록(말소 유지) & 제원 병합`이 겹침

# ## 27.0s
# ## 등록(말소 유지) & 제원 병합
# cse = cs.copy()
# cse.shape

In [696]:
cs.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '차량관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식',
       '차종분류', '총중량', '적재중량', '배기량', '엔진출력'],
      dtype='object')

In [697]:
cs['차량말소YN'].unique()

array(['Y'], dtype=object)

In [698]:
### ----- 2024.02.05 수정3
hisr['차량말소YN'].unique()

array(['N', 'Y'], dtype=object)

In [699]:
### ----- 2024.02.05 수정3
cse['차량말소YN'].unique

<bound method Series.unique of 0           Y
1           N
2           N
3           N
4           N
           ..
80982390    N
80982391    N
80982392    N
80982393    N
80982394    N
Name: 차량말소YN, Length: 80982395, dtype: object>

`여기서 끊기네` (Kernel crashed)

In [700]:
## 2m 32s
## 등록&제원&이력 병합
ersr = cse.merge(hisr, on='차량관리번호', how='left')

MemoryError: Unable to allocate 5.49 GiB for an array with shape (4, 184113846) and data type int64

In [ ]:
cse.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '차량관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식',
       '차종분류', '총중량', '적재중량', '배기량', '엔진출력'],
      dtype='object')

In [ ]:
cse['차량말소YN'].unique()

array(['N', 'Y'], dtype=object)

In [ ]:
## 47.3s
### ----- 2024.02.05 수정3
ersr

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN_x,제원관리번호,차량연식,차종,용도,최초등록일자,...,차명,자동차형식,엔진형식,차종분류,총중량,적재중량,배기량,엔진출력,차량말소YN_y,변경일자
0,W1KEG2BB5PF010533,2820010100,1,NaN,N,00820008600011222,2023,승용,개인용,20231229.0,...,Mercedes-Benz EQE350 +,295121,EM0027,A2826,2680.0,0.0,271.0,215.0,NaN,NaN
1,WAUZZZF12PD050674,4420037031,3,MMY-AD-14-15,N,02420007300681322,2023,승용,개인용,20231229.0,...,Q8 50 TDI quattro,F1,DPX,A2819,2685.0,0.0,2967.0,286.0,NaN,NaN
2,LYVUZ0621RB786732,4812312700,1,PMY-PA-12-12,N,00920004000241223,2024,승용,개인용,20231229.0,...,볼보 XC60B6 AWD,UZ06,B420T,A2819,2290.0,0.0,1969.0,300.0,NaN,NaN
3,WAUZZZF1XPD047683,2820010100,3,MMY-AD-14-15,N,02420007300671322,2023,승용,개인용,20231229.0,...,Q8 45 TDI quattro,F1,DPX,A2819,2630.0,0.0,2967.0,231.0,NaN,NaN
4,VF3MRHNSCPS071152,3017011600,NaN,NaN,N,00220009500051223,2023,승용,개인용,20231229.0,...,Peugeot 5008 1.2 Puretech,MRHNSC,HN05,A2819,2045.0,0.0,1199.0,131.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182981285,10781114373EOGUS,2714012400,5,NaN,N,10004400070000,1983,승합,개인용,NaN,...,현대버스,HD160-91000,HDT 6 3544,A2826,10445.0,0.0,5800.0,155.0,NaN,NaN
182981286,03751356719EOWL,2711013900,5,NaN,N,10002200030002,1975,승용,개인용,NaN,...,레코드로얄,OVP69,1.9S,A2826,1436.0,0.0,1897.0,102.0,NaN,NaN
182981287,02781160480EOQN,2717010600,5,NaN,N,30220100000533,1978,승합,개인용,NaN,...,지프페밀리,CJ-5MC,232CID,A2826,2210.0,0.0,2775.0,100.0,NaN,NaN
182981288,KLAER1981PB156096,4122025000,NaN,NaN,N,99934103001471203,1993,승용,개인용,NaN,...,DAEWOO PRINCE,ER198,C18LE,A2826,1515.0,0.0,1796.0,110.0,NaN,NaN


### ----- 2024.02.05 수정3 (차량말소YN 조건 변경)

`ersr['차량말소YN_x'] == 'N'` => `ersr['차량말소YN_x'] == 'Y'`

In [ ]:
ersr['차량말소YN_x'].unique()

array(['N', 'Y'], dtype=object)

In [ ]:
### ----- 2024.02.05 수정3 (차량말소YN 조건 변경)
# `ersr['차량말소YN_x'] == 'N'` => `ersr['차량말소YN_x'] == 'Y'`

## 16.2s
# 1. 등록 차량말소와 등록이력 차량 말소 둘 모두 해당되는 데이터 추출
# ====================
# 원본 코드
# ersr = ersr.loc[(c) & (ersr['차량말소YN_y'] == 'N')].reset_index(drop=True)
# ersr.shape
# ====================
ersr = ersr.loc[(ersr['차량말소YN_x'] == 'Y') & (ersr['차량말소YN_y'] == 'Y')].reset_index(drop=True)
ersr.shape

(21768928, 24)

In [ ]:
## 14.8s
# 2. 변경일자 기준 최신 데이터만 남기고 차대번호 중복 제거
ersr = ersr.sort_values('변경일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)

In [ ]:
# 3. 변경일자 2019.01.01 이후만 추출
ersr = ersr[ersr['변경일자'] >= 20190101].reset_index(drop=True)

In [ ]:
## 11.1s
ersr['법정동코드'] = ersr['법정동코드'].astype('str')
ersr['법정동코드'] = ersr['법정동코드'].str[:5] + '00000'
ersr['법정동코드'] = pd.to_numeric(ersr['법정동코드'])

In [ ]:
## 28.7s
ersr

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN_x,제원관리번호,차량연식,차종,용도,최초등록일자,...,차명,자동차형식,엔진형식,차종분류,총중량,적재중량,배기량,엔진출력,차량말소YN_y,변경일자
0,WBA51FL01RCP08560,2635000000,NaN,NaN,Y,00120009300081223,2024,승용,개인용,20231031.0,...,BMW 523d xDrive,51FL,B47D20B,A2826,2290.0,-9999.0,1995.0,197.0,Y,20231101.0
1,KMJTA18FPEC018092,4413300000,2,DMY-HD-13-30,Y,A0810005918012314,2014,승합,영업용,20140917.0,...,뉴슈퍼에어로시티,HCY5421-14CFM-SRCS,C6AF,A2826,14600.0,-9999.0,11670.0,290.0,Y,20231101.0
2,KMHFB41BP6A055029,1168000000,3,5MY-HD-24-09,Y,A0810006600111305,2006,승용,개인용,20051130.0,...,그랜저(GRANDEUR),TG-27GL-A2,G6EA,A2826,1945.0,0.0,2656.0,192.0,Y,20231101.0
3,KNFSTZ76AGK098819,1126000000,3,BMY-KM-14-59,Y,A0110003128293115,2016,화물,영업용,20160718.0,...,봉고Ⅲ 냉동차,PUL6AJ-E-RTSS,D4CB,A2814,3210.0,1000.0,2497.0,133.0,Y,20231101.0
4,KNANE81BBMS040400,2723000000,3,JMY-KM-14-54,Y,A0110007100031320,2021,승용,개인용,20201203.0,...,카니발 리무진,KA47BBA-E-9DO-H1N,D4HE,A2821,2550.0,-9999.0,2151.0,202.0,Y,20231101.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4018004,KMFFE17APVU320407,4776000000,NaN,NaN,Y,10015500680003,1997,화물,개인용,19970203.0,...,포터더블캡,HD100-22,D4BA,A2809,2820.0,1000.0,2476.0,80.0,Y,20190125.0
4018005,KN3HAP53A1K738305,2917000000,NaN,NaN,Y,10080700440000,2001,화물,개인용,20010222.0,...,봉고프런티어,SDE22B,JT,A2809,2980.0,1000.0,2957.0,90.0,Y,20190116.0
4018006,KMJWWH7BPYU243657,2920000000,NaN,NaN,Y,10080400670001,2000,승합,개인용,20000527.0,...,스타렉스장축12인승,HA12LP-1,D4BB,A2826,2670.0,0.0,2607.0,83.0,Y,20190116.0
4018007,KNFWF0A429K351723,4682000000,NaN,NaN,Y,A0110003301493108,2009,화물,개인용,20080930.0,...,봉고Ⅲ 냉동차,WFK12K-RTSS,J3,A2814,3355.0,900.0,2902.0,133.0,Y,20190114.0


In [ ]:
## 16.9s
## 등록&제원&이력&법정동코드 병합(errc)
# 매칭 후 '시도' 빈값 없음
errc = ersr.merge(coder, on='법정동코드', how='left')

In [ ]:
errc.shape

(4018009, 28)

In [ ]:
errc['시도'].isnull().sum()

1

In [ ]:
errc['시도'].unique()

array(['부산광역시', '충청남도', '서울특별시', '대구광역시', '인천광역시', '경기도', '강원특별자치도',
       '전라남도', '전라북도', '충청북도', '대전광역시', '광주광역시', '경상남도', '세종특별자치시',
       '울산광역시', '경상북도', '제주특별자치도', nan], dtype=object)

In [ ]:
df['시도'].unique()

array(['부산광역시', '인천광역시', '서울특별시', '경기도', '광주광역시', '충청북도', '대구광역시',
       '대전광역시', nan, '강원특별자치도', '충청남도', '경상남도', '전라남도', '제주특별자치도', '경상북도',
       '전라북도', '울산광역시', '세종특별자치시'], dtype=object)

In [ ]:
## 13.0s
errc['시도'] = errc['시도'].replace({'강원도':'강원특별자치도'})
errc['시도'].unique()

array(['부산광역시', '충청남도', '서울특별시', '대구광역시', '인천광역시', '경기도', '강원특별자치도',
       '전라남도', '전라북도', '충청북도', '대전광역시', '광주광역시', '경상남도', '세종특별자치시',
       '울산광역시', '경상북도', '제주특별자치도', nan], dtype=object)

In [ ]:
df['연료'].unique()

array(['휘발유 하이브리드', '휘발유', '경유', '경유 하이브리드', '전기', '9999', 'LPG(액화석유가스)',
       '수소', '기타연료', '알코올', 'LPG 하이브리드', 'CNG(압축천연가스)', nan, '태양열',
       'LNG(액화천연가스)', 'CNG 하이브리드'], dtype=object)

In [ ]:
df['연료'].value_counts(dropna=False)

연료
휘발유            22129483
경유              2376516
LPG(액화석유가스)      604249
휘발유 하이브리드         38361
전기                20949
9999              19027
CNG(압축천연가스)       13616
LPG 하이브리드          4693
경유 하이브리드           3441
기타연료               3223
수소                  243
CNG 하이브리드            45
태양열                   2
알코올                   1
NaN                   1
LNG(액화천연가스)           1
Name: count, dtype: int64

In [ ]:
df['연료'].isnull().sum()

1

In [ ]:
## 19.4s
## 분석
## 연료 하이브리드 수정
df.loc[(df['연료'] == '휘발유') | (df['연료'] == '휘발유 하이브리드'), 'fuel'] = '휘발유'
df.loc[(df['연료'] == '경유') | (df['연료'] == '경유 하이브리드'), 'fuel'] = '경유'
df.loc[(df['연료'] == 'LPG(액화석유가스)') | (df['연료'] == 'LPG 하이브리드'), 'fuel'] = 'LPG'
df.loc[(df['연료'] == 'CNG(압축천연가스)') | (df['연료'] == 'CNG 하이브리드'), 'fuel'] = 'CNG'
df.loc[(df['연료'] == 'LNG(액화천연가스)') | (df['연료'] == 'LNG 하이브리드'), 'fuel'] = 'LNG'
df.loc[df['연료'] == '전기', 'fuel'] = '전기'
df.loc[df['연료'] == '수소', 'fuel'] = '수소'
df.loc[df['연료'] == '태양열', 'fuel'] = '태양열'
df.loc[df['연료'] == '알코올', 'fuel'] = '알코올'
df.loc[df['연료'] == '등유', 'fuel'] = '등유'
df.loc[df['연료'] == '기타연료', 'fuel'] = '기타연료'
df.loc[df['연료'].isnull() == True, 'fuel'] = np.nan

In [ ]:
df['fuel'].value_counts(dropna=False)

fuel
휘발유     22167844
경유       2379957
LPG       608942
전기         20949
NaN        19028
CNG        13661
기타연료        3223
수소           243
태양열            2
알코올            1
LNG            1
Name: count, dtype: int64

In [ ]:
## 19.2s
df.loc[(df['연료'] == '휘발유') | (df['연료'] == '휘발유 하이브리드'), 'fuel2'] = '휘발유'
df.loc[(df['연료'] == '경유') | (df['연료'] == '경유 하이브리드'), 'fuel2'] = '경유'
df.loc[(df['연료'] == 'LPG(액화석유가스)') | (df['연료'] == 'LPG 하이브리드'), 'fuel2'] = 'LPG(액화석유가스)'
df.loc[(df['연료'] == 'CNG(압축천연가스)') | (df['연료'] == 'CNG 하이브리드'), 'fuel2'] = '기타'
df.loc[(df['연료'] == 'LNG(액화천연가스)') | (df['연료'] == 'LNG 하이브리드'), 'fuel2'] = '기타'
df.loc[(df['연료'] == '전기') | (df['연료'] == '수소'), 'fuel2'] = '전기수소'
df.loc[df['연료'] == '태양열', 'fuel2'] = '기타'
df.loc[df['연료'] == '알코올', 'fuel2'] = '기타'
df.loc[df['연료'] == '등유', 'fuel2'] = '기타'
df.loc[df['연료'] == '기타연료', 'fuel2'] = '기타'
df.loc[df['연료'].isnull() == True, 'fuel2'] = '기타'

In [ ]:
df['fuel2'].value_counts(dropna=False)

fuel2
휘발유            22167844
경유              2379957
LPG(액화석유가스)      608942
전기수소              21192
NaN               19027
기타                16889
Name: count, dtype: int64

In [ ]:
## 36.1s
errc.loc[(errc['연료'] == '휘발유') | (errc['연료'] == '휘발유 하이브리드'), 'fuel'] = '휘발유'
errc.loc[(errc['연료'] == '경유') | (errc['연료'] == '경유 하이브리드'), 'fuel'] = '경유'
errc.loc[(errc['연료'] == 'LPG(액화석유가스)') | (errc['연료'] == 'LPG 하이브리드'), 'fuel'] = 'LPG'
errc.loc[(errc['연료'] == 'CNG(압축천연가스)') | (errc['연료'] == 'CNG 하이브리드'), 'fuel'] = 'CNG'
errc.loc[(errc['연료'] == 'LNG(액화천연가스)') | (errc['연료'] == 'LNG 하이브리드'), 'fuel'] = 'LNG'
errc.loc[errc['연료'] == '전기', 'fuel'] = '전기'
errc.loc[errc['연료'] == '수소', 'fuel'] = '수소'
errc.loc[errc['연료'] == '태양열', 'fuel'] = '태양열'
errc.loc[errc['연료'] == '알코올', 'fuel'] = '알코올'
errc.loc[errc['연료'] == '등유', 'fuel'] = '등유'
errc.loc[errc['연료'] == '기타연료', 'fuel'] = '기타연료'
errc.loc[errc['연료'].isnull() == True, 'fuel'] = np.nan

In [ ]:
## 27.0s
### ----- 2024.02.05 수정3
errc

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN_x,제원관리번호,차량연식,차종,용도,최초등록일자,...,적재중량,배기량,엔진출력,차량말소YN_y,변경일자,시도,시군구,시도코드,시군구코드,fuel
0,WBA51FL01RCP08560,2635000000,NaN,NaN,Y,00120009300081223,2024,승용,개인용,20231031.0,...,-9999.0,1995.0,197.0,Y,20231101.0,부산광역시,해운대구,26.0,350.0,경유
1,KMJTA18FPEC018092,4413300000,2,DMY-HD-13-30,Y,A0810005918012314,2014,승합,영업용,20140917.0,...,-9999.0,11670.0,290.0,Y,20231101.0,충청남도,천안시,44.0,133.0,CNG
2,KMHFB41BP6A055029,1168000000,3,5MY-HD-24-09,Y,A0810006600111305,2006,승용,개인용,20051130.0,...,0.0,2656.0,192.0,Y,20231101.0,서울특별시,강남구,11.0,680.0,휘발유
3,KNFSTZ76AGK098819,1126000000,3,BMY-KM-14-59,Y,A0110003128293115,2016,화물,영업용,20160718.0,...,1000.0,2497.0,133.0,Y,20231101.0,서울특별시,중랑구,11.0,260.0,경유
4,KNANE81BBMS040400,2723000000,3,JMY-KM-14-54,Y,A0110007100031320,2021,승용,개인용,20201203.0,...,-9999.0,2151.0,202.0,Y,20231101.0,대구광역시,북구,27.0,230.0,경유
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4018004,KMFFE17APVU320407,4776000000,NaN,NaN,Y,10015500680003,1997,화물,개인용,19970203.0,...,1000.0,2476.0,80.0,Y,20190125.0,경상북도,영양군,47.0,760.0,경유
4018005,KN3HAP53A1K738305,2917000000,NaN,NaN,Y,10080700440000,2001,화물,개인용,20010222.0,...,1000.0,2957.0,90.0,Y,20190116.0,광주광역시,북구,29.0,170.0,경유
4018006,KMJWWH7BPYU243657,2920000000,NaN,NaN,Y,10080400670001,2000,승합,개인용,20000527.0,...,0.0,2607.0,83.0,Y,20190116.0,광주광역시,광산구,29.0,200.0,경유
4018007,KNFWF0A429K351723,4682000000,NaN,NaN,Y,A0110003301493108,2009,화물,개인용,20080930.0,...,900.0,2902.0,133.0,Y,20190114.0,전라남도,해남군,46.0,820.0,경유


In [ ]:
## 10.4s
errc['fuel'].value_counts(dropna=False)

fuel
경유      1794756
휘발유     1741661
LPG      447162
전기        16160
CNG        9878
NaN        6469
기타연료       1735
수소          184
태양열           2
알코올           1
LNG           1
Name: count, dtype: int64

In [ ]:
### 40.1s
errc.loc[(errc['연료'] == '휘발유') | (errc['연료'] == '휘발유 하이브리드'), 'fuel2'] = '휘발유'
errc.loc[(errc['연료'] == '경유') | (errc['연료'] == '경유 하이브리드'), 'fuel2'] = '경유'
errc.loc[(errc['연료'] == 'LPG(액화석유가스)') | (errc['연료'] == 'LPG 하이브리드'), 'fuel2'] = 'LPG(액화석유가스)'
errc.loc[(errc['연료'] == 'CNG(압축천연가스)') | (errc['연료'] == 'CNG 하이브리드'), 'fuel2'] = '기타'
errc.loc[(errc['연료'] == 'LNG(액화천연가스)') | (errc['연료'] == 'LNG 하이브리드'), 'fuel2'] = '기타'
errc.loc[(errc['연료'] == '전기') | (errc['연료'] == '수소'), 'fuel2'] = '전기수소'
errc.loc[errc['연료'] == '태양열', 'fuel2'] = '기타'
errc.loc[errc['연료'] == '알코올', 'fuel2'] = '기타'
errc.loc[errc['연료'] == '등유', 'fuel2'] = '기타'
errc.loc[errc['연료'] == '기타연료', 'fuel2'] = '기타'
errc.loc[errc['연료'].isnull() == True, 'fuel2'] = '기타'

In [ ]:
errc['fuel2'].value_counts(dropna=False)

fuel2
경유             1794756
휘발유            1741661
LPG(액화석유가스)     447162
전기수소             16344
기타               11617
NaN               6469
Name: count, dtype: int64

In [ ]:
## 12.9s
## 수도권 : 서울특별시, 인천광역시, 경기도
## 비수도권 : 수도권 외
df.loc[(df['시도'] == '서울특별시') | (df['시도'] == '인천광역시') | (df['시도'] == '경기도'), '지역'] = '수도권'
df['지역'] = df['지역'].fillna('비수도권')
df.shape

(25213851, 29)

In [ ]:
## 33.0s
dfm = df.copy()
dfm['최초등록일자'] = dfm['최초등록일자'].astype('str')
dfm['최초등록일자_년'] = dfm['최초등록일자'].str[:4]
dfm['최초등록일자_월'] = dfm['최초등록일자'].str[4:6]
dfm['최초등록일자_일'] = dfm['최초등록일자'].str[6:8]

In [ ]:
## 34.8s
## 시군구명 앞쪽 지역명만 남기기(dfm)
dfm['시군구_수정'] = dfm['시군구'].str.split(' ').str[0]

In [ ]:
## 53.0s
## 수도권 : 서울특별시, 인천광역시, 경기도
## 비수도권 : 수도권 외
errc.loc[(errc['시도'] == '서울특별시') | (errc['시도'] == '인천광역시') | (errc['시도'] == '경기도'), '지역'] = '수도권'
errc['지역'] = errc['지역'].fillna('비수도권')
errc['변경일자'] = errc['변경일자'].astype('str')
errc['변경일자_년'] = errc['변경일자'].str[:4]
errc['변경일자_월'] = errc['변경일자'].str[4:6]
errc['변경일자_일'] = errc['변경일자'].str[6:8]
errc['변경일자'] = errc['변경일자_년'] + errc['변경일자_월'] + errc['변경일자_일']

In [ ]:
## 1m 2.4s
## 시군구명 앞쪽 지역명만 남기기(errc)
errc['시군구_수정'] = errc['시군구'].str.split(' ').str[0]

In [ ]:
## 2m 41.5s
## 경유, 휘발유, LPG, 전기, 수소만 추출
dfm2 = dfm.loc[(dfm['fuel'] == '경유') | (dfm['fuel'] == '휘발유') | (dfm['fuel'] == 'LPG') | (dfm['fuel'] == '전기') | (dfm['fuel'] == '수소')].reset_index(drop=True)
errc2 = errc.loc[(errc['fuel'] == '경유') | (errc['fuel'] == '휘발유') | (errc['fuel'] == 'LPG') | (errc['fuel'] == '전기') | (errc['fuel'] == '수소')].reset_index(drop=True)

In [ ]:
dfm2.shape, errc2.shape

((25177935, 33), (3999923, 35))

In [ ]:
## 등급, 지역별 차량현황
## 연도 설정
today_date = datetime.today().strftime("%Y%m%d")
## year = 2022
year = int(today_date[:4])
year

2024

In [ ]:
## 13.1s
## 2022년 차량 대수
grp1 = dfm.groupby(['지역', '시도', '배출가스등급'], as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = f'{year}'
grp1 = grp1[['연도', '지역', '시도', '배출가스등급', '차량대수']]
grp1

,연도,지역,시도,배출가스등급,차량대수
0,2024,비수도권,강원특별자치도,1,789
1,2024,비수도권,강원특별자치도,2,9300
2,2024,비수도권,강원특별자치도,3,76168
3,2024,비수도권,강원특별자치도,4,4632
4,2024,비수도권,강원특별자치도,5,5041
...,...,...,...,...,...
80,2024,수도권,인천광역시,1,19015
81,2024,수도권,인천광역시,2,703114
82,2024,수도권,인천광역시,3,412412
83,2024,수도권,인천광역시,4,46643


In [ ]:
grp1['배출가스등급'].unique()

array(['1', '2', '3', '4', '5'], dtype=object)

In [ ]:
# sl = grp1[['지역', '시도', '배출가스등급']].drop_duplicates(['지역', '시도', '배출가스등급']).reset_index(drop=True)
# sl.head(10)

In [ ]:
# sl.shape

In [ ]:
# sl.shape[0] * 4

In [ ]:
# periods = 5 # !!! 수정(2023.08.23)
# y_plist = list(pd.date_range(end=today_date, periods=periods, freq="Y").year) # !!! 수정(2023.08.23)
# # mth_plist = list(pd.date_range(end=today_date, periods=periods, freq="MS").month) # !!! 수정(2023.08.23)
# y_plist = y_plist[:-1]
# # mth_plist = mth_plist[:-1]
# y_plist

In [ ]:
# slt = sl
# for _ in range(periods - 2):
#     slt = pd.concat([slt, sl], ignore_index=False)
# slt.shape

In [ ]:
# slt = slt.sort_values(['지역', '시도', '배출가스등급']).reset_index(drop=True)
# slt.head(20)

In [ ]:
# y_plist01 = [str(x) for x in y_plist]

In [ ]:
# ym = pd.DataFrame({'연도':y_plist01})
# ym

In [ ]:
# ymt = ym
# for _ in range(slt.shape[0]//ym.shape[0] - 1):
#     ymt = pd.concat([ymt, ym], ignore_index=False)
# ymt.shape

In [ ]:
# slt = slt.reset_index(drop=True)
# ymt = ymt.reset_index(drop=True)
# slt.shape, ymt.shape

In [ ]:
# base = pd.concat([ymt, slt], axis=1)
# base.head(20)

In [ ]:
## 차량 통계 기본 데이터셋
yr_list = []
rgn_list = []
ctpv_list = []
grd_list = []
for ctpv in grp1['시도'].unique():
    if ('서울특별시' in ctpv) or ('인천광역시' in ctpv) or ('경기도' in ctpv):
        rgn = '수도권'
    else:
        rgn = '비수도권'
    for grd in ['1', '2', '3', '4', '5', '미분류']: # !!! 수정(2023.12.22)
        for yr in range(2019, year + 1):
            yr_list.append(str(yr))
            rgn_list.append(rgn)
            ctpv_list.append(ctpv)
            grd_list.append(grd)
base = pd.DataFrame({'연도':yr_list, '지역':rgn_list, '시도':ctpv_list, '배출가스등급':grd_list})

In [ ]:
## 15.1s
## 연도별 등록대수
grp2 = dfm.groupby(['최초등록일자_년', '지역', '시도', '배출가스등급'], dropna=False)['차대번호'].count().reset_index()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,지역,시도,배출가스등급,등록대수
0,1920,비수도권,강원특별자치도,NaN,1
1,1920,비수도권,대구광역시,NaN,2
2,1920,비수도권,대전광역시,NaN,1
3,1920,비수도권,부산광역시,NaN,1
4,1920,비수도권,충청남도,3,1
...,...,...,...,...,...
3322,2023,수도권,인천광역시,2,216
3323,2023,수도권,인천광역시,3,130
3324,2023,수도권,인천광역시,4,1
3325,2023,수도권,인천광역시,NaN,1216


In [ ]:
## 26.3s
## 연도별 말소대수
grp3 = errc.groupby(['변경일자_년', '지역', '시도', '배출가스등급'], dropna=False)['차대번호'].count().reset_index()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,지역,시도,배출가스등급,말소대수
0,2019,비수도권,강원특별자치도,2,1
1,2019,비수도권,경상남도,1,588
2,2019,비수도권,경상남도,2,8744
3,2019,비수도권,경상남도,3,16564
4,2019,비수도권,경상남도,4,4493
...,...,...,...,...,...
487,2023,수도권,인천광역시,2,21155
488,2023,수도권,인천광역시,3,20530
489,2023,수도권,인천광역시,4,7519
490,2023,수도권,인천광역시,5,5805


In [ ]:
grp1['시도'].unique()

array(['강원특별자치도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시',
       '세종특별자치시', '울산광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도',
       '경기도', '서울특별시', '인천광역시'], dtype=object)

In [ ]:
grp2['시도'].unique()

array(['강원특별자치도', '대구광역시', '대전광역시', '부산광역시', '충청남도', '충청북도', '경기도',
       '서울특별시', '인천광역시', '제주특별자치도', '경상북도', '경상남도', '전라남도', '울산광역시',
       '전라북도', '광주광역시', '세종특별자치시', nan], dtype=object)

In [ ]:
grp3['시도'].unique()

array(['강원특별자치도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시',
       '세종특별자치시', '울산광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도', nan,
       '경기도', '서울특별시', '인천광역시'], dtype=object)

In [ ]:
base

,연도,지역,시도,배출가스등급
0,2019,비수도권,강원특별자치도,1
1,2020,비수도권,강원특별자치도,1
2,2021,비수도권,강원특별자치도,1
3,2022,비수도권,강원특별자치도,1
4,2023,비수도권,강원특별자치도,1
...,...,...,...,...
607,2020,수도권,인천광역시,미분류
608,2021,수도권,인천광역시,미분류
609,2022,수도권,인천광역시,미분류
610,2023,수도권,인천광역시,미분류


In [ ]:
grp1

,연도,지역,시도,배출가스등급,차량대수
0,2024,비수도권,강원특별자치도,1,789
1,2024,비수도권,강원특별자치도,2,9300
2,2024,비수도권,강원특별자치도,3,76168
3,2024,비수도권,강원특별자치도,4,4632
4,2024,비수도권,강원특별자치도,5,5041
...,...,...,...,...,...
80,2024,수도권,인천광역시,1,19015
81,2024,수도권,인천광역시,2,703114
82,2024,수도권,인천광역시,3,412412
83,2024,수도권,인천광역시,4,46643


In [ ]:
base.dtypes

연도        object
지역        object
시도        object
배출가스등급    object
dtype: object

In [ ]:
grp1.dtypes

연도        object
지역        object
시도        object
배출가스등급    object
차량대수       int64
dtype: object

In [ ]:
base1 = base.merge(grp1, on=['연도', '지역', '시도', '배출가스등급'], how='left')

In [ ]:
base2 = base1.merge(grp2, on=['연도', '지역', '시도', '배출가스등급'], how='left')

In [ ]:
base3 = base2.merge(grp3, on=['연도', '지역', '시도', '배출가스등급'], how='left')
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
base3

,연도,지역,시도,배출가스등급,차량대수,등록대수,말소대수
0,2019,비수도권,강원특별자치도,1,0.0,44.0,0.0
1,2020,비수도권,강원특별자치도,1,0.0,38.0,0.0
2,2021,비수도권,강원특별자치도,1,0.0,90.0,0.0
3,2022,비수도권,강원특별자치도,1,0.0,21.0,0.0
4,2023,비수도권,강원특별자치도,1,0.0,12.0,533.0
...,...,...,...,...,...,...,...
607,2020,수도권,인천광역시,미분류,0.0,0.0,0.0
608,2021,수도권,인천광역시,미분류,0.0,0.0,0.0
609,2022,수도권,인천광역시,미분류,0.0,0.0,0.0
610,2023,수도권,인천광역시,미분류,0.0,0.0,0.0


In [ ]:
n = len(base3['연도'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
df1 = base3[['연도', '지역', '시도', '배출가스등급', '차량대수']]
df1['기준연월'] = df1['연도'] + '12'

C:\Users\kbjung\AppData\Local\Temp\ipykernel_17964\1105122844.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['기준연월'] = df1['연도'] + '12'


In [ ]:
df1.head()

,연도,지역,시도,배출가스등급,차량대수,기준연월
0,2019,비수도권,강원특별자치도,1,1161.0,201912
1,2020,비수도권,강원특별자치도,1,1199.0,202012
2,2021,비수도권,강원특별자치도,1,1289.0,202112
3,2022,비수도권,강원특별자치도,1,1310.0,202212
4,2023,비수도권,강원특별자치도,1,789.0,202312


In [ ]:
df1['배출가스등급'].unique()

array(['1', '2', '3', '4', '5', '미분류'], dtype=object)

In [ ]:
df1['배출가스등급'] = df1['배출가스등급'].replace({'1':'1', '2':'2', '3':'3', '4':'4', '5':'5', 'X':'미분류'}) # !!! 수정(2023.12.22)

C:\Users\kbjung\AppData\Local\Temp\ipykernel_17964\1640257978.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['배출가스등급'] = df1['배출가스등급'].replace({'1':'1', '2':'2', '3':'3', '4':'4', '5':'5', 'X':'미분류'}) # !!! 수정(2023.12.22)


In [ ]:
df1['배출가스등급'].unique()

array(['1', '2', '3', '4', '5', '미분류'], dtype=object)

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
df1['테이블생성일자'] = today_date

In [ ]:
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '기준연월':'CRTR_YM', 
    '연도':'YR', 
    '지역':'RGN', 
    '시도':'CTPV_NM', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '차량대수':'VHCL_MKCNT',
}
STD_BD_CAR_CURSTT_MOD = df1.rename(columns=cdict)

In [ ]:
STD_BD_CAR_CURSTT_MOD.columns

Index(['YR', 'RGN', 'CTPV_NM', 'EXHST_GAS_GRD_CD', 'VHCL_MKCNT', 'CRTR_YM',
       'LOAD_DT'],
      dtype='object')

In [ ]:
STD_BD_CAR_CURSTT_MOD

,YR,RGN,CTPV_NM,EXHST_GAS_GRD_CD,VHCL_MKCNT,CRTR_YM,LOAD_DT
0,2019,비수도권,강원특별자치도,1,1161.0,201912,20240206
1,2020,비수도권,강원특별자치도,1,1199.0,202012,20240206
2,2021,비수도권,강원특별자치도,1,1289.0,202112,20240206
3,2022,비수도권,강원특별자치도,1,1310.0,202212,20240206
4,2023,비수도권,강원특별자치도,1,789.0,202312,20240206


### ----- 2024.02.13 수정 (STD_BD_CAR_CURSTT_MOD 데이터 확인을 위한 info 코드 추가)

`STD_BD_CAR_CURSTT_MOD.info()` 추가 => 여기서 데이터가 얼마나 존재하는지 확인

In [ ]:
STD_BD_CAR_CURSTT_MOD.info()

### [출력] STD_BD_CAR_CURSTT_MOD

In [ ]:
# expdf = STD_BD_CAR_CURSTT_MOD
# table_nm = 'STD_BD_CAR_CURSTT_MOD'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [ ]:
year

2024

In [ ]:
## 12.7s
## 등급, 연료별 차량현황
## 2022년 차량 대수
grp1 = dfm.groupby(['fuel2', '배출가스등급'], dropna=False)['차대번호'].count().reset_index()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = f'{year}'
grp1 = grp1[['연도', 'fuel2', '배출가스등급', '차량대수']]

In [ ]:
grp1.head()

,연도,fuel2,배출가스등급,차량대수
0,2024,LPG(액화석유가스),1,98363
1,2024,LPG(액화석유가스),2,232564
2,2024,LPG(액화석유가스),3,262975
3,2024,LPG(액화석유가스),4,14632
4,2024,LPG(액화석유가스),5,24


In [ ]:
# sl = grp1.drop_duplicates(['fuel2', '배출가스등급']).reset_index(drop=True)

In [ ]:
# sl.shape

In [ ]:
# periods = 5 # !!! 수정(2023.08.23)
# y_plist = list(pd.date_range(end=today_date, periods=periods, freq="Y").year) # !!! 수정(2023.08.23)
# # mth_plist = list(pd.date_range(end=today_date, periods=periods, freq="MS").month) # !!! 수정(2023.08.23)
# y_plist = y_plist[:-1]
# # mth_plist = mth_plist[:-1]
# y_plist

In [ ]:
# sl.shape[0] * 4

In [ ]:
# slt = sl
# for _ in range(periods - 2):
#     slt = pd.concat([slt, sl], ignore_index=False)
# slt.shape

In [ ]:
# slt = slt.sort_values(['fuel2', '배출가스등급']).reset_index(drop=True)
# slt.head(20)

In [ ]:
# y_plist01 = [str(x) for x in y_plist]

In [ ]:
# ym = pd.DataFrame({'연도':y_plist01})
# ym

In [ ]:
# ymt = ym
# for _ in range(slt.shape[0]//ym.shape[0] - 1):
#     ymt = pd.concat([ymt, ym], ignore_index=False)
# ymt.shape

In [ ]:
# slt = slt.reset_index(drop=True)
# ymt = ymt.reset_index(drop=True)
# slt.shape, ymt.shape

In [ ]:
# base = pd.concat([ymt, slt], axis=1)
# base.head(20)

In [ ]:
## 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
grd_list = []
for fuel in grp1['fuel2'].unique():
    for grd in ['1', '2', '3', '4', '5', '미분류']: # !!! 수정(2023.12.22)
        for yr in range(2019, year + 1):
            yr_list.append(str(yr))
            fuel_list.append(fuel)
            grd_list.append(grd)
base = pd.DataFrame({'연도':yr_list, 'fuel2':fuel_list, '배출가스등급':grd_list})

In [ ]:
## 14.0s
## 연도별 등록대수
grp2 = dfm.groupby(['최초등록일자_년', 'fuel2', '배출가스등급'], dropna=False)['차대번호'].count().reset_index()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})

In [ ]:
## 10.2s
## 연도별 말소대수
grp3 = errc.groupby(['변경일자_년', 'fuel2', '배출가스등급'], dropna=False)['차대번호'].count().reset_index()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
base1 = base.merge(grp1, on=['연도', 'fuel2', '배출가스등급'], how='left')
base2 = base1.merge(grp2, on=['연도', 'fuel2', '배출가스등급'], how='left')
base3 = base2.merge(grp3, on=['연도', 'fuel2', '배출가스등급'], how='left')
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
n = len(base3['연도'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
df2 = base3[['연도', 'fuel2', '배출가스등급', '차량대수']]
df2 = df2.rename(columns={'fuel2':'연료'})

In [ ]:
df2['배출가스등급'].unique()

array(['1', '2', '3', '4', '5', '미분류'], dtype=object)

In [ ]:
df2['배출가스등급'] = df2['배출가스등급'].replace({'1':'1', '2':'2', '3':'3', '4':'4', '5':'5', 'X':'미분류'}) # !!! 수정(2023.12.22)

In [ ]:
df2['배출가스등급'].unique()

array(['1', '2', '3', '4', '5', '미분류'], dtype=object)

In [ ]:
df2['기준연월'] = df2['연도'] + '12'
today_date = datetime.today().strftime("%Y%m%d")
df2['테이블생성일자'] = today_date

In [ ]:
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '기준연월':'CRTR_YM', 
    '연도':'YR', 
    '연료':'FUEL_CD', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '차량대수':'VHCL_MKCNT',
}
STD_BD_CAR_CURSTT_MOD2 = df2.rename(columns=cdict)

In [ ]:
STD_BD_CAR_CURSTT_MOD2.columns

Index(['YR', 'FUEL_CD', 'EXHST_GAS_GRD_CD', 'VHCL_MKCNT', 'CRTR_YM',
       'LOAD_DT'],
      dtype='object')

In [ ]:
STD_BD_CAR_CURSTT_MOD2.head()

,YR,FUEL_CD,EXHST_GAS_GRD_CD,VHCL_MKCNT,CRTR_YM,LOAD_DT
0,2019,LPG(액화석유가스),1,160685.0,201912,20240206
1,2020,LPG(액화석유가스),1,157029.0,202012,20240206
2,2021,LPG(액화석유가스),1,137691.0,202112,20240206
3,2022,LPG(액화석유가스),1,118177.0,202212,20240206
4,2023,LPG(액화석유가스),1,98363.0,202312,20240206


### [출력] STD_BD_CAR_CURSTT_MOD2

In [ ]:
# expdf = STD_BD_CAR_CURSTT_MOD2
# table_nm = 'STD_BD_CAR_CURSTT_MOD2'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [ ]:
## 11.3s
## 전체 차량 현황(등급, 차종, 지역별 차량현황)
today_date = datetime.today().strftime("%Y%m%d")
dfm2['테이블생성일자'] = today_date
df3 = dfm2[[
    '테이블생성일자', 
    '법정동코드', 
    '차종',
    '용도', 
    '차대번호', 
    '제원관리번호', 
    '배출가스인증번호', 
    '배출가스등급', 
    '차량말소YN', 
    'fuel', 
    '시도', 
    '시군구_수정'
]]

In [ ]:
df3['배출가스등급'].unique()

array(['2', nan, '3', '1', '4', '5'], dtype=object)

In [ ]:
df3['배출가스등급'] = df3['배출가스등급'].fillna('X')

C:\Users\kbjung\AppData\Local\Temp\ipykernel_17964\3944720476.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['배출가스등급'] = df3['배출가스등급'].fillna('X')


In [ ]:
df3['배출가스등급'].unique()

array(['2', 'X', '3', '1', '4', '5'], dtype=object)

In [ ]:
df3['배출가스등급'].value_counts(dropna=False)

배출가스등급
3    17108686
2     5684457
5     1413479
4      764171
1      179759
X       27383
Name: count, dtype: int64

In [ ]:
## 12.0s
cdict = {
    '테이블생성일자':'LOAD_DT',
    '법정동코드':'BSPL_STDG_CD', 
    '차종':'VHCTY_CD',
    '용도':'PURPS_CD2', 
    '차대번호':'VIN', 
    '제원관리번호':'MANG_MNG_NO', 
    '배출가스인증번호':'EXHST_GAS_CERT_NO', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '차량말소YN':'VHCL_ERSR_YN', 
    'fuel':'FUEL_CD', 
    '시도':'CTPV', 
    '시군구_수정':'SGG', 
}
STD_BD_CAR_CURSTT = df3.rename(columns=cdict)

In [ ]:
STD_BD_CAR_CURSTT.columns

Index(['LOAD_DT', 'BSPL_STDG_CD', 'VHCTY_CD', 'PURPS_CD2', 'VIN',
       'MANG_MNG_NO', 'EXHST_GAS_CERT_NO', 'EXHST_GAS_GRD_CD', 'VHCL_ERSR_YN',
       'FUEL_CD', 'CTPV', 'SGG'],
      dtype='object')

### [출력] STD_BD_CAR_CURSTT

In [ ]:
# expdf = STD_BD_CAR_CURSTT
# table_nm = 'STD_BD_CAR_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 3m 51.1s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [ ]:
## 13.2s
## 지역, 연료, 연도별 차량 현황 분석
## 2022년 차량 대수
grp1 = dfm2.groupby(['fuel', '시도'], dropna=False)['차대번호'].count().reset_index()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = f'{year}'
grp1 = grp1[['연도', 'fuel', '시도', '차량대수']]

In [ ]:
# sl = grp1.drop_duplicates(['fuel', '시도']).reset_index(drop=True)

In [ ]:
# sl.shape

In [ ]:
# periods = 5 # !!! 수정(2023.08.23)
# y_plist = list(pd.date_range(end=today_date, periods=periods, freq="Y").year) # !!! 수정(2023.08.23)
# # mth_plist = list(pd.date_range(end=date, periods=periods, freq="MS").month) # !!! 수정(2023.08.23)
# y_plist = y_plist[:-1]
# # mth_plist = mth_plist[:-1]
# y_plist

In [ ]:
# sl.shape[0] * 4

In [ ]:
# slt = sl
# for _ in range(periods - 2):
#     slt = pd.concat([slt, sl], ignore_index=False)
# slt.shape

In [ ]:
# slt = slt.sort_values(['fuel', '시도']).reset_index(drop=True)
# slt.head(20)

In [ ]:
# y_plist01 = [str(x) for x in y_plist]

In [ ]:
# ym = pd.DataFrame({'연도':y_plist01})
# ym

In [ ]:
# ymt = ym
# for _ in range(slt.shape[0]//ym.shape[0] - 1):
#     ymt = pd.concat([ymt, ym], ignore_index=False)
# ymt.shape

In [ ]:
# slt = slt.reset_index(drop=True)
# ymt = ymt.reset_index(drop=True)
# slt.shape, ymt.shape

In [ ]:
# base = pd.concat([ymt, slt], axis=1)
# base.head(20)

In [ ]:
# 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
ctpv_list = []
for fuel in grp1['fuel'].unique():
    for ctpv in grp1['시도'].unique():
        for yr in range(2019, year + 1):
            yr_list.append(str(yr))
            fuel_list.append(fuel)
            ctpv_list.append(ctpv)
base = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '시도':ctpv_list})

In [ ]:
base

,연도,fuel,시도
0,2019,LPG,강원특별자치도
1,2020,LPG,강원특별자치도
2,2021,LPG,강원특별자치도
3,2022,LPG,강원특별자치도
4,2023,LPG,강원특별자치도
...,...,...,...
535,2020,휘발유,NaN
536,2021,휘발유,NaN
537,2022,휘발유,NaN
538,2023,휘발유,NaN


In [ ]:
## 14.2s
## 연도별 등록대수
grp2 = dfm2.groupby(['최초등록일자_년', 'fuel', '시도'], dropna=False)['차대번호'].count().reset_index()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})

In [ ]:
## 10.2s
## 연도별 말소대수
grp3 = errc2.groupby(['변경일자_년', 'fuel', '시도'], dropna=False)['차대번호'].count().reset_index()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})

In [ ]:
base1 = base.merge(grp1, on=['연도', 'fuel', '시도'], how='left')
base2 = base1.merge(grp2, on=['연도', 'fuel', '시도'], how='left')
base3 = base2.merge(grp3, on=['연도', 'fuel', '시도'], how='left')
base3[['차량대수', '등록대수', '말소대수']].isnull().sum()
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
n = len(base3['연도'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
base3

,연도,fuel,시도,차량대수,등록대수,말소대수
0,2019,LPG,강원특별자치도,4612.0,27.0,1.0
1,2020,LPG,강원특별자치도,4620.0,8.0,0.0
2,2021,LPG,강원특별자치도,4632.0,13.0,1.0
3,2022,LPG,강원특별자치도,4638.0,6.0,0.0
4,2023,LPG,강원특별자치도,2775.0,0.0,1863.0
...,...,...,...,...,...,...
535,2020,휘발유,NaN,524.0,7.0,0.0
536,2021,휘발유,NaN,527.0,3.0,0.0
537,2022,휘발유,NaN,532.0,5.0,0.0
538,2023,휘발유,NaN,537.0,5.0,0.0


In [ ]:
base3.groupby(['연도', '시도', 'fuel'], as_index=False)['차량대수'].sum()

,연도,시도,fuel,차량대수
0,2019,강원특별자치도,LPG,4612.0
1,2019,강원특별자치도,경유,21034.0
2,2019,강원특별자치도,수소,1.0
3,2019,강원특별자치도,전기,150.0
4,2019,강원특별자치도,휘발유,87788.0
...,...,...,...,...
505,2024,충청북도,LPG,22279.0
506,2024,충청북도,경유,107576.0
507,2024,충청북도,수소,10.0
508,2024,충청북도,전기,518.0


In [ ]:
gp = base3.groupby(['연도', '시도', 'fuel'], dropna=False)['차량대수'].sum().reset_index()
gp['연료비율'] = round((gp['차량대수'] / gp.groupby(['연도', '시도'])['차량대수'].transform('sum')), 2)

In [ ]:
gp

,연도,시도,fuel,차량대수,연료비율
0,2019,강원특별자치도,LPG,4612.0,0.04
1,2019,강원특별자치도,경유,21034.0,0.19
2,2019,강원특별자치도,수소,1.0,0.00
3,2019,강원특별자치도,전기,150.0,0.00
4,2019,강원특별자치도,휘발유,87788.0,0.77
...,...,...,...,...,...
535,2024,NaN,LPG,68.0,NaN
536,2024,NaN,경유,305.0,NaN
537,2024,NaN,수소,0.0,NaN
538,2024,NaN,전기,1.0,NaN


In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
gp['테이블생성일자'] = today_date
gp1 = gp[['테이블생성일자', '연도', 'fuel', '시도', '차량대수', '연료비율']]
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '연도':'YR', 
    'fuel':'FUEL_CD', 
    '시도':'CTPV', 
    '차량대수':'VHCL_MKCNT', 
    '연료비율':'FUEL_RT'
}
STD_BD_CAR_REG_MKCNT = gp1.rename(columns=cdict)

In [ ]:
STD_BD_CAR_REG_MKCNT.columns

Index(['LOAD_DT', 'YR', 'FUEL_CD', 'CTPV', 'VHCL_MKCNT', 'FUEL_RT'], dtype='object')

In [ ]:
STD_BD_CAR_REG_MKCNT.YR.unique()

array(['2019', '2020', '2021', '2022', '2023', '2024'], dtype=object)

### [출력] STD_BD_CAR_REG_MKCNT

In [ ]:
# expdf = STD_BD_CAR_REG_MKCNT
# table_nm = 'STD_BD_CAR_REG_MKCNT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# print(f'data export : {table_nm}')

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [ ]:
## 23.8s
## 내연차 연료, 연도별 차량 현황 예측
## - 경유, 휘발유, LPG
dfm2dgl = dfm2.loc[(dfm2['연료'] == '경유') | (dfm2['연료'] == '휘발유') | (dfm2['연료'] == 'LPG(액화석유가스)')].reset_index(drop=True)
errc2dgl = errc2.loc[(errc2['연료'] == '경유') | (errc2['연료'] == '휘발유') | (errc2['연료'] == 'LPG(액화석유가스)')].reset_index(drop=True)

In [ ]:
## 11.8s
## 2022년 차량 대수
grp1 = dfm2dgl.groupby('연료', dropna=False)['차대번호'].count().reset_index()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = f'{year}'
grp1 = grp1[['연도', '연료', '차량대수']]

In [ ]:
# sl = grp1.drop_duplicates('연료').reset_index(drop=True)

In [ ]:
# sl.shape

In [ ]:
# periods = 5 # !!! 수정(2023.08.23)
# y_plist = list(pd.date_range(end=today_date, periods=periods, freq="Y").year) # !!! 수정(2023.08.23)
# # mth_plist = list(pd.date_range(end=date, periods=periods, freq="MS").month) # !!! 수정(2023.08.23)
# y_plist = y_plist[:-1]
# # mth_plist = mth_plist[:-1]
# y_plist

In [ ]:
# sl.shape[0] * 4

In [ ]:
# slt = sl
# for _ in range(periods - 2):
#     slt = pd.concat([slt, sl], ignore_index=False)
# slt.shape

In [ ]:
# slt = slt.sort_values('연료').reset_index(drop=True)
# slt.head(20)

In [ ]:
# y_plist01 = [str(x) for x in y_plist]

In [ ]:
# ym = pd.DataFrame({'연도':y_plist01})
# ym

In [ ]:
# ymt = ym
# for _ in range(slt.shape[0]//ym.shape[0] - 1):
#     ymt = pd.concat([ymt, ym], ignore_index=False)
# ymt.shape

In [ ]:
# slt = slt.reset_index(drop=True)
# ymt = ymt.reset_index(drop=True)
# slt.shape, ymt.shape

In [ ]:
# base = pd.concat([ymt, slt], axis=1)
# base.head(20)

In [ ]:
# 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
for fuel in grp1['연료'].unique():
    for yr in range(2019, year + 1):
        yr_list.append(str(yr))
        fuel_list.append(fuel)
base = pd.DataFrame({'연도':yr_list, '연료':fuel_list})

In [ ]:
## 13.4s
## 연도별 등록대수
grp2 = dfm2dgl.groupby(['최초등록일자_년', '연료'], dropna=False)['차대번호'].count().reset_index()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})

In [ ]:
## 10.1s
## 연도별 말소대수
grp3 = errc2dgl.groupby(['변경일자_년', '연료'], dropna=False)['차대번호'].count().reset_index()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
base1 = base.merge(grp1, on=['연도', '연료'], how='left')
base2 = base1.merge(grp2, on=['연도', '연료'], how='left')
base3 = base2.merge(grp3, on=['연도', '연료'], how='left')
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
n = len(base3['연도'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
die = base3.loc[base3['연료'] == '경유', ['연도', '연료', '차량대수']].reset_index(drop=True)
gas = base3.loc[base3['연료'] == '휘발유', ['연도', '연료', '차량대수']].reset_index(drop=True)
lpg = base3.loc[base3['연료'] == 'LPG(액화석유가스)', ['연도', '연료', '차량대수']].reset_index(drop=True)

In [ ]:
die['연도'] = die['연도'].astype('int')
gas['연도'] = gas['연도'].astype('int')
lpg['연도'] = lpg['연도'].astype('int')

In [ ]:
## 선형 예측
fit1 = np.polyfit(die['연도'], die['차량대수'], 1)
fit2 = np.polyfit(gas['연도'], gas['차량대수'], 1)
fit3 = np.polyfit(lpg['연도'], lpg['차량대수'], 1)
a1, b1 = fit1
a2, b2 = fit2
a3, b3 = fit3

In [ ]:
## BSpline 예측
spl1 = intp.BSpline(die['연도'], die['차량대수'], 1, extrapolate=True)
spl2 = intp.BSpline(gas['연도'], gas['차량대수'], 1, extrapolate=True)
spl3 = intp.BSpline(lpg['연도'], lpg['차량대수'], 1, extrapolate=True)
spl1pred = spl1(range(year + 1, 2036))
spl2pred = spl2(range(year + 1, 2036))
spl3pred = spl3(range(year + 1, 2036))

In [ ]:
## akima 예측
aki1 = intp.Akima1DInterpolator(die['연도'], die['차량대수'])
aki2 = intp.Akima1DInterpolator(gas['연도'], gas['차량대수'])
aki3 = intp.Akima1DInterpolator(lpg['연도'], lpg['차량대수'])
aki1pred = aki1([x for x in range(year + 1, 2036)], extrapolate=True)
aki2pred = aki2([x for x in range(year + 1, 2036)], extrapolate=True)
aki3pred = aki3([x for x in range(year + 1, 2036)], extrapolate=True)

In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '경유'
for yr in range(year + 1, 2036): # 차량 예측 연도 설정
    pred = a1 * yr + b1
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
die_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, '경유_예측':pred_list})
die_pred['경유_예측_BSpline'] = spl1pred
die_pred['경유_예측_Akima'] = aki1pred

In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '휘발유'
for yr in range(year + 1, 2036): # 차량 예측 연도 설정
    pred = a2 * yr + b2
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
gas_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, '휘발유_예측':pred_list})
gas_pred['휘발유_예측_BSpline'] = spl2pred
gas_pred['휘발유_예측_Akima'] = aki2pred

In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = 'LPG'
for yr in range(year + 1, 2036): # 차량 예측 연도 설정
    pred = a3 * yr + b3
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
lpg_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, 'LPG_예측':pred_list})
lpg_pred['LPG_예측_BSpline'] = spl3pred
lpg_pred['LPG_예측_Akima'] = aki3pred

In [ ]:
die_t = pd.concat([die, die_pred], ignore_index=True)
gas_t = pd.concat([gas, gas_pred], ignore_index=True)
lpg_t = pd.concat([lpg, lpg_pred], ignore_index=True)
die_t = die_t.rename(columns={'차량대수':'경유_대수'})
gas_t = gas_t.rename(columns={'차량대수':'휘발유_대수'})
lpg_t = lpg_t.rename(columns={'차량대수':'LPG_대수'})

In [ ]:
df5 = pd.concat([die_t[['연도', '경유_대수', '경유_예측', '경유_예측_BSpline', '경유_예측_Akima']], gas_t[['휘발유_대수', '휘발유_예측', '휘발유_예측_BSpline', '휘발유_예측_Akima']], lpg_t[['LPG_대수', 'LPG_예측', 'LPG_예측_BSpline', 'LPG_예측_Akima']]], axis=1)

In [ ]:
## 음수 0으로 처리
df5.loc[df5['경유_예측'] < 0, '경유_예측'] = 0
df5.loc[df5['경유_예측_BSpline'] < 0, '경유_예측_BSpline'] = 0
df5.loc[df5['경유_예측_Akima'] < 0, '경유_예측_Akima'] = 0
df5.loc[df5['휘발유_예측'] < 0, '휘발유_예측'] = 0
df5.loc[df5['휘발유_예측_BSpline'] < 0, '휘발유_예측_BSpline'] = 0
df5.loc[df5['휘발유_예측_Akima'] < 0, '휘발유_예측_Akima'] = 0
df5.loc[df5['LPG_예측'] < 0, 'LPG_예측'] = 0
df5.loc[df5['LPG_예측_BSpline'] < 0, 'LPG_예측_BSpline'] = 0
df5.loc[df5['LPG_예측_Akima'] < 0, 'LPG_예측_Akima'] = 0

In [ ]:
## 첫째자리까지 반올림
df5[['경유_대수', '휘발유_대수', 'LPG_대수', '경유_예측', '경유_예측_BSpline','경유_예측_Akima', '휘발유_예측', '휘발유_예측_BSpline', '휘발유_예측_Akima', 'LPG_예측', 'LPG_예측_BSpline', 'LPG_예측_Akima']] = df5[['경유_대수', '휘발유_대수', 'LPG_대수', '경유_예측', '경유_예측_BSpline','경유_예측_Akima', '휘발유_예측', '휘발유_예측_BSpline', '휘발유_예측_Akima', 'LPG_예측', 'LPG_예측_BSpline', 'LPG_예측_Akima']].round(0)

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
df5['테이블생성일자'] = today_date
df5 = df5[[
   '테이블생성일자',
   '연도',
   '경유_대수',
   '휘발유_대수',
   'LPG_대수',
   '경유_예측',
   '경유_예측_BSpline',
   '경유_예측_Akima',
   '휘발유_예측',
   '휘발유_예측_BSpline',
   '휘발유_예측_Akima',
   'LPG_예측',
   'LPG_예측_BSpline',
   'LPG_예측_Akima',
    ]]
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '연도':'YR', 
    '경유_대수':'DSL', 
    '휘발유_대수':'GSL', 
    'LPG_대수':'LPG', 
    '경유_예측':'DSL_PRET', 
    '경유_예측_BSpline':'DSL_PRET_BSPLN', 
    '경유_예측_Akima':'DSL_PRET_AKM', 
    '휘발유_예측':'GSL_PRET', 
    '휘발유_예측_BSpline':'GSL_PRET_BSPLN', 
    '휘발유_예측_Akima':'GSL_PRET_AKM', 
    'LPG_예측':'LPG_PRET',
    'LPG_예측_BSpline':'LPG_PRET_BSPLN',
    'LPG_예측_Akima':'LPG_PRET_AKM',
}
STD_BD_CAR_PRET = df5.rename(columns=cdict)

In [ ]:
STD_BD_CAR_PRET.columns

Index(['LOAD_DT', 'YR', 'DSL', 'GSL', 'LPG', 'DSL_PRET', 'DSL_PRET_BSPLN',
       'DSL_PRET_AKM', 'GSL_PRET', 'GSL_PRET_BSPLN', 'GSL_PRET_AKM',
       'LPG_PRET', 'LPG_PRET_BSPLN', 'LPG_PRET_AKM'],
      dtype='object')

In [ ]:
STD_BD_CAR_PRET.tail()

,LOAD_DT,YR,DSL,GSL,LPG,DSL_PRET,DSL_PRET_BSPLN,DSL_PRET_AKM,GSL_PRET,GSL_PRET_BSPLN,GSL_PRET_AKM,LPG_PRET,LPG_PRET_BSPLN,LPG_PRET_AKM
12,20240206,2031,NaN,NaN,NaN,16233.0,218693.0,0.0,19809533.0,19633864.0,0.0,5880.0,0.0,0.0
13,20240206,2032,NaN,NaN,NaN,0.0,0.0,0.0,19499273.0,19270057.0,0.0,0.0,0.0,0.0
14,20240206,2033,NaN,NaN,NaN,0.0,0.0,0.0,19189013.0,18906250.0,0.0,0.0,0.0,0.0
15,20240206,2034,NaN,NaN,NaN,0.0,0.0,0.0,18878753.0,18542443.0,0.0,0.0,0.0,0.0
16,20240206,2035,NaN,NaN,NaN,0.0,0.0,0.0,18568494.0,18178636.0,0.0,0.0,0.0,0.0


### [출력] STD_BD_CAR_PRET

In [ ]:
# expdf = STD_BD_CAR_PRET
# table_nm = 'STD_BD_CAR_PRET'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [ ]:
## 12.0s
## 하이브리드 연료, 연도별 차량 현황 예측
# - 경유 하이브리드, 휘발유 하이브리드, LPG 하이브리드
dfm2h = dfm2.loc[(dfm2['연료'] == '경유 하이브리드') | (dfm2['연료'] == '휘발유 하이브리드') | (dfm2['연료'] == 'LPG 하이브리드')].reset_index(drop=True)
errc2h = errc2.loc[(errc2['연료'] == '경유 하이브리드') | (errc2['연료'] == '휘발유 하이브리드') | (errc2['연료'] == 'LPG 하이브리드')].reset_index(drop=True)

In [ ]:
## 2022년 차량 대수
grp1 = dfm2h.groupby('연료', as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = f'{year}'
grp1 = grp1[['연도', '연료', '차량대수']]

In [ ]:
# sl = grp1.drop_duplicates('연료').reset_index(drop=True)

In [ ]:
# sl.shape

In [ ]:
# periods = 5 # !!! 수정(2023.08.23)
# y_plist = list(pd.date_range(end=today_date, periods=periods, freq="Y").year) # !!! 수정(2023.08.23)
# # mth_plist = list(pd.date_range(end=today_date, periods=periods, freq="MS").month) # !!! 수정(2023.08.23)
# y_plist = y_plist[:-1]
# # mth_plist = mth_plist[:-1]
# y_plist

In [ ]:
# sl.shape[0] * 4

In [ ]:
# slt = sl
# for _ in range(periods - 2):
#     slt = pd.concat([slt, sl], ignore_index=False)
# slt.shape

In [ ]:
# slt = slt.sort_values(['연료']).reset_index(drop=True)
# # slt = slt[['fuel', '시도', '차량대수']]
# slt.head(20)

In [ ]:
# y_plist01 = [str(x) for x in y_plist]
# # mth_plist01 = [f'{x:0>2}' for x in mth_plist]

In [ ]:
# ym = pd.DataFrame({'연도':y_plist01})
# ym

In [ ]:
# ymt = ym
# for _ in range(slt.shape[0]//ym.shape[0] - 1):
#     ymt = pd.concat([ymt, ym], ignore_index=False)
# ymt.shape

In [ ]:
# slt = slt.reset_index(drop=True)
# ymt = ymt.reset_index(drop=True)
# slt.shape, ymt.shape

In [ ]:
# base = pd.concat([ymt, slt], axis=1)
# base.head(20)

In [ ]:
## 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
for fuel in grp1['연료'].unique():
    for yr in range(2019, year + 1):
        yr_list.append(str(yr))
        fuel_list.append(fuel)
base = pd.DataFrame({'연도':yr_list, '연료':fuel_list})

In [ ]:
## 연도별 등록대수
grp2 = dfm2h.groupby(['최초등록일자_년', '연료'], dropna=False)['차대번호'].count().reset_index()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})

In [ ]:
## 연도별 말소대수
grp3 = errc2h.groupby(['변경일자_년', '연료'], dropna=False)['차대번호'].count().reset_index()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
base1 = base.merge(grp1, on=['연도', '연료'], how='left')
base2 = base1.merge(grp2, on=['연도', '연료'], how='left')
base3 = base2.merge(grp3, on=['연도', '연료'], how='left')
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
n = len(base3['연도'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
die = base3.loc[base3['연료'] == '경유 하이브리드', ['연도', '연료', '차량대수']].reset_index(drop=True)
gas = base3.loc[base3['연료'] == '휘발유 하이브리드', ['연도', '연료', '차량대수']].reset_index(drop=True)
lpg = base3.loc[base3['연료'] == 'LPG 하이브리드', ['연도', '연료', '차량대수']].reset_index(drop=True)
die['연도'] = die['연도'].astype('int')
gas['연도'] = gas['연도'].astype('int')
lpg['연도'] = lpg['연도'].astype('int')

In [ ]:
## 선형예측
fit1 = np.polyfit(die['연도'], die['차량대수'], 1)
fit2 = np.polyfit(gas['연도'], gas['차량대수'], 1)
fit3 = np.polyfit(lpg['연도'], lpg['차량대수'], 1)
a1, b1 = fit1
a2, b2 = fit2
a3, b3 = fit3

In [ ]:
## BSpline 예측
spl1 = intp.BSpline(die['연도'], die['차량대수'], 1, extrapolate=True)
spl2 = intp.BSpline(gas['연도'], gas['차량대수'], 1, extrapolate=True)
spl3 = intp.BSpline(lpg['연도'], lpg['차량대수'], 1, extrapolate=True)
spl1pred = spl1(range(year + 1, 2036)) # 차량 예측 연도 설정
spl2pred = spl2(range(year + 1, 2036))
spl3pred = spl3(range(year + 1, 2036))

In [ ]:
## akima 예측
aki1 = intp.Akima1DInterpolator(die['연도'], die['차량대수'])
aki2 = intp.Akima1DInterpolator(gas['연도'], gas['차량대수'])
aki3 = intp.Akima1DInterpolator(lpg['연도'], lpg['차량대수'])
aki1pred = aki1([x for x in range(year + 1, 2036)], extrapolate=True) # 차량 예측 연도 설정
aki2pred = aki2([x for x in range(year + 1, 2036)], extrapolate=True)
aki3pred = aki3([x for x in range(year + 1, 2036)], extrapolate=True)

In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '경유_하이브리드'
for yr in range(year + 1, 2036): # 차량 예측 연도 설정
    pred = a1 * yr + b1
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
die_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, f'{fuel}_예측':pred_list})
die_pred[f'{fuel}_예측_BSpline'] = spl1pred
die_pred[f'{fuel}_예측_Akima'] = aki1pred

In [ ]:
die_pred.columns

Index(['연도', '연료', '경유_하이브리드_예측', '경유_하이브리드_예측_BSpline', '경유_하이브리드_예측_Akima'], dtype='object')

In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '휘발유_하이브리드'
for yr in range(year + 1, 2036): # 차량 예측 연도 설정
    pred = a2 * yr + b2
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
gas_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, f'{fuel}_예측':pred_list})
gas_pred[f'{fuel}_예측_BSpline'] = spl2pred
gas_pred[f'{fuel}_예측_Akima'] = aki2pred

In [ ]:
gas_pred.columns

Index(['연도', '연료', '휘발유_하이브리드_예측', '휘발유_하이브리드_예측_BSpline',
       '휘발유_하이브리드_예측_Akima'],
      dtype='object')

In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = 'LPG_하이브리드'
for yr in range(year + 1, 2036): # 차량 예측 연도 설정
    pred = a3 * yr + b3
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
lpg_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, f'{fuel}_예측':pred_list})
lpg_pred[f'{fuel}_예측_BSpline'] = spl3pred
lpg_pred[f'{fuel}_예측_Akima'] = aki3pred

In [ ]:
lpg_pred.columns

Index(['연도', '연료', 'LPG_하이브리드_예측', 'LPG_하이브리드_예측_BSpline',
       'LPG_하이브리드_예측_Akima'],
      dtype='object')

In [ ]:
die_t = pd.concat([die, die_pred], ignore_index=True)
gas_t = pd.concat([gas, gas_pred], ignore_index=True)
lpg_t = pd.concat([lpg, lpg_pred], ignore_index=True)
die_t = die_t.rename(columns={'차량대수':'경유_하이브리드_대수'})
gas_t = gas_t.rename(columns={'차량대수':'휘발유_하이브리드_대수'})
lpg_t = lpg_t.rename(columns={'차량대수':'LPG_하이브리드_대수'})

In [ ]:
die_t.columns

Index(['연도', '연료', '경유_하이브리드_대수', '경유_하이브리드_예측', '경유_하이브리드_예측_BSpline',
       '경유_하이브리드_예측_Akima'],
      dtype='object')

In [ ]:
die_t = die_t.drop('연료', axis=1)
gas_t = gas_t.drop(['연도', '연료'], axis=1)
lpg_t = lpg_t.drop(['연도', '연료'], axis=1)

In [ ]:
df5 = pd.concat([die_t, gas_t, lpg_t], axis=1)

In [ ]:
df5.columns

Index(['연도', '경유_하이브리드_대수', '경유_하이브리드_예측', '경유_하이브리드_예측_BSpline',
       '경유_하이브리드_예측_Akima', '휘발유_하이브리드_대수', '휘발유_하이브리드_예측',
       '휘발유_하이브리드_예측_BSpline', '휘발유_하이브리드_예측_Akima', 'LPG_하이브리드_대수',
       'LPG_하이브리드_예측', 'LPG_하이브리드_예측_BSpline', 'LPG_하이브리드_예측_Akima'],
      dtype='object')

In [ ]:
df5.head()

,연도,경유_하이브리드_대수,경유_하이브리드_예측,경유_하이브리드_예측_BSpline,경유_하이브리드_예측_Akima,휘발유_하이브리드_대수,휘발유_하이브리드_예측,휘발유_하이브리드_예측_BSpline,휘발유_하이브리드_예측_Akima,LPG_하이브리드_대수,LPG_하이브리드_예측,LPG_하이브리드_예측_BSpline,LPG_하이브리드_예측_Akima
0,2019,2565.0,NaN,NaN,NaN,55573.0,NaN,NaN,NaN,7793.0,NaN,NaN,NaN
1,2020,3145.0,NaN,NaN,NaN,57682.0,NaN,NaN,NaN,7531.0,NaN,NaN,NaN
2,2021,3777.0,NaN,NaN,NaN,55409.0,NaN,NaN,NaN,6643.0,NaN,NaN,NaN
3,2022,3963.0,NaN,NaN,NaN,48633.0,NaN,NaN,NaN,5804.0,NaN,NaN,NaN
4,2023,3441.0,NaN,NaN,NaN,38361.0,NaN,NaN,NaN,4693.0,NaN,NaN,NaN


In [ ]:
df5.iloc[:, 1:].head()

,경유_하이브리드_대수,경유_하이브리드_예측,경유_하이브리드_예측_BSpline,경유_하이브리드_예측_Akima,휘발유_하이브리드_대수,휘발유_하이브리드_예측,휘발유_하이브리드_예측_BSpline,휘발유_하이브리드_예측_Akima,LPG_하이브리드_대수,LPG_하이브리드_예측,LPG_하이브리드_예측_BSpline,LPG_하이브리드_예측_Akima
0,2565.0,NaN,NaN,NaN,55573.0,NaN,NaN,NaN,7793.0,NaN,NaN,NaN
1,3145.0,NaN,NaN,NaN,57682.0,NaN,NaN,NaN,7531.0,NaN,NaN,NaN
2,3777.0,NaN,NaN,NaN,55409.0,NaN,NaN,NaN,6643.0,NaN,NaN,NaN
3,3963.0,NaN,NaN,NaN,48633.0,NaN,NaN,NaN,5804.0,NaN,NaN,NaN
4,3441.0,NaN,NaN,NaN,38361.0,NaN,NaN,NaN,4693.0,NaN,NaN,NaN


In [ ]:
df5.iloc[:, 1:].tail()

,경유_하이브리드_대수,경유_하이브리드_예측,경유_하이브리드_예측_BSpline,경유_하이브리드_예측_Akima,휘발유_하이브리드_대수,휘발유_하이브리드_예측,휘발유_하이브리드_예측_BSpline,휘발유_하이브리드_예측_Akima,LPG_하이브리드_대수,LPG_하이브리드_예측,LPG_하이브리드_예측_BSpline,LPG_하이브리드_예측_Akima
12,NaN,4869.038095,5451.0,33528.570732,NaN,8072.009524,-5575.0,-6.648860e+05,NaN,-552.980952,-908.0,-96301.157628
13,NaN,5024.866667,5637.0,44966.736585,NaN,3763.466667,-12351.0,-1.047809e+06,NaN,-1263.066667,-1747.0,-149420.292842
14,NaN,5180.695238,5823.0,58900.317073,NaN,-545.076190,-19127.0,-1.546846e+06,NaN,-1973.152381,-2586.0,-218278.193059
15,NaN,5336.523810,6009.0,75566.121951,NaN,-4853.619048,-25903.0,-2.177162e+06,NaN,-2683.238095,-3425.0,-304896.830080
16,NaN,5492.352381,6195.0,95200.960976,NaN,-9162.161905,-32679.0,-2.953925e+06,NaN,-3393.323810,-4264.0,-411298.175705


In [ ]:
df5.iloc[:, 1:].tail()

,경유_하이브리드_대수,경유_하이브리드_예측,경유_하이브리드_예측_BSpline,경유_하이브리드_예측_Akima,휘발유_하이브리드_대수,휘발유_하이브리드_예측,휘발유_하이브리드_예측_BSpline,휘발유_하이브리드_예측_Akima,LPG_하이브리드_대수,LPG_하이브리드_예측,LPG_하이브리드_예측_BSpline,LPG_하이브리드_예측_Akima
12,NaN,4869.038095,5451.0,33528.570732,NaN,8072.009524,-5575.0,-6.648860e+05,NaN,-552.980952,-908.0,-96301.157628
13,NaN,5024.866667,5637.0,44966.736585,NaN,3763.466667,-12351.0,-1.047809e+06,NaN,-1263.066667,-1747.0,-149420.292842
14,NaN,5180.695238,5823.0,58900.317073,NaN,-545.076190,-19127.0,-1.546846e+06,NaN,-1973.152381,-2586.0,-218278.193059
15,NaN,5336.523810,6009.0,75566.121951,NaN,-4853.619048,-25903.0,-2.177162e+06,NaN,-2683.238095,-3425.0,-304896.830080
16,NaN,5492.352381,6195.0,95200.960976,NaN,-9162.161905,-32679.0,-2.953925e+06,NaN,-3393.323810,-4264.0,-411298.175705


In [ ]:
## 음수 확인
print(df5.loc[df5['경유_하이브리드_대수'] < 0, '경유_하이브리드_대수'].shape)
print(df5.loc[df5['경유_하이브리드_예측'] < 0, '경유_하이브리드_예측'].shape)
print(df5.loc[df5['경유_하이브리드_예측_BSpline'] < 0, '경유_하이브리드_예측_BSpline'].shape)
print(df5.loc[df5['경유_하이브리드_예측_Akima'] < 0, '경유_하이브리드_예측_Akima'].shape)
print(df5.loc[df5['휘발유_하이브리드_대수'] < 0, '휘발유_하이브리드_대수'].shape)
print(df5.loc[df5['휘발유_하이브리드_예측'] < 0, '휘발유_하이브리드_예측'].shape)
print(df5.loc[df5['휘발유_하이브리드_예측_BSpline'] < 0, '휘발유_하이브리드_예측_BSpline'].shape)
print(df5.loc[df5['휘발유_하이브리드_예측_Akima'] < 0, '휘발유_하이브리드_예측_Akima'].shape)
print(df5.loc[df5['LPG_하이브리드_대수'] < 0, 'LPG_하이브리드_대수'].shape)
print(df5.loc[df5['LPG_하이브리드_예측'] < 0, 'LPG_하이브리드_예측'].shape)
print(df5.loc[df5['LPG_하이브리드_예측_BSpline'] < 0, 'LPG_하이브리드_예측_BSpline'].shape)
print(df5.loc[df5['LPG_하이브리드_예측_Akima'] < 0, 'LPG_하이브리드_예측_Akima'].shape)

(0,)
(0,)
(0,)
(0,)
(0,)
(3,)
(5,)
(8,)
(0,)
(5,)
(6,)
(9,)


In [ ]:
## 음수 0으로 처리
df5.loc[df5['경유_하이브리드_대수'] < 0, '경유_하이브리드_대수'] = 0
df5.loc[df5['경유_하이브리드_예측'] < 0, '경유_하이브리드_예측'] = 0
df5.loc[df5['경유_하이브리드_예측_BSpline'] < 0, '경유_하이브리드_예측_BSpline'] = 0
df5.loc[df5['경유_하이브리드_예측_Akima'] < 0, '경유_하이브리드_예측_Akima'] = 0
df5.loc[df5['휘발유_하이브리드_대수'] < 0, '휘발유_하이브리드_대수'] = 0
df5.loc[df5['휘발유_하이브리드_예측'] < 0, '휘발유_하이브리드_예측'] = 0
df5.loc[df5['휘발유_하이브리드_예측_BSpline'] < 0, '휘발유_하이브리드_예측_BSpline'] = 0
df5.loc[df5['휘발유_하이브리드_예측_Akima'] < 0, '휘발유_하이브리드_예측_Akima'] = 0
df5.loc[df5['LPG_하이브리드_대수'] < 0, 'LPG_하이브리드_대수'] = 0
df5.loc[df5['LPG_하이브리드_예측'] < 0, 'LPG_하이브리드_예측'] = 0
df5.loc[df5['LPG_하이브리드_예측_BSpline'] < 0, 'LPG_하이브리드_예측_BSpline'] = 0
df5.loc[df5['LPG_하이브리드_예측_Akima'] < 0, 'LPG_하이브리드_예측_Akima'] = 0

In [ ]:
## 음수 확인
print(df5.loc[df5['경유_하이브리드_대수'] < 0, '경유_하이브리드_대수'].shape)
print(df5.loc[df5['경유_하이브리드_예측'] < 0, '경유_하이브리드_예측'].shape)
print(df5.loc[df5['경유_하이브리드_예측_BSpline'] < 0, '경유_하이브리드_예측_BSpline'].shape)
print(df5.loc[df5['경유_하이브리드_예측_Akima'] < 0, '경유_하이브리드_예측_Akima'].shape)
print(df5.loc[df5['휘발유_하이브리드_대수'] < 0, '휘발유_하이브리드_대수'].shape)
print(df5.loc[df5['휘발유_하이브리드_예측'] < 0, '휘발유_하이브리드_예측'].shape)
print(df5.loc[df5['휘발유_하이브리드_예측_BSpline'] < 0, '휘발유_하이브리드_예측_BSpline'].shape)
print(df5.loc[df5['휘발유_하이브리드_예측_Akima'] < 0, '휘발유_하이브리드_예측_Akima'].shape)
print(df5.loc[df5['LPG_하이브리드_대수'] < 0, 'LPG_하이브리드_대수'].shape)
print(df5.loc[df5['LPG_하이브리드_예측'] < 0, 'LPG_하이브리드_예측'].shape)
print(df5.loc[df5['LPG_하이브리드_예측_BSpline'] < 0, 'LPG_하이브리드_예측_BSpline'].shape)
print(df5.loc[df5['LPG_하이브리드_예측_Akima'] < 0, 'LPG_하이브리드_예측_Akima'].shape)

(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)


In [ ]:
## 첫째자리까지 반올림
df5.iloc[:, 1:] = df5.iloc[:, 1:].round(0)

In [ ]:
df5.head()

,연도,경유_하이브리드_대수,경유_하이브리드_예측,경유_하이브리드_예측_BSpline,경유_하이브리드_예측_Akima,휘발유_하이브리드_대수,휘발유_하이브리드_예측,휘발유_하이브리드_예측_BSpline,휘발유_하이브리드_예측_Akima,LPG_하이브리드_대수,LPG_하이브리드_예측,LPG_하이브리드_예측_BSpline,LPG_하이브리드_예측_Akima
0,2019,2565.0,NaN,NaN,NaN,55573.0,NaN,NaN,NaN,7793.0,NaN,NaN,NaN
1,2020,3145.0,NaN,NaN,NaN,57682.0,NaN,NaN,NaN,7531.0,NaN,NaN,NaN
2,2021,3777.0,NaN,NaN,NaN,55409.0,NaN,NaN,NaN,6643.0,NaN,NaN,NaN
3,2022,3963.0,NaN,NaN,NaN,48633.0,NaN,NaN,NaN,5804.0,NaN,NaN,NaN
4,2023,3441.0,NaN,NaN,NaN,38361.0,NaN,NaN,NaN,4693.0,NaN,NaN,NaN


In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
df5['테이블생성일자'] = today_date

df5 = df5[[
    '테이블생성일자', 
    '연도', 
    '휘발유_하이브리드_대수', 
    '휘발유_하이브리드_예측',
    '휘발유_하이브리드_예측_BSpline',
    '휘발유_하이브리드_예측_Akima',
    '경유_하이브리드_대수', 
    '경유_하이브리드_예측',
    '경유_하이브리드_예측_BSpline',
    '경유_하이브리드_예측_Akima',
    'LPG_하이브리드_대수', 
    'LPG_하이브리드_예측',
    'LPG_하이브리드_예측_BSpline', 
    'LPG_하이브리드_예측_Akima', 
    ]]
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '연도':'YR', 
    '휘발유_하이브리드_대수':'GSLH', 
    '휘발유_하이브리드_예측':'GSLH_PRET', 
    '휘발유_하이브리드_예측_BSpline':'GSLH_PRET_BSPLN', 
    '휘발유_하이브리드_예측_Akima':'GSLH_PRET_AKM', 
    '경유_하이브리드_대수':'DSLH', 
    '경유_하이브리드_예측':'DSLH_PRET', 
    '경유_하이브리드_예측_BSpline':'DSLH_PRET_BSPLN', 
    '경유_하이브리드_예측_Akima':'DSLH_PRET_AKM', 
    'LPG_하이브리드_대수':'LPGH', 
    'LPG_하이브리드_예측':'LPGH_PRET',
    'LPG_하이브리드_예측_BSpline':'LPGH_PRET_BSPLN',
    'LPG_하이브리드_예측_Akima':'LPGH_PRET_AKM',
}
STD_BD_HYBRD_CAR_PRET = df5.rename(columns=cdict)

In [ ]:
STD_BD_HYBRD_CAR_PRET.columns

Index(['LOAD_DT', 'YR', 'GSLH', 'GSLH_PRET', 'GSLH_PRET_BSPLN',
       'GSLH_PRET_AKM', 'DSLH', 'DSLH_PRET', 'DSLH_PRET_BSPLN',
       'DSLH_PRET_AKM', 'LPGH', 'LPGH_PRET', 'LPGH_PRET_BSPLN',
       'LPGH_PRET_AKM'],
      dtype='object')

### [출력] STD_BD_HYBRD_CAR_PRET

In [ ]:
# expdf = STD_BD_HYBRD_CAR_PRET
# table_nm = 'STD_BD_HYBRD_CAR_PRET'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [ ]:
## 내연차 연료, 등급, 연도별 차량 현황 예측
# - 경유, 휘발유, LPG
# 2022년 차량 대수
grp1 = dfm2dgl.groupby(['시도', 'fuel', '배출가스등급'], as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = f'{year}'
grp1 = grp1[['연도', '시도', 'fuel', '배출가스등급', '차량대수']]

In [ ]:
# sl = grp1.drop_duplicates(['시도', 'fuel', '배출가스등급']).reset_index(drop=True)

In [ ]:
# sl.shape

In [ ]:
# periods = 5 # !!! 수정(2023.08.23)
# y_plist = list(pd.date_range(end=today_date, periods=periods, freq="Y").year) # !!! 수정(2023.08.23)
# # mth_plist = list(pd.date_range(end=today_date, periods=periods, freq="MS").month) # !!! 수정(2023.08.23)
# y_plist = y_plist[:-1]
# # mth_plist = mth_plist[:-1]
# y_plist

In [ ]:
# sl.shape[0] * 4

In [ ]:
# slt = sl
# for _ in range(periods - 2):
#     slt = pd.concat([slt, sl], ignore_index=False)
# slt.shape

In [ ]:
# slt = slt.sort_values(['시도', 'fuel', '배출가스등급']).reset_index(drop=True)
# # slt = slt[['시도', 'fuel', '배출가스등급']]
# slt.head(20)

In [ ]:
# y_plist01 = [str(x) for x in y_plist]
# # mth_plist01 = [f'{x:0>2}' for x in mth_plist]

In [ ]:
# ym = pd.DataFrame({'연도':y_plist01})
# ym

In [ ]:
# ymt = ym
# for _ in range(slt.shape[0]//ym.shape[0] - 1):
#     ymt = pd.concat([ymt, ym], ignore_index=False)
# ymt.shape

In [ ]:
# slt = slt.reset_index(drop=True)
# ymt = ymt.reset_index(drop=True)
# slt.shape, ymt.shape

In [ ]:
# base = pd.concat([ymt, slt], axis=1)
# base.head(20)

In [ ]:
## 차량 통계 기본 데이터셋
yr_list = []
ctpv_list = []
fuel_list = []
grd_list = []
for ctpv in grp1['시도'].unique():
    for fuel in grp1['fuel'].unique():
        for grd in grp1['배출가스등급'].unique():
            for yr in range(2019, year + 1):
                yr_list.append(str(yr))
                ctpv_list.append(ctpv)
                fuel_list.append(fuel)
                grd_list.append(grd)
base = pd.DataFrame({'연도':yr_list, '시도':ctpv_list, 'fuel':fuel_list, '배출가스등급':grd_list})

In [ ]:
## 연도별 등록대수
grp2 = dfm2dgl.groupby(['최초등록일자_년', '시도', 'fuel', '배출가스등급'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})

In [ ]:
## 연도별 말소대수
grp3 = errc2dgl.groupby(['변경일자_년', '시도', 'fuel', '배출가스등급'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
base1 = base.merge(grp1, on=['연도', '시도', 'fuel', '배출가스등급'], how='left')
base2 = base1.merge(grp2, on=['연도', '시도', 'fuel', '배출가스등급'], how='left')
base3 = base2.merge(grp3, on=['연도', '시도', 'fuel', '배출가스등급'], how='left')
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
n = len(base3['연도'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
base3['연도'] = base3['연도'].astype('int') # !!! 수정

In [ ]:
total = pd.DataFrame()
for ctpv in base3['시도'].unique():
    for fuel in base3['fuel'].unique():
        for grd in base3['배출가스등급'].unique():
            temp = base3.loc[(base3['시도'] == ctpv) & (base3['fuel'] == fuel) & (base3['배출가스등급'] == grd)].reset_index(drop=True)
            a, b = np.polyfit(temp['연도'], temp['차량대수'], 1)
            yr_list, ctpv_list, fuel_list, grd_list, pred_list = [], [], [], [], []
            for yr in range(year + 1, 2036):
                pred = a * yr + b
                yr_list.append(yr)
                ctpv_list.append(ctpv)
                fuel_list.append(fuel)
                grd_list.append(grd)
                pred_list.append(pred)
            temp_pred = pd.DataFrame({'연도':yr_list, '시도':ctpv_list, 'fuel':fuel_list, '배출가스등급':grd_list, '차량예측':pred_list})
            ttemp = pd.concat([temp, temp_pred], ignore_index=True)
            total = pd.concat([total, ttemp], ignore_index=True)

In [ ]:
total.shape

(4335, 8)

In [ ]:
total.head()

,연도,시도,fuel,배출가스등급,차량대수,등록대수,말소대수,차량예측
0,2019,강원특별자치도,LPG,1,561.0,11.0,0.0,NaN
1,2020,강원특별자치도,LPG,1,566.0,5.0,0.0,NaN
2,2021,강원특별자치도,LPG,1,570.0,4.0,0.0,NaN
3,2022,강원특별자치도,LPG,1,571.0,1.0,0.0,NaN
4,2023,강원특별자치도,LPG,1,340.0,0.0,231.0,NaN


In [ ]:
total.loc[(total['시도'] == '강원특별자치도') & (total['fuel'] == 'LPG') & (total['배출가스등급'] == '2')]

c:\Users\kbjung\anaconda3\Lib\site-packages\IPython\core\displayhook.py:281: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


,연도,시도,fuel,배출가스등급,차량대수,등록대수,말소대수,차량예측
17,2019,강원특별자치도,LPG,2,2546.0,16.0,1.0,NaN
18,2020,강원특별자치도,LPG,2,2549.0,3.0,0.0,NaN
19,2021,강원특별자치도,LPG,2,2557.0,9.0,1.0,NaN
20,2022,강원특별자치도,LPG,2,2562.0,5.0,0.0,NaN
21,2023,강원특별자치도,LPG,2,1540.0,0.0,1022.0,NaN
22,2024,강원특별자치도,LPG,2,1540.0,0.0,0.0,NaN
23,2025,강원특별자치도,LPG,2,NaN,NaN,NaN,1410.466667
24,2026,강원특별자치도,LPG,2,NaN,NaN,NaN,1180.409524
25,2027,강원특별자치도,LPG,2,NaN,NaN,NaN,950.352381
26,2028,강원특별자치도,LPG,2,NaN,NaN,NaN,720.295238


In [ ]:
# die = base3.loc[base3['fuel'] == '경유', ['연도', 'fuel', '배출가스등급', '차량대수']].reset_index(drop=True)
# gas = base3.loc[base3['fuel'] == '휘발유', ['연도', 'fuel', '배출가스등급', '차량대수']].reset_index(drop=True)
# lpg = base3.loc[base3['fuel'] == 'LPG', ['연도', 'fuel', '배출가스등급', '차량대수']].reset_index(drop=True)

In [ ]:
# die['연도'] = die['연도'].astype('int')
# gas['연도'] = gas['연도'].astype('int')
# lpg['연도'] = lpg['연도'].astype('int')

In [ ]:
# die1 = die.loc[die['배출가스등급'] == '1'].reset_index(drop=True)
# die2 = die.loc[die['배출가스등급'] == '2'].reset_index(drop=True)
# die3 = die.loc[die['배출가스등급'] == '3'].reset_index(drop=True)
# die4 = die.loc[die['배출가스등급'] == '4'].reset_index(drop=True)
# die5 = die.loc[die['배출가스등급'] == '5'].reset_index(drop=True)
# gas1 = gas.loc[gas['배출가스등급'] == '1'].reset_index(drop=True)
# gas2 = gas.loc[gas['배출가스등급'] == '2'].reset_index(drop=True)
# gas3 = gas.loc[gas['배출가스등급'] == '3'].reset_index(drop=True)
# gas4 = gas.loc[gas['배출가스등급'] == '4'].reset_index(drop=True)
# gas5 = gas.loc[gas['배출가스등급'] == '5'].reset_index(drop=True)
# lpg1 = lpg.loc[lpg['배출가스등급'] == '1'].reset_index(drop=True)
# lpg2 = lpg.loc[lpg['배출가스등급'] == '2'].reset_index(drop=True)
# lpg3 = lpg.loc[lpg['배출가스등급'] == '3'].reset_index(drop=True)
# lpg4 = lpg.loc[lpg['배출가스등급'] == '4'].reset_index(drop=True)
# lpg5 = lpg.loc[lpg['배출가스등급'] == '5'].reset_index(drop=True)

In [ ]:
# fit_d1 = np.polyfit(die1['연도'], die1['차량대수'], 1)
# fit_d2 = np.polyfit(die2['연도'], die2['차량대수'], 1)
# fit_d3 = np.polyfit(die3['연도'], die3['차량대수'], 1)
# fit_d4 = np.polyfit(die4['연도'], die4['차량대수'], 1)
# fit_d5 = np.polyfit(die5['연도'], die5['차량대수'], 1)
# fit_g1 = np.polyfit(gas1['연도'], gas1['차량대수'], 1)
# fit_g2 = np.polyfit(gas2['연도'], gas2['차량대수'], 1)
# fit_g3 = np.polyfit(gas3['연도'], gas3['차량대수'], 1)
# fit_g4 = np.polyfit(gas4['연도'], gas4['차량대수'], 1)
# fit_g5 = np.polyfit(gas5['연도'], gas5['차량대수'], 1)
# fit_l1 = np.polyfit(lpg1['연도'], lpg1['차량대수'], 1)
# fit_l2 = np.polyfit(lpg2['연도'], lpg2['차량대수'], 1)
# fit_l3 = np.polyfit(lpg3['연도'], lpg3['차량대수'], 1)
# fit_l4 = np.polyfit(lpg4['연도'], lpg4['차량대수'], 1)
# fit_l5 = np.polyfit(lpg5['연도'], lpg5['차량대수'], 1)

In [ ]:
# ad1, bd1 = fit_d1
# ad2, bd2 = fit_d2
# ad3, bd3 = fit_d3
# ad4, bd4 = fit_d4
# ad5, bd5 = fit_d5
# ag1, bg1 = fit_g1
# ag2, bg2 = fit_g2
# ag3, bg3 = fit_g3
# ag4, bg4 = fit_g4
# ag5, bg5 = fit_g5
# al1, bl1 = fit_l1
# al2, bl2 = fit_l2
# al3, bl3 = fit_l3
# al4, bl4 = fit_l4
# al5, bl5 = fit_l5

In [ ]:
# # 경유 1등급 예측
# yr_list = []
# fuel_list = []
# grd_list = []
# pred_list = []
# fuel = '경유'
# grd = '1'
# for yr in range(year + 1, 2036):
#     pred = ad1 * yr + bd1
#     yr_list.append(yr)
#     fuel_list.append(fuel)
#     grd_list.append(grd)
#     pred_list.append(pred)
# die1_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'경유_예측':pred_list})
# # 경유 2등급 예측
# yr_list = []
# fuel_list = []
# grd_list = []
# pred_list = []
# fuel = '경유'
# grd = '2'
# for yr in range(year + 1, 2036):
#     pred = ad2 * yr + bd2
#     yr_list.append(yr)
#     fuel_list.append(fuel)
#     grd_list.append(grd)
#     pred_list.append(pred)
# die2_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'경유_예측':pred_list})
# # 경유 3등급 예측
# yr_list = []
# fuel_list = []
# grd_list = []
# pred_list = []
# fuel = '경유'
# grd = '3'
# for yr in range(year + 1, 2036):
#     pred = ad3 * yr + bd3
#     yr_list.append(yr)
#     fuel_list.append(fuel)
#     grd_list.append(grd)
#     pred_list.append(pred)
# die3_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'경유_예측':pred_list})
# # 경유 4등급 예측
# yr_list = []
# fuel_list = []
# grd_list = []
# pred_list = []
# fuel = '경유'
# grd = '4'
# for yr in range(year + 1, 2036):
#     pred = ad4 * yr + bd4
#     yr_list.append(yr)
#     fuel_list.append(fuel)
#     grd_list.append(grd)
#     pred_list.append(pred)
# die4_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'경유_예측':pred_list})
# # 경유 5등급 예측
# yr_list = []
# fuel_list = []
# grd_list = []
# pred_list = []
# fuel = '경유'
# grd = '5'
# for yr in range(year + 1, 2036):
#     pred = ad5 * yr + bd5
#     yr_list.append(yr)
#     fuel_list.append(fuel)
#     grd_list.append(grd)
#     pred_list.append(pred)
# die5_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'경유_예측':pred_list})

In [ ]:
# # 휘발유 1등급 예측
# yr_list = []
# fuel_list = []
# grd_list = []
# pred_list = []
# fuel = '휘발유'
# grd = '1'
# for yr in range(year + 1, 2036):
#     pred = ag1 * yr + bg1
#     yr_list.append(yr)
#     fuel_list.append(fuel)
#     grd_list.append(grd)
#     pred_list.append(pred)
# gas1_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'휘발유_예측':pred_list})
# # 휘발유 2등급 예측
# yr_list = []
# fuel_list = []
# grd_list = []
# pred_list = []
# fuel = '휘발유'
# grd = '2'
# for yr in range(year + 1, 2036):
#     pred = ag2 * yr + bg2
#     yr_list.append(yr)
#     fuel_list.append(fuel)
#     grd_list.append(grd)
#     pred_list.append(pred)
# gas2_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'휘발유_예측':pred_list})
# # 휘발유 3등급 예측
# yr_list = []
# fuel_list = []
# grd_list = []
# pred_list = []
# fuel = '휘발유'
# grd = '3'
# for yr in range(year + 1, 2036):
#     pred = ag3 * yr + bg3
#     yr_list.append(yr)
#     fuel_list.append(fuel)
#     grd_list.append(grd)
#     pred_list.append(pred)
# gas3_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'휘발유_예측':pred_list})
# # 휘발유 4등급 예측
# yr_list = []
# fuel_list = []
# grd_list = []
# pred_list = []
# fuel = '휘발유'
# grd = '4'
# for yr in range(year + 1, 2036):
#     pred = ag4 * yr + bg4
#     yr_list.append(yr)
#     fuel_list.append(fuel)
#     grd_list.append(grd)
#     pred_list.append(pred)
# gas4_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'휘발유_예측':pred_list})
# # 휘발유 5등급 예측
# yr_list = []
# fuel_list = []
# grd_list = []
# pred_list = []
# fuel = '휘발유'
# grd = '5'
# for yr in range(year + 1, 2036):
#     pred = ag5 * yr + bg5
#     yr_list.append(yr)
#     fuel_list.append(fuel)
#     grd_list.append(grd)
#     pred_list.append(pred)
# gas5_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'휘발유_예측':pred_list})

In [ ]:
# # LPG 1등급 예측
# yr_list = []
# fuel_list = []
# grd_list = []
# pred_list = []
# fuel = 'LPG'
# grd = '1'
# for yr in range(year + 1, 2036):
#     pred = al1 * yr + bl1
#     yr_list.append(yr)
#     fuel_list.append(fuel)
#     grd_list.append(grd)
#     pred_list.append(pred)
# lpg1_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'LPG_예측':pred_list})
# # LPG 2등급 예측
# yr_list = []
# fuel_list = []
# grd_list = []
# pred_list = []
# fuel = 'LPG'
# grd = '2'
# for yr in range(year + 1, 2036):
#     pred = al2 * yr + bl2
#     yr_list.append(yr)
#     fuel_list.append(fuel)
#     grd_list.append(grd)
#     pred_list.append(pred)
# lpg2_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'LPG_예측':pred_list})
# # LPG 3등급 예측
# yr_list = []
# fuel_list = []
# grd_list = []
# pred_list = []
# fuel = 'LPG'
# grd = '3'
# for yr in range(year + 1, 2036):
#     pred = al3 * yr + bl3
#     yr_list.append(yr)
#     fuel_list.append(fuel)
#     grd_list.append(grd)
#     pred_list.append(pred)
# lpg3_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'LPG_예측':pred_list})
# # LPG 4등급 예측
# yr_list = []
# fuel_list = []
# grd_list = []
# pred_list = []
# fuel = 'LPG'
# grd = '4'
# for yr in range(year + 1, 2036):
#     pred = al4 * yr + bl4
#     yr_list.append(yr)
#     fuel_list.append(fuel)
#     grd_list.append(grd)
#     pred_list.append(pred)
# lpg4_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'LPG_예측':pred_list})
# # LPG 5등급 예측
# yr_list = []
# fuel_list = []
# grd_list = []
# pred_list = []
# fuel = 'LPG'
# grd = '5'
# for yr in range(year + 1, 2036):
#     pred = al5 * yr + bl5
#     yr_list.append(yr)
#     fuel_list.append(fuel)
#     grd_list.append(grd)
#     pred_list.append(pred)
# lpg5_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'LPG_예측':pred_list})

In [ ]:
# die_t = pd.concat([die1, die1_pred, die2, die2_pred, die3, die3_pred, die4, die4_pred, die5, die5_pred], ignore_index=True)
# gas_t = pd.concat([gas1, gas1_pred, gas2, gas2_pred, gas3, gas3_pred, gas4, gas4_pred, gas5, gas5_pred], ignore_index=True)
# lpg_t = pd.concat([lpg1, lpg1_pred, lpg2, lpg2_pred, lpg3, lpg3_pred, lpg4, lpg4_pred, lpg5, lpg5_pred], ignore_index=True)
# die_t = die_t.rename(columns={'경유_예측':'차량예측'})
# gas_t = gas_t.rename(columns={'휘발유_예측':'차량예측'})
# lpg_t = lpg_t.rename(columns={'LPG_예측':'차량예측'})

In [ ]:
# df6 = pd.concat([die_t, gas_t, lpg_t], ignore_index=True)

In [ ]:
# df6.shape

In [ ]:
# # 음수 차량 대수 수정
# df6.loc[df6['차량예측'] < 0, '차량예측'] = 0

In [ ]:
# # 첫째자리까지 반올림
# df6[['차량대수', '차량예측']] = df6[['차량대수', '차량예측']].round(0)

In [ ]:
## 음수 차량 대수 수정
total.loc[total['차량예측'] < 0, '차량예측'] = 0

In [ ]:
total.loc[total['차량예측'] < 0].shape

(0, 8)

In [ ]:
## 첫째자리까지 반올림
total[['차량대수', '차량예측']] = total[['차량대수', '차량예측']].round(0)

In [ ]:
total.loc[(total['시도'] == '충청북도') & (total['배출가스등급'] == '1')]

,연도,시도,fuel,배출가스등급,차량대수,등록대수,말소대수,차량예측
4080,2019,충청북도,LPG,1,4435.0,130.0,144.0,NaN
4081,2020,충청북도,LPG,1,4287.0,66.0,214.0,NaN
4082,2021,충청북도,LPG,1,3764.0,34.0,557.0,NaN
4083,2022,충청북도,LPG,1,3232.0,3.0,535.0,NaN
4084,2023,충청북도,LPG,1,2650.0,0.0,582.0,NaN
4085,2024,충청북도,LPG,1,2650.0,0.0,0.0,NaN
4086,2025,충청북도,LPG,1,NaN,NaN,NaN,2066.0
4087,2026,충청북도,LPG,1,NaN,NaN,NaN,1656.0
4088,2027,충청북도,LPG,1,NaN,NaN,NaN,1245.0
4089,2028,충청북도,LPG,1,NaN,NaN,NaN,835.0


In [ ]:
## 데이터 확인
for ctpv in total['시도'].unique():
    for grd in total['배출가스등급'].unique():
        if total.loc[(total['시도'] == ctpv) & (total['배출가스등급'] == grd)].shape[0] != 51:
            print(ctpv, grd)

In [ ]:
total.shape

(4335, 8)

In [ ]:
len(total['시도'].unique()), len(total['배출가스등급'].unique())

(17, 5)

In [ ]:
4335 / 51 / 5 # 17개 시도

17.0

In [ ]:
df6 = total[['연도', '시도', 'fuel', '배출가스등급', '차량대수', '차량예측']]

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
df6['테이블생성일자'] = today_date
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '연도':'YR', 
    '시도':'CTPV',
    'fuel':'FUEL_CD', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '차량대수':'VHCL_MKCNT', 
    '차량예측':'VHCL_PRET', 
}
STD_BD_FUEL_GRD_VHCL_CURSTT_PRET = df6.rename(columns=cdict)

C:\Users\kbjung\AppData\Local\Temp\ipykernel_17964\455187394.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['테이블생성일자'] = today_date


In [ ]:
STD_BD_FUEL_GRD_VHCL_CURSTT_PRET.columns

Index(['YR', 'CTPV', 'FUEL_CD', 'EXHST_GAS_GRD_CD', 'VHCL_MKCNT', 'VHCL_PRET',
       'LOAD_DT'],
      dtype='object')

### [출력] STD_BD_FUEL_GRD_VHCL_CURSTT_PRET

In [ ]:
# expdf = STD_BD_FUEL_GRD_VHCL_CURSTT_PRET
# table_nm = 'STD_BD_FUEL_GRD_VHCL_CURSTT_PRET'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [ ]:
## 무공해차 연료, 연도별 차량 현황 예측
# - 전기, 수소
dfm2bh = dfm2.loc[(dfm2['fuel'] == '전기') | (dfm2['fuel'] == '수소')].reset_index(drop=True)
errc2bh = errc2.loc[(errc2['fuel'] == '전기') | (errc2['fuel'] == '수소')].reset_index(drop=True)

In [ ]:
## 2022년 차량 대수
grp1 = dfm2bh.groupby('fuel', as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = f'{year}'
grp1 = grp1[['연도', 'fuel', '차량대수']]

In [ ]:
# sl = grp1.drop_duplicates('fuel').reset_index(drop=True)

In [ ]:
# sl.shape

In [ ]:
# periods = 5 # !!! 수정(2023.08.23)
# y_plist = list(pd.date_range(end=today_date, periods=periods, freq="Y").year) # !!! 수정(2023.08.23)
# # mth_plist = list(pd.date_range(end=date, periods=periods, freq="MS").month) # !!! 수정(2023.08.23)
# y_plist = y_plist[:-1]
# # mth_plist = mth_plist[:-1]
# y_plist

In [ ]:
# sl.shape[0] * 4

In [ ]:
# slt = sl
# for _ in range(periods - 2):
#     slt = pd.concat([slt, sl], ignore_index=False)
# slt.shape

In [ ]:
# slt = slt.sort_values('fuel').reset_index(drop=True)
# # slt = slt[['fuel', '시도', '차량대수']]
# slt.head(20)

In [ ]:
# y_plist01 = [str(x) for x in y_plist]
# # mth_plist01 = [f'{x:0>2}' for x in mth_plist]

In [ ]:
# ym = pd.DataFrame({'연도':y_plist01})
# ym

In [ ]:
# ymt = ym
# for _ in range(slt.shape[0]//ym.shape[0] - 1):
#     ymt = pd.concat([ymt, ym], ignore_index=False)
# ymt.shape

In [ ]:
# slt = slt.reset_index(drop=True)
# ymt = ymt.reset_index(drop=True)
# slt.shape, ymt.shape

In [ ]:
# base = pd.concat([ymt, slt], axis=1)
# base.head(20)

In [ ]:
## 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
for fuel in grp1['fuel'].unique():
    for yr in range(2019, year + 1):
        yr_list.append(str(yr))
        fuel_list.append(fuel)
base = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list})

In [ ]:
## 연도별 등록대수
grp2 = dfm2bh.groupby(['최초등록일자_년', 'fuel'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})

In [ ]:
## 연도별 말소대수
grp3 = errc2bh.groupby(['변경일자_년', 'fuel'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
base1 = base.merge(grp1, on=['연도', 'fuel'], how='left')
base2 = base1.merge(grp2, on=['연도', 'fuel'], how='left')
base3 = base2.merge(grp3, on=['연도', 'fuel'], how='left')
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
n = len(base3['연도'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
bt = base3.loc[base3['fuel'] == '전기', ['연도', 'fuel', '차량대수']].reset_index(drop=True)
hy = base3.loc[base3['fuel'] == '수소', ['연도', 'fuel', '차량대수']].reset_index(drop=True)
bt['연도'] = bt['연도'].astype('int')
hy['연도'] = hy['연도'].astype('int')

In [ ]:
## 선형예측
fit1 = np.polyfit(bt['연도'], bt['차량대수'], 1)
fit2 = np.polyfit(hy['연도'], hy['차량대수'], 1)
a1, b1 = fit1
a2, b2 = fit2

In [ ]:
## BSpline 예측
spl1 = intp.BSpline(bt['연도'], bt['차량대수'], 1, extrapolate=True)
spl2 = intp.BSpline(hy['연도'], hy['차량대수'], 1, extrapolate=True)
spl1pred = spl1(range(year + 1, 2036))
spl2pred = spl2(range(year + 1, 2036))

In [ ]:
## akima 예측
aki1 = intp.Akima1DInterpolator(bt['연도'], bt['차량대수'])
aki2 = intp.Akima1DInterpolator(hy['연도'], hy['차량대수'])
aki1pred = aki1([x for x in range(year + 1, 2036)], extrapolate=True)
aki2pred = aki2([x for x in range(year + 1, 2036)], extrapolate=True)

In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '전기'
for yr in range(year + 1, 2036):
    pred = a1 * yr + b1
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
bt_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, f'{fuel}_예측':pred_list})
bt_pred[f'{fuel}_예측_BSpline'] = spl1pred
bt_pred[f'{fuel}_예측_Akima'] = aki1pred

In [ ]:
bt_pred.columns

Index(['연도', '연료', '전기_예측', '전기_예측_BSpline', '전기_예측_Akima'], dtype='object')

In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '수소'
for yr in range(year + 1, 2036):
    pred = a2 * yr + b2
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
hy_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, f'{fuel}_예측':pred_list})
hy_pred[f'{fuel}_예측_BSpline'] = spl2pred
hy_pred[f'{fuel}_예측_Akima'] = aki2pred

In [ ]:
hy_pred.columns

Index(['연도', '연료', '수소_예측', '수소_예측_BSpline', '수소_예측_Akima'], dtype='object')

In [ ]:
bt_t = pd.concat([bt, bt_pred], ignore_index=True)
hy_t = pd.concat([hy, hy_pred], ignore_index=True)
bt_t = bt_t.rename(columns={'차량대수':'전기_대수'})
hy_t = hy_t.rename(columns={'차량대수':'수소_대수'})

In [ ]:
bt_t = bt_t.drop('연료', axis=1)
hy_t = hy_t.drop(['연료', '연도'], axis=1)

In [ ]:
df7 = pd.concat([bt_t, hy_t], axis=1)

In [ ]:
df7.columns

Index(['연도', 'fuel', '전기_대수', '전기_예측', '전기_예측_BSpline', '전기_예측_Akima', 'fuel',
       '수소_대수', '수소_예측', '수소_예측_BSpline', '수소_예측_Akima'],
      dtype='object')

In [ ]:
df7.head()

,연도,fuel,전기_대수,전기_예측,전기_예측_BSpline,전기_예측_Akima,fuel,수소_대수,수소_예측,수소_예측_BSpline,수소_예측_Akima
0,2019,전기,30903.0,NaN,NaN,NaN,수소,300.0,NaN,NaN,NaN
1,2020,전기,33067.0,NaN,NaN,NaN,수소,354.0,NaN,NaN,NaN
2,2021,전기,31358.0,NaN,NaN,NaN,수소,358.0,NaN,NaN,NaN
3,2022,전기,26904.0,NaN,NaN,NaN,수소,292.0,NaN,NaN,NaN
4,2023,전기,20949.0,NaN,NaN,NaN,수소,243.0,NaN,NaN,NaN


In [ ]:
df7.tail()

,연도,fuel,전기_대수,전기_예측,전기_예측_BSpline,전기_예측_Akima,fuel,수소_대수,수소_예측,수소_예측_BSpline,수소_예측_Akima
12,2031,NaN,NaN,2769.542857,-8728.0,-5.095508e+05,NaN,NaN,112.676190,-236.0,-3041.484848
13,2032,NaN,NaN,181.600000,-13182.0,-7.891866e+05,NaN,NaN,93.133333,-302.0,-4835.181818
14,2033,NaN,NaN,-2406.342857,-17636.0,-1.151801e+06,NaN,NaN,73.590476,-368.0,-7173.818182
15,2034,NaN,NaN,-4994.285714,-22090.0,-1.608066e+06,NaN,NaN,54.047619,-434.0,-10128.666667
16,2035,NaN,NaN,-7582.228571,-26544.0,-2.168654e+06,NaN,NaN,34.504762,-500.0,-13771.000000


In [ ]:
## 음수 확인
print(df7.loc[df7['전기_대수'] < 0, '전기_대수'].shape)
print(df7.loc[df7['전기_예측'] < 0, '전기_예측'].shape)
print(df7.loc[df7['전기_예측_BSpline'] < 0, '전기_예측_BSpline'].shape)
print(df7.loc[df7['전기_예측_Akima'] < 0, '전기_예측_Akima'].shape)
print(df7.loc[df7['수소_대수'] < 0, '수소_대수'].shape)
print(df7.loc[df7['수소_예측'] < 0, '수소_예측'].shape)
print(df7.loc[df7['수소_예측_BSpline'] < 0, '수소_예측_BSpline'].shape)
print(df7.loc[df7['수소_예측_Akima'] < 0, '수소_예측_Akima'].shape)

(0,)
(3,)
(6,)
(9,)
(0,)
(0,)
(8,)
(8,)


In [ ]:
## 음수 0으로 처리
df7.loc[df7['전기_대수'] < 0, '전기_대수'] = 0
df7.loc[df7['전기_예측'] < 0, '전기_예측'] = 0
df7.loc[df7['전기_예측_BSpline'] < 0, '전기_예측_BSpline'] = 0
df7.loc[df7['전기_예측_Akima'] < 0, '전기_예측_Akima'] = 0
df7.loc[df7['수소_대수'] < 0, '수소_대수'] = 0
df7.loc[df7['수소_예측'] < 0, '수소_예측'] = 0
df7.loc[df7['수소_예측_BSpline'] < 0, '수소_예측_BSpline'] = 0
df7.loc[df7['수소_예측_Akima'] < 0, '수소_예측_Akima'] = 0

In [ ]:
# 음수 확인
print(df7.loc[df7['전기_대수'] < 0, '전기_대수'].shape)
print(df7.loc[df7['전기_예측'] < 0, '전기_예측'].shape)
print(df7.loc[df7['전기_예측_BSpline'] < 0, '전기_예측_BSpline'].shape)
print(df7.loc[df7['전기_예측_Akima'] < 0, '전기_예측_Akima'].shape)
print(df7.loc[df7['수소_대수'] < 0, '수소_대수'].shape)
print(df7.loc[df7['수소_예측'] < 0, '수소_예측'].shape)
print(df7.loc[df7['수소_예측_BSpline'] < 0, '수소_예측_BSpline'].shape)
print(df7.loc[df7['수소_예측_Akima'] < 0, '수소_예측_Akima'].shape)

(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)


In [ ]:
## 첫째자리까지 반올림
df7.iloc[:, 1:] = df7.iloc[:, 1:].round(0)

In [ ]:
df7.head()

,연도,fuel,전기_대수,전기_예측,전기_예측_BSpline,전기_예측_Akima,fuel,수소_대수,수소_예측,수소_예측_BSpline,수소_예측_Akima
0,2019,전기,30903.0,NaN,NaN,NaN,수소,300.0,NaN,NaN,NaN
1,2020,전기,33067.0,NaN,NaN,NaN,수소,354.0,NaN,NaN,NaN
2,2021,전기,31358.0,NaN,NaN,NaN,수소,358.0,NaN,NaN,NaN
3,2022,전기,26904.0,NaN,NaN,NaN,수소,292.0,NaN,NaN,NaN
4,2023,전기,20949.0,NaN,NaN,NaN,수소,243.0,NaN,NaN,NaN


In [ ]:
## 현재 연도 차량 대수로 예측 대수 채우기
df7.loc[df7['연도'] == year, '전기_예측'] = df7.loc[df7['연도'] == year, '전기_대수']
df7.loc[df7['연도'] == year, '전기_예측_BSpline'] = df7.loc[df7['연도'] == year, '전기_대수']
df7.loc[df7['연도'] == year, '전기_예측_Akima'] = df7.loc[df7['연도'] == year, '전기_대수']
df7.loc[df7['연도'] == year, '수소_예측'] = df7.loc[df7['연도'] == year, '수소_대수']
df7.loc[df7['연도'] == year, '수소_예측_BSpline'] = df7.loc[df7['연도'] == year, '수소_대수']
df7.loc[df7['연도'] == year, '수소_예측_Akima'] = df7.loc[df7['연도'] == year, '수소_대수']

In [ ]:
df7.head()

,연도,fuel,전기_대수,전기_예측,전기_예측_BSpline,전기_예측_Akima,fuel,수소_대수,수소_예측,수소_예측_BSpline,수소_예측_Akima
0,2019,전기,30903.0,NaN,NaN,NaN,수소,300.0,NaN,NaN,NaN
1,2020,전기,33067.0,NaN,NaN,NaN,수소,354.0,NaN,NaN,NaN
2,2021,전기,31358.0,NaN,NaN,NaN,수소,358.0,NaN,NaN,NaN
3,2022,전기,26904.0,NaN,NaN,NaN,수소,292.0,NaN,NaN,NaN
4,2023,전기,20949.0,NaN,NaN,NaN,수소,243.0,NaN,NaN,NaN


In [ ]:
df7.head(10)

,연도,fuel,전기_대수,전기_예측,전기_예측_BSpline,전기_예측_Akima,fuel,수소_대수,수소_예측,수소_예측_BSpline,수소_예측_Akima
0,2019,전기,30903.0,NaN,NaN,NaN,수소,300.0,NaN,NaN,NaN
1,2020,전기,33067.0,NaN,NaN,NaN,수소,354.0,NaN,NaN,NaN
2,2021,전기,31358.0,NaN,NaN,NaN,수소,358.0,NaN,NaN,NaN
3,2022,전기,26904.0,NaN,NaN,NaN,수소,292.0,NaN,NaN,NaN
4,2023,전기,20949.0,NaN,NaN,NaN,수소,243.0,NaN,NaN,NaN
5,2024,전기,20949.0,20949.0,20949.0,20949.0,수소,243.0,243.0,243.0,243.0
6,2025,NaN,NaN,18297.0,17996.0,23347.0,NaN,NaN,230.0,160.0,268.0
7,2026,NaN,NaN,15709.0,13542.0,17470.0,NaN,NaN,210.0,94.0,247.0
8,2027,NaN,NaN,13121.0,9088.0,0.0,NaN,NaN,191.0,28.0,109.0
9,2028,NaN,NaN,10533.0,4634.0,0.0,NaN,NaN,171.0,0.0,0.0


In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
df7['테이블생성일자'] = today_date
df7 = df7[[
    '테이블생성일자', 
    '연도', 
    '전기_대수', 
    '전기_예측',
    '전기_예측_BSpline',
    '전기_예측_Akima',
    '수소_대수', 
    '수소_예측',
    '수소_예측_BSpline',
    '수소_예측_Akima',
    ]]
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '연도':'YR', 
    '전기_대수':'BTYCR', 
    '전기_예측':'BTYCR_PRET',
    '전기_예측_BSpline':'BTYCR_PRET_BSPLN',
    '전기_예측_Akima':'BTYCR_PRET_AKM',
    '수소_대수':'HY', 
    '수소_예측':'HY_PRET',
    '수소_예측_BSpline':'HY_PRET_BSPLN',
    '수소_예측_Akima':'HY_PRET_AKM',    
}
STD_BD_ECO_CAR_PRET = df7.rename(columns=cdict)

In [ ]:
STD_BD_ECO_CAR_PRET.columns

Index(['LOAD_DT', 'YR', 'BTYCR', 'BTYCR_PRET', 'BTYCR_PRET_BSPLN',
       'BTYCR_PRET_AKM', 'HY', 'HY_PRET', 'HY_PRET_BSPLN', 'HY_PRET_AKM'],
      dtype='object')

### [출력] STD_BD_ECO_CAR_PRET

In [ ]:
# expdf = STD_BD_ECO_CAR_PRET
# table_nm = 'STD_BD_ECO_CAR_PRET'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [ ]:
## 지역, 등급별 말소 차량 현황
errc2['배출가스등급'] = errc2['배출가스등급'].map({'1':'1.0', '2':'2.0', '3':'3.0', '4':'4.0', '5':'5.0', 'X':'X'})

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
errc2['테이블생성일자'] = today_date
df8 = errc2[[
    '테이블생성일자', 
    '법정동코드',
    '시도',
    '시군구', 
    '차대번호', 
    '변경일자', 
    '배출가스등급', 
    '연료'
    ]]
chc_col = {
    '테이블생성일자':'LOAD_DT', 
    '법정동코드':'BSPL_STDG_CD',
    '시도':'CTPV',
    '시군구':'SGG', 
    '차대번호':'VIN', 
    '변경일자':'CHG_YMD', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '연료':'FUEL_CD', 
}
STD_BD_ERSR_RSLT = df8.rename(columns=chc_col)

In [ ]:
STD_BD_ERSR_RSLT.columns

Index(['LOAD_DT', 'BSPL_STDG_CD', 'CTPV', 'SGG', 'VIN', 'CHG_YMD',
       'EXHST_GAS_GRD_CD', 'FUEL_CD'],
      dtype='object')

In [ ]:
STD_BD_ERSR_RSLT.tail()

,LOAD_DT,BSPL_STDG_CD,CTPV,SGG,VIN,CHG_YMD,EXHST_GAS_GRD_CD,FUEL_CD
3999918,20240206,4776000000,경상북도,영양군,KMFFE17APVU320407,20190125,NaN,경유
3999919,20240206,2917000000,광주광역시,북구,KN3HAP53A1K738305,20190116,NaN,경유
3999920,20240206,2920000000,광주광역시,광산구,KMJWWH7BPYU243657,20190116,NaN,경유
3999921,20240206,4682000000,전라남도,해남군,KNFWF0A429K351723,20190114,NaN,경유
3999922,20240206,2914000000,광주광역시,서구,KMHGT41CP3U045985,20190108,3.0,휘발유


### [출력] STD_BD_ERSR_RSLT

In [ ]:
# expdf = STD_BD_ERSR_RSLT
# table_nm = 'STD_BD_ERSR_RSLT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [ ]:
## 등급별현황 테이블
# - 시도, 연도, 월, 등급, 연료, 차종, 차량유형, 용도별 / 차량대수, 말소차량대수, 차량 비율
df9 = dfm.copy()

In [ ]:
# 데이터 연도 설정
# year = 2022
# month = 12
today_date = datetime.today().strftime("%Y%m%d")
year = int(today_date[:4])
month = int(today_date[4:6])

In [ ]:
# 차량 대수
grp1 = df9.groupby(['시도', '배출가스등급', '연료', '차종', '차종유형', '용도'], dropna=False)['차대번호'].count().reset_index() # !!! 수정(2023.11.09)
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = f'{year}'
grp1['월'] = f'{month}'
grp1

,시도,배출가스등급,연료,차종,차종유형,용도,차량대수,연도,월
0,강원특별자치도,1,LPG 하이브리드,승용,중형,개인용,41,2024,2
1,강원특별자치도,1,LPG 하이브리드,승용,중형,관용,1,2024,2
2,강원특별자치도,1,LPG(액화석유가스),승용,중형,개인용,104,2024,2
3,강원특별자치도,1,LPG(액화석유가스),승용,중형,영업용,236,2024,2
4,강원특별자치도,1,수소,승용,중형,개인용,1,2024,2
...,...,...,...,...,...,...,...,...,...
4366,NaN,5,휘발유,승용,소형,개인용,1,2024,2
4367,NaN,NaN,경유,화물,소형,개인용,1,2024,2
4368,NaN,NaN,휘발유,승용,대형,개인용,1,2024,2
4369,NaN,NaN,휘발유,승용,중형,개인용,2,2024,2


In [ ]:
# sl = grp1.drop_duplicates(['시도', '배출가스등급', '연료', '차종', '차종유형']).reset_index(drop=True)

In [ ]:
# sl.shape

In [ ]:
# periods = 5
# y_plist = list(pd.date_range(end=today_date, periods=div, freq="MS").year) # !!! 수정(2023.11.09)
# mth_plist = list(pd.date_range(end=today_date, periods=div, freq="MS").month) # !!! 수정(2023.11.09)
# y_plist = y_plist[:-1]
# mth_plist = mth_plist[:-1]
# y_plist, mth_plist

In [ ]:
# sl.shape[0] * 4

In [ ]:
# slt = sl
# for _ in range(periods - 2):
#     slt = pd.concat([slt, sl], ignore_index=False)
# slt.shape

In [ ]:
# slt = slt.sort_values(['시도', '배출가스등급', '연료', '차종', '차종유형']).reset_index(drop=True)
# # slt = slt[['fuel', '시도', '차량대수']]
# slt.head(20)

In [ ]:
# y_plist01 = [str(x) for x in y_plist]
# mth_plist01 = [f'{x:0>2}' for x in mth_plist]

In [ ]:
# ym = pd.DataFrame({'연도':y_plist01, '월':mth_plist01})
# ym

In [ ]:
# ymt = ym
# for _ in range(slt.shape[0]//ym.shape[0] - 1):
#     ymt = pd.concat([ymt, ym], ignore_index=False)
# ymt.shape

In [ ]:
# slt = slt.reset_index(drop=True)
# ymt = ymt.reset_index(drop=True)
# slt.shape, ymt.shape

In [ ]:
# base = pd.concat([ymt, slt], axis=1)
# base.head(20)

In [ ]:
# 날짜 설정
# date_date = '20221231'
date_date = datetime.today().strftime("%Y%m%d")
# 37.5s

In [ ]:
div = 5
y_plist = list(pd.date_range(end=date_date, periods=div, freq="MS").year) # !!! 수정(2023.11.09)
mth_plist = list(pd.date_range(end=date_date, periods=div, freq="MS").month) # !!! 수정(2023.11.09)

y_plist, mth_plist

([2023, 2023, 2023, 2024, 2024], [10, 11, 12, 1, 2])

In [ ]:
## 37.5s
# 차량 통계 기본 데이터셋
ctpv_list, yr_list, month_list, grd_list, fuel_list, vhcty_list, ty_list, purps_list = [], [], [], [], [], [], [], []
for ctpv in grp1['시도'].unique():
    for yr, month in zip(y_plist, mth_plist):
        for grd in grp1['배출가스등급'].unique():
            for fuel in grp1['연료'].unique():
                for vhcty in grp1['차종'].unique():
                    for ty in grp1['차종유형'].unique():
                        for purps in grp1['용도'].unique():
                            ctpv_list.append(ctpv)
                            yr_list.append(str(yr))
                            month_list.append(f'{month:0>2}')
                            grd_list.append(grd)
                            fuel_list.append(fuel)
                            vhcty_list.append(vhcty)
                            ty_list.append(ty)
                            purps_list.append(purps)
base = pd.DataFrame({
    '시도':ctpv_list, 
    '연도':yr_list, 
    '월':month_list, 
    '배출가스등급':grd_list, 
    '연료':fuel_list, 
    '차종':vhcty_list, 
    '차종유형':ty_list, 
    '용도':purps_list, 
    })

In [ ]:
base

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도
0,강원특별자치도,2023,10,1,LPG 하이브리드,승용,중형,개인용
1,강원특별자치도,2023,10,1,LPG 하이브리드,승용,중형,관용
2,강원특별자치도,2023,10,1,LPG 하이브리드,승용,중형,영업용
3,강원특별자치도,2023,10,1,LPG 하이브리드,승용,경형,개인용
4,강원특별자치도,2023,10,1,LPG 하이브리드,승용,경형,관용
...,...,...,...,...,...,...,...,...
518395,NaN,2024,02,NaN,NaN,특수,대형,관용
518396,NaN,2024,02,NaN,NaN,특수,대형,영업용
518397,NaN,2024,02,NaN,NaN,특수,NaN,개인용
518398,NaN,2024,02,NaN,NaN,특수,NaN,관용


In [ ]:
## 13.6s
# 연도별 등록대수
grp2 = df9.groupby(['시도', '최초등록일자_년', '최초등록일자_월', '배출가스등급', '연료', '차종', '차종유형', '용도'], dropna=False)['차대번호'].count().reset_index() # !!! 수정(2023.11.09)
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '최초등록일자_월':'월', '차대번호':'등록대수'})
grp2

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도,등록대수
0,강원특별자치도,1920,01,NaN,휘발유,승용,대형,개인용,1
1,강원특별자치도,1981,12,5,휘발유,승용,소형,개인용,1
2,강원특별자치도,1983,06,5,경유,화물,소형,개인용,1
3,강원특별자치도,1984,06,5,경유,화물,소형,개인용,1
4,강원특별자치도,1985,01,5,경유,화물,중형,개인용,1
...,...,...,...,...,...,...,...,...,...
174874,NaN,2023,01,NaN,휘발유,승용,중형,개인용,1
174875,NaN,2023,05,NaN,휘발유,승용,대형,개인용,1
174876,NaN,2023,10,NaN,경유,화물,소형,개인용,1
174877,NaN,2023,11,NaN,휘발유 하이브리드,승용,중형,개인용,2


In [ ]:
## 2.5s
# 연도별 말소대수
grp3 = errc.groupby(['시도', '변경일자_년', '변경일자_월', '배출가스등급', '연료', '차종', '차종유형', '용도'], dropna=False)['차대번호'].count().reset_index() # !!! 수정(2023.11.09)
grp3 = grp3.rename(columns={'변경일자_년':'연도', '변경일자_월':'월', '차대번호':'말소대수'})
grp3

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도,말소대수
0,강원특별자치도,2019,07,2,LPG(액화석유가스),승용,중형,개인용,1
1,강원특별자치도,2020,02,2,휘발유,승용,경형,개인용,1
2,강원특별자치도,2021,06,2,LPG(액화석유가스),승용,중형,개인용,1
3,강원특별자치도,2021,08,4,경유,화물,소형,개인용,1
4,강원특별자치도,2022,01,4,경유,화물,소형,개인용,1
...,...,...,...,...,...,...,...,...,...
70865,충청북도,2023,11,5,경유,화물,중형,개인용,4
70866,충청북도,2023,11,5,휘발유,승용,중형,개인용,1
70867,충청북도,2023,11,NaN,휘발유,승용,경형,개인용,1
70868,충청북도,2023,11,NaN,휘발유,승용,대형,개인용,1


In [ ]:
base1 = base.merge(grp1, on=['시도', '연도', '월', '배출가스등급', '연료', '차종', '차종유형', '용도'], how='left')
base2 = base1.merge(grp2, on=['시도', '연도', '월', '배출가스등급', '연료', '차종', '차종유형', '용도'], how='left')
base3 = base2.merge(grp3, on=['시도', '연도', '월', '배출가스등급', '연료', '차종', '차종유형', '용도'], how='left')
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)
base3 = base3.sort_values(['시도', '배출가스등급', '연료', '차종', '차종유형', '용도', '연도', '월']).reset_index(drop=True)
base3

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도,차량대수,등록대수,말소대수
0,강원특별자치도,2023,10,1,9999,승용,경형,개인용,0.0,0.0,0.0
1,강원특별자치도,2023,11,1,9999,승용,경형,개인용,0.0,0.0,0.0
2,강원특별자치도,2023,12,1,9999,승용,경형,개인용,0.0,0.0,0.0
3,강원특별자치도,2024,01,1,9999,승용,경형,개인용,0.0,0.0,0.0
4,강원특별자치도,2024,02,1,9999,승용,경형,개인용,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
518395,NaN,2023,10,NaN,NaN,화물,NaN,영업용,0.0,0.0,0.0
518396,NaN,2023,11,NaN,NaN,화물,NaN,영업용,0.0,0.0,0.0
518397,NaN,2023,12,NaN,NaN,화물,NaN,영업용,0.0,0.0,0.0
518398,NaN,2024,01,NaN,NaN,화물,NaN,영업용,0.0,0.0,0.0


In [ ]:
base3['월'].unique()

array(['10', '11', '12', '01', '02'], dtype=object)

In [ ]:
## 1m 28.6s
n = len(base3['월'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
base3

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도,차량대수,등록대수,말소대수
0,강원특별자치도,2023,10,1,9999,승용,경형,개인용,0.0,0.0,0.0
1,강원특별자치도,2023,11,1,9999,승용,경형,개인용,0.0,0.0,0.0
2,강원특별자치도,2023,12,1,9999,승용,경형,개인용,0.0,0.0,0.0
3,강원특별자치도,2024,01,1,9999,승용,경형,개인용,0.0,0.0,0.0
4,강원특별자치도,2024,02,1,9999,승용,경형,개인용,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
518395,NaN,2023,10,NaN,NaN,화물,NaN,영업용,0.0,0.0,0.0
518396,NaN,2023,11,NaN,NaN,화물,NaN,영업용,0.0,0.0,0.0
518397,NaN,2023,12,NaN,NaN,화물,NaN,영업용,0.0,0.0,0.0
518398,NaN,2024,01,NaN,NaN,화물,NaN,영업용,0.0,0.0,0.0


In [ ]:
base3.loc[[x for x in range(0, base3.shape[0], div)], '차량대수'] = 0 # !!! 수정(2023.11.09)

In [ ]:
anl1 = base3[['시도', '연도', '월', '배출가스등급', '연료', '차종', '차종유형', '용도', '차량대수', '말소대수']]
anl1

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도,차량대수,말소대수
0,강원특별자치도,2023,10,1,9999,승용,경형,개인용,0.0,0.0
1,강원특별자치도,2023,11,1,9999,승용,경형,개인용,0.0,0.0
2,강원특별자치도,2023,12,1,9999,승용,경형,개인용,0.0,0.0
3,강원특별자치도,2024,01,1,9999,승용,경형,개인용,0.0,0.0
4,강원특별자치도,2024,02,1,9999,승용,경형,개인용,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
518395,NaN,2023,10,NaN,NaN,화물,NaN,영업용,0.0,0.0
518396,NaN,2023,11,NaN,NaN,화물,NaN,영업용,0.0,0.0
518397,NaN,2023,12,NaN,NaN,화물,NaN,영업용,0.0,0.0
518398,NaN,2024,01,NaN,NaN,화물,NaN,영업용,0.0,0.0


In [ ]:
anl1['연도_월별_차량대수'] = anl1.groupby(['연도', '월'])['차량대수'].transform('sum') # !!! 수정(2023.11.09)
anl1

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도,차량대수,말소대수,연도_월별_차량대수
0,강원특별자치도,2023,10,1,9999,승용,경형,개인용,0.0,0.0,0.0
1,강원특별자치도,2023,11,1,9999,승용,경형,개인용,0.0,0.0,-333.0
2,강원특별자치도,2023,12,1,9999,승용,경형,개인용,0.0,0.0,0.0
3,강원특별자치도,2024,01,1,9999,승용,경형,개인용,0.0,0.0,0.0
4,강원특별자치도,2024,02,1,9999,승용,경형,개인용,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
518395,NaN,2023,10,NaN,NaN,화물,NaN,영업용,0.0,0.0,0.0
518396,NaN,2023,11,NaN,NaN,화물,NaN,영업용,0.0,0.0,-333.0
518397,NaN,2023,12,NaN,NaN,화물,NaN,영업용,0.0,0.0,0.0
518398,NaN,2024,01,NaN,NaN,화물,NaN,영업용,0.0,0.0,0.0


In [ ]:
anl1['연도_월_연료별_차량대수'] = anl1.groupby(['연도', '월', '연료'])['차량대수'].transform('sum') # !!! 수정(2023.11.09)
anl1

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도,차량대수,말소대수,연도_월별_차량대수,연도_월_연료별_차량대수
0,강원특별자치도,2023,10,1,9999,승용,경형,개인용,0.0,0.0,0.0,0.0
1,강원특별자치도,2023,11,1,9999,승용,경형,개인용,0.0,0.0,-333.0,-1.0
2,강원특별자치도,2023,12,1,9999,승용,경형,개인용,0.0,0.0,0.0,0.0
3,강원특별자치도,2024,01,1,9999,승용,경형,개인용,0.0,0.0,0.0,0.0
4,강원특별자치도,2024,02,1,9999,승용,경형,개인용,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
518395,NaN,2023,10,NaN,NaN,화물,NaN,영업용,0.0,0.0,0.0,NaN
518396,NaN,2023,11,NaN,NaN,화물,NaN,영업용,0.0,0.0,-333.0,NaN
518397,NaN,2023,12,NaN,NaN,화물,NaN,영업용,0.0,0.0,0.0,NaN
518398,NaN,2024,01,NaN,NaN,화물,NaN,영업용,0.0,0.0,0.0,NaN


In [ ]:
anl1['연도_연료차량비율'] = anl1['연도_월_연료별_차량대수'] / anl1['연도_월별_차량대수'] # !!! 수정(2023.11.09)
anl1['연도_연료차량비율'] = anl1['연도_연료차량비율'].round(2) # !!! 수정(2023.11.09)
anl1['연도_연료차량비율'] = anl1['연도_연료차량비율'].fillna(0) # !!! 수정(2023.11.09)
anl1

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도,차량대수,말소대수,연도_월별_차량대수,연도_월_연료별_차량대수,연도_연료차량비율
0,강원특별자치도,2023,10,1,9999,승용,경형,개인용,0.0,0.0,0.0,0.0,0.0
1,강원특별자치도,2023,11,1,9999,승용,경형,개인용,0.0,0.0,-333.0,-1.0,0.0
2,강원특별자치도,2023,12,1,9999,승용,경형,개인용,0.0,0.0,0.0,0.0,0.0
3,강원특별자치도,2024,01,1,9999,승용,경형,개인용,0.0,0.0,0.0,0.0,0.0
4,강원특별자치도,2024,02,1,9999,승용,경형,개인용,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
518395,NaN,2023,10,NaN,NaN,화물,NaN,영업용,0.0,0.0,0.0,NaN,0.0
518396,NaN,2023,11,NaN,NaN,화물,NaN,영업용,0.0,0.0,-333.0,NaN,0.0
518397,NaN,2023,12,NaN,NaN,화물,NaN,영업용,0.0,0.0,0.0,NaN,0.0
518398,NaN,2024,01,NaN,NaN,화물,NaN,영업용,0.0,0.0,0.0,NaN,0.0


In [ ]:
anl1[anl1['연도_연료차량비율'] < 0].shape

(0, 13)

In [ ]:
anl1[anl1['연도_연료차량비율'] > 0.1]

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도,차량대수,말소대수,연도_월별_차량대수,연도_월_연료별_차량대수,연도_연료차량비율
1801,강원특별자치도,2023,11,1,경유,승용,경형,개인용,0.0,0.0,-333.0,-85.0,0.26
1806,강원특별자치도,2023,11,1,경유,승용,경형,관용,0.0,0.0,-333.0,-85.0,0.26
1811,강원특별자치도,2023,11,1,경유,승용,경형,영업용,0.0,0.0,-333.0,-85.0,0.26
1816,강원특별자치도,2023,11,1,경유,승용,대형,개인용,0.0,0.0,-333.0,-85.0,0.26
1821,강원특별자치도,2023,11,1,경유,승용,대형,관용,0.0,0.0,-333.0,-85.0,0.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...
518076,NaN,2023,11,NaN,휘발유 하이브리드,화물,중형,관용,0.0,0.0,-333.0,-85.0,0.26
518081,NaN,2023,11,NaN,휘발유 하이브리드,화물,중형,영업용,0.0,0.0,-333.0,-85.0,0.26
518086,NaN,2023,11,NaN,휘발유 하이브리드,화물,NaN,개인용,0.0,0.0,-333.0,-85.0,0.26
518091,NaN,2023,11,NaN,휘발유 하이브리드,화물,NaN,관용,0.0,0.0,-333.0,-85.0,0.26


In [ ]:
anl1[anl1['배출가스등급'].isnull()].shape

(86400, 13)

In [ ]:
anl1 = anl1[(anl1['배출가스등급'].notnull()) & (anl1['연료'].notnull())] # !!! 수정(2023.11.09)
anl1.shape

(405000, 13)

In [ ]:
anl1[['배출가스등급', '연료']].isnull().sum()

배출가스등급    0
연료        0
dtype: int64

In [ ]:
anl1['연도_연료차량비율'] = anl1['연도_연료차량비율'].round(2) # !!! 수정(2023.11.10)
anl1[['연도_연료차량비율']].head()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_17964\1181954589.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anl1['연도_연료차량비율'] = anl1['연도_연료차량비율'].round(2) # !!! 수정(2023.11.10)


,연도_연료차량비율
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [ ]:
today_date = datetime.today().strftime('%Y%m%d')
today_date

'20240206'

In [ ]:
anl1['테이블생성일자'] = today_date

C:\Users\kbjung\AppData\Local\Temp\ipykernel_17964\3278807475.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anl1['테이블생성일자'] = today_date


In [ ]:
STD_BD_DAT_GRD_CURSTT = anl1[[
    '시도',
    '연도',
    '월',
    '배출가스등급',
    '연료',
    '차종',
    '차종유형',
    '용도',
    '차량대수',
    '말소대수',
    '연도_연료차량비율',
    '테이블생성일자'
    ]]
cdict = {
    '시도':'CTPV', 
    '연도':'YR', 
    '월':'MM', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '연료':'FUEL_CD', 
    '차종':'VHCTY_CD', 
    '차종유형':'VHCTY_TY', 
    '용도':'PURPS_CD2', 
    '차량대수':'VHCL_MKCNT', 
    '말소대수':'ERSR_MKCNT', 
    '연도_연료차량비율':'YR_FUEL_VHCL_RT', 
    '테이블생성일자':'LOAD_DT', 
}
STD_BD_DAT_GRD_CURSTT = STD_BD_DAT_GRD_CURSTT.rename(columns=cdict)

In [ ]:
STD_BD_DAT_GRD_CURSTT.columns

Index(['CTPV', 'YR', 'MM', 'EXHST_GAS_GRD_CD', 'FUEL_CD', 'VHCTY_CD',
       'VHCTY_TY', 'PURPS_CD2', 'VHCL_MKCNT', 'ERSR_MKCNT', 'YR_FUEL_VHCL_RT',
       'LOAD_DT'],
      dtype='object')

### [출력] STD_BD_DAT_GRD_CURSTT

In [ ]:
# expdf = STD_BD_DAT_GRD_CURSTT
# table_nm = 'STD_BD_DAT_GRD_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [ ]:
## 내연기관차 감소추이
grp1 = dfm2dgl.groupby(['배출가스등급', '연료'])['차대번호'].count().reset_index()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})

In [ ]:
grp1

,배출가스등급,연료,차량대수
0,1,LPG(액화석유가스),93672
1,1,휘발유,25132
2,2,LPG(액화석유가스),232564
3,2,휘발유,5450971
4,3,LPG(액화석유가스),262975
5,3,경유,535465
6,3,휘발유,16308844
7,4,LPG(액화석유가스),14632
8,4,경유,422988
9,4,휘발유,326551


In [ ]:
# sl = grp1.drop_duplicates(['배출가스등급', '연료']).reset_index(drop=True)

In [ ]:
# sl.shape

In [ ]:
# periods = 13 # !!! 수정(2023.08.23)
# y_plist = list(pd.date_range(end=date, periods=periods, freq="MS").year) # !!! 수정(2023.08.23)
# mth_plist = list(pd.date_range(end=date, periods=periods, freq="MS").month) # !!! 수정(2023.08.23)
# y_plist = y_plist[:-1]
# mth_plist = mth_plist[:-1]
# y_plist

In [ ]:
# sl.shape[0] * periods

In [ ]:
# slt = sl
# for _ in range(periods - 2):
#     slt = pd.concat([slt, sl], ignore_index=False)
# slt.shape

In [ ]:
# slt = slt.sort_values(['배출가스등급', '연료']).reset_index(drop=True)
# # slt = slt[['fuel', '시도', '차량대수']]
# slt.head(20)

In [ ]:
# y_plist01 = [str(x) for x in y_plist]
# mth_plist01 = [f'{x:0>2}' for x in mth_plist]

In [ ]:
# ym = pd.DataFrame({'연도':y_plist01, '월':mth_plist01})
# ym

In [ ]:
# ymt = ym
# for _ in range(slt.shape[0]//ym.shape[0] - 1):
#     ymt = pd.concat([ymt, ym], ignore_index=False)
# ymt.shape

In [ ]:
# slt = slt.reset_index(drop=True)
# ymt = ymt.reset_index(drop=True)
# slt.shape, ymt.shape

In [ ]:
# base = pd.concat([ymt, slt], axis=1)
# base.head(20)

In [ ]:
## year = '2022'
## month = '12'
today_date = datetime.today().strftime("%Y%m%d")
year = today_date[:4]
month = today_date[4:6]
grp1[['연도', '월']] = [year, month]
year, month

('2024', '02')

In [ ]:
grp1

,배출가스등급,연료,차량대수,연도,월
0,1,LPG(액화석유가스),93672,2024,02
1,1,휘발유,25132,2024,02
2,2,LPG(액화석유가스),232564,2024,02
3,2,휘발유,5450971,2024,02
4,3,LPG(액화석유가스),262975,2024,02
5,3,경유,535465,2024,02
6,3,휘발유,16308844,2024,02
7,4,LPG(액화석유가스),14632,2024,02
8,4,경유,422988,2024,02
9,4,휘발유,326551,2024,02


In [ ]:
sorted(['1', '2', '3', '4', '5', 'X'])

['1', '2', '3', '4', '5', 'X']

In [ ]:
sorted(grp1['배출가스등급'].unique())

['1', '2', '3', '4', '5']

In [ ]:
yr_list, month_list, grd_list, fuel_list = [], [], [], []
for grd in sorted(grp1['배출가스등급'].unique()):
    for fuel in grp1['연료'].unique():
        for yr in range(2019, int(year) + 1):
            for month in range(1, 13):
                yr_list.append(str(yr))
                month_list.append(f'{month:0>2}')
                grd_list.append(grd)
                fuel_list.append(fuel)
base = pd.DataFrame({'연도':yr_list, '월':month_list, '배출가스등급':grd_list, '연료':fuel_list})

In [ ]:
base

,연도,월,배출가스등급,연료
0,2019,01,1,LPG(액화석유가스)
1,2019,02,1,LPG(액화석유가스)
2,2019,03,1,LPG(액화석유가스)
3,2019,04,1,LPG(액화석유가스)
4,2019,05,1,LPG(액화석유가스)
...,...,...,...,...
1075,2024,08,5,경유
1076,2024,09,5,경유
1077,2024,10,5,경유
1078,2024,11,5,경유


In [ ]:
dfm2dgl.shape

(25110248, 34)

In [ ]:
grp2 = dfm2dgl.groupby(['최초등록일자_년', '최초등록일자_월', '배출가스등급', '연료'], dropna=False)['차대번호'].count().reset_index()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '최초등록일자_월':'월', '차대번호':'등록대수'})

In [ ]:
grp2.head()

,연도,월,배출가스등급,연료,등록대수
0,1920,01,1,휘발유,3
1,1920,01,2,휘발유,1
2,1920,01,3,경유,1
3,1920,01,3,휘발유,2
4,1920,01,NaN,경유,8


In [ ]:
grp3 = errc2dgl.groupby(['변경일자_년', '변경일자_월', '배출가스등급', '연료'], dropna=False)['차대번호'].count().reset_index()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '변경일자_월':'월', '차대번호':'말소대수'})

In [ ]:
base1 = base.merge(grp1, on=['연도', '월', '배출가스등급', '연료'], how='left')
base2 = base1.merge(grp2, on=['연도', '월', '배출가스등급', '연료'], how='left')
base3 = base2.merge(grp3, on=['연도', '월', '배출가스등급', '연료'], how='left')
base3 = base3.sort_values(['배출가스등급', '연료', '연도', '월']).reset_index(drop=True)
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
base1

,연도,월,배출가스등급,연료,차량대수
0,2019,01,1,LPG(액화석유가스),NaN
1,2019,02,1,LPG(액화석유가스),NaN
2,2019,03,1,LPG(액화석유가스),NaN
3,2019,04,1,LPG(액화석유가스),NaN
4,2019,05,1,LPG(액화석유가스),NaN
...,...,...,...,...,...
1075,2024,08,5,경유,NaN
1076,2024,09,5,경유,NaN
1077,2024,10,5,경유,NaN
1078,2024,11,5,경유,NaN


In [ ]:
len(base3['연도'].unique()) * len(base3['월'].unique())

72

In [ ]:
n = len(base3['연도'].unique()) * len(base3['월'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
base3.head()

,연도,월,배출가스등급,연료,차량대수,등록대수,말소대수
0,2019,01,1,LPG(액화석유가스),59912.0,434.0,0.0
1,2019,02,1,LPG(액화석유가스),60322.0,410.0,0.0
2,2019,03,1,LPG(액화석유가스),60767.0,445.0,0.0
3,2019,04,1,LPG(액화석유가스),61294.0,527.0,0.0
4,2019,05,1,LPG(액화석유가스),61809.0,515.0,0.0


In [ ]:
mth_plist01

['02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '01']

In [ ]:
pred_grd_list, pred_fuel_list, pred_yr_list, pred_month_list = [], [], [], []
for grd in sorted(grp1['배출가스등급'].unique()):
    for fuel in grp1['연료'].unique():
        for yr in range(int(year) + 1, 2036):
            for month in range(1, 13):
        # for yr in y_plist01:
        #     for month in mth_plist01:
                pred_grd_list.append(grd)
                pred_fuel_list.append(fuel)
                pred_yr_list.append(str(yr))
                pred_month_list.append(f'{month:0>2}')
                # pred_yr_list.append(yr)
                # pred_month_list.append(month)
pred_df = pd.DataFrame({'연도':pred_yr_list, '연료':pred_fuel_list, '월':pred_month_list, '배출가스등급':pred_grd_list})

In [ ]:
pred_df.head()

,연도,연료,월,배출가스등급
0,2025,LPG(액화석유가스),01,1
1,2025,LPG(액화석유가스),02,1
2,2025,LPG(액화석유가스),03,1
3,2025,LPG(액화석유가스),04,1
4,2025,LPG(액화석유가스),05,1


In [ ]:
total_base = pd.concat([base3, pred_df], ignore_index=True)
total_base = total_base.sort_values(['배출가스등급', '연료', '연도', '월']).reset_index(drop=True)

In [ ]:
total_base

,연도,월,배출가스등급,연료,차량대수,등록대수,말소대수
0,2019,01,1,LPG(액화석유가스),59912.0,434.0,0.0
1,2019,02,1,LPG(액화석유가스),60322.0,410.0,0.0
2,2019,03,1,LPG(액화석유가스),60767.0,445.0,0.0
3,2019,04,1,LPG(액화석유가스),61294.0,527.0,0.0
4,2019,05,1,LPG(액화석유가스),61809.0,515.0,0.0
...,...,...,...,...,...,...,...
3055,2035,08,5,휘발유,NaN,NaN,NaN
3056,2035,09,5,휘발유,NaN,NaN,NaN
3057,2035,10,5,휘발유,NaN,NaN,NaN
3058,2035,11,5,휘발유,NaN,NaN,NaN


In [ ]:
total_pred_df = pd.DataFrame()
for fuel in total_base['연료'].unique():
    for grd in total_base['배출가스등급'].unique():
        temp = total_base.loc[(total_base['연료'] == fuel) & (total_base['배출가스등급'] == grd)].reset_index(drop=True).reset_index()
        present = temp[temp['연도'] <= year]
        future = temp.loc[temp['연도'] > year, ['index', '연도', '월', '배출가스등급', '연료']]
        if fuel == 'LPG(액화석유가스)':
            fuel_mod = 'LPG'
        else:
            fuel_mod = fuel
        present = present.rename(columns={'차량대수':f"{fuel_mod}_대수"})
        # 선형예측
        a, b = np.polyfit(present['index'], present[f"{fuel_mod}_대수"], 1)
        future[f'{fuel_mod}_예측'] = a * future['index'] + b
        # BSpline 예측
        spl = intp.BSpline(present['index'], present[f"{fuel_mod}_대수"], 1, extrapolate=True)
        future[f'{fuel_mod}_예측_BSpline'] = spl(future['index'])
        # Akima 예측
        aki = intp.Akima1DInterpolator(present['index'], present[f"{fuel_mod}_대수"])
        future[f'{fuel_mod}_예측_Akima'] = aki(future['index'], extrapolate=True)
        temp2 = pd.concat([present, future], ignore_index=True)
        total_pred_df = pd.concat([total_pred_df, temp2], ignore_index=True)       

In [ ]:
total_pred_df.head()

,index,연도,월,배출가스등급,연료,LPG_대수,등록대수,말소대수,LPG_예측,LPG_예측_BSpline,LPG_예측_Akima,경유_대수,경유_예측,경유_예측_BSpline,경유_예측_Akima,휘발유_대수,휘발유_예측,휘발유_예측_BSpline,휘발유_예측_Akima
0,0,2019,01,1,LPG(액화석유가스),59912.0,434.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2019,02,1,LPG(액화석유가스),60322.0,410.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2019,03,1,LPG(액화석유가스),60767.0,445.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2019,04,1,LPG(액화석유가스),61294.0,527.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2019,05,1,LPG(액화석유가스),61809.0,515.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
total_pred_df.tail()

,index,연도,월,배출가스등급,연료,LPG_대수,등록대수,말소대수,LPG_예측,LPG_예측_BSpline,LPG_예측_Akima,경유_대수,경유_예측,경유_예측_BSpline,경유_예측_Akima,휘발유_대수,휘발유_예측,휘발유_예측_BSpline,휘발유_예측_Akima
3055,199,2035,08,5,휘발유,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2392.527365,0.0,0.0
3056,200,2035,09,5,휘발유,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2410.677889,0.0,0.0
3057,201,2035,10,5,휘발유,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2428.828413,0.0,0.0
3058,202,2035,11,5,휘발유,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2446.978938,0.0,0.0
3059,203,2035,12,5,휘발유,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2465.129462,0.0,0.0


In [ ]:
df5 = total_pred_df[[
    '연도',
    '월',
    '배출가스등급',
    '연료',
    'LPG_대수',
    'LPG_예측',
    'LPG_예측_BSpline',
    'LPG_예측_Akima',
    '경유_대수',
    '경유_예측',
    '경유_예측_BSpline',
    '경유_예측_Akima',
    '휘발유_대수',
    '휘발유_예측',
    '휘발유_예측_BSpline',
    '휘발유_예측_Akima'
]]

In [ ]:
## 음수 0으로 처리
df5.loc[df5['경유_대수'] < 0, '경유_대수'] = 0
df5.loc[df5['휘발유_대수'] < 0, '휘발유_대수'] = 0
df5.loc[df5['LPG_대수'] < 0, 'LPG_대수'] = 0

df5.loc[df5['경유_예측'] < 0, '경유_예측'] = 0
df5.loc[df5['경유_예측_BSpline'] < 0, '경유_예측_BSpline'] = 0
df5.loc[df5['경유_예측_Akima'] < 0, '경유_예측_Akima'] = 0
df5.loc[df5['휘발유_예측'] < 0, '휘발유_예측'] = 0
df5.loc[df5['휘발유_예측_BSpline'] < 0, '휘발유_예측_BSpline'] = 0
df5.loc[df5['휘발유_예측_Akima'] < 0, '휘발유_예측_Akima'] = 0
df5.loc[df5['LPG_예측'] < 0, 'LPG_예측'] = 0
df5.loc[df5['LPG_예측_BSpline'] < 0, 'LPG_예측_BSpline'] = 0
df5.loc[df5['LPG_예측_Akima'] < 0, 'LPG_예측_Akima'] = 0

In [ ]:
## 첫째자리에서 반올림
df5[['경유_대수', '휘발유_대수', 'LPG_대수', '경유_예측', '경유_예측_BSpline','경유_예측_Akima', '휘발유_예측', '휘발유_예측_BSpline', '휘발유_예측_Akima', 'LPG_예측', 'LPG_예측_BSpline', 'LPG_예측_Akima']] = df5[['경유_대수', '휘발유_대수', 'LPG_대수', '경유_예측', '경유_예측_BSpline','경유_예측_Akima', '휘발유_예측', '휘발유_예측_BSpline', '휘발유_예측_Akima', 'LPG_예측', 'LPG_예측_BSpline', 'LPG_예측_Akima']].round(0)

C:\Users\kbjung\AppData\Local\Temp\ipykernel_17964\2976025369.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5[['경유_대수', '휘발유_대수', 'LPG_대수', '경유_예측', '경유_예측_BSpline','경유_예측_Akima', '휘발유_예측', '휘발유_예측_BSpline', '휘발유_예측_Akima', 'LPG_예측', 'LPG_예측_BSpline', 'LPG_예측_Akima']] = df5[['경유_대수', '휘발유_대수', 'LPG_대수', '경유_예측', '경유_예측_BSpline','경유_예측_Akima', '휘발유_예측', '휘발유_예측_BSpline', '휘발유_예측_Akima', 'LPG_예측', 'LPG_예측_BSpline', 'LPG_예측_Akima']].round(0)


In [ ]:
## 분기 정보 추가
df6 = df5.loc[(df5['월'] == '03') | (df5['월'] == '06') | (df5['월'] == '09') | (df5['월'] == '12')]
df6.loc[df6['월'] == '03' , '분기'] = '1'
df6.loc[df6['월'] == '06' , '분기'] = '2'
df6.loc[df6['월'] == '09' , '분기'] = '3'
df6.loc[df6['월'] == '12' , '분기'] = '4'

C:\Users\kbjung\AppData\Local\Temp\ipykernel_17964\3043907949.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6.loc[df6['월'] == '03' , '분기'] = '1'


In [ ]:
df6 = df6[[
    '연도',
    '분기', 
    '배출가스등급',
    '연료',
    'LPG_대수',
    'LPG_예측',
    'LPG_예측_BSpline',
    'LPG_예측_Akima',
    '경유_대수',
    '경유_예측',
    '경유_예측_BSpline',
    '경유_예측_Akima',
    '휘발유_대수',
    '휘발유_예측',
    '휘발유_예측_BSpline',
    '휘발유_예측_Akima',
]]

today_date = datetime.today().strftime("%Y%m%d")
df6['테이블생성일자'] = today_date
cdict = {
    '연도':'YR',
    '분기':'QRT',
    '배출가스등급':'EXHST_GAS_GRD_CD',
    '연료':'FUEL_CD',
    'LPG_대수':'LPG_MKCNT',
    'LPG_예측':'LPG_PRET',
    'LPG_예측_BSpline':'LPG_PRET_BSPLN',
    'LPG_예측_Akima':'LPG_PRET_AKM',
    '경유_대수':'DSL_MKCNT',
    '경유_예측':'DSL_PRET',
    '경유_예측_BSpline':'DSL_PRET_BSPLN',
    '경유_예측_Akima':'DSL_PRET_AKM',
    '휘발유_대수':'GSL_MKCNT',
    '휘발유_예측':'GSL_PRET',
    '휘발유_예측_BSpline':'GSL_PRET_BSPLN',
    '휘발유_예측_Akima':'GSL_PRET_AKM',
    '테이블생성일자':'LOAD_DT', 
}
STD_BD_DAT_FUEL_CAR_DEC = df6.rename(columns=cdict)

In [ ]:
STD_BD_DAT_FUEL_CAR_DEC.columns

Index(['YR', 'QRT', 'EXHST_GAS_GRD_CD', 'FUEL_CD', 'LPG_MKCNT', 'LPG_PRET',
       'LPG_PRET_BSPLN', 'LPG_PRET_AKM', 'DSL_MKCNT', 'DSL_PRET',
       'DSL_PRET_BSPLN', 'DSL_PRET_AKM', 'GSL_MKCNT', 'GSL_PRET',
       'GSL_PRET_BSPLN', 'GSL_PRET_AKM', 'LOAD_DT'],
      dtype='object')

In [ ]:
STD_BD_DAT_FUEL_CAR_DEC.head()

,YR,QRT,EXHST_GAS_GRD_CD,FUEL_CD,LPG_MKCNT,LPG_PRET,LPG_PRET_BSPLN,LPG_PRET_AKM,DSL_MKCNT,DSL_PRET,DSL_PRET_BSPLN,DSL_PRET_AKM,GSL_MKCNT,GSL_PRET,GSL_PRET_BSPLN,GSL_PRET_AKM,LOAD_DT
2,2019,1,1,LPG(액화석유가스),60767.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20240206
5,2019,2,1,LPG(액화석유가스),62124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20240206
8,2019,3,1,LPG(액화석유가스),61107.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20240206
11,2019,4,1,LPG(액화석유가스),59222.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20240206
14,2020,1,1,LPG(액화석유가스),56565.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20240206


### [출력] STD_BD_DAT_FUEL_CAR_DEC

In [ ]:
# expdf = STD_BD_DAT_FUEL_CAR_DEC
# table_nm = 'STD_BD_DAT_FUEL_CAR_DEC'.upper()

# # 테이블 생성
# try:
#     sql = 'create or replace table ' + table_nm + '( \n'

#     for idx,column in enumerate(expdf.columns):
#         # if 'float' in expdf[column].dtype.name:
#         #     sql += column + ' float'
#         # elif 'int' in expdf[column].dtype.name:
#         #     sql += column + ' number'
#         # else:
#         sql += column + ' varchar(255)'

#         if len(expdf.columns) - 1 != idx:
#             sql += ','
#         sql += '\n'
#     sql += ')'    
#     we.execute(sql)
#     we.import_from_pandas(expdf, table_nm)
# except:
#     # 데이터 추가
#     # 7s
#     we.import_from_pandas(expdf, table_nm)
    
# print(f'data export : {table_nm}')

### !!! 수정 시작(2023.11.27)

### [로드] STD_BD_CAR_CURSTT_MOD_4RD_INS

In [ ]:
ins_4rd = we.export_to_pandas("SELECT * FROM STD_BD_CAR_CURSTT_MOD_4RD_INS;")
ins_4rd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   YR          561 non-null    int64 
 1   CRTR_YM     561 non-null    int64 
 2   FUEL_CD     561 non-null    object
 3   RGN         561 non-null    object
 4   CTPV_NM     561 non-null    object
 5   VHCL_MKCNT  561 non-null    int64 
 6   LOAD_DT     561 non-null    object
dtypes: int64(3), object(4)
memory usage: 30.8+ KB


### [로드] STD_BD_CAR_CURSTT_MOD2_4RD_INS

In [ ]:
ins2_4rd = we.export_to_pandas("SELECT * FROM STD_BD_CAR_CURSTT_MOD2_4RD_INS;")
ins2_4rd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   YR           176 non-null    int64  
 1   CRTR_YM      176 non-null    int64  
 2   VHCTY_CD_NM  176 non-null    object 
 3   FUEL_CD      176 non-null    object 
 4   VHCL_MKCNT   154 non-null    float64
 5   LOAD_DT      176 non-null    object 
dtypes: float64(1), int64(2), object(3)
memory usage: 8.4+ KB


In [ ]:
ins_4rd['CRTR_YM'] = ins_4rd['CRTR_YM'].astype('str')
ins_4rd['MM'] = ins_4rd['CRTR_YM'].str[4:6]
ins_4rd['CRTR_YM'] = ins_4rd['CRTR_YM'].astype('int')

In [ ]:
ins2_4rd['CRTR_YM'] = ins2_4rd['CRTR_YM'].astype('str')
ins2_4rd['MM'] = ins2_4rd['CRTR_YM'].str[4:6]
ins2_4rd['CRTR_YM'] = ins2_4rd['CRTR_YM'].astype('int')

In [ ]:
ins_4rd = ins_4rd[['YR', 'MM', 'CRTR_YM', 'FUEL_CD', 'RGN', 'CTPV_NM', 'VHCL_MKCNT', 'LOAD_DT']]
ins2_4rd = ins2_4rd[['YR', 'MM', 'CRTR_YM', 'VHCTY_CD_NM', 'FUEL_CD', 'VHCL_MKCNT', 'LOAD_DT']]

### [출력] STD_BD_DAT_GRD4_INS_CURSTT

In [ ]:
STD_BD_DAT_GRD4_INS_CURSTT = ins_4rd.copy()
STD_BD_DAT_GRD4_INS_CURSTT.columns

Index(['YR', 'MM', 'CRTR_YM', 'FUEL_CD', 'RGN', 'CTPV_NM', 'VHCL_MKCNT',
       'LOAD_DT'],
      dtype='object')

In [ ]:
STD_BD_DAT_GRD4_INS_CURSTT.head()

,YR,MM,CRTR_YM,FUEL_CD,RGN,CTPV_NM,VHCL_MKCNT,LOAD_DT
0,2023,06,202306,경유,수도권,서울,99250,2023-10-23
1,2023,06,202306,경유,수도권,인천,54184,2023-10-23
2,2023,06,202306,경유,수도권,경기,237064,2023-10-23
3,2023,06,202306,경유,비수도권,부산,52885,2023-10-23
4,2023,06,202306,경유,비수도권,대구,47224,2023-10-23


In [ ]:
# expdf = STD_BD_DAT_GRD4_INS_CURSTT
# table_nm = 'STD_BD_DAT_GRD4_INS_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx, column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)
    
# # print(f'data export : {table_nm}')

### [출력] STD_BD_DAT_GRD4_INS_CURSTT2

In [ ]:
STD_BD_DAT_GRD4_INS_CURSTT2 = ins2_4rd.copy()
STD_BD_DAT_GRD4_INS_CURSTT2.columns

Index(['YR', 'MM', 'CRTR_YM', 'VHCTY_CD_NM', 'FUEL_CD', 'VHCL_MKCNT',
       'LOAD_DT'],
      dtype='object')

In [ ]:
STD_BD_DAT_GRD4_INS_CURSTT2.head()

,YR,MM,CRTR_YM,VHCTY_CD_NM,FUEL_CD,VHCL_MKCNT,LOAD_DT
0,2023,06,202306,승용,경유,476669.0,2023-10-23
1,2023,06,202306,승합,경유,118636.0,2023-10-23
2,2023,06,202306,화물,경유,447152.0,2023-10-23
3,2023,06,202306,특수,경유,5955.0,2023-10-23
4,2023,06,202306,승용,휘발유,47012.0,2023-10-23


In [ ]:
# expdf = STD_BD_DAT_GRD4_INS_CURSTT2
# table_nm = 'STD_BD_DAT_GRD4_INS_CURSTT2'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx, column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)
    
# # print(f'data export : {table_nm}')

### !!! 수정 끝(2023.11.27)

## 3-1 code end ##################################################################

## ---------- 3-2 start ----------

In [ ]:
# 2024.02.13 수정 (데이터 테이블 변경 STD_CEG_CAR_MIG => VSYSZ.STD_CEG_GRD_LST)
## 39.1s
## 등록정보(VSYSZ.STD_CEG_GRD_LST) 5등급만
car = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO FROM VSYSZ.STD_CEG_GRD_LST WHERE EXHST_GAS_GRD_CD = 'A0505' OR EXHST_GAS_GRD_CD = 'A05T5';")
car.info()
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호',
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자',
    'VHCL_MNG_NO':'차량관리번호'
}
carr = car.rename(columns=car_ch_col)

## 중복 차대번호 제거
carr['최초등록일자'] = pd.to_numeric(carr['최초등록일자'], errors='coerce')
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)

## 차종 코드 변환
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)

## 용도 코드 변환
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)

## 등록정보 말소 제거
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)

c:\Users\kbjung\anaconda3\Lib\site-packages\pyexasol\callback.py:42: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(pipe, skip_blank_lines=False, **kwargs)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2314358 entries, 0 to 2314357
Data columns (total 10 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   VIN             object
 1   BSPL_STDG_CD    int64 
 2   VHCL_ERSR_YN    object
 3   MANP_MNG_NO     object
 4   YRIDNW          int64 
 5   VHCTY_CD        object
 6   PURPS_CD2       object
 7   FRST_REG_YMD    object
 8   VHCL_FBCTN_YMD  object
 9   VHCL_MNG_NO     object
dtypes: int64(2), object(8)
memory usage: 176.6+ MB


In [ ]:
# # 8.6s
# cs_raw = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO, VHRNO, EXTGAS_INSP_VLD_YMD, VHCL_OWNR_CL_CD , FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY, VHCTY_CL_CD, TOTL_WGHT, CRYNG_WGHT, DSPLVL, EGIN_OTPT FROM STD_CEG_GRD_LST WHERE EXHST_GAS_GRD_CD = 'A0505';")
# cs_raw_dict = {
#     'VIN':'차대번호', 
#     'BSPL_STDG_CD':'법정동코드', 
#     'EXHST_GAS_GRD_CD':'배출가스등급', 
#     'EXHST_GAS_CERT_NO':'배출가스인증번호',
#     'VHCL_ERSR_YN':'차량말소YN',
#     'MANP_MNG_NO':'제원관리번호', 
#     'YRIDNW':'차량연식', 
#     'VHCTY_CD':'차종', 
#     'PURPS_CD2':'용도', 
#     'FRST_REG_YMD':'최초등록일자',
#     'VHCL_FBCTN_YMD':'제작일자', 
#     'VHCL_MNG_NO':'차량관리번호', 
#     'VHRNO':'자동차등록번호',
#     'EXTGAS_INSP_VLD_YMD':'검사유효일',
#     'VHCL_OWNR_CL_CD':'소유자구분',
    
#     'FUEL_CD':'연료',
#     'VHCTY_TY_CD2':'차종유형', 
#     'MNFCTR_NM':'제작사명', 
#     'VHCNM':'차명', 
#     'VHCL_FRM':'자동차형식', 
#     'EGIN_TY':'엔진형식', 
#     'VHCTY_CL_CD':'차종분류',
#     'TOTL_WGHT':'총중량',
#     'CRYNG_WGHT':'적재중량',
#     'DSPLVL':'배기량', 
#     'EGIN_OTPT':'엔진출력',
# }
# cs = cs_raw.rename(columns=cs_raw_dict)
# print(cs.columns)
# print(cs.shape)
# print('data load : STD_CEG_GRD_LST')

In [ ]:
# cs.shape

In [ ]:
# cs['배출가스등급'].unique()

In [ ]:
# ## 배출가스등급 코드 변환
# grd_dict = {
#     'A0501':'1', 
#     'A0502':'2', 
#     'A0503':'3', 
#     'A0504':'4', 
#     'A0505':'5', 
#     'A05T2':'2',
#     'A05T3':'3', 
#     'A05T4':'4', 
#     'A05T5':'5', 
#     'A05X':'X', 
# }
# cs['배출가스등급'] = cs['배출가스등급'].replace(grd_dict)
# cs['배출가스등급'].unique()

In [ ]:
# cs['차종'].unique()

In [ ]:
# ## 차종 코드 변환
# cd_dict = {
#     'A31M':'이륜', 
#     'A31P':'승용', 
#     'A31S':'특수', 
#     'A31T':'화물', 
#     'A31V':'승합'
# }
# cs['차종'] = cs['차종'].replace(cd_dict)
# cs['차종'].unique()

In [ ]:
# cs['용도'].unique()

In [ ]:
# ## 용도 코드 변환
# purps_dict = {
#     'A08P':'개인용', 
#     'A08B':'영업용', 
#     'A08O':'관용',
# }
# cs['용도'] = cs['용도'].replace(purps_dict)
# cs['용도'].unique()

In [ ]:
# cs['소유자구분'].unique()

In [ ]:
# ## 소유자구분 코드 변환
# ownr_dict = {
#     'A27B':'사업자', 
#     'A27F':'외국인', 
#     'A27L':'법인', 
#     'A27O':'기타', 
#     'A27R':'주민', 
# }
# cs['소유자구분'] = cs['소유자구분'].replace(ownr_dict)
# cs['소유자구분'].unique()

In [ ]:
# cs['연료'].unique()

In [ ]:
# ## 연료 코드 변환
# fuel_dict = {
#     'A90GS':'휘발유', 
#     'A91DS':'경유',
#     'A92LP':'LPG(액화석유가스)', 
#     'A90GH':'휘발유 하이브리드', 
#     'A93EV':'전기', 
#     'A91DH':'경유 하이브리드', 
#     'A92CN':'CNG(압축천연가스)', 
#     'A93HD':'수소', 
#     'A92LH':'LPG 하이브리드', 
#     'A94OT':'기타연료', 
#     'A92CH':'CNG 하이브리드',
#     'A90AC':'알코올', 
#     'A93SH':'태양열', 
#     'A91KS':'등유', 
#     'A92LN':'LNG(액화천연가스)', 
#     'A90PH':'플러그인 하이브리드', 
# }
# cs['연료'] = cs['연료'].replace(fuel_dict)
# cs['연료'].unique()

In [ ]:
# cs['차종유형'].unique()

In [ ]:
# ## 차종유형 코드 변환
# ty_dict = {
#     'A30C':'경형', 
#     'A30L':'대형', 
#     'A30M':'중형', 
#     'A30S':'소형',
# }
# cs['차종유형'] = cs['차종유형'].replace(ty_dict)
# cs['차종유형'].unique()

In [ ]:
# ## 등록정보 말소 제거
# csm = cs[cs['차량말소YN'] == 'N'].reset_index(drop=True)

In [ ]:
# csm.shape

In [ ]:
## 19.4s
## 등록&제원 병합
cs = carm.merge(srcr, on='제원관리번호', how='left')

In [ ]:
## 11.7s
## 등록&제원&저감이력 병합
csa = cs.merge(attr[['차대번호', 'DPF_YN']], on='차대번호', how='left')
# csa = csm.merge(attr[['차대번호', 'DPF_YN']], on='차대번호', how='left')

In [ ]:
csa['법정동코드'] = csa['법정동코드'].astype('str')
csa['법정동코드'] = csa['법정동코드'].str[:5] + '00000'
csa['법정동코드'] = pd.to_numeric(csa['법정동코드'])

In [ ]:
## 지역정보 병합
csac = csa.merge(coder, on='법정동코드', how='left')

In [ ]:
csac['시도'].isnull().sum()

10195

In [ ]:
# ### 매칭 안되는 지역 처리
# # 주소 수정
# csac.loc[csac['법정동코드'] == 5172035031, ['시도', '시군구']] = ['강원특별자치도', '홍천군'] # 강원특별자치도 홍천군
# csac.loc[csac['법정동코드'] == 5180031023, ['시도', '시군구']] = ['강원특별자치도', '양구군'] # 
# csac.loc[csac['법정동코드'] == 5180031031, ['시도', '시군구']] = ['강원특별자치도', '양구군'] # 
# csac.loc[csac['법정동코드'] == 5172035030, ['시도', '시군구']] = ['강원특별자치도', '홍천군'] # 
# csac.loc[csac['법정동코드'] == 5180031028, ['시도', '시군구']] = ['강원특별자치도', '양구군'] # 

# csac.loc[csac['법정동코드'] == 5172035021, ['시도', '시군구']] = ['강원특별자치도', '홍천군'] # 
# csac.loc[csac['법정동코드'] == 5180031025, ['시도', '시군구']] = ['강원특별자치도', '양구군'] # 
# csac.loc[csac['법정동코드'] == 4165052000, ['시도', '시군구']] = ['경기도', '포천시'] # 경기도 포천시 선단동
# csac.loc[csac['법정동코드'] == 5172035023, ['시도', '시군구']] = ['강원특별자치도', '홍천군'] # 
# csac.loc[csac['법정동코드'] == 5180031027, ['시도', '시군구']] = ['강원특별자치도', '양구군'] # 

# csac.loc[csac['법정동코드'] == 5172035024, ['시도', '시군구']] = ['강원특별자치도', '홍천군'] # 
# csac.loc[csac['법정동코드'] == 5175037022, ['시도', '시군구']] = ['강원특별자치도', '영월군'] # 
# csac.loc[csac['법정동코드'] == 4163055000, ['시도', '시군구']] = ['경기도', '양주시'] # 경기도 양주시 회천3동
# csac.loc[csac['법정동코드'] == 5180031033, ['시도', '시군구']] = ['강원특별자치도', '양구군'] # 경기도 양주시 회천3동

In [ ]:
## 조기폐차 정보들 병합
elp = pd.concat([aear, lgvr], ignore_index=True)
elpm = elp.sort_values('조기폐차최종승인YN', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
elpm = elpm[elpm['조기폐차최종승인YN'] == 'Y'].reset_index(drop=True)

In [ ]:
## 11.6s
## 조기폐차 정보 추가
df = csac.merge(elpm, on='차대번호', how='left')

In [ ]:
## 41.3s
## 비상시 및 계절제 병합
isdp= isdisr.merge(isper, on='적발번호', how='left')

In [ ]:
## 9.5s
isdpi = isdp.merge(isisr, on='발령번호', how='left')

In [ ]:
### 지역정보 추가
coder_dup = coder.sort_values('법정동코드', ascending=True).drop_duplicates(['시도코드', '시군구코드']).reset_index(drop=True)

In [ ]:
coder_dup.head()

,법정동코드,시도,시군구,시도코드,시군구코드
0,1100000000,서울특별시,NaN,11,0
1,1111000000,서울특별시,종로구,11,110
2,1114000000,서울특별시,중구,11,140
3,1117000000,서울특별시,용산구,11,170
4,1120000000,서울특별시,성동구,11,200


In [ ]:
coder_dup[coder_dup['시도'] == '강원특별자치도'].head()

,법정동코드,시도,시군구,시도코드,시군구코드
462,5100000000,강원특별자치도,NaN,51,0
463,5111000000,강원특별자치도,춘천시,51,110
464,5113000000,강원특별자치도,원주시,51,130
465,5115000000,강원특별자치도,강릉시,51,150
466,5117000000,강원특별자치도,동해시,51,170


In [ ]:
coder_dup[coder_dup['시도코드'] == 36]

,법정동코드,시도,시군구,시도코드,시군구코드
123,3611000000,세종특별자치시,NaN,36,110


In [ ]:
isdpi.columns

Index(['적발번호', '발령번호', '차대번호', '등록시도코드', '등록시군구코드', '적발지역코드', '단속일', '유형기준아이디',
       '농도기준아이디'],
      dtype='object')

In [ ]:
isdpi.loc[isdpi['등록시도코드'] == 42].shape

(1078758, 9)

In [ ]:
isdpi.loc[isdpi['등록시도코드'] == 42, '등록시도코드'] = 51 # !!! 수정

In [ ]:
isdpi.loc[isdpi['등록시도코드'] == 42].shape

(0, 9)

In [ ]:
isdpi.loc[isdpi['등록시도코드'] == 36].shape

(646809, 9)

In [ ]:
isdpi.loc[isdpi['등록시도코드'] == 36, '등록시군구코드'].unique()

array([110], dtype=int64)

In [ ]:
## 28.4s
is_total1 = isdpi.merge(coder_dup[['시도코드', '시군구코드', '시도', '시군구']], left_on=['등록시도코드', '등록시군구코드'], right_on=['시도코드', '시군구코드'], how='left')
is_total1 = is_total1.drop(['시도코드', '시군구코드'], axis=1)
is_total1 = is_total1.rename(columns={'시도':'등록시도', '시군구':'등록시군구'})

In [ ]:
is_total1['등록시도'].isnull().sum()

0

In [ ]:
is_total1['적발지역코드'].head()

0    50110
1    50110
2    27000
3    31000
4    31000
Name: 적발지역코드, dtype: int64

In [ ]:
## 1m 19.2s
is_total1['적발지역코드'] = is_total1['적발지역코드'].astype('str')
is_total1['적발시도코드'] = is_total1['적발지역코드'].str[:2]
is_total1['적발시군구코드'] = is_total1['적발지역코드'].str[2:5]
is_total1[['적발시도코드', '적발시군구코드']] = is_total1[['적발시도코드', '적발시군구코드']].astype('int')

In [ ]:
is_total1.loc[is_total1['적발시도코드'] == 42].shape

(671237, 13)

In [ ]:
is_total1.loc[is_total1['적발시도코드'] == 42, '적발시도코드'] = 51 # !!! 수정

In [ ]:
is_total1.loc[is_total1['적발시도코드'] == 42].shape

(0, 13)

In [ ]:
is_total1.loc[is_total1['적발시도코드'] == 36].shape

(1313666, 13)

In [ ]:
is_total1.loc[is_total1['적발시도코드'] == 36, '적발시군구코드'] = 110 # !!! 수정

In [ ]:
## 16.5s
is_total = is_total1.merge(coder_dup[['시도코드', '시군구코드', '시도', '시군구']], left_on=['적발시도코드', '적발시군구코드'], right_on=['시도코드', '시군구코드'], how='left')

In [ ]:
is_total['시도'].isnull().sum()

0

In [ ]:
## 1m 12.8s
is_total = is_total.drop(['적발시도코드', '적발시군구코드', '시도코드', '시군구코드'], axis=1)
is_total = is_total.rename(columns={'시도':'적발시도', '시군구':'적발시군구'})

In [ ]:
is_total['적발시도'].isnull().sum()

0

In [ ]:
is_total['적발시도'].unique()

array(['제주특별자치도', '대구광역시', '울산광역시', '충청북도', '충청남도', '부산광역시', '서울특별시',
       '대전광역시', '경상북도', '광주광역시', '전라북도', '전라남도', '경기도', '경상남도', '강원특별자치도',
       '인천광역시', '세종특별자치시'], dtype=object)

In [ ]:
## 30s
### !!! 수정(2023.09.01)
is_total.loc[(is_total['적발시도'] == '서울특별시') | (is_total['적발시도'] == '경기도') | (is_total['적발시도'] == '인천광역시'), '적발지역'] = '수도권'
is_total.loc[is_total['적발지역'].isnull(), '적발지역'] = '수도권외'

In [ ]:
is_total['적발지역'].unique()

array(['수도권외', '수도권'], dtype=object)

In [ ]:
## 상시 병합
usdp = usdisr.merge(usper, on='번호', how='left')

In [ ]:
usdp.columns

Index(['번호', '차대번호', '적발유형', '단속지역코드', '등록시도코드', '등록시군구코드', '적발건수', '적발년월'], dtype='object')

In [ ]:
usdp.loc[usdp['등록시도코드'] == 42].shape

(32448, 8)

In [ ]:
usdp.loc[usdp['등록시도코드'] == 42, '등록시도코드'] = 51 # !!! 수정

In [ ]:
usdp.loc[usdp['등록시도코드'] == 36].shape

(2919, 8)

In [ ]:
usdp.loc[usdp['등록시도코드'] == 36, '등록시군구코드'] = 110 # !!! 수정

In [ ]:
### 지역정보 추가
# !!! 수정(2023.09.01)
us_total1 = usdp.merge(coder_dup, left_on=['등록시도코드', '등록시군구코드'], right_on=['시도코드', '시군구코드'], how='left')
us_total1 = us_total1.drop(['시도코드', '시군구코드'], axis=1)
us_total1 = us_total1.rename(columns={'시도':'등록시도', '시군구':'등록시군구'}) # !!! 수정(2023.09.01)
us_total1.head()

,번호,차대번호,적발유형,단속지역코드,등록시도코드,등록시군구코드,적발건수,적발년월,법정동코드,등록시도,등록시군구
0,100674,KMJWWH7JP3U559055,S,41430,41,150,NaN,NaN,4115000000,경기도,의정부시
1,100675,KMFGA17EPYC136311,S,41430,11,0,NaN,NaN,1100000000,서울특별시,NaN
2,100927,KMJHD17PP7C037417,S,41410,41,650,NaN,NaN,4165000000,경기도,포천시
3,100878,KN3HAP8N24K042273,S,41410,41,410,NaN,NaN,4141000000,경기도,군포시
4,100879,KMFGA17PP7C069979,S,41280,41,800,NaN,NaN,4180000000,경기도,연천군


In [ ]:
us_total1['등록시도'].isnull().sum()

0

In [ ]:
## 4s
# !!! 수정(2023.09.01)
us_total1['단속지역코드'] = us_total1['단속지역코드'].astype('str')
us_total1['단속시도코드'] = us_total1['단속지역코드'].str[:2]
us_total1['단속시군구코드'] = us_total1['단속지역코드'].str[2:5]
us_total1[['단속시도코드', '단속시군구코드']] = us_total1[['단속시도코드', '단속시군구코드']].astype('int')

In [ ]:
us_total1[us_total1['단속시도코드'] == 42].shape, us_total1[us_total1['단속시도코드'] == 51].shape

((0, 13), (0, 13))

In [ ]:
us_total1[us_total1['단속시도코드'] == 36].shape, us_total1.loc[(us_total1['단속시도코드'] == 36) & (us_total1['단속시군구코드'] == 110)].shape

((0, 13), (0, 13))

In [ ]:
# us_total1.loc[us_total1['단속시도코드'] == 42, '단속시도코드'] = 51
# us_total1.loc[us_total1['단속시도코드'] == 36, '단속시군구코드'] = 110

In [ ]:
coder_dup[['시도코드', '시군구코드']].dtypes

시도코드     int64
시군구코드    int64
dtype: object

In [ ]:
## 2s
# !!! 수정(2023.09.01)
us_total = us_total1.merge(coder_dup[['시도코드', '시군구코드', '시도', '시군구']], left_on=['단속시도코드', '단속시군구코드'], right_on=['시도코드', '시군구코드'], how='left')
us_total = us_total.drop(['시도코드', '시군구코드'], axis=1)
us_total = us_total.rename(columns={'시도':'단속시도', '시군구':'단속시군구'})

In [ ]:
us_total['단속시도'].isnull().sum()

0

In [ ]:
us_total.head()

,번호,차대번호,적발유형,단속지역코드,등록시도코드,등록시군구코드,적발건수,적발년월,법정동코드,등록시도,등록시군구,단속시도코드,단속시군구코드,단속시도,단속시군구
0,100674,KMJWWH7JP3U559055,S,41430,41,150,NaN,NaN,4115000000,경기도,의정부시,41,430,경기도,의왕시
1,100675,KMFGA17EPYC136311,S,41430,11,0,NaN,NaN,1100000000,서울특별시,NaN,41,430,경기도,의왕시
2,100927,KMJHD17PP7C037417,S,41410,41,650,NaN,NaN,4165000000,경기도,포천시,41,410,경기도,군포시
3,100878,KN3HAP8N24K042273,S,41410,41,410,NaN,NaN,4141000000,경기도,군포시,41,410,경기도,군포시
4,100879,KMFGA17PP7C069979,S,41280,41,800,NaN,NaN,4180000000,경기도,연천군,41,280,경기도,고양시


In [ ]:
us_total['단속시도'].isnull().sum()

0

In [ ]:
us_total['단속시도'].unique()

array(['경기도', '서울특별시', '인천광역시'], dtype=object)

In [ ]:
## 1s
# !!! 수정(2023.09.01)
us_total.loc[(us_total['단속시도'] == '서울특별시') | (us_total['단속시도'] == '경기도') | (us_total['단속시도'] == '인천광역시'), '단속지역'] = '수도권'
# us_total.loc[us_total['단속지역'].isnull(), '단속지역'] = '수도권외'

In [ ]:
## 6s
us_total['단속지역'].unique()

array(['수도권'], dtype=object)

In [ ]:
us_total['단속지역'].value_counts(dropna=False)

단속지역
수도권    3178626
Name: count, dtype: int64

In [ ]:
us_total.columns

Index(['번호', '차대번호', '적발유형', '단속지역코드', '등록시도코드', '등록시군구코드', '적발건수', '적발년월',
       '법정동코드', '등록시도', '등록시군구', '단속시도코드', '단속시군구코드', '단속시도', '단속시군구', '단속지역'],
      dtype='object')

In [ ]:
is_total.shape, us_total.shape

((110000000, 14), (3178626, 16))

In [ ]:
## 15.3s
## 등록(말소 유지) & 제원 병합
cse = carr.merge(srcr, on='제원관리번호', how='left')
# cse = cs.copy()

In [ ]:
## 2m 6.0s
## 등록&제원&이력 병합
ersr = cse.merge(hisr, on='차량관리번호', how='left')

In [ ]:
## 1. 등록 차량말소와 등록이력 차량 말소 둘 모두 해당되는 데이터 추출
ersr = ersr.loc[(ersr['차량말소YN_x'] == 'Y') & (ersr['차량말소YN_y'] == 'Y')].reset_index(drop=True)

In [ ]:
## 2. 변경일자 기준 최신 데이터만 남기고 차대번호 중복 제거
ersr = ersr.sort_values('변경일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)

In [ ]:
## 3. 변경일자 2019.01.01 이후만 추출
ersr = ersr[ersr['변경일자'] >= 20190101].reset_index(drop=True)

In [ ]:
ersr['법정동코드'] = ersr['법정동코드'].astype('str')
ersr['법정동코드'] = ersr['법정동코드'].str[:5] + '00000'
ersr['법정동코드'] = pd.to_numeric(ersr['법정동코드'])

In [ ]:
## 등록&제원&이력&법정동코드 병합(errc)
# 매칭 후 '시도' 빈값 없음
errc = ersr.merge(coder, on='법정동코드', how='left')

In [ ]:
errc['시도'].isnull().sum()

1

## Load

### 5등급 지역별 조기폐차(STD_BD_GRD5_ELPDSRC_CURSTT)(한글파일 내용 입력)

In [ ]:
## 11.1s
df1 = we.export_to_pandas("SELECT * FROM STD_BD_GRD5_ELPDSRC;")

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   RGN         68 non-null     object 
 1   YR          68 non-null     int64  
 2   VHCL_MKCNT  68 non-null     int64  
 3   VHCL_REDE   68 non-null     float64
 4   ELPDSRC     68 non-null     float64
 5   NTRL_DCLN   68 non-null     float64
dtypes: float64(3), int64(2), object(1)
memory usage: 3.3+ KB


In [ ]:
df1[df1['RGN'].isnull() == True]

,RGN,YR,VHCL_MKCNT,VHCL_REDE,ELPDSRC,NTRL_DCLN


In [ ]:
df1.isnull().sum()

RGN           0
YR            0
VHCL_MKCNT    0
VHCL_REDE     0
ELPDSRC       0
NTRL_DCLN     0
dtype: int64

In [ ]:
df1 = df1.dropna()

In [ ]:
df1.shape

(68, 6)

In [ ]:
df1.tail()

,RGN,YR,VHCL_MKCNT,VHCL_REDE,ELPDSRC,NTRL_DCLN
63,경남,2022,98885,24199.000000,14765.000000,9434.0
64,제주,2019,44978,9339.666667,5424.333333,3915.0
65,제주,2020,37016,7962.000000,4562.000000,3400.0
66,제주,2021,29057,7959.000000,4501.000000,3458.0
67,제주,2022,23163,5894.000000,3238.000000,2656.0


### 5등급 저공해 미조치(STD_BD_GRD5_LEM_N_MOD)(한글파일 내용 입력)

In [ ]:
## 11.1s
no_dpf = we.export_to_pandas("SELECT * FROM STD_BD_GRD5_LEM_N;")

In [ ]:
no_dpf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      12 non-null     object
 1   구분      12 non-null     object
 2   차량대수    12 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 420.0+ bytes


In [ ]:
# 분석
## 5등급 지역별 조기폐차 현황
# dfm = df.copy()
# dfm['최초등록일자'] = dfm['최초등록일자'].astype('str')
# dfm['최초등록일자_년'] = dfm['최초등록일자'].str[:4]
# dfm['최초등록일자_월'] = dfm['최초등록일자'].str[4:6]
# dfm['최초등록일자_일'] = dfm['최초등록일자'].str[6:8]
# errc['변경일자'] = errc['변경일자'].astype('str')
# errc['변경일자_년'] = errc['변경일자'].str[:4]
# errc['변경일자_월'] = errc['변경일자'].str[4:6]
# errc['변경일자_일'] = errc['변경일자'].str[6:8]

In [ ]:
# ## 시도, 연도별 차량 현황 분석
# # 2022년 차량 대수
# grp1 = dfm.groupby(['시도'], as_index=False)['차대번호'].count()
# grp1 = grp1.rename(columns={'차대번호':'차량대수'})
# grp1['연도'] = f'{year}'
# grp1 = grp1[['연도', '시도', '차량대수']]

In [ ]:
# # 차량 통계 기본 데이터셋
# yr_list = []
# ctpv_list = []
# for ctpv in grp1['시도'].unique():
#     for yr in range(2019, year + 1):
#         yr_list.append(str(yr))
#         ctpv_list.append(ctpv)
# base = pd.DataFrame({'연도':yr_list, '시도':ctpv_list})

In [ ]:
# # 연도별 등록대수
# grp2 = dfm.groupby(['최초등록일자_년', '시도'], as_index=False)['차대번호'].count()
# grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})

In [ ]:
# # 연도별 말소대수
# grp3 = errc.groupby(['변경일자_년', '시도'], as_index=False)['차대번호'].count()
# grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
# base1 = base.merge(grp1, on=['연도', '시도'], how='left')
# base2 = base1.merge(grp2, on=['연도', '시도'], how='left')
# base3 = base2.merge(grp3, on=['연도', '시도'], how='left')
# base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
# n = len(base3['연도'].unique())
# for i in range(base3.shape[0] // n):
#     for j in range(2, n+1):
#         base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
# dfm['말소일자'] = dfm['말소일자'].astype('str')
# dfm['말소일자_년'] = dfm['말소일자'].str[:4]
# dfm['말소일자_월'] = dfm['말소일자'].str[4:6]
# dfm['말소일자_일'] = dfm['말소일자'].str[6:8]
# grp4 = dfm.loc[dfm['조기폐차최종승인YN'] == 'Y'].groupby(['말소일자_년'], as_index=False)['차대번호'].count()
# grp4 = grp4.rename(columns={'말소일자_년':'연도', '차대번호':'조기폐차대수'})
# base4 = base3.merge(grp4, on='연도', how='left')
# base4['조기폐차대수'] = base4['조기폐차대수'].fillna(0)
# base4 = base4.drop(['등록대수', '말소대수'], axis=1)

In [ ]:
# n = len(base4['연도'].unique())
# for i in range(base4.shape[0] // n):
#     for j in range(n-1):
#         base4.loc[i*4 + j+1, '감소대수'] = base4.loc[i*4 + j, '차량대수'] - base4.loc[i*4 + j+1, '차량대수']
# base4['자연감소대수'] = base4['감소대수'] - base4['조기폐차대수']

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
df1['LOAD_DT'] = today_date
STD_BD_GRD5_ELPDSRC_CURSTT = df1.copy()

In [ ]:
STD_BD_GRD5_ELPDSRC_CURSTT.columns

Index(['RGN', 'YR', 'VHCL_MKCNT', 'VHCL_REDE', 'ELPDSRC', 'NTRL_DCLN',
       'LOAD_DT'],
      dtype='object')

In [ ]:
STD_BD_GRD5_ELPDSRC_CURSTT.shape

(68, 7)

In [ ]:
STD_BD_GRD5_ELPDSRC_CURSTT.head()

,RGN,YR,VHCL_MKCNT,VHCL_REDE,ELPDSRC,NTRL_DCLN,LOAD_DT
0,서울,2019,199898,46949.666667,31792.000000,15157.666667,20240206
1,서울,2020,161864,38034.000000,23045.000000,14989.000000,20240206
2,서울,2021,138381,23483.000000,8148.000000,15335.000000,20240206
3,서울,2022,120996,17385.000000,2476.000000,14909.000000,20240206
4,인천,2019,90619,20596.666667,16421.666667,4175.000000,20240206


### ----- 2024.02.06 수정 1 (데이터 타입 변경 코드 수정)

`.str.replace(',', '') => .astype(str).str.replace(',', '')`

In [ ]:
### ----- 2024.02.06 수정 1 (데이터 타입 변경 코드 수정)
# `.str.replace(',', '') => .astype(str).str.replace(',', '')`

STD_BD_GRD5_ELPDSRC_CURSTT['VHCL_MKCNT'] = STD_BD_GRD5_ELPDSRC_CURSTT['VHCL_MKCNT'].astype(str).str.replace(',', '')
STD_BD_GRD5_ELPDSRC_CURSTT['VHCL_REDE'] = STD_BD_GRD5_ELPDSRC_CURSTT['VHCL_REDE'].astype(str).str.replace(',', '')
STD_BD_GRD5_ELPDSRC_CURSTT['ELPDSRC'] = STD_BD_GRD5_ELPDSRC_CURSTT['ELPDSRC'].astype(str).str.replace(',', '')
STD_BD_GRD5_ELPDSRC_CURSTT['NTRL_DCLN'] = STD_BD_GRD5_ELPDSRC_CURSTT['NTRL_DCLN'].astype(str).str.replace(',', '')
STD_BD_GRD5_ELPDSRC_CURSTT['VHCL_MKCNT'] = pd.to_numeric(STD_BD_GRD5_ELPDSRC_CURSTT['VHCL_MKCNT'], errors='coerce')
STD_BD_GRD5_ELPDSRC_CURSTT['VHCL_REDE'] = pd.to_numeric(STD_BD_GRD5_ELPDSRC_CURSTT['VHCL_REDE'], errors='coerce')
STD_BD_GRD5_ELPDSRC_CURSTT['ELPDSRC'] = pd.to_numeric(STD_BD_GRD5_ELPDSRC_CURSTT['ELPDSRC'], errors='coerce')
STD_BD_GRD5_ELPDSRC_CURSTT['NTRL_DCLN'] = pd.to_numeric(STD_BD_GRD5_ELPDSRC_CURSTT['NTRL_DCLN'], errors='coerce')

In [ ]:
STD_BD_GRD5_ELPDSRC_CURSTT.head()

,RGN,YR,VHCL_MKCNT,VHCL_REDE,ELPDSRC,NTRL_DCLN,LOAD_DT
0,서울,2019,199898,46949.666667,31792.000000,15157.666667,20240206
1,서울,2020,161864,38034.000000,23045.000000,14989.000000,20240206
2,서울,2021,138381,23483.000000,8148.000000,15335.000000,20240206
3,서울,2022,120996,17385.000000,2476.000000,14909.000000,20240206
4,인천,2019,90619,20596.666667,16421.666667,4175.000000,20240206


In [ ]:
STD_BD_GRD5_ELPDSRC_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   RGN         68 non-null     object 
 1   YR          68 non-null     int64  
 2   VHCL_MKCNT  68 non-null     int64  
 3   VHCL_REDE   68 non-null     float64
 4   ELPDSRC     68 non-null     float64
 5   NTRL_DCLN   68 non-null     float64
 6   LOAD_DT     68 non-null     object 
dtypes: float64(3), int64(2), object(2)
memory usage: 3.8+ KB


### [출력] [D] STD_BD_GRD5_ELPDSRC_CURSTT

In [ ]:
# expdf = STD_BD_GRD5_ELPDSRC_CURSTT
# table_nm = 'STD_BD_GRD5_ELPDSRC_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [ ]:
## 5등급 지역별 저공해미조치 차량현황
today_date = datetime.today().strftime("%Y%m%d")
no_dpf['테이블생성일자'] = today_date
cdict = {
    '테이블생성일자':'LOAD_DT',
    '지역':'RGN',
    '구분':'SEASON',
    '차량대수':'VHCL_MKCNT', 
}
STD_BD_GRD5_LEM_N_MOD = no_dpf.rename(columns=cdict)

In [ ]:
STD_BD_GRD5_LEM_N_MOD.columns

Index(['RGN', 'SEASON', 'VHCL_MKCNT', 'LOAD_DT'], dtype='object')

In [ ]:
STD_BD_GRD5_LEM_N_MOD.head()

,RGN,SEASON,VHCL_MKCNT,LOAD_DT
0,수도권,2차계절제_시작,539748,20240206
1,수도권,2차계절제_종료,515615,20240206
2,수도권,3차계절제_시작,459110,20240206
3,수도권,3차계절제_종료,436921,20240206
4,수도권,4차계절제_시작,394533,20240206


### ----- 2024.02.06 수정 2 (데이터 타입 변경 코드 수정)

`.str.replace(',', '') => .astype(str).str.replace(',', '')`

In [ ]:
### ----- 2024.02.06 수정 2 (데이터 타입 변경 코드 수정)
# `.str.replace(',', '') => .astype(str).str.replace(',', '')`

STD_BD_GRD5_LEM_N_MOD['VHCL_MKCNT'] = STD_BD_GRD5_LEM_N_MOD['VHCL_MKCNT'].astype(str).str.replace(',', '')
STD_BD_GRD5_LEM_N_MOD['VHCL_MKCNT'] = pd.to_numeric(STD_BD_GRD5_LEM_N_MOD['VHCL_MKCNT'], errors='coerce')

In [ ]:
STD_BD_GRD5_LEM_N_MOD.head()

,RGN,SEASON,VHCL_MKCNT,LOAD_DT
0,수도권,2차계절제_시작,539748,20240206
1,수도권,2차계절제_종료,515615,20240206
2,수도권,3차계절제_시작,459110,20240206
3,수도권,3차계절제_종료,436921,20240206
4,수도권,4차계절제_시작,394533,20240206


In [ ]:
STD_BD_GRD5_LEM_N_MOD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   RGN         12 non-null     object
 1   SEASON      12 non-null     object
 2   VHCL_MKCNT  12 non-null     int64 
 3   LOAD_DT     12 non-null     object
dtypes: int64(1), object(3)
memory usage: 516.0+ bytes


### [출력] [D] STD_BD_GRD5_LEM_N_MOD

In [ ]:
# expdf = STD_BD_GRD5_LEM_N_MOD
# table_nm = 'STD_BD_GRD5_LEM_N_MOD'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [ ]:
## 11.3s
## 차대번호별 운행제한 적발 현황
# DNSTY_STDR_ID(농도기준아이디) : 실발령(C011), 모의발령(C012)
# TY_STDR_ID(유형기준아이디) : 비상시(T001), 계절제(T002)
is_season = is_total.loc[(is_total['농도기준아이디'] == 'C011') & (is_total['유형기준아이디'] == 'T002')].reset_index(drop=True)

In [ ]:
is_season.head()

,적발번호,발령번호,차대번호,등록시도코드,등록시군구코드,적발지역코드,단속일,유형기준아이디,농도기준아이디,등록시도,등록시군구,적발시도,적발시군구,적발지역
0,110896090,2427,KMFGA17LP2C168386,27,140,27000,NaN,T002,C011,대구광역시,동구,대구광역시,NaN,수도권외
1,110896111,2427,KPALC4A81WP035279,11,740,26000,NaN,T002,C011,서울특별시,강동구,부산광역시,NaN,수도권외
2,110896112,2427,KL3P9CNF35K000043,26,470,26000,NaN,T002,C011,부산광역시,연제구,부산광역시,NaN,수도권외
3,110896387,2427,KMFGA17PP7C061069,41,170,11000,NaN,T002,C011,경기도,안양시,서울특별시,NaN,수도권
4,110896403,2427,KMFLA18AP2C035441,11,740,11000,NaN,T002,C011,서울특별시,강동구,서울특별시,NaN,수도권


In [ ]:
is_season['차대번호'].shape, len(is_season['차대번호'].unique())

((12005413,), 402061)

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
for yr in range(2019, int(today_date[:4])):
    start_date = f'{yr}1130'
    end_date = f'{yr+1}0401'
    is_season.loc[(is_season['단속일'] > int(start_date)) & (is_season['단속일'] < int(end_date)), f'계절제_{yr-2018}차여부'] = 'Y'

In [ ]:
agg_dict = {x:'count' for x in is_season.columns if '계절제' in x}
limit_season_rename_dict = {x:x.replace('여부','') for x in agg_dict.keys()}

In [ ]:
limit_season = is_season.groupby(['차대번호'], as_index=False).agg(agg_dict)
limit_season = limit_season.rename(columns=limit_season_rename_dict)

In [ ]:
limit_season.shape, len(limit_season['차대번호'].unique())

((402061, 6), 402061)

In [ ]:
# 11.0s
# DNSTY_STDR_ID(농도기준아이디) : 실발령(C011), 모의발령(C012)
# TY_STDR_ID(유형기준아이디) : 비상시(T001), 계절제(T002)
is_high = is_total.loc[(is_total['농도기준아이디'] == 'C011') & (is_total['유형기준아이디'] == 'T001')].reset_index(drop=True)
limit_high = is_high.groupby(['차대번호'], as_index=False).agg({'단속일':'count'})
limit_high = limit_high.rename(columns={'단속일':'비상시'})

In [ ]:
limit_alw = us_total.groupby('차대번호', as_index=False).agg({'적발건수':'sum'})
limit_alw = limit_alw.rename(columns={'적발건수':'상시'})
limit_sh = limit_season.merge(limit_high, on='차대번호', how='left')
limit = limit_sh.merge(limit_alw, on='차대번호', how='left')

In [ ]:
limit.shape

(402061, 8)

In [ ]:
limit.iloc[:, 1:] = limit.iloc[:, 1:].fillna(0)
limit['비상시'] = limit['비상시'].astype('int')
limit['상시'] = limit['상시'].astype('int')

In [ ]:
df.shape, len(df['차대번호'].unique())

((910028, 30), 900911)

In [ ]:
lmt1 = df.merge(limit, on='차대번호', how='left')
lmt1.loc[(lmt1['시도'] == '서울특별시') | (lmt1['시도'] == '경기도') | (lmt1['시도'] == '인천광역시'), '지역'] = '수도권'
lmt1['지역'] = lmt1['지역'].fillna('수도권외')
lmt1['DPF_YN'] = lmt1['DPF_YN'].fillna('무')

In [ ]:
lmt1.shape

(910028, 38)

In [ ]:
season_start_date = datetime(2020, 12, 1)
season_end_date = datetime(2021, 3, 31)
days = (season_end_date - season_start_date).days
for one in [x for x in limit_season_rename_dict.values()]:
    lmt1[one + '_일평균'] = lmt1[one] / days

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
lmt1['테이블생성일자'] = today_date
season_col = ['테이블생성일자', '차대번호'] + ['지역', '시도', 'DPF_YN', '차종', '차종유형'] + [x for x in limit_season_rename_dict.values()]
lmt1[[x for x in limit_season_rename_dict.values()] + [x + '_일평균' for x in limit_season_rename_dict.values()]] = lmt1[[x for x in limit_season_rename_dict.values()] + [x + '_일평균' for x in limit_season_rename_dict.values()]].fillna(0)
season = lmt1[season_col]
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '차대번호':'VIN', 
    '지역':'RGN', 
    '시도':'CTPV', 
    'DPF_YN':'DPF_EXTRNS_YN', 
    '차종':'VHCTY_CD', 
    '차종유형':'VHCTY_TY', 
}
for one in limit_season_rename_dict.values():
    cdict[one] = one.replace('계절제', 'SEASON').replace('차', 'ODR_CRDN_NOCS')
STD_BD_SEASON_CRDN_NOCS_CURSTT = season.rename(columns=cdict)

In [ ]:
STD_BD_SEASON_CRDN_NOCS_CURSTT.columns

Index(['LOAD_DT', 'VIN', 'RGN', 'CTPV', 'DPF_EXTRNS_YN', 'VHCTY_CD',
       'VHCTY_TY', 'SEASON_1ODR_CRDN_NOCS', 'SEASON_2ODR_CRDN_NOCS',
       'SEASON_3ODR_CRDN_NOCS', 'SEASON_4ODR_CRDN_NOCS',
       'SEASON_5ODR_CRDN_NOCS'],
      dtype='object')

In [ ]:
STD_BD_SEASON_CRDN_NOCS_CURSTT['CTPV'].isnull().sum()

10195

### [출력] STD_BD_SEASON_CRDN_NOCS_CURSTT

In [ ]:
# expdf = STD_BD_SEASON_CRDN_NOCS_CURSTT
# table_nm = 'STD_BD_SEASON_CRDN_NOCS_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [ ]:
### !!! 수정 시작(2023.09.06)
#### 계절관리제 적발 차량 현황(건)

In [ ]:
is_season.head()

,적발번호,발령번호,차대번호,등록시도코드,등록시군구코드,적발지역코드,단속일,유형기준아이디,농도기준아이디,등록시도,등록시군구,적발시도,적발시군구,적발지역,계절제_1차여부,계절제_2차여부,계절제_3차여부,계절제_4차여부,계절제_5차여부
0,110896090,2427,KMFGA17LP2C168386,27,140,27000,NaN,T002,C011,대구광역시,동구,대구광역시,NaN,수도권외,NaN,NaN,NaN,NaN,NaN
1,110896111,2427,KPALC4A81WP035279,11,740,26000,NaN,T002,C011,서울특별시,강동구,부산광역시,NaN,수도권외,NaN,NaN,NaN,NaN,NaN
2,110896112,2427,KL3P9CNF35K000043,26,470,26000,NaN,T002,C011,부산광역시,연제구,부산광역시,NaN,수도권외,NaN,NaN,NaN,NaN,NaN
3,110896387,2427,KMFGA17PP7C061069,41,170,11000,NaN,T002,C011,경기도,안양시,서울특별시,NaN,수도권,NaN,NaN,NaN,NaN,NaN
4,110896403,2427,KMFLA18AP2C035441,11,740,11000,NaN,T002,C011,서울특별시,강동구,서울특별시,NaN,수도권,NaN,NaN,NaN,NaN,NaN


In [ ]:
agg_dict.keys()

dict_keys(['계절제_1차여부', '계절제_2차여부', '계절제_3차여부', '계절제_4차여부', '계절제_5차여부'])

In [ ]:
lmt2 = is_season.sort_values(list(agg_dict.keys()))
lmt2.head()

,적발번호,발령번호,차대번호,등록시도코드,등록시군구코드,적발지역코드,단속일,유형기준아이디,농도기준아이디,등록시도,등록시군구,적발시도,적발시군구,적발지역,계절제_1차여부,계절제_2차여부,계절제_3차여부,계절제_4차여부,계절제_5차여부
3114992,11787654,494,KMFGA17PP6C028152,45,210,41110,20210310.0,T002,C011,전라북도,김제시,경기도,수원시,수도권,NaN,Y,NaN,NaN,NaN
3114994,11787659,494,KMHSJ81WP7U128716,31,170,41110,20210310.0,T002,C011,울산광역시,동구,경기도,수원시,수도권,NaN,Y,NaN,NaN,NaN
3115104,11722872,494,KMFZCD7JP7U253562,43,750,41550,20210310.0,T002,C011,충청북도,진천군,경기도,안성시,수도권,NaN,Y,NaN,NaN,NaN
3115131,11728008,494,KMFWVH7JP6U743666,41,150,11000,20210310.0,T002,C011,경기도,의정부시,서울특별시,NaN,수도권,NaN,Y,NaN,NaN,NaN
3115136,11728025,494,KMHMH81VP3U184335,51,760,11000,20210310.0,T002,C011,강원특별자치도,평창군,서울특별시,NaN,수도권,NaN,Y,NaN,NaN,NaN


In [ ]:
lmt2.shape

(12005413, 19)

In [ ]:
agg_dict

{'계절제_1차여부': 'count',
 '계절제_2차여부': 'count',
 '계절제_3차여부': 'count',
 '계절제_4차여부': 'count',
 '계절제_5차여부': 'count'}

In [ ]:
lmt2['적발지역'].value_counts(dropna=False)

적발지역
수도권     10835572
수도권외     1169841
Name: count, dtype: int64

In [ ]:
limit_season_rename_dict

{'계절제_1차여부': '계절제_1차',
 '계절제_2차여부': '계절제_2차',
 '계절제_3차여부': '계절제_3차',
 '계절제_4차여부': '계절제_4차',
 '계절제_5차여부': '계절제_5차'}

In [ ]:
grp2 = lmt2.groupby(['적발지역']).agg(agg_dict).reset_index()
grp2 = grp2.rename(columns=limit_season_rename_dict)
grp2

,적발지역,계절제_1차,계절제_2차,계절제_3차,계절제_4차,계절제_5차
0,수도권,0,82445,84104,48395,0
1,수도권외,0,0,0,58169,0


In [ ]:
season_start_date = datetime(2020, 12, 1)
season_end_date = datetime(2021, 3, 31)
days = (season_end_date - season_start_date).days
days

120

In [ ]:
len(limit_season_rename_dict)

5

In [ ]:
for i, one in zip([x for x in range(len(limit_season_rename_dict))], [x for x in limit_season_rename_dict.values()]):
    season_start_date = datetime(2019 + i, 12, 1)
    season_end_date = datetime(2020 + i, 3, 31)
    days = (season_end_date - season_start_date).days
    grp2[one + '_일평균'] = grp2[one] / days
    print(days)

121
120
120
120
121


In [ ]:
grp2

,적발지역,계절제_1차,계절제_2차,계절제_3차,계절제_4차,계절제_5차,계절제_1차_일평균,계절제_2차_일평균,계절제_3차_일평균,계절제_4차_일평균,계절제_5차_일평균
0,수도권,0,82445,84104,48395,0,0.0,687.041667,700.866667,403.291667,0.0
1,수도권외,0,0,0,58169,0,0.0,0.000000,0.000000,484.741667,0.0


In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20240206'

In [ ]:
grp2['테이블생성일자'] = today_date

In [ ]:
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '적발지역':'DSCL_RGN', 
}
for one in limit_season_rename_dict.values():
    cdict[one] = one.replace('계절제', 'SEASON').replace('차', 'ODR_CRDN_NOCS')
for one in limit_season_rename_dict.values():
    cdict[one+ '_일평균'] = (one+ '_일평균').replace('계절제', 'SEASON').replace('차', 'ODR').replace('일평균', 'DY_AVRG_CRDN_NOCS')
cdict

{'테이블생성일자': 'LOAD_DT',
 '적발지역': 'DSCL_RGN',
 '계절제_1차': 'SEASON_1ODR_CRDN_NOCS',
 '계절제_2차': 'SEASON_2ODR_CRDN_NOCS',
 '계절제_3차': 'SEASON_3ODR_CRDN_NOCS',
 '계절제_4차': 'SEASON_4ODR_CRDN_NOCS',
 '계절제_5차': 'SEASON_5ODR_CRDN_NOCS',
 '계절제_1차_일평균': 'SEASON_1ODR_DY_AVRG_CRDN_NOCS',
 '계절제_2차_일평균': 'SEASON_2ODR_DY_AVRG_CRDN_NOCS',
 '계절제_3차_일평균': 'SEASON_3ODR_DY_AVRG_CRDN_NOCS',
 '계절제_4차_일평균': 'SEASON_4ODR_DY_AVRG_CRDN_NOCS',
 '계절제_5차_일평균': 'SEASON_5ODR_DY_AVRG_CRDN_NOCS'}

In [ ]:
STD_BD_SEASON_DSCL_VHCL_CURSTT = grp2.rename(columns=cdict)
STD_BD_SEASON_DSCL_VHCL_CURSTT.columns

Index(['DSCL_RGN', 'SEASON_1ODR_CRDN_NOCS', 'SEASON_2ODR_CRDN_NOCS',
       'SEASON_3ODR_CRDN_NOCS', 'SEASON_4ODR_CRDN_NOCS',
       'SEASON_5ODR_CRDN_NOCS', 'SEASON_1ODR_DY_AVRG_CRDN_NOCS',
       'SEASON_2ODR_DY_AVRG_CRDN_NOCS', 'SEASON_3ODR_DY_AVRG_CRDN_NOCS',
       'SEASON_4ODR_DY_AVRG_CRDN_NOCS', 'SEASON_5ODR_DY_AVRG_CRDN_NOCS',
       'LOAD_DT'],
      dtype='object')

In [ ]:
list(STD_BD_SEASON_DSCL_VHCL_CURSTT.columns)

['DSCL_RGN',
 'SEASON_1ODR_CRDN_NOCS',
 'SEASON_2ODR_CRDN_NOCS',
 'SEASON_3ODR_CRDN_NOCS',
 'SEASON_4ODR_CRDN_NOCS',
 'SEASON_5ODR_CRDN_NOCS',
 'SEASON_1ODR_DY_AVRG_CRDN_NOCS',
 'SEASON_2ODR_DY_AVRG_CRDN_NOCS',
 'SEASON_3ODR_DY_AVRG_CRDN_NOCS',
 'SEASON_4ODR_DY_AVRG_CRDN_NOCS',
 'SEASON_5ODR_DY_AVRG_CRDN_NOCS',
 'LOAD_DT']

In [ ]:
STD_BD_SEASON_DSCL_VHCL_CURSTT.shape

(2, 12)

In [ ]:
STD_BD_SEASON_DSCL_VHCL_CURSTT.head()

,DSCL_RGN,SEASON_1ODR_CRDN_NOCS,SEASON_2ODR_CRDN_NOCS,SEASON_3ODR_CRDN_NOCS,SEASON_4ODR_CRDN_NOCS,SEASON_5ODR_CRDN_NOCS,SEASON_1ODR_DY_AVRG_CRDN_NOCS,SEASON_2ODR_DY_AVRG_CRDN_NOCS,SEASON_3ODR_DY_AVRG_CRDN_NOCS,SEASON_4ODR_DY_AVRG_CRDN_NOCS,SEASON_5ODR_DY_AVRG_CRDN_NOCS,LOAD_DT
0,수도권,0,82445,84104,48395,0,0.0,687.041667,700.866667,403.291667,0.0,20240206
1,수도권외,0,0,0,58169,0,0.0,0.000000,0.000000,484.741667,0.0,20240206


### [출력] STD_BD_SEASON_DSCL_VHCL_CURSTT

In [ ]:
# expdf = STD_BD_SEASON_DSCL_VHCL_CURSTT
# table_nm = 'STD_BD_SEASON_DSCL_VHCL_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20240206'

In [ ]:
## 0s
# STD_BD_SEASON_DSCL_VHCL_CURSTT.to_csv(os.path.join(df3_fold, f'STD_BD_SEASON_DSCL_VHCL_CURSTT({today_date}).csv'), index=False)

#### 운행제한 현황
- 운행제한 현황(KPI)
- 등록지별 계절관리제 적발차량 현황(대)
- 등록지별 저공해 미조치 차량 현황(대)
- 등록지역별 현황(대)

In [ ]:
is_season.head()

,적발번호,발령번호,차대번호,등록시도코드,등록시군구코드,적발지역코드,단속일,유형기준아이디,농도기준아이디,등록시도,등록시군구,적발시도,적발시군구,적발지역,계절제_1차여부,계절제_2차여부,계절제_3차여부,계절제_4차여부,계절제_5차여부
0,110896090,2427,KMFGA17LP2C168386,27,140,27000,NaN,T002,C011,대구광역시,동구,대구광역시,NaN,수도권외,NaN,NaN,NaN,NaN,NaN
1,110896111,2427,KPALC4A81WP035279,11,740,26000,NaN,T002,C011,서울특별시,강동구,부산광역시,NaN,수도권외,NaN,NaN,NaN,NaN,NaN
2,110896112,2427,KL3P9CNF35K000043,26,470,26000,NaN,T002,C011,부산광역시,연제구,부산광역시,NaN,수도권외,NaN,NaN,NaN,NaN,NaN
3,110896387,2427,KMFGA17PP7C061069,41,170,11000,NaN,T002,C011,경기도,안양시,서울특별시,NaN,수도권,NaN,NaN,NaN,NaN,NaN
4,110896403,2427,KMFLA18AP2C035441,11,740,11000,NaN,T002,C011,서울특별시,강동구,서울특별시,NaN,수도권,NaN,NaN,NaN,NaN,NaN


In [ ]:
agg_dict

{'계절제_1차여부': 'count',
 '계절제_2차여부': 'count',
 '계절제_3차여부': 'count',
 '계절제_4차여부': 'count',
 '계절제_5차여부': 'count'}

In [ ]:
is_season.sort_values((list(agg_dict.keys()))).head()

,적발번호,발령번호,차대번호,등록시도코드,등록시군구코드,적발지역코드,단속일,유형기준아이디,농도기준아이디,등록시도,등록시군구,적발시도,적발시군구,적발지역,계절제_1차여부,계절제_2차여부,계절제_3차여부,계절제_4차여부,계절제_5차여부
3114992,11787654,494,KMFGA17PP6C028152,45,210,41110,20210310.0,T002,C011,전라북도,김제시,경기도,수원시,수도권,NaN,Y,NaN,NaN,NaN
3114994,11787659,494,KMHSJ81WP7U128716,31,170,41110,20210310.0,T002,C011,울산광역시,동구,경기도,수원시,수도권,NaN,Y,NaN,NaN,NaN
3115104,11722872,494,KMFZCD7JP7U253562,43,750,41550,20210310.0,T002,C011,충청북도,진천군,경기도,안성시,수도권,NaN,Y,NaN,NaN,NaN
3115131,11728008,494,KMFWVH7JP6U743666,41,150,11000,20210310.0,T002,C011,경기도,의정부시,서울특별시,NaN,수도권,NaN,Y,NaN,NaN,NaN
3115136,11728025,494,KMHMH81VP3U184335,51,760,11000,20210310.0,T002,C011,강원특별자치도,평창군,서울특별시,NaN,수도권,NaN,Y,NaN,NaN,NaN


In [ ]:
is_season.shape

(12005413, 19)

In [ ]:
limit.shape

(402061, 8)

In [ ]:
## 18.3s
is_season3 = is_season.sort_values((list(agg_dict.keys()))).drop_duplicates(['등록시도', '적발시도', '차대번호']).reset_index(drop=True)
is_season3.shape

(666810, 19)

In [ ]:
lmt31 = is_season3.merge(df[['차대번호', '차종', '차종유형', 'DPF_YN']], on='차대번호', how='left')
lmt31.shape

(669246, 22)

In [ ]:
lmt3 = lmt31.merge(limit, on='차대번호', how='left')
lmt3.shape

(669246, 29)

In [ ]:
lmt3.head()

,적발번호,발령번호,차대번호,등록시도코드,등록시군구코드,적발지역코드,단속일,유형기준아이디,농도기준아이디,등록시도,...,차종,차종유형,DPF_YN,계절제_1차,계절제_2차,계절제_3차,계절제_4차,계절제_5차,비상시,상시
0,11787654,494,KMFGA17PP6C028152,45,210,41110,20210310.0,T002,C011,전라북도,...,화물,중형,유,0,1,0,0,0,0,0
1,11787659,494,KMHSJ81WP7U128716,31,170,41110,20210310.0,T002,C011,울산광역시,...,NaN,NaN,NaN,0,1,0,0,0,0,0
2,11722872,494,KMFZCD7JP7U253562,43,750,41550,20210310.0,T002,C011,충청북도,...,화물,중형,유,0,5,0,0,0,1,0
3,11728008,494,KMFWVH7JP6U743666,41,150,11000,20210310.0,T002,C011,경기도,...,화물,중형,유,0,24,0,0,0,3,0
4,11728025,494,KMHMH81VP3U184335,51,760,11000,20210310.0,T002,C011,강원특별자치도,...,승용,중형,NaN,0,1,0,0,0,0,0


In [ ]:
lmt3['테이블생성일자'] = today_date

In [ ]:
list(lmt3.columns)

['적발번호',
 '발령번호',
 '차대번호',
 '등록시도코드',
 '등록시군구코드',
 '적발지역코드',
 '단속일',
 '유형기준아이디',
 '농도기준아이디',
 '등록시도',
 '등록시군구',
 '적발시도',
 '적발시군구',
 '적발지역',
 '계절제_1차여부',
 '계절제_2차여부',
 '계절제_3차여부',
 '계절제_4차여부',
 '계절제_5차여부',
 '차종',
 '차종유형',
 'DPF_YN',
 '계절제_1차',
 '계절제_2차',
 '계절제_3차',
 '계절제_4차',
 '계절제_5차',
 '비상시',
 '상시',
 '테이블생성일자']

In [ ]:
lmt3m = lmt3[[
    '테이블생성일자',
    '차대번호',
    '차종',
    '차종유형',
    'DPF_YN',
    '등록시도',
    '적발시도',
    '계절제_1차',
    '계절제_2차',
    '계절제_3차',
    '계절제_4차',
    '비상시', 
    '상시', 
]]
lmt3m.shape

(669246, 13)

In [ ]:
lmt3m.head()

,테이블생성일자,차대번호,차종,차종유형,DPF_YN,등록시도,적발시도,계절제_1차,계절제_2차,계절제_3차,계절제_4차,비상시,상시
0,20240206,KMFGA17PP6C028152,화물,중형,유,전라북도,경기도,0,1,0,0,0,0
1,20240206,KMHSJ81WP7U128716,NaN,NaN,NaN,울산광역시,경기도,0,1,0,0,0,0
2,20240206,KMFZCD7JP7U253562,화물,중형,유,충청북도,경기도,0,5,0,0,1,0
3,20240206,KMFWVH7JP6U743666,화물,중형,유,경기도,서울특별시,0,24,0,0,3,0
4,20240206,KMHMH81VP3U184335,승용,중형,NaN,강원특별자치도,서울특별시,0,1,0,0,0,0


In [ ]:
limit_season_rename_dict

{'계절제_1차여부': '계절제_1차',
 '계절제_2차여부': '계절제_2차',
 '계절제_3차여부': '계절제_3차',
 '계절제_4차여부': '계절제_4차',
 '계절제_5차여부': '계절제_5차'}

In [ ]:
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '차대번호':'VIN',
    '차종':'VHCTY_CD', 
    '차종유형':'VHCTY_TY', 
    'DPF_YN':'DPF_EXTRNS_YN', 
    '등록시도':'REG_CTPV', 
    '적발시도':'DSCL_CTPV', 
    '비상시':'EMGN_CRDN_NOCS', 
    '상시':'ORDITM_CRDN_NOCS', 
}
for one in limit_season_rename_dict.values():
    cdict[one] = one.replace('계절제', 'SEASON').replace('차', 'ODR_CRDN_NOCS')
cdict

{'테이블생성일자': 'LOAD_DT',
 '차대번호': 'VIN',
 '차종': 'VHCTY_CD',
 '차종유형': 'VHCTY_TY',
 'DPF_YN': 'DPF_EXTRNS_YN',
 '등록시도': 'REG_CTPV',
 '적발시도': 'DSCL_CTPV',
 '비상시': 'EMGN_CRDN_NOCS',
 '상시': 'ORDITM_CRDN_NOCS',
 '계절제_1차': 'SEASON_1ODR_CRDN_NOCS',
 '계절제_2차': 'SEASON_2ODR_CRDN_NOCS',
 '계절제_3차': 'SEASON_3ODR_CRDN_NOCS',
 '계절제_4차': 'SEASON_4ODR_CRDN_NOCS',
 '계절제_5차': 'SEASON_5ODR_CRDN_NOCS'}

In [ ]:
STD_BD_RUN_LMT_CURSTT = lmt3m.rename(columns=cdict)
STD_BD_RUN_LMT_CURSTT.columns

Index(['LOAD_DT', 'VIN', 'VHCTY_CD', 'VHCTY_TY', 'DPF_EXTRNS_YN', 'REG_CTPV',
       'DSCL_CTPV', 'SEASON_1ODR_CRDN_NOCS', 'SEASON_2ODR_CRDN_NOCS',
       'SEASON_3ODR_CRDN_NOCS', 'SEASON_4ODR_CRDN_NOCS', 'EMGN_CRDN_NOCS',
       'ORDITM_CRDN_NOCS'],
      dtype='object')

In [ ]:
list(STD_BD_RUN_LMT_CURSTT.columns)

['LOAD_DT',
 'VIN',
 'VHCTY_CD',
 'VHCTY_TY',
 'DPF_EXTRNS_YN',
 'REG_CTPV',
 'DSCL_CTPV',
 'SEASON_1ODR_CRDN_NOCS',
 'SEASON_2ODR_CRDN_NOCS',
 'SEASON_3ODR_CRDN_NOCS',
 'SEASON_4ODR_CRDN_NOCS',
 'EMGN_CRDN_NOCS',
 'ORDITM_CRDN_NOCS']

In [ ]:
STD_BD_RUN_LMT_CURSTT.shape

(669246, 13)

In [ ]:
STD_BD_RUN_LMT_CURSTT.head()

,LOAD_DT,VIN,VHCTY_CD,VHCTY_TY,DPF_EXTRNS_YN,REG_CTPV,DSCL_CTPV,SEASON_1ODR_CRDN_NOCS,SEASON_2ODR_CRDN_NOCS,SEASON_3ODR_CRDN_NOCS,SEASON_4ODR_CRDN_NOCS,EMGN_CRDN_NOCS,ORDITM_CRDN_NOCS
0,20240206,KMFGA17PP6C028152,화물,중형,유,전라북도,경기도,0,1,0,0,0,0
1,20240206,KMHSJ81WP7U128716,NaN,NaN,NaN,울산광역시,경기도,0,1,0,0,0,0
2,20240206,KMFZCD7JP7U253562,화물,중형,유,충청북도,경기도,0,5,0,0,1,0
3,20240206,KMFWVH7JP6U743666,화물,중형,유,경기도,서울특별시,0,24,0,0,3,0
4,20240206,KMHMH81VP3U184335,승용,중형,NaN,강원특별자치도,서울특별시,0,1,0,0,0,0


### [출력] STD_BD_RUN_LMT_CURSTT

In [ ]:
# expdf = STD_BD_RUN_LMT_CURSTT
# table_nm = 'STD_BD_RUN_LMT_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20240206'

In [ ]:
## 2.9s
# STD_BD_RUN_LMT_CURSTT.to_csv(os.path.join(df3_fold, f'STD_BD_RUN_LMT_CURSTT({today_date}).csv'), index=False)

In [ ]:
### !!! 수정 끝(2023.09.06)

In [ ]:
## 계절제별 적발건수
season_tot = lmt1[[x + '_일평균' for x in limit_season_rename_dict.values()]].sum().reset_index()
season_tot = season_tot.rename(columns={'index':'계절제차수', 0:'일평균적발건수'})
season_tot['계절제차수'] = season_tot['계절제차수'].str.replace('계절제_', '').str.replace('_일평균', '')
today_date = datetime.today().strftime("%Y%m%d")
season_tot['테이블생성일자'] = today_date
cdict = {
    '계절제차수':'SEASON_ORD', 
    '일평균적발건수':'DY_AVRG_CRDN_NOCS', 
    '테이블생성일자':'LOAD_DT', 
    }
STD_BD_SEASON_DY_AVRG_CRDN_NOCS = season_tot.rename(columns=cdict)

In [ ]:
STD_BD_SEASON_DY_AVRG_CRDN_NOCS.columns

Index(['SEASON_ORD', 'DY_AVRG_CRDN_NOCS', 'LOAD_DT'], dtype='object')

In [ ]:
STD_BD_SEASON_DY_AVRG_CRDN_NOCS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SEASON_ORD         5 non-null      object 
 1   DY_AVRG_CRDN_NOCS  5 non-null      float64
 2   LOAD_DT            5 non-null      object 
dtypes: float64(1), object(2)
memory usage: 252.0+ bytes


### [출력] STD_BD_SEASON_DY_AVRG_CRDN_NOCS

In [ ]:
# expdf = STD_BD_SEASON_DY_AVRG_CRDN_NOCS
# table_nm = 'STD_BD_SEASON_DY_AVRG_CRDN_NOCS'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [ ]:
us_total.shape

(3178626, 16)

In [ ]:
## 지역별 상시운행제한 단속 현황
us_total2 = us_total.merge(limit_alw, on='차대번호', how='left')
us_total2.shape

(3178626, 17)

In [ ]:
us_total2.columns

Index(['번호', '차대번호', '적발유형', '단속지역코드', '등록시도코드', '등록시군구코드', '적발건수', '적발년월',
       '법정동코드', '등록시도', '등록시군구', '단속시도코드', '단속시군구코드', '단속시도', '단속시군구', '단속지역',
       '상시'],
      dtype='object')

In [ ]:
us_total2['등록시도'].unique()

array(['경기도', '서울특별시', '인천광역시', '전라북도', '강원특별자치도', '경상남도', '대전광역시',
       '경상북도', '부산광역시', '충청남도', '대구광역시', '전라남도', '충청북도', '광주광역시', '울산광역시',
       '세종특별자치시', '제주특별자치도'], dtype=object)

In [ ]:
us_total2.loc[(us_total2['등록시도'] == '서울특별시') | (us_total2['등록시도'] == '경기도') | (us_total2['등록시도'] == '인천광역시'), '등록지역'] = '수도권'
us_total2['등록지역'] = us_total2['등록지역'].fillna('수도권외')

In [ ]:
us_total2['등록지역'].unique()

array(['수도권', '수도권외'], dtype=object)

In [ ]:
us_total2['적발년월'].isnull().sum()

2729903

In [ ]:
us_total2['적발년월'] = us_total2['적발년월'].astype('str')
us_total2['적발년월_년'] = us_total2['적발년월'].str[:4]

In [ ]:
us_total2['적발년월_년'].isnull().sum()

0

In [ ]:
us_total2 = us_total2.sort_values('적발년월_년', ascending=True).drop_duplicates('차대번호').reset_index(drop=True)
us_total2.shape

(86834, 19)

In [ ]:
## 14.2s
### !!! 수정 시작(2023.11.03)
## 지역별 상시운행제한 단속 현황
us_total2 = us_total.merge(limit_alw, on='차대번호', how='left')
us_total2.loc[(us_total2['등록시도'] == '서울특별시') | (us_total2['등록시도'] == '경기도') | (us_total2['등록시도'] == '인천광역시'), '등록지역'] = '수도권'
us_total2['등록지역'] = us_total2['등록지역'].fillna('수도권외')
us_total2['적발년월'] = us_total2['적발년월'].astype('str')
us_total2['적발년월_년'] = us_total2['적발년월'].str[:4]
us_total2 = us_total2.sort_values('적발년월_년', ascending=True).drop_duplicates('차대번호').reset_index(drop=True)
us_total2 = us_total2.drop(['적발건수'], axis=1)
us_total2 = us_total2.rename(columns={'적발년월_년':'적발년도', '상시':'적발건수'})
### !!! 수정 끝(2023.11.03)

In [ ]:
us_total2['적발건수'].dtype

dtype('float64')

In [ ]:
us_total2.columns

Index(['번호', '차대번호', '적발유형', '단속지역코드', '등록시도코드', '등록시군구코드', '적발년월', '법정동코드',
       '등록시도', '등록시군구', '단속시도코드', '단속시군구코드', '단속시도', '단속시군구', '단속지역', '적발건수',
       '등록지역', '적발년도'],
      dtype='object')

In [ ]:
us_total2['등록지역'].unique()

array(['수도권', '수도권외'], dtype=object)

In [ ]:
us_total2['등록시도'].unique()

array(['경기도', '서울특별시', '대구광역시', '울산광역시', '부산광역시', '충청북도', '인천광역시', '경상남도',
       '전라북도', '경상북도', '충청남도', '광주광역시', '대전광역시', '전라남도', '강원특별자치도',
       '제주특별자치도', '세종특별자치시'], dtype=object)

In [ ]:
orditm = us_total2.loc[(us_total2['적발건수'] > 0) & (us_total2['적발년도'].isnull() == False), [
    '차대번호',
    '적발년도',
    '적발건수',
    '등록지역', # !!! 수정(2023.11.03)
    '등록시도', # !!! 수정(2023.11.03)
]]

today_date = datetime.today().strftime("%Y%m%d")
orditm['테이블생성일자'] = today_date
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '차대번호':'VIN', 
    '적발년도':'DSCL_YR', 
    '적발건수':'DSCL_NOCS', 
    '등록지역':'RGN',
    '등록시도':'CTPV', 
}
STD_BD_ORDITM_DSCL_CURSTT = orditm.rename(columns=cdict)

In [ ]:
STD_BD_ORDITM_DSCL_CURSTT.columns

Index(['VIN', 'DSCL_YR', 'DSCL_NOCS', 'RGN', 'CTPV', 'LOAD_DT'], dtype='object')

In [ ]:
STD_BD_ORDITM_DSCL_CURSTT['CTPV'].unique()

array(['경기도', '서울특별시', '대구광역시', '울산광역시', '부산광역시', '충청북도', '인천광역시', '경상남도',
       '전라북도', '경상북도', '충청남도', '광주광역시', '대전광역시', '전라남도', '강원특별자치도',
       '제주특별자치도', '세종특별자치시'], dtype=object)

### [출력] STD_BD_ORDITM_DSCL_CURSTT

In [ ]:
# expdf = STD_BD_ORDITM_DSCL_CURSTT
# table_nm = 'STD_BD_ORDITM_DSCL_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

### !!! 수정 (2023.11.27)

In [ ]:
ust = us_total2.loc[(us_total2['적발건수'] > 0) & (us_total2['적발년도'].isnull() == False)].reset_index(drop=True)
ust.shape

(84653, 18)

In [ ]:
us_total2.loc[us_total2['차대번호'].str.contains(' '), '차대번호'].shape

(55,)

In [ ]:
us_total2.loc[us_total2['차대번호'].str.contains(' '), '차대번호'].head()

2280     290043301816     
14164    290234202899     
17579    450134313666     
22483    419243396001     
30167    280142207551     
Name: 차대번호, dtype: object

In [ ]:
ust.loc[ust['차대번호'].str.contains('.', regex=False), '차대번호'].shape

(0,)

In [ ]:
ust.columns

Index(['번호', '차대번호', '적발유형', '단속지역코드', '등록시도코드', '등록시군구코드', '적발년월', '법정동코드',
       '등록시도', '등록시군구', '단속시도코드', '단속시군구코드', '단속시도', '단속시군구', '단속지역', '적발건수',
       '등록지역', '적발년도'],
      dtype='object')

In [ ]:
len(set(us_total['차대번호'].to_list()))

86834

In [ ]:
len(set(usdisr['차대번호'].to_list()))

86834

In [ ]:
len(set(usdisr['차대번호'].to_list()) - set(carr['차대번호'].to_list()))

1872

In [ ]:
len(set(us_total['차대번호'].to_list()) - set(carr['차대번호'].to_list()))

1872

In [ ]:
len(set(ust['차대번호'].to_list()) - set(carr['차대번호'].to_list()))

1659

In [ ]:
ust['적발년월_월'] = ust['적발년월'].str[4:6]

### !!! 수정 (2023.11.27)

In [ ]:
us_total2.columns

Index(['번호', '차대번호', '적발유형', '단속지역코드', '등록시도코드', '등록시군구코드', '적발년월', '법정동코드',
       '등록시도', '등록시군구', '단속시도코드', '단속시군구코드', '단속시도', '단속시군구', '단속지역', '적발건수',
       '등록지역', '적발년도'],
      dtype='object')

In [ ]:
us_total2.shape

(86834, 18)

In [ ]:
carr.shape

(2314308, 10)

In [ ]:
bd32_dat_orditm01 = ust.merge(carr[['차대번호', '제원관리번호', '차종']], on='차대번호', how='left')
bd32_dat_orditm01.shape

(84653, 21)

In [ ]:
bd32_dat_orditm01['제원관리번호'].isnull().sum()

1659

In [ ]:
bd32_dat_orditm01['제원관리번호'].notnull().sum()

82994

In [ ]:
bd32_dat_orditm02 = bd32_dat_orditm01.merge(srcr[['제원관리번호', '차종유형']], on='제원관리번호', how='left')
bd32_dat_orditm02.shape

(85063, 22)

In [ ]:
bd32_dat_orditm02.columns

Index(['번호', '차대번호', '적발유형', '단속지역코드', '등록시도코드', '등록시군구코드', '적발년월', '법정동코드',
       '등록시도', '등록시군구', '단속시도코드', '단속시군구코드', '단속시도', '단속시군구', '단속지역', '적발건수',
       '등록지역', '적발년도', '적발년월_월', '제원관리번호', '차종', '차종유형'],
      dtype='object')

In [ ]:
bd32_dat_orditm = bd32_dat_orditm02.groupby(['적발년도', '적발년월_월', '등록지역', '등록시도', '차종', '차종유형'], dropna=False).agg({'적발건수':'sum', '차대번호':'count'}).reset_index()
bd32_dat_orditm = bd32_dat_orditm.rename(columns={'차대번호':'차량대수'})
bd32_dat_orditm

,적발년도,적발년월_월,등록지역,등록시도,차종,차종유형,적발건수,차량대수
0,2019,01,수도권,경기도,승용,대형,9.0,3
1,2019,01,수도권,경기도,승용,중형,26.0,2
2,2019,01,수도권,경기도,승합,중형,20.0,2
3,2019,01,수도권,경기도,특수,중형,5.0,1
4,2019,01,수도권,경기도,화물,대형,1.0,1
...,...,...,...,...,...,...,...,...
1825,2023,09,수도권,인천광역시,승용,중형,29.0,5
1826,2023,09,수도권,인천광역시,승합,대형,1.0,1
1827,2023,09,수도권,인천광역시,화물,대형,21.0,5
1828,2023,09,수도권,인천광역시,화물,소형,65.0,6


In [ ]:
bd32_dat_orditm.loc[bd32_dat_orditm['적발년월_월'].isnull()]

,적발년도,적발년월_월,등록지역,등록시도,차종,차종유형,적발건수,차량대수


In [ ]:
bd32_dat_orditm['차량대수'].sum()

85063

In [ ]:
today_date

'20240206'

In [ ]:
bd32_dat_orditm['테이블생성일자'] = today_date

In [ ]:
cdict = {
    '적발년도':'DSCL_Y', 
    '적발년월_월':'DSCL_MM', 
    '등록지역':'REG_RGN', 
    '등록시도':'REG_CTPV', 
    '차종':'VHCTY_CD', 
    '차종유형':'VHCTY_TY',    
    '적발건수':'DSCL_NOCS', 
    '차량대수':'VHCL_MKCNT', 
    '테이블생성일자':'LOAD_DT', 
}
STD_BD_DAT_ORDITM_RUN_LMT_CURSTT = bd32_dat_orditm.rename(columns=cdict)
STD_BD_DAT_ORDITM_RUN_LMT_CURSTT.columns

Index(['DSCL_Y', 'DSCL_MM', 'REG_RGN', 'REG_CTPV', 'VHCTY_CD', 'VHCTY_TY',
       'DSCL_NOCS', 'VHCL_MKCNT', 'LOAD_DT'],
      dtype='object')

### [출력] STD_BD_DAT_ORDITM_RUN_LMT_CURSTT

In [ ]:
# expdf = STD_BD_DAT_ORDITM_RUN_LMT_CURSTT
# table_nm = 'STD_BD_DAT_ORDITM_RUN_LMT_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

### !!! 수정 끝(2023.11.27)

In [ ]:
## 적발지역별 계절제 단속 현황 출력
is_season2 = is_season.sort_values(['적발시도', '적발시군구']).drop_duplicates(['차대번호', '적발시도']).reset_index(drop=True) # !!! 수정(2023.08.10)

In [ ]:
is_lmt = is_season2.merge(limit_season, on='차대번호', how='left')
is_lmt.loc[(is_lmt['적발시도'] == '서울특별시') | (is_lmt['적발시도'] == '경기도') | (is_lmt['적발시도'] == '인천광역시'), '적발지역'] = '수도권'
is_lmt['적발지역'] = is_lmt['적발지역'].fillna('수도권외')

In [ ]:
is_lmt['등록시도'].unique()

array(['경기도', '서울특별시', '강원특별자치도', '인천광역시', '전라북도', '전라남도', '광주광역시',
       '경상북도', '대전광역시', '경상남도', '부산광역시', '충청남도', '세종특별자치시', '충청북도',
       '대구광역시', '울산광역시', '제주특별자치도'], dtype=object)

In [ ]:
is_lmt['적발시도코드'] = is_lmt['적발지역코드'].str[:2] # !!! 수정
is_lmt['적발시도코드'].head()

0    41
1    41
2    41
3    41
4    41
Name: 적발시도코드, dtype: object

In [ ]:
is_lmt2 = is_lmt[[
    '적발지역',
    '적발시도',
    '적발시도코드',  # !!! 수정
    '등록시도',  # !!! 수정
    '차대번호', 
    '계절제_1차', 
    '계절제_2차', 
    '계절제_3차', 
    '계절제_4차', 
]]

In [ ]:
dfm = df.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)

In [ ]:
slimit = is_lmt2.merge(dfm[['차대번호', '차종', '차종유형']], on='차대번호', how='left')  # !!! 수정
today_date = datetime.today().strftime("%Y%m%d")

In [ ]:
# 시도명 2글자로 수정
slimit['적발시도'] = slimit['적발시도'].map({'경기도':'경기', '대구광역시':'대구', '부산광역시':'부산', '서울특별시':'서울', '인천광역시':'인천'})

In [ ]:
# 건수 0 -> nan
slimit.loc[slimit['계절제_1차'] == 0, '계절제_1차'] = np.nan
slimit.loc[slimit['계절제_2차'] == 0, '계절제_2차'] = np.nan
slimit.loc[slimit['계절제_3차'] == 0, '계절제_3차'] = np.nan
slimit.loc[slimit['계절제_4차'] == 0, '계절제_4차'] = np.nan

In [ ]:
slimit['테이블생성일자'] = today_date
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '적발지역':'DSCL_RGN', 
    '적발시도':'DSCL_CTPV', 
    '적발시도코드':'DSCL_CTPV_CD', # !!! 수정
    '등록시도':'REG_CTPV', # !!! 수정
    '차대번호':'VIN', 
    '계절제_1차':'SEASON_1ODR_CRDN_NOCS', 
    '계절제_2차':'SEASON_2ODR_CRDN_NOCS', 
    '계절제_3차':'SEASON_3ODR_CRDN_NOCS', 
    '계절제_4차':'SEASON_4ODR_CRDN_NOCS', 
    '차종':'VHCTY_CD',
    '차종유형':'VHCTY_TY', 
}
STD_BD_SEASON_DSCL_RGN_CURSTT = slimit.rename(columns=cdict)

In [ ]:
STD_BD_SEASON_DSCL_RGN_CURSTT.columns

Index(['DSCL_RGN', 'DSCL_CTPV', 'DSCL_CTPV_CD', 'REG_CTPV', 'VIN',
       'SEASON_1ODR_CRDN_NOCS', 'SEASON_2ODR_CRDN_NOCS',
       'SEASON_3ODR_CRDN_NOCS', 'SEASON_4ODR_CRDN_NOCS', 'VHCTY_CD',
       'VHCTY_TY', 'LOAD_DT'],
      dtype='object')

In [ ]:
STD_BD_SEASON_DSCL_RGN_CURSTT[['DSCL_RGN', 'DSCL_CTPV', 'DSCL_CTPV_CD']].isnull().sum()

DSCL_RGN        0
DSCL_CTPV       0
DSCL_CTPV_CD    0
dtype: int64

### [출력] STD_BD_SEASON_DSCL_RGN_CURSTT

In [ ]:
# expdf = STD_BD_SEASON_DSCL_RGN_CURSTT
# table_nm = 'STD_BD_SEASON_DSCL_RGN_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [ ]:
## 14.5s
## 5등급 저감사업
ce = carr.merge(elpm, on='차대번호', how='left')
dfe = ce.merge(attr, on='차대번호', how='left')

In [ ]:
## 13.3s
dfe['최초등록일자'] = dfe['최초등록일자'].astype('str')
dfe['최초등록일자_년'] = dfe['최초등록일자'].str[:4]
dfe['최초등록일자_월'] = dfe['최초등록일자'].str[4:6]
dfe['최초등록일자_일'] = dfe['최초등록일자'].str[6:8]
dfe['말소일자_년'] = dfe['말소일자'].astype('str').str[:4]
dfe['말소일자_월'] = dfe['말소일자'].astype('str').str[4:6]
dfe['말소일자_일'] = dfe['말소일자'].astype('str').str[6:8]

In [ ]:
errc['변경일자'] = errc['변경일자'].astype('str')
errc['변경일자_년'] = errc['변경일자'].str[:4]
errc['변경일자_월'] = errc['변경일자'].str[4:6]
errc['변경일자_일'] = errc['변경일자'].str[6:8]

In [ ]:
ere = errc.merge(elpm, on='차대번호', how='left')
erea = ere.merge(attr, on='차대번호', how='left')

In [ ]:
# 연도 설정
# year = '2022'
year = today_date[:4]

dfe['연도'] = year

In [ ]:
def knd1(x):
    if '1종' in x.unique():
        return x.value_counts()['1종']
    else:
        return 0
def knd2(x):
    if '1종+SCR' in x.unique():
        return x.value_counts()['1종+SCR']
    else:
        return 0

In [ ]:
## 올해 차량 대수
grp1 = dfe[dfe['차량말소YN'] == 'N'].groupby(['연도']).agg({'차대번호':'count', '저감장치구분':[knd1, knd2]}).reset_index()
grp1.columns = ['연도', '차량대수', '저감장치(1종)', '저감장치(1종+SCR)']

In [ ]:
## 연도별 등록대수
grp2 = dfe[dfe['차량말소YN'] == 'N'].groupby(['최초등록일자_년']).agg({'차대번호':'count', '저감장치구분':[knd1, knd2]}).reset_index()
grp2.columns = ['연도', '등록대수', '등록저감장치(1종)', '등록저감장치(1종+SCR)']

In [ ]:
## 연도별 말소대수
grp3 = erea.groupby('변경일자_년').agg({'차대번호':'count', '저감장치구분':[knd1, knd2]}).reset_index()
grp3.columns = ['연도', '말소대수', '말소저감장치(1종)', '말소저감장치(1종+SCR)']

In [ ]:
## 연도별 조기폐차 대수
grp4 = dfe.groupby('말소일자_년').agg({'조기폐차최종승인YN':'count'}).reset_index()
grp4 = grp4.rename(columns={'말소일자_년':'연도', '조기폐차최종승인YN':'조기폐차'})

In [ ]:
## 차량 통계 기본 데이터셋
yr_list = []
for yr in range(2019, int(year) + 1):
    yr_list.append(str(yr))
base = pd.DataFrame({'연도':yr_list})

In [ ]:
base1 = base.merge(grp1, on='연도', how='left')
base2 = base1.merge(grp2, on='연도', how='left')
base3 = base2.merge(grp3, on='연도', how='left')
base4 = base3.merge(grp4, on='연도', how='left')

In [ ]:
base4[['차량대수', '조기폐차', '저감장치(1종)', '저감장치(1종+SCR)', '등록대수', '등록저감장치(1종)', '등록저감장치(1종+SCR)', '말소대수', '말소저감장치(1종)', '말소저감장치(1종+SCR)']] = base4[['차량대수', '조기폐차', '저감장치(1종)', '저감장치(1종+SCR)', '등록대수', '등록저감장치(1종)', '등록저감장치(1종+SCR)', '말소대수', '말소저감장치(1종)', '말소저감장치(1종+SCR)']].fillna(0)

In [ ]:
n = len(base4['연도'].unique())
for i in range(base4.shape[0] // n):
    for j in range(2, n+1):
        base4.loc[(i+1)*n - j, '차량대수'] = base4.loc[(i+1)*n - (j-1), '차량대수'] + base4.loc[(i+1)*n - (j-1), '말소대수'] - base4.loc[(i+1)*n - (j-1), '등록대수']
        base4.loc[(i+1)*n - j, '저감장치(1종)'] = base4.loc[(i+1)*n - (j-1), '저감장치(1종)'] + base4.loc[(i+1)*n - (j-1), '말소저감장치(1종)'] - base4.loc[(i+1)*n - (j-1), '등록저감장치(1종)']
        base4.loc[(i+1)*n - j, '저감장치(1종+SCR)'] = base4.loc[(i+1)*n - (j-1), '저감장치(1종+SCR)'] + base4.loc[(i+1)*n - (j-1), '말소저감장치(1종+SCR)'] - base4.loc[(i+1)*n - (j-1), '등록저감장치(1종+SCR)']

In [ ]:
base5 = base4[['연도', '차량대수', '조기폐차', '저감장치(1종)', '저감장치(1종+SCR)']]
base5['감소대수'] = base5['차량대수'].shift() - base5['차량대수']
base5['자연감소'] = base5['감소대수'] - base5['조기폐차']
base5['미장착'] = base5['차량대수'] - base5['저감장치(1종)'] - base5['저감장치(1종+SCR)']

C:\Users\kbjung\AppData\Local\Temp\ipykernel_17964\4145734907.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base5['감소대수'] = base5['차량대수'].shift() - base5['차량대수']
C:\Users\kbjung\AppData\Local\Temp\ipykernel_17964\4145734907.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base5['자연감소'] = base5['감소대수'] - base5['조기폐차']
C:\Users\kbjung\AppData\Local\Temp\ipykernel_17964\4145734907.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [ ]:
base5['테이블생성일자'] = today_date
base5 = base5[[
    '연도',
    '차량대수',
    '자연감소',
    '조기폐차',
    '저감장치(1종)',
    '저감장치(1종+SCR)',
    '미장착',
    '테이블생성일자'
    ]]
cdict = {
    '연도':'YR', 
    '차량대수':'VHCL_MKCNT', 
    '자연감소':'NTRL_DCLN', 
    '조기폐차':'ELPDSRC', 
    '저감장치(1종)':'RDCDVC_1KND', 
    '저감장치(1종+SCR)':'RDCDVC_1KND_SCR', 
    '미장착':'UNMNTNG', 
    '테이블생성일자':'LOAD_DT', 
}
STD_BD_DAT_GRD5_REDUC_BIZ = base5.rename(columns=cdict)

C:\Users\kbjung\AppData\Local\Temp\ipykernel_17964\2988678351.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base5['테이블생성일자'] = today_date


In [ ]:
STD_BD_DAT_GRD5_REDUC_BIZ.columns

Index(['YR', 'VHCL_MKCNT', 'NTRL_DCLN', 'ELPDSRC', 'RDCDVC_1KND',
       'RDCDVC_1KND_SCR', 'UNMNTNG', 'LOAD_DT'],
      dtype='object')

In [ ]:
STD_BD_DAT_GRD5_REDUC_BIZ

,YR,VHCL_MKCNT,NTRL_DCLN,ELPDSRC,RDCDVC_1KND,RDCDVC_1KND_SCR,UNMNTNG,LOAD_DT
0,2019,1725913.0,NaN,79951,351869.0,2827.0,1371217.0,20240206
1,2020,1500670.0,127751.0,97492,344444.0,2785.0,1153441.0,20240206
2,2021,1181098.0,206777.0,112795,325447.0,2668.0,852983.0,20240206
3,2022,1029311.0,62578.0,89209,298503.0,2533.0,728275.0,20240206
4,2023,900911.0,72828.0,55572,265987.0,2355.0,632569.0,20240206
5,2024,900911.0,-33.0,33,265987.0,2355.0,632569.0,20240206


### [출력] STD_BD_DAT_GRD5_REDUC_BIZ

In [ ]:
# expdf = STD_BD_DAT_GRD5_REDUC_BIZ
# table_nm = 'STD_BD_DAT_GRD5_REDUC_BIZ'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [ ]:
## 운행제한현황
run_lmt1 = lmt1.copy()
run_lmt1.loc[run_lmt1['DPF_YN'] == '무', '저감장치미장착'] = '미장착'

In [ ]:
total_grp_lmt = pd.DataFrame()
for one in limit_season_rename_dict.values():
    temp1 = run_lmt1.loc[run_lmt1[one] > 0, ['차대번호', '지역', '시도', '차종', '차종유형', '저감장치미장착'] + [one]]
    temp2 = run_lmt1.loc[run_lmt1[one] > 1, ['차대번호', '지역', '시도', '차종', '차종유형'] + [one]]
    if temp1.shape[0] > 0 and temp2.shape[0] > 0:
        grp1 = temp1.groupby(['지역', '시도', '차종', '차종유형']).agg({'차대번호':'count', '저감장치미장착':'count'}).reset_index()
        grp1 = grp1.rename(columns={'차대번호':'적발차량대수', '저감장치미장착':'저공해미조치'})
        grp2 = temp2.groupby(['지역', '시도', '차종', '차종유형'])['차대번호'].count().reset_index()
        grp2 = grp2.rename(columns={'차대번호':'중복적발대수'})
        grp = grp1.merge(grp2, on=['지역', '시도', '차종', '차종유형'], how='left')
        grp['계절관리제'] = one
        total_grp_lmt = pd.concat([total_grp_lmt, grp], ignore_index=True)
    else:
        pass

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
total_grp_lmt['테이블생성일자'] = today_date
total_grp_lmt = total_grp_lmt[[
    '계절관리제', 
    '지역', 
    '시도', 
    '차종', 
    '차종유형', 
    '적발차량대수', 
    '저공해미조치', 
    '중복적발대수', 
    '테이블생성일자', 
]]
cdict = {
    '계절관리제':'SEASON', 
    '지역':'RGN', 
    '시도':'CTPV', 
    '차종':'VHCTY_CD', 
    '차종유형':'VHCTY_TY', 
    '적발차량대수':'DSCL_VHCL_MKCNT', 
    '저공해미조치':'UNLEM', 
    '중복적발대수':'DUP_DSCL_MKCNT', 
    '테이블생성일자':'LOAD_DT', 
}
STD_BD_DAT_RUN_LMT_CURSTT = total_grp_lmt.rename(columns=cdict)

In [ ]:
STD_BD_DAT_RUN_LMT_CURSTT.columns

Index(['SEASON', 'RGN', 'CTPV', 'VHCTY_CD', 'VHCTY_TY', 'DSCL_VHCL_MKCNT',
       'UNLEM', 'DUP_DSCL_MKCNT', 'LOAD_DT'],
      dtype='object')

### [출력] STD_BD_DAT_RUN_LMT_CURSTT

In [ ]:
# expdf = STD_BD_DAT_RUN_LMT_CURSTT
# table_nm = 'STD_BD_DAT_RUN_LMT_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

## 3-2 code end ##################################################################

## ---------- 3-3 start ----------

In [ ]:
# 2024.02.13 수정 (데이터 테이블 변경 STD_CEG_CAR_MIG => VSYSZ.STD_CEG_GRD_LST)
## 48.9s
## 등록정보(VSYSZ.STD_CEG_GRD_LST) 4, 5등급만
car = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, VHCL_ERSR_YN, MANP_MNG_NO, EXHST_GAS_GRD_CD, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHRNO FROM VSYSZ.STD_CEG_GRD_LST WHERE EXHST_GAS_GRD_CD = 'A0504' OR EXHST_GAS_GRD_CD = 'A05T4' OR EXHST_GAS_GRD_CD = 'A0505' OR EXHST_GAS_GRD_CD = 'A05T5';")
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자',
    'VHRNO':'차량번호',
}
carr = car.rename(columns=car_ch_col)

## 중복 차대번호 제거
carr['최초등록일자'] = pd.to_numeric(carr['최초등록일자'], errors='coerce')
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=
True)

c:\Users\kbjung\anaconda3\Lib\site-packages\pyexasol\callback.py:42: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(pipe, skip_blank_lines=False, **kwargs)


In [ ]:
carr['배출가스등급'].unique()

array(['A0504', 'A0505'], dtype=object)

In [ ]:
## 12.4s
## 배출가스등급 코드 변환
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
carr['배출가스등급'] = carr['배출가스등급'].replace(grd_dict)

## 차종 코드 변환
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)

## 용도 코드 변환
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)

## 등록정보 말소 제거
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)

In [ ]:
## 14.0s
## 등록&제원 병합
cs = carm.merge(srcr, on='제원관리번호', how='left')

In [ ]:
## 2m 0.5s
## 등록&제원&정기&정밀 병합
csi = cs.merge(insm, on='차대번호', how='left')

In [ ]:
## 11.5s
csi['법정동코드'] = csi['법정동코드'].astype('str')
csi['법정동코드'] = csi['법정동코드'].str[:5] + '00000'
csi['법정동코드'] = pd.to_numeric(csi['법정동코드'])

In [ ]:
## 등록&제원&정기&정밀&법정동 병합
csic = csi.merge(coder, on='법정동코드', how='left')

In [ ]:
csic['시도'].isnull().sum()

28672

In [ ]:
# # 주소 수정
# csic.loc[csic['법정동코드'] == 5172035031, ['시도', '시군구']] = ['강원특별자치도', '홍천군'] # 강원특별자치도 홍천군
# csic.loc[csic['법정동코드'] == 5180031023, ['시도', '시군구']] = ['강원특별자치도', '양구군'] # 
# csic.loc[csic['법정동코드'] == 5180031031, ['시도', '시군구']] = ['강원특별자치도', '양구군'] # 
# csic.loc[csic['법정동코드'] == 5172035030, ['시도', '시군구']] = ['강원특별자치도', '홍천군'] # 
# csic.loc[csic['법정동코드'] == 5180031028, ['시도', '시군구']] = ['강원특별자치도', '양구군'] # 

# csic.loc[csic['법정동코드'] == 5172035021, ['시도', '시군구']] = ['강원특별자치도', '홍천군'] # 
# csic.loc[csic['법정동코드'] == 5180031025, ['시도', '시군구']] = ['강원특별자치도', '양구군'] # 
# csic.loc[csic['법정동코드'] == 4165052000, ['시도', '시군구']] = ['경기도', '포천시'] # 경기도 포천시 선단동
# csic.loc[csic['법정동코드'] == 5172035023, ['시도', '시군구']] = ['강원특별자치도', '홍천군'] # 
# csic.loc[csic['법정동코드'] == 5180031027, ['시도', '시군구']] = ['강원특별자치도', '양구군'] # 

# csic.loc[csic['법정동코드'] == 5172035024, ['시도', '시군구']] = ['강원특별자치도', '홍천군'] # 
# csic.loc[csic['법정동코드'] == 5175037022, ['시도', '시군구']] = ['강원특별자치도', '영월군'] # 
# csic.loc[csic['법정동코드'] == 4163055000, ['시도', '시군구']] = ['경기도', '양주시'] # 경기도 양주시 회천3동
# csic.loc[csic['법정동코드'] == 5180031033, ['시도', '시군구']] = ['강원특별자치도', '양구군'] # 경기도 양주시 회천3동

In [ ]:
## 12.0s
## 등록&제원&정기&정밀&법정동&저감 병합
attr = attr.drop_duplicates('차대번호').reset_index(drop=True)
csica = csic.merge(attr, on='차대번호', how='left')

In [ ]:
## 운행제한 건수
limit['운행제한건수'] = limit[['계절제_1차', '계절제_2차', '계절제_3차', '계절제_4차', '비상시', '상시']].sum(axis=1)

In [ ]:
## 11.5s
## 등록&제원&정기&정밀&법정동&저감&운행제한건수 병합
df = csica.merge(limit[['차대번호', '운행제한건수']], on='차대번호', how='left')

In [ ]:
## 14.3s
## 저감장치 부착 여부 result로 수정
## 4등급 result 파일 참고하여 DPF유무 수정
rdf = df.copy()
rs = rs.drop_duplicates('차대번호').reset_index(drop=True)
rdf1 = rdf.merge(rs, on='차대번호', how='left')
rdf1.loc[(rdf1['DPF_YN'] == '유') | (rdf1['DPF유무_수정'] == '유'), 'DPF_YN'] = '유'
rdf1.loc[(rdf1['DPF유무_수정'] == '무'), 'DPF_YN'] = '무'
rdf1.loc[(rdf1['DPF유무_수정'] == '확인불가'), 'DPF_YN'] = '확인불가'
df = rdf1.drop('DPF유무_수정', axis=1)

In [ ]:
# 전처리
## 일일평균주행거리 계산
df['최초등록일자'] = pd.to_datetime(df['최초등록일자'], format="%Y%m%d", errors='coerce')
df['검사일자'] = pd.to_datetime(df['검사일자'], format="%Y%m%d", errors='coerce')

In [ ]:
today_date = datetime.today().strftime("%Y-%m-%d")
df['현재날짜'] = today_date
df['현재날짜'] = pd.to_datetime(df['현재날짜'], format='%Y-%m-%d', errors='coerce')
df['최근검사경과일'] = df['현재날짜'] - df['검사일자']
df['최근검사경과일'] = df['최근검사경과일'].astype('str')
df['최근검사경과일'] = df['최근검사경과일'].str.split(' ').str[0]
df['최근검사경과일'] = pd.to_numeric(df['최근검사경과일'], errors='coerce')
df['등록일기준검사일'] = df['검사일자'] - df['최초등록일자']
df['등록일기준검사일'] = df['등록일기준검사일'].astype('str')
df['등록일기준검사일'] = df['등록일기준검사일'].str.split(' ').str[0]
df['등록일기준검사일'] = pd.to_numeric(df['등록일기준검사일'], errors='coerce')
df['일일평균주행거리'] = df['주행거리'] / df['등록일기준검사일']

In [ ]:
## KOSIS 데이터 활용 일일평균주행거리 수정
### 빈 값 kosis로 대체
df1y = df[df['일일평균주행거리'].isnull() == False]
df1n = df[df['일일평균주행거리'].isnull() == True]

In [ ]:
df1n = df1n.drop('일일평균주행거리', axis=1)
df1nm = df1n.merge(kosisr, on=['시도', '시군구', '차종'], how='left')

In [ ]:
df2y = df1nm[df1nm['일일평균주행거리'].isnull() == False]
df2n = df1nm[df1nm['일일평균주행거리'].isnull() == True]
df2n = df2n.drop('일일평균주행거리', axis=1)
df2nm = df2n.merge(kosisr.drop_duplicates(['시도', '차종'])[['시도', '차종', '일일평균주행거리']], on=['시도', '차종'], how='left')

In [ ]:
df3y = df2nm[df2nm['일일평균주행거리'].isnull() == False]
df3n = df2nm[df2nm['일일평균주행거리'].isnull() == True]

In [ ]:
for ctpv, sgg, cd in df3n.loc[df3n['일일평균주행거리'].isnull() == True, ['시도', '시군구', '차종']].values:
    try:
        df3n.loc[(df3n['일일평균주행거리'].isnull() == True) & (df3n['시도'] == ctpv) & (df3n['시군구'] == sgg), '일일평균주행거리'] = kosisr.loc[(kosisr['시도'] == ctpv) & (kosisr['시군구'] == '소계') & (kosisr['차종'] == '합계'), '일일평균주행거리'].values[0]
    except:
        df3n.loc[(df3n['일일평균주행거리'].isnull() == True) & (df3n['시도'].isnull() == True) & (df3n['시군구'].isnull() == True), '일일평균주행거리'] = kosisr.loc[(kosisr['시도'] == '서울특별시') & (kosisr['시군구'] == '소계') & (kosisr['차종'] == '합계'), '일일평균주행거리'].values[0]

In [ ]:
df2nm = pd.concat([df3y, df3n], ignore_index=True)
df1nm = pd.concat([df2y, df2nm], ignore_index=True)
df = pd.concat([df1y, df1nm], ignore_index=True)

In [ ]:
## 11.6s
## 4, 5등급 분리
g4 = df.loc[df['배출가스등급'] == '4'].reset_index(drop=True)
g5 = df.loc[df['배출가스등급'] == '5'].reset_index(drop=True)

In [ ]:
g4.loc[g4['차량번호'] == '31고7134', '연료']

1286537    경유
Name: 연료, dtype: object

In [ ]:
### 코란KJ 연료 휘발유로 수정
# - 차량번호 : 31고7134
# - 연식 : 1996
# - 연료 : 휘발유
g4.loc[g4['차량번호'] == '31고7134', '연료'] = '휘발유'

In [ ]:
# 전처리
gm4d = g4.loc[g4['연료'] == '경유'].reset_index(drop=True)
gm4r = g4.loc[g4['연료'] != '경유'].reset_index(drop=True)

In [ ]:
### 5등급 저감장치 변환
# - 1종 -> DPF
# - 1종+SCR -> PM-NOx
g5.loc[g5['저감장치구분'] == '1종', '저감장치'] = 'DPF'
g5.loc[g5['저감장치구분'] == '1종+SCR', '저감장치'] = 'PM-NOx'

In [ ]:
## 5등급 경유차 추출(gm5d)
gm5d = g5.loc[g5['연료'] == '경유'].reset_index(drop=True)
gm5r = g5.loc[g5['연료'] != '경유'].reset_index(drop=True)

In [ ]:
## 필수 컬럼 추출
gm4d = gm4d.rename(columns={'차량연식':'연식', 'DPF_YN':'저감장치부착유무'})
gm4d = gm4d[[
    '차대번호', 
    '차량번호', 
    '법정동코드', 
    '시도', 
    '시군구', 
    '연식', 
    '용도', 
    '차종', 
    '차종유형', 
    '저감장치부착유무', 
    '무부하매연측정치1', 
    '일일평균주행거리',
    '최근검사경과일', 
    '운행제한건수', 
    ]]

In [ ]:
gm5d = gm5d.rename(columns={'차량등록번호':'차량번호', '본거지법정동코드':'법정동코드', '차량연식':'연식', 'DPF_YN':'저감장치부착유무'})
gm5d = gm5d[[
    '차대번호', 
    '차량번호', 
    '법정동코드', 
    '시도', 
    '시군구', 
    '연식', 
    '용도', 
    '차종', 
    '차종유형', 
    '저감장치', 
    '저감장치부착유무',
    '무부하매연측정치1', 
    '일일평균주행거리',
    '최근검사경과일', 
    '운행제한건수', 
    ]]

In [ ]:
## 이상값 추출
gm4d['운행제한건수'] = gm4d['운행제한건수'].fillna(0)
gm5d['운행제한건수'] = gm5d['운행제한건수'].fillna(0)
idx4 = set(gm4d.index)
idx5 = set(gm5d.index)

In [ ]:
## A급
# - 최근검사경과일 365*3 = 1095 초과 또는 최근검사경과일 없는 경우
# - 운행제한건수 1이상
idx4a = gm4d.loc[((gm4d['최근검사경과일'] > 1095) | (gm4d['최근검사경과일'].isnull() == True)) & (gm4d['운행제한건수'] >= 1)].index
gm4da = gm4d.loc[idx4a]
gm4da['우선순위'] = 1

In [ ]:
idx5a = gm5d.loc[((gm5d['최근검사경과일'] > 1095) | (gm5d['최근검사경과일'].isnull() == True)) & (gm5d['운행제한건수'] >= 1)].index
gm5da = gm5d.loc[idx5a]
gm5da['우선순위'] = 1

In [ ]:
## B급
# - 최근검사경과일 365*3 = 1095 초과 또는 최근검사경과일 없는 경우
# - 운행제한건수 1미만
idx4b = gm4d.loc[((gm4d['최근검사경과일'] > 1095) | (gm4d['최근검사경과일'].isnull() == True)) & (gm4d['운행제한건수'] < 1)].index
gm4db = gm4d.loc[idx4b]
gm4db['우선순위'] = 2

In [ ]:
idx5b = gm5d.loc[((gm5d['최근검사경과일'] > 1095) | (gm5d['최근검사경과일'].isnull() == True)) & (gm5d['운행제한건수'] < 1)].index
gm5db = gm5d.loc[idx5b]
gm5db['우선순위'] = 2

In [ ]:
## C급
# - 최근검사경과일 365*3 = 1095 이하
idx4c = gm4d.loc[gm4d['최근검사경과일'] <= 1095].index
gm4dc = gm4d.loc[idx4c]
gm4dc['우선순위'] = 3

In [ ]:
idx5c = gm5d.loc[gm5d['최근검사경과일'] <= 1095].index
gm5dc = gm5d.loc[idx5c]
gm5dc['우선순위'] = 3

In [ ]:
# 분석
## 가중치 설정을 위한 상관계수 확인
# - 선별 조건(4가지)
#     - 매연
#     - 일일평균주행거리
#     - 최근검사경과일
#     - 운행제한단속건수
gm4di = gm4dc.copy()
gm5di = gm5dc.copy()

In [ ]:
### 상관계수 계산
#### 4등급 경유 C급
# 선별조건선정
col = ['무부하매연측정치1', '일일평균주행거리', '최근검사경과일', '운행제한건수']
gm4di[col] = gm4di[col].fillna(0)
gm4di_corr = gm4di[col].corr()

In [ ]:
##### 4등급 경유차 선별조건 상관계수
t4 = pd.concat([gm4di_corr, gm4di_corr.sum()], axis=1)
t4 = t4.rename(columns={0:'합계'})
t4 = t4.reset_index()

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
t4['테이블생성일자'] = today_date
chc_col = {
    '테이블생성일자':'LOAD_DT', 
    'index':'LIST', 
    '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
    '일일평균주행거리':'DY_AVRG_DRVNG_DSTNC', 
    '최근검사경과일':'RCNT_INSP_ELPSD_WHL', 
    '운행제한건수':'RUN_LMT_NOCS', 
    '합계':'TOT_CRRLTN_CFFCNT',
}
STD_BD_GRD4_DS_CRRLTN_CFFCNT = t4.rename(columns=chc_col)

In [ ]:
STD_BD_GRD4_DS_CRRLTN_CFFCNT.columns

Index(['LIST', 'NOLOD_SMO_MEVLU1', 'DY_AVRG_DRVNG_DSTNC',
       'RCNT_INSP_ELPSD_WHL', 'RUN_LMT_NOCS', 'TOT_CRRLTN_CFFCNT', 'LOAD_DT'],
      dtype='object')

##### [출력] STD_BD_GRD4_DS_CRRLTN_CFFCNT

In [ ]:
# expdf = STD_BD_GRD4_DS_CRRLTN_CFFCNT
# table_nm = 'STD_BD_GRD4_DS_CRRLTN_CFFCNT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [ ]:
c1, c2, c3, c4 = gm4di_corr.sum()[col]
sc1 = c1 + c2 + c3 + c4 
w1, w2, w3, w4 = c1/sc1, c2/sc1, c3/sc1, c4/sc1
gm4di['선별포인트'] = np.round(w1 * gm4di['무부하매연측정치1'] + w2 * gm4di['일일평균주행거리'] + w3 * gm4di['최근검사경과일'] + w4 * gm4di['운행제한건수'] , 0)

In [ ]:
#### 4등급 경유차 선별포인트 샘플
gm4da['선별포인트'] = np.nan
gm4db['선별포인트'] = np.nan
total4d = pd.concat([gm4da, gm4db, gm4di], ignore_index=True)

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
total4d['테이블생성일자'] = today_date
STD_BD_GRD4_LEM_PRIO_ORD_SELCT_CURSTT = total4d[[
    '테이블생성일자', 
    '차대번호', 
    '차량번호', 
    '법정동코드',
    '시도', 
    '시군구', 
    '연식', 
    '용도', 
    '차종', 
    '차종유형', 
    '우선순위',
    '선별포인트',
    '무부하매연측정치1', 
    '일일평균주행거리',
    '최근검사경과일', 
    '운행제한건수', 
    ]]
chc_col = {
    '테이블생성일자':'LOAD_DT', 
    '차대번호':'VIN', 
    '차량번호':'VHRNO', # 자동차등록번호
    '법정동코드':'STDG_CD', 
    '시도':'CTPV_NM', 
    '시군구':'SGG_NM', 
    '연식':'YRIDNW', 
    '용도':'PURPS_CD2', 
    '차종':'VHCTY_CD', # 차종코드
    '차종유형':'VHCTY_TY', 
    '우선순위':'PRIO_GRD',
    '선별포인트':'SELCT_PNT',
    '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
    '일일평균주행거리':'DY_AVRG_DRVNG_DSTNC',
    '최근검사경과일':'RCNT_INSP_ELPSD_WHL', 
    '운행제한건수':'RUN_LMT_NOCS', 
    '지원비용_백만원':'SPRT_CST',
    '배기량_리터':'DSPLVL',
    '총중량_톤':'TOTL_WGHT',
}
STD_BD_GRD4_LEM_PRIO_ORD_SELCT_CURSTT = STD_BD_GRD4_LEM_PRIO_ORD_SELCT_CURSTT.rename(columns=chc_col)

In [ ]:
STD_BD_GRD4_LEM_PRIO_ORD_SELCT_CURSTT.columns

c:\Users\kbjung\anaconda3\Lib\site-packages\IPython\core\displayhook.py:281: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


Index(['LOAD_DT', 'VIN', 'VHRNO', 'STDG_CD', 'CTPV_NM', 'SGG_NM', 'YRIDNW',
       'PURPS_CD2', 'VHCTY_CD', 'VHCTY_TY', 'PRIO_GRD', 'SELCT_PNT',
       'NOLOD_SMO_MEVLU1', 'DY_AVRG_DRVNG_DSTNC', 'RCNT_INSP_ELPSD_WHL',
       'RUN_LMT_NOCS'],
      dtype='object')

##### [출력] STD_BD_GRD4_LEM_PRIO_ORD_SELCT_CURSTT

In [ ]:
# expdf = STD_BD_GRD4_LEM_PRIO_ORD_SELCT_CURSTT
# table_nm = 'STD_BD_GRD4_LEM_PRIO_ORD_SELCT_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [ ]:
#### 5등급 경유 C급
###### 조기폐차 선별포인트
# 선별조건선정
col = ['무부하매연측정치1', '일일평균주행거리', '최근검사경과일', '운행제한건수']
# nan 값 0으로 채우기
gm5di[col] = gm5di[col].fillna(0)
gm5di_corr = gm5di[col].corr()

In [ ]:
###### 5등급 경유차 선별조건 상관계수
t5 = pd.concat([gm5di_corr, gm5di_corr.sum()], axis=1)
t5 = t5.rename(columns={0:'합계'})
t5 = t5.reset_index()
today_date = datetime.today().strftime("%Y%m%d")
t5['테이블생성일자'] = today_date

chc_col = {
    '테이블생성일자':'LOAD_DT', 
    'index':'LIST', 
    '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
    '일일평균주행거리':'DY_AVRG_DRVNG_DSTNC', 
    '최근검사경과일':'RCNT_INSP_ELPSD_WHL', 
    '운행제한건수':'RUN_LMT_NOCS', 
    '합계':'TOT_CRRLTN_CFFCNT',
}
STD_BD_GRD5_DS_CRRLTN_CFFCNT = t5.rename(columns=chc_col)

In [ ]:
STD_BD_GRD5_DS_CRRLTN_CFFCNT.columns

Index(['LIST', 'NOLOD_SMO_MEVLU1', 'DY_AVRG_DRVNG_DSTNC',
       'RCNT_INSP_ELPSD_WHL', 'RUN_LMT_NOCS', 'TOT_CRRLTN_CFFCNT', 'LOAD_DT'],
      dtype='object')

##### [출력] STD_BD_GRD5_DS_CRRLTN_CFFCNT

In [ ]:
# expdf = STD_BD_GRD5_DS_CRRLTN_CFFCNT
# table_nm = 'STD_BD_GRD5_DS_CRRLTN_CFFCNT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [ ]:
c1, c2, c3, c4 = gm5di_corr.sum()[col]
sc1 = c1 + c2 + c3 + c4
w1, w2, w3, w4 = c1/sc1, c2/sc1, c3/sc1, c4/sc1
gm5di['선별포인트'] = np.round(w1 * gm5di['무부하매연측정치1'] + w2 * gm5di['일일평균주행거리'] + w3 * gm5di['최근검사경과일'] + w4 * gm5di['운행제한건수'], 0)

In [ ]:
#### 5등급 경유차 선별포인트 샘플

gm5da['선별포인트'] = np.nan
gm5db['선별포인트'] = np.nan

total5d = pd.concat([gm5da, gm5db, gm5di], ignore_index=True)

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
total5d['테이블생성일자'] = today_date
STD_BD_GRD5_LEM_PRIO_ORD_SELCT_CURSTT = total5d[[
    '차대번호',
    '차량번호',
    '법정동코드',
    '시도',
    '시군구',
    '연식',
    '용도',
    '차종',
    '차종유형',
    '저감장치',
    '우선순위',
    '선별포인트',
    '무부하매연측정치1',
    '일일평균주행거리',
    '최근검사경과일',
    '운행제한건수',
    '테이블생성일자', 
]]
chc_col = {
    '테이블생성일자':'LOAD_DT', 
    '차대번호':'VIN', 
    '차량번호':'VHRNO', # 자동차등록번호
    '법정동코드':'STDG_CD', 
    '시도':'CTPV_NM', 
    '시군구':'SGG_NM', 
    '연식':'YRIDNW', # 연식
    '용도':'PURPS_CD2', 
    '차종':'VHCTY_CD', # 차종코드
    '차종유형':'VHCTY_TY', 
    '저감장치':'RDCDVC',
    '우선순위':'PRIO_GRD',
    '선별포인트':'SELCT_PNT',
    '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
    '일일평균주행거리':'DY_AVRG_DRVNG_DSTNC',
    '최근검사경과일':'RCNT_INSP_ELPSD_WHL', 
    '운행제한건수':'RUN_LMT_NOCS', 
}
STD_BD_GRD5_LEM_PRIO_ORD_SELCT_CURSTT = STD_BD_GRD5_LEM_PRIO_ORD_SELCT_CURSTT.rename(columns=chc_col)

In [ ]:
STD_BD_GRD5_LEM_PRIO_ORD_SELCT_CURSTT.columns

Index(['VIN', 'VHRNO', 'STDG_CD', 'CTPV_NM', 'SGG_NM', 'YRIDNW', 'PURPS_CD2',
       'VHCTY_CD', 'VHCTY_TY', 'RDCDVC', 'PRIO_GRD', 'SELCT_PNT',
       'NOLOD_SMO_MEVLU1', 'DY_AVRG_DRVNG_DSTNC', 'RCNT_INSP_ELPSD_WHL',
       'RUN_LMT_NOCS', 'LOAD_DT'],
      dtype='object')

##### [출력] STD_BD_GRD5_LEM_PRIO_ORD_SELCT_CURSTT

In [ ]:
# expdf = STD_BD_GRD5_LEM_PRIO_ORD_SELCT_CURSTT
# table_nm = 'STD_BD_GRD5_LEM_PRIO_ORD_SELCT_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [ ]:
## 저공해조치선별
# lem4d = total4d.merge(coder[['법정동코드', '시도']], on='법정동코드', how='left')
# lem5d = total5d.merge(coder[['법정동코드', '시도']], on='법정동코드', how='left')
lem4d = total4d.copy()
lem5d = total5d.copy()

In [ ]:
lem4d['시도'].isnull().sum(), lem5d['시도'].isnull().sum()

(12044, 9855)

In [ ]:
# lem4d.loc[lem4d['법정동코드'] == 5180031023, '시도'] = '강원특별자치도' # 강원특별자치도 양구군 남면 야촌리
# lem4d.loc[lem4d['법정동코드'] == 5172035030, '시도'] = '강원특별자치도' # 강원특별자치도 홍천군 동면 노천리
# lem4d.loc[lem4d['법정동코드'] == 5172035021, '시도'] = '강원특별자치도' # 강원특별자치도 홍천군 동면 속초리

In [ ]:
# # lem5d.loc[lem5d['법정동코드'] == 4165052000, '시도'] = '경기도' # 경기도 포천시 선단동
# lem5d.loc[lem5d['법정동코드'] == 5180031023, '시도'] = '강원특별자치도' # 강원특별자치도 양구군 남면 야촌리
# lem5d.loc[lem5d['법정동코드'] == 5180031027, '시도'] = '강원특별자치도' # 강원특별자치도 양구군 남면 황강리
# lem5d.loc[lem5d['법정동코드'] == 5180031025, '시도'] = '강원특별자치도' # 강원특별자치도 양구군 남면 청리
# lem5d.loc[lem5d['법정동코드'] == 5180031028, '시도'] = '강원특별자치도' # 강원특별자치도 양구군 남면 창리
# lem5d.loc[lem5d['법정동코드'] == 5172035023, '시도'] = '강원특별자치도' # 강원특별자치도 홍천군 동면 덕치리
# lem5d.loc[lem5d['법정동코드'] == 5180031031, '시도'] = '강원특별자치도' # 강원특별자치도 양구군 남면 죽리
# lem5d.loc[lem5d['법정동코드'] == 5175037022, '시도'] = '강원특별자치도' # 강원특별자치도 홍천군 서면 어유포리
# lem5d.loc[lem5d['법정동코드'] == 5180031033, '시도'] = '강원특별자치도' # 강원특별자치도 양구군 남면 원리

In [ ]:
lem4d['배출가스등급'] = '4'
lem5d['배출가스등급'] = '5'
lem = pd.concat([lem4d, lem5d], ignore_index=True)

In [ ]:
grp1 = lem.groupby(['시도', '배출가스등급', '차종', '차종유형', '우선순위']).agg({'차대번호':'count'}).unstack('우선순위').reset_index()
grp1.columns = ['시도', '배출가스등급', '차종', '차종유형', '1순위(대수)', '2순위(대수)', '3순위(대수)']
grp1[['1순위(대수)', '2순위(대수)', '3순위(대수)']] = grp1[['1순위(대수)', '2순위(대수)', '3순위(대수)']].fillna(0)
grp1['합계'] = grp1.iloc[:, -3:].sum(axis=1)
grp1['1순위(비율)'] = grp1['1순위(대수)'] / grp1['합계']
grp1['2순위(비율)'] = grp1['2순위(대수)'] / grp1['합계']
grp1['3순위(비율)'] = grp1['3순위(대수)'] / grp1['합계']
grp1 = grp1[['시도', '배출가스등급', '차종', '차종유형', '합계', '1순위(대수)', '1순위(비율)', '2순위(대수)', '2순위(비율)', '3순위(대수)', '3순위(비율)']]
grp1 = grp1.rename(columns={'합계':'선별대수'})

In [ ]:
grp2 = lem.groupby(['시도', '배출가스등급', '차종', '차종유형']).agg({'저감장치':'count'}).reset_index()
grp2 = grp2.rename(columns={'저감장치':'저감장치부착대수'})

In [ ]:
grp = grp1.merge(grp2, on=['시도', '배출가스등급', '차종', '차종유형'], how='left')
today_date = datetime.today().strftime("%Y%m%d")
grp['테이블생성일자'] = today_date
cdict = {
    '시도':'CTPV',
    '배출가스등급':'EXHST_GAS_GRD_CD',
    '차종':'VHCTY_CD',
    '차종유형':'VHCTY_TY',
    '선별대수':'SELCT_MKCNT',
    '1순위(대수)':'SENO1_MKCNT',
    '1순위(비율)':'SENO1_RT',
    '2순위(대수)':'SENO2_MKCNT',
    '2순위(비율)':'SENO2_RT',
    '3순위(대수)':'SENO3_MKCNT',
    '3순위(비율)':'SENO3_RT',
    '저감장치부착대수':'RDCDVC_EXTRNS_MKCNT',
    '테이블생성일자':'LOAD_DT', 
}
STD_BD_DAT_LEM_SELCT = grp.rename(columns=cdict)

In [ ]:
STD_BD_DAT_LEM_SELCT.columns

Index(['CTPV', 'EXHST_GAS_GRD_CD', 'VHCTY_CD', 'VHCTY_TY', 'SELCT_MKCNT',
       'SENO1_MKCNT', 'SENO1_RT', 'SENO2_MKCNT', 'SENO2_RT', 'SENO3_MKCNT',
       'SENO3_RT', 'RDCDVC_EXTRNS_MKCNT', 'LOAD_DT'],
      dtype='object')

### [출력] STD_BD_DAT_LEM_SELCT

In [ ]:
# expdf = STD_BD_DAT_LEM_SELCT
# table_nm = 'STD_BD_DAT_LEM_SELCT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

In [ ]:
lem['선별포인트'].describe()

count    173870.000000
mean         74.503204
std          81.285034
min       -2433.000000
25%          40.000000
50%          62.000000
75%          94.000000
max         499.000000
Name: 선별포인트, dtype: float64

In [ ]:
for n in range(0, 300, 50):
    print(n)

0
50
100
150
200
250


### !!! 수정 시작(2023.11.10)

In [ ]:
## 선별포인트현황
for n in range(0, 300, 50):
    lem.loc[(lem['선별포인트'] >= n) & (lem['선별포인트'] < n + 50), '선별포인트구간'] = f'{n} ~ {n+49}'
n += 50
lem.loc[lem['선별포인트'] >= n, '선별포인트구간'] = f'{n} ~ {n+49}'

### !!! 수정 끝(2023.11.10)

In [ ]:
set(lem['선별포인트구간'].unique())

{'0 ~ 49',
 '100 ~ 149',
 '150 ~ 199',
 '200 ~ 249',
 '250 ~ 299',
 '300 ~ 349',
 '50 ~ 99',
 nan}

In [ ]:
stat = lem.groupby(['배출가스등급', '선별포인트구간', '차종', '차종유형']).agg({'차대번호':'count', '무부하매연측정치1':'mean', '일일평균주행거리':'mean', '최근검사경과일':'mean', '운행제한건수':'mean'}).reset_index()
stat = stat.rename(columns={'차대번호':'차량대수', '무부하매연측정치1':'매연측정값'})
today_date = datetime.today().strftime("%Y%m%d")
stat['테이블생성일자'] = today_date
cdict = {
    '배출가스등급':'EXHST_GAS_GRD_CD',
    '선별포인트구간':'SELCT_PNT_RNG',
    '차종':'VHCTY_CD',
    '차종유형':'VHCTY_TY',
    '차량대수':'VHCL_MKCNT',
    '매연측정값':'SMO_MSRMT_VAL',
    '일일평균주행거리':'DY_AVRG_DRVNG_DSTNC',
    '최근검사경과일':'RCNT_INSP_ELPSD_WHL',
    '운행제한건수':'RUN_LMT_NOCS',
    '테이블생성일자':'LOAD_DT', 
}
STD_BD_DAT_SELCT_PNT_CURSTT = stat.rename(columns=cdict)

In [ ]:
STD_BD_DAT_SELCT_PNT_CURSTT.columns

Index(['EXHST_GAS_GRD_CD', 'SELCT_PNT_RNG', 'VHCTY_CD', 'VHCTY_TY',
       'VHCL_MKCNT', 'SMO_MSRMT_VAL', 'DY_AVRG_DRVNG_DSTNC',
       'RCNT_INSP_ELPSD_WHL', 'RUN_LMT_NOCS', 'LOAD_DT'],
      dtype='object')

In [ ]:
STD_BD_DAT_SELCT_PNT_CURSTT.head()

,EXHST_GAS_GRD_CD,SELCT_PNT_RNG,VHCTY_CD,VHCTY_TY,VHCL_MKCNT,SMO_MSRMT_VAL,DY_AVRG_DRVNG_DSTNC,RCNT_INSP_ELPSD_WHL,RUN_LMT_NOCS,LOAD_DT
0,4,0 ~ 49,승용,대형,466,5.263948,43.501729,91.896996,0.0,20240206
1,4,0 ~ 49,승용,소형,49,9.979592,35.204774,92.285714,0.0,20240206
2,4,0 ~ 49,승용,중형,2454,3.814181,34.425328,99.759984,0.0,20240206
3,4,0 ~ 49,승합,대형,51,2.098039,47.966291,95.705882,0.0,20240206
4,4,0 ~ 49,승합,중형,4310,7.224594,40.965912,91.780278,0.0,20240206


### [출력] STD_BD_DAT_SELCT_PNT_CURSTT

In [ ]:
# expdf = STD_BD_DAT_SELCT_PNT_CURSTT
# table_nm = 'STD_BD_DAT_SELCT_PNT_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# print(f'data export : {table_nm}')

## 3-3 code end ##################################################################

In [ ]:
sec = time.time() - start_time
print(str(dt.timedelta(seconds=sec)).split('.')[0])

1:09:55


In [ ]:
print('code end')

code end


# code end ##################################################################